<a href="https://colab.research.google.com/github/varun1724/Pison-GPU-conversion/blob/main/Research_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!rm -rf sample_data/

In [ ]:
!mkdir general

In [ ]:
!mkdir serialSpec

In [ ]:
!mkdir parallelSpec

In [ ]:
!mkdir testing

In [ ]:
!mkdir dataset

In [ ]:
%%writefile /content/general/Bitmap.h

#ifndef BITMAP_H
#define BITMAP_H

#include <iostream>

#define MAX_LEVEL 22
#define SEQUENTIAL 1
#define PARALLEL 2

class Bitmap {
    friend class BitmapConstructor;
  private:
    int type;
  public:
    Bitmap() {
        type = SEQUENTIAL;
    }
    virtual void setRecordLength(double length) {}
    virtual void indexConstruction() {}
    virtual void setStreamFlag(bool flag) {}
    virtual ~Bitmap() {}
};
#endif

Writing /content/general/Bitmap.h


In [ ]:
%%writefile /content/general/BitmapConstructor.cpp

#include "BitmapConstructor.h"

Bitmap* BitmapConstructor::construct(Record* record, int thread_num, int level_num) {
    Bitmap* bm = NULL;
    if (thread_num == 1) {
        bm = SerialBitmapConstructor::construct(record, level_num);
        bm->type = SEQUENTIAL;
    } else {
        bm = ParallelBitmapConstructor::construct(record, thread_num, level_num);
        bm->type = PARALLEL;
    }
    return bm;
}

BitmapIterator* BitmapConstructor::getIterator(Bitmap* bm) {
    BitmapIterator* bi = NULL;
    if (bm->type == SEQUENTIAL) {
        bi = new SerialBitmapIterator((SerialBitmap*)bm);
        bi->type = SEQUENTIAL;
    } else {
        bi = new ParallelBitmapIterator((ParallelBitmap*)bm);
        bi->type = PARALLEL;
    }
    return bi;
}


Writing /content/general/BitmapConstructor.cpp


In [ ]:
%%writefile /content/general/BitmapConstructor.h

#ifndef BITMAPCONSTRUCTOR_H
#define BITMAPCONSTRUCTOR_H

#include <string>
#include "Bitmap.h"
#include "BitmapIterator.h"
#include "../serialSpec/SerialBitmapConstructor.h"
#include "../parallelSpec/ParallelBitmapConstructor.h"
#include "../serialSpec/SerialBitmapIterator.h"
#include "../parallelSpec/ParallelBitmapIterator.h"
#include "Records.h"

class BitmapConstructor {
  public:
    // construct leveled bitmaps for a JSON record
    static Bitmap* construct(Record* record, int thread_num = 1, int level_num = MAX_LEVEL);
    // get bitmap iterator for given bitmap index
    static BitmapIterator* getIterator(Bitmap* bi);
};

#endif

Writing /content/general/BitmapConstructor.h


In [ ]:
%%writefile /content/general/BitmapIterator.h

#ifndef BITMAPITERATOR_H
#define BITMAPITERATOR_H

#include "Bitmap.h"
#include <unordered_set>

class Bitmap;

#define OBJECT 1
#define ARRAY 2
#define PRIMITIVE 3

#define ERR -1
#define MAX_FIELD_SIZE 1000

// Metadata for parsing and querying, saves context information at each level when iterating leveled bitmap.
struct IterCtxInfo {
    // current thread id for parsing and querying. Only used during leveled bitmap iteration.
    int thread_id;
    // OBJECT or ARRAY
    int type;
    // position array for colon and comma
    long* positions;
    // start index of the record position array at the current level
    long start_idx;
    // end index of the record position array at the current level
    long end_idx;
    // current index of the record position array at the current level
    long cur_idx;
    // the current level
    int level;
};

struct KeyPos {
    long start;
    long end;
};

class BitmapIterator {
    friend class BitmapConstructor;
    friend class BitmapConstructor1;
  private:
    int type;
  public:
    int mVisitedFields;
  public:
    // Creates a copy of iterator. Often used for parallel querying.
    virtual BitmapIterator* getCopy() = 0;
    // Moves back to the object or array which contains the current nested record.
    // Often used when the current nested record has been processed.
    // Valid except for the first level of the record.
    virtual bool up() = 0;
    // Moves to the start of the nested object or array.
    // Gets all colon or comma positions from leveled bitmap indexes for current nested record.
    // Valid if we are at { or [.
    virtual bool down() = 0;
    // Whether the iterator points to an object.
    virtual bool isObject() = 0;
    // Whether the iterator points to an array.
    virtual bool isArray() = 0;
    // Moves iterator to the next array item.
    virtual bool moveNext() = 0;
    // Moves to the corresponding key field inside the current object.
    virtual bool moveToKey(char* key) = 0;
    // Moves to the corresponding key fields inside the current object, returns the current key name.
    // After this operation, the current key field will be removed from key_set.
    virtual char* moveToKey(std::unordered_set<char*>& key_set) = 0;
    // Returns the number of elements inside current array.
    virtual int numArrayElements() = 0;
    // If the current record is an array, moves to an item based on index.
    // Returns false if the index is out of the boundary.
    virtual bool moveToIndex(int index) = 0;
    // Gets the content of the current value inside an object or array.
    virtual char* getValue() = 0;
    virtual ~BitmapIterator() {

    }
};
#endif

Writing /content/general/BitmapIterator.h


In [ ]:
%%writefile /content/general/LocalBitmap.cpp

#include "LocalBitmap.h"
#include <immintrin.h>
#include <emmintrin.h>
#include <string.h>
#include <sys/time.h>
#include <stdio.h>
#include <string.h>
#include <stdlib.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdint.h>
#include <ctype.h>
#include <pthread.h>
#include <malloc.h>
#include <sys/time.h>
#include <sys/file.h>
#include <unistd.h>
#include <sched.h>
#include <unordered_map>

LocalBitmap::LocalBitmap() {

}

LocalBitmap::LocalBitmap(char* record, int level_num) {
    this->mThreadId = 0;
    this->mRecord = record;
    this->mDepth = level_num - 1;
    this->mStartWordId = 0;
    this->mEndWordId = 0;
    this->mQuoteBitmap = NULL;
    this->mEscapeBitmap = NULL;
    this->mColonBitmap = NULL;
    this->mCommaBitmap = NULL;
    this->mStrBitmap = NULL;
    this->mLbraceBitmap = NULL;
    this->mRbraceBitmap = NULL;
    this->mLbracketBitmap = NULL;
    this->mRbracketBitmap = NULL;
    for (int i = 0; i < MAX_LEVEL; ++i) {
        this->mLevColonBitmap[i] = NULL;
        this->mLevCommaBitmap[i] = NULL;
        this->mNegLevColonBitmap[i] = NULL;
        this->mNegLevCommaBitmap[i] = NULL;
        this->mFinalLevColonBitmap[i] = NULL;
        this->mFinalLevCommaBitmap[i] = NULL;
    }
    this->mStartInStrBitmap = 0ULL;
    this->mEndInStrBitmap = 0ULL;
    this->mMaxPositiveLevel = 0;
    this->mMinNegativeLevel = -1;

    this->mNumTknErr = 0;
    this->mNumTrial = 0;
}

void LocalBitmap::freeMemory()
{
    for(int m = 0; m < MAX_LEVEL; ++m){
        if (mLevColonBitmap[m]) {
            free(mLevColonBitmap[m]);
            mLevColonBitmap[m] = NULL;
        }
        if (mLevCommaBitmap[m]) {
            free(mLevCommaBitmap[m]);
            mLevCommaBitmap[m] = NULL;
        }
        if (mNegLevColonBitmap[m]) {
            free(mNegLevColonBitmap[m]);
            mNegLevColonBitmap[m] = NULL;
        }
        if (mNegLevCommaBitmap[m]) {
            free(mNegLevCommaBitmap[m]);
            mNegLevCommaBitmap[m] = NULL;
        }
    }
    if (mQuoteBitmap) {
        free(mQuoteBitmap);
        mQuoteBitmap = NULL;
    }
    if (mEscapeBitmap) {
        free(mEscapeBitmap);
        mEscapeBitmap = NULL;
    }
    if (mStrBitmap) {
        free(mStrBitmap);
        mStrBitmap = NULL;
    }
    if (mColonBitmap) {
        free(mColonBitmap);
        mColonBitmap = NULL;
    }
    if (mCommaBitmap) {
        free(mCommaBitmap);
        mCommaBitmap = NULL;
    }
    if (mLbraceBitmap) {
        free(mLbraceBitmap);
        mLbraceBitmap = NULL;
    }
    if (mRbraceBitmap) {
        free(mRbraceBitmap);
        mRbraceBitmap = NULL;
    }
    if (mLbracketBitmap) {
        free(mLbracketBitmap);
        mLbracketBitmap = NULL;
    }
    if (mRbracketBitmap) {
        free(mRbracketBitmap);
        mRbracketBitmap = NULL;
    }
}

LocalBitmap::~LocalBitmap()
{
    freeMemory();
}

void LocalBitmap::setRecordLength(long length) {
    this->mRecordLength = length;
    this->mNumTmpWords = length / 32;
    this->mNumWords = length / 64;
    this->mQuoteBitmap = (unsigned long*)malloc((mNumWords) * sizeof(unsigned long));
}

int LocalBitmap::contextInference() {
    Tokenizer tkn;
    int start_states[2] = {OUT, IN};
    bool getStartState = false;
    int start_state = OUT;
    for (int j = 0; j < 2; ++j) {
        ++mNumTrial;
        int state = start_states[j];
        tkn.createIterator(mRecord, state);
        while (true) {
            int tkn_status = tkn.hasNextToken();
            if (tkn_status == END)
                break;
            if (tkn_status == ERROR) {
                ++mNumTknErr;
                start_state = tkn.oppositeState(state);
                getStartState = true;
                break;
            }
            tkn.nextToken();
        }
        if (getStartState == true) break;
    }
    if (start_state == IN) {
        mStartInStrBitmap = 0xffffffffffffffffULL;
    } else {
        mStartInStrBitmap = 0ULL;
    }
    //cout<<"inference result num of trails: "<<mNumTrial<<" num of token error "<<mNumTknErr<<endl;
    //cout<<"inference result "<<start_state<<" "<<getStartState<<endl;
    if (getStartState == true) return start_state;
    return UNKNOWN;
}

void LocalBitmap::nonSpecIndexConstruction() {
    // vectors for structural characters
    __m256i v_quote = _mm256_set1_epi8(0x22);
    __m256i v_colon = _mm256_set1_epi8(0x3a);
    __m256i v_escape = _mm256_set1_epi8(0x5c);
    __m256i v_lbrace = _mm256_set1_epi8(0x7b);
    __m256i v_rbrace = _mm256_set1_epi8(0x7d);
    __m256i v_comma = _mm256_set1_epi8(0x2c);
    __m256i v_lbracket = _mm256_set1_epi8(0x5b);
    __m256i v_rbracket = _mm256_set1_epi8(0x5d);

    // variables for saving temporary results in the first four steps
    unsigned long colonbit0, quotebit0, escapebit0, stringbit0, lbracebit0, rbracebit0, commabit0, lbracketbit0, rbracketbit0;
    unsigned long colonbit, quotebit, escapebit, stringbit, lbracebit, rbracebit, commabit, lbracketbit, rbracketbit;
    unsigned long str_mask;

    // variables for saving temporary results in the last step
    unsigned long lb_mask, rb_mask, cb_mask;
    unsigned long lb_bit, rb_bit, cb_bit;
    unsigned long first, second;
    int cur_level = -1;

    // variables for saving context information among different words
    int top_word = -1;
    uint64_t prev_iter_ends_odd_backslash = 0ULL;
    uint64_t prev_iter_inside_quote = mStartInStrBitmap;
    const uint64_t even_bits = 0x5555555555555555ULL;
    const uint64_t odd_bits = ~even_bits;

    for (int j = 0; j < mNumTmpWords; ++j) {
        colonbit = 0, quotebit = 0, escapebit = 0, stringbit = 0, lbracebit = 0, rbracebit = 0, commabit = 0, lbracketbit = 0, rbracketbit = 0;
        unsigned long i = j * 32;
        // step 1: build structural character bitmaps
        __m256i v_text = _mm256_loadu_si256(reinterpret_cast<const __m256i *>(mRecord + i));
        colonbit = (unsigned)_mm256_movemask_epi8(_mm256_cmpeq_epi8(v_text, v_colon));
        quotebit = (unsigned)_mm256_movemask_epi8(_mm256_cmpeq_epi8(v_text, v_quote));
        escapebit = (unsigned)_mm256_movemask_epi8(_mm256_cmpeq_epi8(v_text, v_escape));
        lbracebit  = (unsigned)_mm256_movemask_epi8(_mm256_cmpeq_epi8(v_text, v_lbrace));
        rbracebit  = (unsigned)_mm256_movemask_epi8(_mm256_cmpeq_epi8(v_text, v_rbrace));
        commabit = (unsigned)_mm256_movemask_epi8(_mm256_cmpeq_epi8(v_text, v_comma));
	lbracketbit = (unsigned)_mm256_movemask_epi8(_mm256_cmpeq_epi8(v_text, v_lbracket));
	rbracketbit = (unsigned)_mm256_movemask_epi8(_mm256_cmpeq_epi8(v_text, v_rbracket));
        // first half of the word (lowest 32 bits)
        if(j % 2 == 0) {
            colonbit0 = colonbit;
            quotebit0 = quotebit;
            escapebit0 = escapebit;
            lbracebit0 = lbracebit;
            rbracebit0 = rbracebit;
            commabit0 = commabit;
            lbracketbit0 = lbracketbit;
            rbracketbit0 = rbracketbit;
            continue;
        } else {
            // highest 32 bits inside a word
            colonbit = (colonbit << 32) | colonbit0;
            quotebit = (quotebit << 32) | quotebit0;
            escapebit = (escapebit << 32) | escapebit0;
            lbracebit = (lbracebit << 32) | lbracebit0;
            rbracebit = (rbracebit << 32) | rbracebit0;
            commabit = (commabit << 32) | commabit0;
            lbracketbit = (lbracketbit << 32) | lbracketbit0;
            rbracketbit = (rbracketbit << 32) | rbracketbit0;

            // step 2: update structural quote bitmaps
            uint64_t bs_bits = escapebit;
            uint64_t start_edges = bs_bits & ~(bs_bits << 1);
            int64_t even_start_mask = even_bits ^ prev_iter_ends_odd_backslash;
            uint64_t even_starts = start_edges & even_start_mask;
            uint64_t odd_starts = start_edges & ~even_start_mask;
            uint64_t even_carries = bs_bits + even_starts;
            int64_t odd_carries;
            bool iter_ends_odd_backslash = __builtin_uaddll_overflow(bs_bits, odd_starts,
                (unsigned long long *)(&odd_carries));
            odd_carries |= prev_iter_ends_odd_backslash;
            prev_iter_ends_odd_backslash = iter_ends_odd_backslash ? 0x1ULL : 0x0ULL;
            uint64_t even_carry_ends = even_carries & ~bs_bits;
            uint64_t odd_carry_ends = odd_carries & ~bs_bits;
            uint64_t even_start_odd_end = even_carry_ends & odd_bits;
            uint64_t odd_start_even_end = odd_carry_ends & even_bits;
            uint64_t odd_ends = even_start_odd_end | odd_start_even_end;
            int64_t quote_bits = quotebit & ~odd_ends;
            mQuoteBitmap[++top_word] = quote_bits;

            str_mask = _mm_cvtsi128_si64(_mm_clmulepi64_si128(
                _mm_set_epi64x(0ULL, quote_bits), _mm_set1_epi8(0xFFu), 0));
            str_mask ^= prev_iter_inside_quote;
            prev_iter_inside_quote = static_cast<uint64_t>(static_cast<int64_t>(str_mask) >> 63);

            // step 4: update structural character bitmaps
            unsigned long tmp = (~str_mask);
            colonbit = colonbit & tmp;
            lbracebit = lbracebit & tmp;
            rbracebit = rbracebit & tmp;
            commabit = commabit & tmp;
            lbracketbit = lbracketbit & tmp;
            rbracketbit = rbracketbit & tmp;

            // step 5: generate leveled bitmaps
            lb_mask = lbracebit | lbracketbit;
            rb_mask = rbracebit | rbracketbit;
            cb_mask = lb_mask | rb_mask;
            lb_bit = lb_mask & (-lb_mask);
            rb_bit = rb_mask & (-rb_mask);
            if (!cb_mask) {
                if (cur_level >= 0 && cur_level <= mDepth) {
                    if (!mLevColonBitmap[cur_level]) {
                        mLevColonBitmap[cur_level] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                    }
                    if (!mLevCommaBitmap[cur_level]) {
                        mLevCommaBitmap[cur_level] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                    }
                    if (colonbit) {
                        mLevColonBitmap[cur_level][top_word] = colonbit;
                    } else {
                        mLevCommaBitmap[cur_level][top_word] = commabit;
	            }
	        } else if (cur_level < 0) {
                    if (!mNegLevColonBitmap[-cur_level]) {
                        mNegLevColonBitmap[-cur_level] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                        // before finding the first bracket, update minimum negative level
                        if (cur_level < mMinNegativeLevel) {
                            mMinNegativeLevel = cur_level;
                        }
                    }
                    if (!mNegLevCommaBitmap[-cur_level]) {
                        mNegLevCommaBitmap[-cur_level] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                    }
                    if (colonbit) {
                        mNegLevColonBitmap[-cur_level][top_word] = colonbit;
                    } else {
                        mNegLevCommaBitmap[-cur_level][top_word] = commabit;
                    }
                }
            } else {
                first = 1;
                while (cb_mask || first) {
                    if (!cb_mask) {
                        second = 1UL<<63;
                    } else {
                        cb_bit = cb_mask & (-cb_mask);
                        second = cb_bit;
                    }
                    if (cur_level >= 0 && cur_level <= mDepth) {
                        if (!mLevColonBitmap[cur_level]) {
                            mLevColonBitmap[cur_level] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                        }
                        if (!mLevCommaBitmap[cur_level]) {
                            mLevCommaBitmap[cur_level] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                        }
                        unsigned long mask = second - first;
                        if (!cb_mask) mask = mask | second;
                        unsigned long colon_mask = mask & colonbit;
                        if (colon_mask) {
                            mLevColonBitmap[cur_level][top_word] |= colon_mask;
                        } else {
                            mLevCommaBitmap[cur_level][top_word] |= (commabit & mask);
                        }
                        if (cb_mask) {
                            if (cb_bit == rb_bit) {
                                mLevColonBitmap[cur_level][top_word] |= cb_bit;
                                mLevCommaBitmap[cur_level][top_word] |= cb_bit;
                            }
                            else if (cb_bit == lb_bit && cur_level + 1 <= mDepth) {
                                if (!mLevCommaBitmap[cur_level + 1]) {
                                     mLevCommaBitmap[cur_level + 1] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                                }
                                mLevCommaBitmap[cur_level + 1][top_word] |= cb_bit;
                            }
                        }
                    } else if (cur_level < 0) {
                        if (!mNegLevColonBitmap[-cur_level]) {
                            mNegLevColonBitmap[-cur_level] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                        }
                        if (!mNegLevCommaBitmap[-cur_level]) {
                            mNegLevCommaBitmap[-cur_level] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                        }
                        unsigned long mask = second - first;
                        if (!cb_mask) mask = mask | second;
                        unsigned long colon_mask = mask & colonbit;
                        if (colon_mask) {
                            mNegLevColonBitmap[-cur_level][top_word] |= colon_mask;
                        } else {
                            mNegLevCommaBitmap[-cur_level][top_word] |= (commabit & mask);
                        }
                        if (cb_mask) {
                            if (cb_bit == rb_bit) {
                                mNegLevColonBitmap[-cur_level][top_word] |= cb_bit;
                                mNegLevCommaBitmap[-cur_level][top_word] |= cb_bit;
                            }
                            else if (cb_bit == lb_bit) {
                                if (cur_level + 1 == 0) {
                                    if (!mLevCommaBitmap[0]) {
                                         mLevCommaBitmap[0] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                                    }
                                    mLevCommaBitmap[0][top_word] |= cb_bit;
                                } else {
                                    if (!mNegLevCommaBitmap[-(cur_level + 1)]) {
                                        mNegLevCommaBitmap[-(cur_level + 1)] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                                    }
                                    mNegLevCommaBitmap[-(cur_level + 1)][top_word] |= cb_bit;
                                }
                            }
                        }
                    }
                    if (cb_mask) {
                        if (cb_bit == lb_bit) {
                            lb_mask = lb_mask & (lb_mask - 1);
                            lb_bit = lb_mask & (-lb_mask);
                            ++cur_level;
                            if (mThreadId == 0 && cur_level == 0) {
                                // JSON record at the top level could be an array
                                if (!mLevCommaBitmap[cur_level]) {
                                     mLevCommaBitmap[cur_level] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                                }
                                mLevCommaBitmap[cur_level][top_word] |= cb_bit;
                            }
                        } else if (cb_bit == rb_bit) {
                            rb_mask = rb_mask & (rb_mask - 1);
                            rb_bit = rb_mask & (-rb_mask);
                            --cur_level;
                        }
                        first = second;
                        cb_mask = cb_mask & (cb_mask - 1);
                        if (cur_level > mMaxPositiveLevel) {
                            mMaxPositiveLevel = cur_level;
                        } else if (cur_level < mMinNegativeLevel) {
                            mMinNegativeLevel = cur_level;
                        }
                    } else {
                        first = 0;
                    }
                }
	    }
        }
    }
    if (mDepth == MAX_LEVEL - 1) mDepth = mMaxPositiveLevel;
    mEndLevel = cur_level;
}

void LocalBitmap::buildStringMaskBitmap() {
    // allocate memory space for saving results
    if (!mQuoteBitmap) {
        mQuoteBitmap = (unsigned long*)malloc((mNumWords) * sizeof(unsigned long));
    }
    if (!mColonBitmap) {
        mColonBitmap = (unsigned long*)malloc((mNumWords) * sizeof(unsigned long));
    }
    if (!mCommaBitmap) {
        mCommaBitmap = (unsigned long*)malloc((mNumWords) * sizeof(unsigned long));
    }
    if (!mStrBitmap) {
        mStrBitmap = (unsigned long*)malloc((mNumWords) * sizeof(unsigned long));
    }
    if (!mLbraceBitmap) {
        mLbraceBitmap = (unsigned long*)malloc((mNumWords) * sizeof(unsigned long));
    }
    if (!mRbraceBitmap) {
        mRbraceBitmap = (unsigned long*)malloc((mNumWords) * sizeof(unsigned long));
    }
    if (!mLbracketBitmap) {
        mLbracketBitmap = (unsigned long*)malloc((mNumWords) * sizeof(unsigned long));
    }
    if (!mRbracketBitmap) {
        mRbracketBitmap = (unsigned long*)malloc((mNumWords) * sizeof(unsigned long));
    }

    // vectors for structural characters
    __m256i v_quote = _mm256_set1_epi8(0x22);
    __m256i v_colon = _mm256_set1_epi8(0x3a);
    __m256i v_escape = _mm256_set1_epi8(0x5c);
    __m256i v_lbrace = _mm256_set1_epi8(0x7b);
    __m256i v_rbrace = _mm256_set1_epi8(0x7d);
    __m256i v_comma = _mm256_set1_epi8(0x2c);
    __m256i v_lbracket = _mm256_set1_epi8(0x5b);
    __m256i v_rbracket = _mm256_set1_epi8(0x5d);

    // variables for saving temporary results
    unsigned long colonbit0, quotebit0, escapebit0, stringbit0, lbracebit0, rbracebit0, commabit0, lbracketbit0, rbracketbit0;
    unsigned long colonbit, quotebit, escapebit, stringbit, lbracebit, rbracebit, commabit, lbracketbit, rbracketbit;
    unsigned long str_mask;

    // variables for saving context information among different words
    int top_word = -1;
    uint64_t prev_iter_ends_odd_backslash = 0ULL;
    uint64_t prev_iter_inside_quote = mStartInStrBitmap;
    const uint64_t even_bits = 0x5555555555555555ULL;
    const uint64_t odd_bits = ~even_bits;

    for (int j = 0; j < mNumTmpWords; ++j) {
        colonbit = 0, quotebit = 0, escapebit = 0, stringbit = 0, lbracebit = 0, rbracebit = 0, commabit = 0, lbracketbit = 0, rbracketbit = 0;
        unsigned long i = j * 32;
        // step 1: build structural character bitmaps
        __m256i v_text = _mm256_loadu_si256(reinterpret_cast<const __m256i *>(mRecord + i));
        colonbit = (unsigned)_mm256_movemask_epi8(_mm256_cmpeq_epi8(v_text, v_colon));
        quotebit = (unsigned)_mm256_movemask_epi8(_mm256_cmpeq_epi8(v_text, v_quote));
        escapebit = (unsigned)_mm256_movemask_epi8(_mm256_cmpeq_epi8(v_text, v_escape));
        lbracebit  = (unsigned)_mm256_movemask_epi8(_mm256_cmpeq_epi8(v_text, v_lbrace));
        rbracebit  = (unsigned)_mm256_movemask_epi8(_mm256_cmpeq_epi8(v_text, v_rbrace));
        commabit = (unsigned)_mm256_movemask_epi8(_mm256_cmpeq_epi8(v_text, v_comma));
	lbracketbit = (unsigned)_mm256_movemask_epi8(_mm256_cmpeq_epi8(v_text, v_lbracket));
	rbracketbit = (unsigned)_mm256_movemask_epi8(_mm256_cmpeq_epi8(v_text, v_rbracket));
        // first half of the word (lowest 32 bits)
        if(j % 2 == 0) {
            colonbit0 = colonbit;
            quotebit0 = quotebit;
            escapebit0 = escapebit;
            lbracebit0 = lbracebit;
            rbracebit0 = rbracebit;
            commabit0 = commabit;
            lbracketbit0 = lbracketbit;
            rbracketbit0 = rbracketbit;
            continue;
        } else {
            // highest 32 bits inside a word
            colonbit = (colonbit << 32) | colonbit0;
            quotebit = (quotebit << 32) | quotebit0;
            escapebit = (escapebit << 32) | escapebit0;
            lbracebit = (lbracebit << 32) | lbracebit0;
            rbracebit = (rbracebit << 32) | rbracebit0;
            commabit = (commabit << 32) | commabit0;
            lbracketbit = (lbracketbit << 32) | lbracketbit0;
            rbracketbit = (rbracketbit << 32) | rbracketbit0;
            mColonBitmap[++top_word] = colonbit;
            mCommaBitmap[top_word] = commabit;
            mLbraceBitmap[top_word] = lbracebit;
            mRbraceBitmap[top_word] = rbracebit;
            mLbracketBitmap[top_word] = lbracketbit;
            mRbracketBitmap[top_word] = rbracketbit;

            // step 2: update structural quote bitmaps
            uint64_t bs_bits = escapebit;
            uint64_t start_edges = bs_bits & ~(bs_bits << 1);
            int64_t even_start_mask = even_bits ^ prev_iter_ends_odd_backslash;
            uint64_t even_starts = start_edges & even_start_mask;
            uint64_t odd_starts = start_edges & ~even_start_mask;
            uint64_t even_carries = bs_bits + even_starts;
            int64_t odd_carries;
            bool iter_ends_odd_backslash = __builtin_uaddll_overflow(bs_bits, odd_starts,
                (unsigned long long *)(&odd_carries));
            odd_carries |= prev_iter_ends_odd_backslash;
            prev_iter_ends_odd_backslash = iter_ends_odd_backslash ? 0x1ULL : 0x0ULL;
            uint64_t even_carry_ends = even_carries & ~bs_bits;
            uint64_t odd_carry_ends = odd_carries & ~bs_bits;
            uint64_t even_start_odd_end = even_carry_ends & odd_bits;
            uint64_t odd_start_even_end = odd_carry_ends & even_bits;
            uint64_t odd_ends = even_start_odd_end | odd_start_even_end;
            int64_t quote_bits = quotebit & ~odd_ends;
            mQuoteBitmap[top_word] = quote_bits;

            // step 3: build string mask bitmaps
            str_mask = _mm_cvtsi128_si64(_mm_clmulepi64_si128(
                _mm_set_epi64x(0ULL, quote_bits), _mm_set1_epi8(0xFFu), 0));
            str_mask ^= prev_iter_inside_quote;
            mStrBitmap[top_word] = str_mask;
            prev_iter_inside_quote = static_cast<uint64_t>(static_cast<int64_t>(str_mask) >> 63);
        }
    }
    mEndInStrBitmap = prev_iter_inside_quote;
}

void LocalBitmap::buildLeveledBitmap() {
    // variables for saving temporary results in the first four steps
    unsigned long colonbit, quotebit, escapebit, stringbit, lbracebit, rbracebit, commabit, lbracketbit, rbracketbit;
    unsigned long str_mask;

     // variables for saving temporary results in the last step
    unsigned long lb_mask, rb_mask, cb_mask;
    unsigned long lb_bit, rb_bit, cb_bit;
    unsigned long first, second;
    int cur_level = -1;

    for (int j = 0; j < mNumWords; ++j) {
        // get input info
        colonbit = mColonBitmap[j];
        commabit = mCommaBitmap[j];
        lbracebit = mLbraceBitmap[j];
        rbracebit = mRbraceBitmap[j];
        lbracketbit = mLbracketBitmap[j];
        rbracketbit = mRbracketBitmap[j];
        str_mask = mStrBitmap[j];

        // step 4: update structural character bitmaps
        unsigned long tmp = (~str_mask);
        colonbit = colonbit & tmp;
        lbracebit = lbracebit & tmp;
        rbracebit = rbracebit & tmp;
        commabit = commabit & tmp;
        lbracketbit = lbracketbit & tmp;
        rbracketbit = rbracketbit & tmp;

        // step 5: generate leveled bitmaps
        lb_mask = lbracebit | lbracketbit;
        rb_mask = rbracebit | rbracketbit;
        cb_mask = lb_mask | rb_mask;
        lb_bit = lb_mask & (-lb_mask);
        rb_bit = rb_mask & (-rb_mask);
        int top_word = j;
        if (!cb_mask) {
            if (cur_level >= 0 && cur_level <= mDepth) {
                if (!mLevColonBitmap[cur_level]) {
                    mLevColonBitmap[cur_level] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                }
                if (!mLevCommaBitmap[cur_level]) {
                    mLevCommaBitmap[cur_level] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                }
                if (colonbit) {
                    mLevColonBitmap[cur_level][top_word] = colonbit;
                } else {
                    mLevCommaBitmap[cur_level][top_word] = commabit;
                }
            } else if (cur_level < 0) {
                if (!mNegLevColonBitmap[-cur_level]) {
                    mNegLevColonBitmap[-cur_level] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                }
                if (!mNegLevCommaBitmap[-cur_level]) {
                    mNegLevCommaBitmap[-cur_level] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                }
                if (colonbit) {
                    mNegLevColonBitmap[-cur_level][top_word] = colonbit;
                } else {
                    mNegLevCommaBitmap[-cur_level][top_word] = commabit;
                }
            }
        } else {
            first = 1;
            while (cb_mask || first) {
                if (!cb_mask) {
                    second = 1UL<<63;
                } else {
                    cb_bit = cb_mask & (-cb_mask);
                    second = cb_bit;
                }
                if (cur_level >= 0 && cur_level <= mDepth) {
                    if (!mLevColonBitmap[cur_level]) {
                        mLevColonBitmap[cur_level] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                    }
                    if (!mLevCommaBitmap[cur_level]) {
                        mLevCommaBitmap[cur_level] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                    }
                    unsigned long mask = second - first;
                    if (!cb_mask) mask = mask | second;
                    unsigned long colon_mask = mask & colonbit;
                    if (colon_mask) {
                        mLevColonBitmap[cur_level][top_word] |= colon_mask;
                    } else {
                        mLevCommaBitmap[cur_level][top_word] |= (commabit & mask);
                    }
                    if (cb_mask) {
                        if (cb_bit == rb_bit) {
                            mLevColonBitmap[cur_level][top_word] |= cb_bit;
                            mLevCommaBitmap[cur_level][top_word] |= cb_bit;
                        }
                        else if (cb_bit == lb_bit && cur_level + 1 <= mDepth) {
                            if (!mLevCommaBitmap[cur_level + 1]) {
                                mLevCommaBitmap[cur_level + 1] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                            }
                            mLevCommaBitmap[cur_level + 1][top_word] |= cb_bit;
                        }
                    }
                } else if (cur_level < 0) {
                    if (!mNegLevColonBitmap[-cur_level]) {
                        mNegLevColonBitmap[-cur_level] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                    }
                    if (!mNegLevCommaBitmap[-cur_level]) {
                        mNegLevCommaBitmap[-cur_level] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                    }
                    unsigned long mask = second - first;
                    if (!cb_mask) mask = mask | second;
                    unsigned long colon_mask = mask & colonbit;
                    if (colon_mask) {
                        mNegLevColonBitmap[-cur_level][top_word] |= colon_mask;
                    } else {
                        mNegLevCommaBitmap[-cur_level][top_word] |= (commabit & mask);
                    }
                    if (cb_mask) {
                        if (cb_bit == rb_bit) {
                            mNegLevColonBitmap[-cur_level][top_word] |= cb_bit;
                            mNegLevCommaBitmap[-cur_level][top_word] |= cb_bit;
                        }
                        else if (cb_bit == lb_bit) {
                            if (cur_level + 1 == 0) {
                                if (!mLevCommaBitmap[0]) {
                                    mLevCommaBitmap[0] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                                }
                                mLevCommaBitmap[0][top_word] |= cb_bit;
                            } else {
                                if (!mNegLevCommaBitmap[-(cur_level + 1)]) {
                                    mNegLevCommaBitmap[-(cur_level + 1)] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                                }
                                mNegLevCommaBitmap[-(cur_level + 1)][top_word] |= cb_bit;
                            }
                        }
                    }
                }
                if (cb_mask) {
                    if (cb_bit == lb_bit) {
                        lb_mask = lb_mask & (lb_mask - 1);
                        lb_bit = lb_mask & (-lb_mask);
                        ++cur_level;
                        if (mThreadId == 0 && cur_level == 0) {
                            // JSON record at the top level could be an array
                            if (!mLevCommaBitmap[cur_level]) {
                                mLevCommaBitmap[cur_level] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                            }
                            mLevCommaBitmap[cur_level][top_word] |= cb_bit;
                        }
                    } else if (cb_bit == rb_bit) {
                        rb_mask = rb_mask & (rb_mask - 1);
                        rb_bit = rb_mask & (-rb_mask);
                        --cur_level;
                    }
                    first = second;
                    cb_mask = cb_mask & (cb_mask - 1);
                    if (cur_level > mMaxPositiveLevel) {
                        mMaxPositiveLevel = cur_level;
                    } else if (cur_level < mMinNegativeLevel) {
                        mMinNegativeLevel = cur_level;
                    }
                } else {
                    first = 0;
                }
            }
        }
    }
    if (mDepth == MAX_LEVEL - 1) mDepth = mMaxPositiveLevel;
    mEndLevel = cur_level;
}

Writing /content/general/LocalBitmap.cpp


In [ ]:
%%writefile /content/general/LocalBitmap.h

#ifndef LOCALBITMAP_H
#define LOCALBITMAP_H
#include <string>
#include <iostream>
#include <vector>
#include <bitset>
#include <cassert>
#include <stack>
#include <algorithm>
#include <unordered_map>
#include <functional>
#include <math.h>
#include <immintrin.h>
#include "Bitmap.h"
#include "Tokenizer.h"

class LocalBitmap : public Bitmap {
    friend class ParallelBitmap;
    friend class ParallelBitmapIterator;
  private:
    int mNumTknErr;
    int mNumTrial;
    int mThreadId;
    char* mRecord;
    // for a single large record, stream length equals to record length
    long mRecordLength;
    // each temp word has 32 bytes
    long mNumTmpWords;
    // each word has 64 bytes
    long mNumWords;
    // the deepest level of leveled bitmap indexes (starting from 0)
    int mDepth;
    // structural character bitmaps
    unsigned long* mEscapeBitmap, *mStrBitmap, *mColonBitmap, *mCommaBitmap, *mLbracketBitmap, *mRbracketBitmap, *mLbraceBitmap, *mRbraceBitmap;

    // following two variables are used for validating inference results of Step 3 (build string mask bitmap)
    // marks whether current chunk starts inside string or not
    unsigned long mStartInStrBitmap;
    // marks whether current chunk ends inside string or not
    unsigned long mEndInStrBitmap;

    // following variables are used for merging phase (after Step 5, merge leveled bitmap)
    // each thread starts with level 0, following two arrays save bitmaps for levels higher than 0 (temporary result)
    unsigned long *mLevColonBitmap[MAX_LEVEL];
    unsigned long *mLevCommaBitmap[MAX_LEVEL];
    // each thread starts with level 0, following two arrays save bitmaps for levels less than 0 (temporary result)
    unsigned long *mNegLevColonBitmap[MAX_LEVEL];
    unsigned long *mNegLevCommaBitmap[MAX_LEVEL];
    // each thread starts with level 0
    // mMaxPositiveLevel saves the maximum positive level in current thread
    int mMaxPositiveLevel;
    // mMinNegativeLevel saves the minimum negative level in current thread
    int mMinNegativeLevel;
    // saves the level after processing the whole chunk, used for parallel index construction
    int mEndLevel;

    // following variables are used by ParallelBitmapIterator
    // temporary leveled colon bitmap is mapped to the correct level, which happens during the merging phase
    unsigned long *mFinalLevColonBitmap[MAX_LEVEL];
    unsigned long *mFinalLevCommaBitmap[MAX_LEVEL];
    // structural quote bitmap, used for getting the key field when iterating bitmaps
    unsigned long *mQuoteBitmap;
    // word ids for the first and last words, often used when iterating leveled bitmap to get some information like colon, comma and key field positions
    long mStartWordId;
    long mEndWordId;

  public:
    LocalBitmap();
    LocalBitmap(char* record, int level_num);
    ~LocalBitmap();
    // context inference for parallel index construction (step 3).
    // if it context information couldn't be inferred, return SPECULATIVE; else return NOSPECULATIVE
    int contextInference();
    // function for non-speculative parallel index construction
    void nonSpecIndexConstruction();
    // following two functions are used for speculative parallel index construction
    void buildStringMaskBitmap();
    void buildLeveledBitmap();
    void setRecordLength(long length);
    void setThreadId(int thread_id) {mThreadId = thread_id;}

  private:
    void freeMemory();
};
#endif

Writing /content/general/LocalBitmap.h


In [ ]:
%%writefile /content/general/RecordLoader.cpp

#include <sys/time.h>
#include <fstream>
#include "RecordLoader.h"

#define MAX_PAD 64

void* aligned_malloc(size_t, size_t);

// Opens binary files
// First have to convert the data to binary files??
Record* RecordLoader::loadSingleRecord(const char* file_path) {
    unsigned long size;
    FILE* fp = fopen(file_path, "r");
    if (fp == NULL) {
        return NULL;
    }
    fseek(fp, 0, SEEK_END);
    size = ftell(fp);
    rewind(fp);

    void* p = aligned_malloc(64, (size + MAX_PAD) * sizeof(char));
    if (p == NULL) {
        std::cout << "Fail to allocate memory space for input record." << std::endl;
    }
    char* record_text = static_cast<char*>(p);
    size_t load_size = fread(record_text, 1, size, fp);
    if (load_size == 0) {
        std::cout << "Fail to load the input record into memory" << std::endl;
    }
    int remain = 64 - (size % 64);
    int counter = 0;
    // pad the input data where its size can be divided by 64
    while (counter < remain) {
        record_text[size + counter] = 'd';
        counter++;
    }
    record_text[size + counter] = '\0';
    fclose(fp);

    // only one single record
    Record* record = new Record();
    record->text = record_text;
    record->rec_start_pos = 0;
    record->rec_length = strlen(record_text);
    return record;
}



RecordSet* RecordLoader::loadRecords(const char* file_path) {
    FILE *fp = fopen(file_path, "r");
    RecordSet* rs = new RecordSet();
    if (fp) {
        char line[MAX_RECORD_SIZE];
        std::string str;
        int start_pos = 0;
        while (fgets(line, sizeof(line), fp) != NULL) {
            if (strlen(line) <= MIN_RECORD_SIZE) continue;
            int remain = 64 - strlen(line) % 64;
            int top = strlen(line);
            while (remain > 0) {
                line[top++] = 'd';
                --remain;
            }
            line[top] = '\0';
            if (strlen(line) > MIN_RECORD_SIZE) {
                // concating a sequence of record texts into one single string generates the best performance for indexing and querying
                str.append(line);
                Record* record = new Record();
                record->rec_start_pos = start_pos;
                record->rec_length = strlen(line);
                start_pos += strlen(line);
                rs->recs.push_back(record);
                ++rs->num_recs;
            }
        }
        void* p = aligned_malloc(64, str.size()*sizeof(char));
        //void* p;
        if (p == NULL) {
            std::cout<<"Fail to allocate memory space for records from input file."<<std::endl;
        }
        for (int i = 0; i < rs->recs.size(); ++i) {
            // all record objects points to the same input text which contacts a sequence of JSON records
            rs->recs[i]->text = (char*) p;
            if (i == 0) strcpy(rs->recs[0]->text, str.c_str());
            // deconstructor in the last record object can delete input text
            if (i < rs->recs.size() - 1) rs->recs[i]->can_delete_text = false;
        }
        fclose(fp);
        return rs;
    }
    printf("Error: %d (%s)\n", errno, strerror(errno));
    std::cout<<"Fail open the file."<<std::endl;
    return rs;
}


void* aligned_malloc(size_t alignment, size_t size) {

    uintptr_t mask = alignment - 1;
    void* ptr = malloc(size + alignment + sizeof(void*));
    if (ptr) {
        void* aligned_ptr = reinterpret_cast<void*>((reinterpret_cast<uintptr_t>(ptr) & ~mask) + alignment);
        *(reinterpret_cast<void**>(aligned_ptr) - 1) = ptr;
        return aligned_ptr;
    }

    return nullptr;
}

Writing /content/general/RecordLoader.cpp


In [ ]:
%%writefile /content/general/RecordLoader.h

#ifndef _RECORDLOADER_H
#define _RECORDLOADER_H

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <ctype.h>
#include <thread>
#include <sys/time.h>
#include <sys/file.h>
#include <malloc.h>
#include <iostream>
#include <string>
#include <vector>
#include "Records.h"

class RecordLoader{
  public:
    static Record* loadSingleRecord(const char* file_path);
    static RecordSet* loadRecords(const char* file_path);

};
#endif

Writing /content/general/RecordLoader.h


In [ ]:
%%writefile /content/general/Records.h

#ifndef RECORDS_H
#define RECORDS_H

#include <stdlib.h>
#include <vector>
#include <iostream>

#define MIN_RECORD_SIZE 5
#define MAX_RECORD_SIZE 1000000

// information for a single JSON record
struct Record {
    // for line-delimited JSON stream with a sequence of records,
    // contacting them into one single string generates the best
    // performance for indexing and querying
    char* text;
    long rec_start_pos;
    long rec_length;
    // text could be shared among different Record objects
    // (e.g. line-delimited JSON stream with a sequence of records)
    bool can_delete_text;

    Record() {
        text = NULL;
        rec_start_pos = 0;
        rec_length = 0;
        can_delete_text = true;
    }

    ~Record() {
        if (can_delete_text == true && text != NULL) {
            free(text);
            text = NULL;
            can_delete_text = false;
        }
    }
};

// information for a sequence of JSON records
class RecordSet {
    friend class RecordLoader;
  public:
    std::vector<Record*> recs;
    long num_recs;

  public:
    RecordSet() {
        num_recs = 0;
    }

    // record can be accessed in array style.
    Record*& operator[] (long idx) {
        if (idx >= 0 && idx < num_recs)
            return recs[idx];
        std::cout << "Array index in RecordSet out of bound."<< std::endl;
        exit(0);
    }

    long size() {
        return num_recs;
    }

    ~RecordSet() {
        for (long i = 0; i < num_recs; ++i) {
            if (recs[i] != NULL)
                delete recs[i];
        }
    }
};
#endif

Writing /content/general/Records.h


In [ ]:
%%writefile /content/general/Tokenizer.cpp

#include "Tokenizer.h"
#include <iostream>

int Tokenizer::getStringToken(int& pos) {
    while (pos < 64) {
        int escape_cnt = 0;
        while (pos < 64 &&mChunk[pos] == '\\') {
            ++escape_cnt;
            ++pos;
        }
        if (mChunk[pos] == '"') {
            ++pos;
            if (escape_cnt % 2 == 0) {
                mCurTknType = STRING;
                mNextTknPos = pos;
                return TRUE;
            }
        } else {
            ++pos;
        }
    }
    return END;
}

int Tokenizer::hasNextToken() {
    if (mCurPos >= 64) return END;
    // current chunk starts inside string
    if (mCurPos == 0 && mStartState == IN) {
        int pos = mCurPos;
        return getStringToken(pos);
    }
    int pos = mCurPos;
    while (pos < 64) {
        switch(mChunk[pos]) {
            case '\t':
            case '\n':
                ++pos;
                break;
            case '{':
                ++pos;
                mCurTknType = LCB;
                mNextTknPos = pos;
                return TRUE;
            case '}':
                ++pos;
                mCurTknType = RCB;
                mNextTknPos = pos;
                return TRUE;
            case '[':
                ++pos;
                mCurTknType = LB;
                mNextTknPos = pos;
                return TRUE;
            case ']':
                ++pos;
                mCurTknType = RB;
                mNextTknPos = pos;
                return TRUE;
            case ',':
                ++pos;
                mCurTknType = COM;
                mNextTknPos = pos;
                return TRUE;
            case ':':
                ++pos;
                mCurTknType = COLON;
                mNextTknPos = pos;
                return TRUE;
            case '"':
                ++pos;
                return getStringToken(pos);
            case 't':
            {
                if (mChunk[pos + 1] == 'r' && mChunk[pos + 2] == 'u' && mChunk[pos + 3] == 'e') {
                    pos += 4;
                    mCurTknType = PRI;
                    mNextTknPos = pos;
                    return TRUE;
                } else {
                    return ERROR;
                }
            }
            case 'r':
            {
                if (mChunk[pos + 2] == 'u' && mChunk[pos + 3] == 'e') {
                    pos += 3;
                    mCurTknType = PRI;
                    mNextTknPos = pos;
                    return TRUE;
                } else {
                    return ERROR;
                }
            }
            case 'e':
            {
               if (mChunk[pos + 1] == ',' || mChunk[pos + 1] == ']' || mChunk[pos + 1] == '}') {
                   ++pos;
                   mCurTknType = PRI;
                   mNextTknPos = pos;
                   return TRUE;
               } else {
                   return ERROR;
               }
            }
            case 'f':
            {
                if (mChunk[pos + 1] == 'a' && mChunk[pos + 2] == 'l' && mChunk[pos + 3] == 's' && mChunk[pos + 4] == 'e') {
                    pos += 5;
                    mCurTknType = PRI;
                    mNextTknPos = pos;
                    return TRUE;
                } else {
                    return ERROR;
                }
            }
            case 'a':
            {
                if (mChunk[pos + 1] == 'l' && mChunk[pos + 2] == 's' && mChunk[pos + 3] == 'e') {
                    pos += 4;
                    mCurTknType = PRI;
                    mNextTknPos = pos;
                    return TRUE;
                } else {
                    return ERROR;
                }
            }
            case 's':
            {
                if (mChunk[pos + 1] == 'e') {
                    pos += 2;
                    mCurTknType = PRI;
                    mNextTknPos = pos;
                    return TRUE;
                } else {
                    return ERROR;
                }
            }
            case 'n':
            {
               if (mChunk[pos + 1] == 'u' && mChunk[pos + 2] == 'l' && mChunk[pos + 3] == 'l') {
                   pos += 4;
                   mCurTknType = PRI;
                   mNextTknPos = pos;
                   return TRUE;
               } else {
                   return ERROR;
               }
            }
            case 'u':
            {
               if (mChunk[pos + 1] == 'l' && mChunk[pos + 2] == 'l') {
                   pos += 3;
                   mCurTknType = PRI;
                   mNextTknPos = pos;
                   return TRUE;
               } else if (mChunk[pos + 1] == 'e') {
                   pos += 2;
                   mCurTknType = PRI;
                   mNextTknPos = pos;
                   return TRUE;
               } else {
                   return ERROR;
               }
            }
            case 'l':
            {
               if (mChunk[pos + 1] == 'l' || mChunk[pos + 1] == ',' || mChunk[pos + 1] == ']' || mChunk[pos + 1] == '}') {
                   ++pos;
                   if (mChunk[pos] == 'l') ++pos;
                   mCurTknType = PRI;
                   mNextTknPos = pos;
                   return TRUE;
               } else if (mChunk[pos + 1] == 's' && mChunk[pos + 2] == 'e') {
                   pos += 3;
                   mCurTknType = PRI;
                   mNextTknPos = pos;
                   return TRUE;
               } else {
                   return ERROR;
               }
            }
            case '-':
            case '0':
            case '1':
            case '2':
            case '3':
            case '4':
            case '5':
            case '6':
            case '7':
            case '8':
            case '9':
            case '.':
            {
                ++pos;
                char ch = mChunk[pos];
                if (ch == '}' || ch == ']' || ch == ' ' || ch == '\t' || ch == ',') {
                    mCurTknType = PRI;
                    mNextTknPos = pos;
                    return TRUE;
                }
                if (ch == '"') {
                    return ERROR;
                }
                break;
            }
            case ' ':
                ++pos;
                break;
            default:
                return ERROR;
        }
    }
    return END;
}

int Tokenizer::nextToken() {
    mCurPos = mNextTknPos;
    return mCurTknType;
}

Writing /content/general/Tokenizer.cpp


In [ ]:
%%writefile /content/general/Tokenizer.h

#ifndef TOKENIZER_H
#define TONENIZER_H

#define IN 10
#define OUT 11

#define TRUE 100
#define ERROR 101
#define END 102

// token type
#define LCB 1   //'{'
#define RCB 2   //'}'
#define LB 3    //'['
#define RB 4    //']'
#define COM 5   //','
#define COLON 6 //':'
#define PRI 7   //primitive
#define STRING 8 // "abc"
#define UNKNOWN 9
#define INVALID -1  //invalid

class Tokenizer {
  private:
    char* mChunk;
    // starting state of the separated chunk
    // IN -- inside string
    // OUT -- outside string
    int mStartState;
    // current position of mText
    long mCurPos;
    // current token type
    int mCurTknType;
    // starting position of the next token
    long mNextTknPos;
  public:
    Tokenizer() {}

    Tokenizer(char* chunk, int state) {
        mChunk = chunk;
        mStartState = state;
        mCurPos = 0;
        mCurTknType = UNKNOWN;
        mNextTknPos = 0;
    }

    // create iterator based on start state
    void createIterator(char* chunk, int state) {
        mChunk = chunk;
        mStartState = state;
        mCurPos = 0;
        mCurTknType = UNKNOWN;
        mNextTknPos = 0;
    }

    int oppositeState(int state) {
        if (state == IN) return OUT;
        if (state == OUT) return IN;
        return UNKNOWN;
    }

    // whether the next valid token exists, only iterates through the first word (64 bits)
    int hasNextToken();
    // returns the type of the next token, only iterates through the first word (64 bits)
    int nextToken();

  private:
    int getStringToken(int& pos);
};
#endif


Writing /content/general/Tokenizer.h


In [ ]:
%%writefile /content/parallelSpec/ParallelBitmap.cpp

#include "ParallelBitmap.h"

ParallelBitmap::ParallelBitmap(char* record, int thread_num, int depth) {
    mRecord = record;
    mDepth = depth;
    mThreadNum = thread_num;
    char* start_chunk = record;
    int rec_len = strlen(record);
    mRecordLength = rec_len;
    int chunk_len = rec_len / thread_num;
    if (chunk_len % 64 > 0) {
        chunk_len = chunk_len + 64 - chunk_len % 64;
    }
    int cur_len = 0;
    mParallelMode = NONSPECULATIVE;
    for (int i = 0; i < thread_num; ++i) {
        mBitmaps[i] = new LocalBitmap(start_chunk, depth);
        mBitmaps[i]->setThreadId(i);
        if (i < thread_num - 1) {
            int pad_len = 0;
            // escaped backslashes are not separated into different chunks
            while (start_chunk[chunk_len + pad_len - 1] == '\\') {
                pad_len += 64;
            }
            mBitmaps[i]->setRecordLength(chunk_len + pad_len);
            start_chunk = start_chunk + chunk_len + pad_len;
            cur_len += (chunk_len + pad_len);
        } else {
            int last_chunk_len = rec_len - cur_len;
            mBitmaps[i]->setRecordLength(last_chunk_len);
        }
        // perform context inference and decide whether the program runs in speculative mode
        if (mBitmaps[i]->contextInference() == UNKNOWN) {
            mParallelMode = SPECULATIVE;
        }
    }
}

ParallelBitmap::ParallelBitmap(char* record, long rec_len, int thread_num, int depth) {
    mRecord = record;
    mDepth = depth;
    mThreadNum = thread_num;
    char* start_chunk = record;
    mRecordLength = rec_len;
    int chunk_len = rec_len / thread_num;
    if (chunk_len % 64 > 0) {
        chunk_len = chunk_len + 64 - chunk_len % 64;
    }
    int cur_len = 0;
    mParallelMode = NONSPECULATIVE;
    for (int i = 0; i < thread_num; ++i) {
        mBitmaps[i] = new LocalBitmap(start_chunk, depth);
        if (i < thread_num - 1) {
            int pad_len = 0;
            // escaped backslashes are not separated into different chunks
            while (start_chunk[chunk_len + pad_len - 1] == '\\') {
                pad_len += 64;
            }
            mBitmaps[i]->setRecordLength(chunk_len + pad_len);
            start_chunk = start_chunk + chunk_len + pad_len;
            cur_len += (chunk_len + pad_len);
        } else {
            int last_chunk_len = rec_len - cur_len;
            mBitmaps[i]->setRecordLength(last_chunk_len);
        }
        // perform context inference and decide whether the program runs in speculative mode
        if (mBitmaps[i]->contextInference() == UNKNOWN) {
            mParallelMode = SPECULATIVE;
        }
    }
}

ParallelBitmap::~ParallelBitmap() {
    for (int i = 0; i < mThreadNum; ++i) {
        delete mBitmaps[i];
    }
}

int ParallelBitmap::parallelMode() {
    return mParallelMode;
}

void ParallelBitmap::setRecordLength(long length) {
    mRecordLength = length;
}

void ParallelBitmap::rectifyStringMaskBitmaps() {
    //cout<<"start verification"<<endl;
    unsigned long prev_iter_inside_quote = mBitmaps[0]->mEndInStrBitmap;
    for (int i = 1; i < mThreadNum; ++i) {
       if (prev_iter_inside_quote != mBitmaps[i]->mStartInStrBitmap) {
           mBitmaps[i]->mStartInStrBitmap = prev_iter_inside_quote;
           // flip string mask bitmaps
           //cout<<"flip for "<<i<<"th thread "<<endl;
           for (int j = 0; j < mBitmaps[i]->mNumWords; ++j) {
               mBitmaps[i]->mStrBitmap[j] = ~mBitmaps[i]->mStrBitmap[j];
           }
           if (mBitmaps[i]->mEndInStrBitmap == 0) {
               mBitmaps[i]->mEndInStrBitmap = 0xffffffffffffffffULL;
           } else {
               mBitmaps[i]->mEndInStrBitmap = 0ULL;
           }
       }
       prev_iter_inside_quote = mBitmaps[i]->mEndInStrBitmap;
    }
    //cout<<"end verification"<<endl;
}

void ParallelBitmap::mergeBitmaps() {
    //cout<<"start merge"<<endl;
    int cur_level = mBitmaps[0]->mEndLevel;
    long offset = 0;
    for (int i = 0; i <= mBitmaps[0]->mMaxPositiveLevel; ++i) {
        mBitmaps[0]->mFinalLevColonBitmap[i] = mBitmaps[0]->mLevColonBitmap[i];
        mBitmaps[0]->mFinalLevCommaBitmap[i] = mBitmaps[0]->mLevCommaBitmap[i];
    }
    offset += mBitmaps[0]->mNumWords;
    mBitmaps[0]->mStartWordId = 0;
    mBitmaps[0]->mEndWordId = offset;
    // link leveled colon and comma bitmaps generated from different threads
    for (int i = 1; i < mThreadNum; ++i) {
        mBitmaps[i]->mStartWordId = offset;
        mBitmaps[i]->mEndWordId = offset + mBitmaps[i]->mNumWords;
        for(int j = 1; j <= -mBitmaps[i]->mMinNegativeLevel && (cur_level - j + 1) >= 0; ++j) {
            mBitmaps[i]->mFinalLevColonBitmap[cur_level - j + 1] = mBitmaps[i]->mNegLevColonBitmap[j];
            mBitmaps[i]->mFinalLevCommaBitmap[cur_level - j + 1] = mBitmaps[i]->mNegLevCommaBitmap[j];
        }
        for(int j = 0; j <= mBitmaps[i]->mMaxPositiveLevel && (cur_level + j + 1) >= 0; ++j) {
            mBitmaps[i]->mFinalLevColonBitmap[cur_level + j + 1] = mBitmaps[i]->mLevColonBitmap[j];
            mBitmaps[i]->mFinalLevCommaBitmap[cur_level + j + 1] = mBitmaps[i]->mLevCommaBitmap[j];
        }
        cur_level += (mBitmaps[i]->mEndLevel + 1);
        offset += mBitmaps[i]->mNumWords;
    }
    //cout<<"final level after merge "<<cur_level<<" "<<endl;
}

Writing /content/parallelSpec/ParallelBitmap.cpp


In [ ]:
%%writefile /content/parallelSpec/ParallelBitmap.h

#ifndef PARALLELBITMAP_H
#define PARALLELBITMAP_H
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <ctype.h>
#include <pthread.h>
#include <malloc.h>
#include <sys/time.h>
#include <sys/file.h>
#include <unistd.h>
#include <sched.h>
#include "../general/Bitmap.h"
#include "../general/LocalBitmap.h"
using namespace std;

#define MAX_THREAD 70
#define SPECULATIVE 10
#define NONSPECULATIVE 11

class ParallelBitmap : public Bitmap {
    friend class ParallelBitmapConstructor;
    friend class ParallelBitmapIterator;
  private:
    LocalBitmap* mBitmaps[MAX_THREAD];
    int mThreadNum;
    char* mRecord;
    long mRecordLength;
    int mDepth;
    int mParallelMode;

  public:
    ParallelBitmap(char* record, int thread_num, int depth);
    ParallelBitmap(char* record, long rec_len, int thread_num, int depth);
    ~ParallelBitmap();
    void setRecordLength(long length);
    // SPECULATIVE or NONSPECULATIVE
    int parallelMode();
    // validation after step 3
    void rectifyStringMaskBitmaps();
    // validation after step 5
    void mergeBitmaps();
};
#endif

Writing /content/parallelSpec/ParallelBitmap.h


In [ ]:
%%writefile /content/parallelSpec/ParallelBitmapConstructor.cpp

#include "ParallelBitmapConstructor.h"

ParallelBitmap* ParallelBitmapConstructor::mParallelBitmap = NULL;
pthread_t ParallelBitmapConstructor::mThreads[MAX_THREAD];
int ParallelBitmapConstructor::mThreadArgs[MAX_THREAD];

ParallelBitmap* ParallelBitmapConstructor::construct(Record* record, int thread_num, int level_num) {
    char* record_text = NULL;
    long length = 0;
    if (record->rec_start_pos > 0) record_text = record->text + record->rec_start_pos;
    else record_text = record->text;
    if (record->rec_length > 0) length = record->rec_length;
    else length = strlen(record->text);
    mParallelBitmap = new ParallelBitmap(record_text, length, thread_num, level_num);
    int mode = mParallelBitmap->parallelMode();
    if (mode == NONSPECULATIVE) {
        for (int i = 0; i < thread_num; ++i) {
            mThreadArgs[i] = i;
            int rc=pthread_create(&mThreads[i], NULL, &ParallelBitmapConstructor::nonSpecIndexConstruction, &mThreadArgs[i]);
            if (rc)
            {
                cout<<"Thread Error; return code is "<<rc<<endl;
                return NULL;
            }
        }
        for(int i = 0; i < thread_num; ++i)
        {
            int rc = pthread_join(mThreads[i], NULL);
            if (rc)
            {
                cout<<"Thread Error; return code is "<<rc<<endl;
                return NULL;
            }
        }
        mParallelBitmap->mergeBitmaps();
    } else {
        for (int i = 0; i < thread_num; ++i) {
            mThreadArgs[i] = i;
            int rc=pthread_create(&mThreads[i], NULL, &ParallelBitmapConstructor::buildStringMaskBitmap, &mThreadArgs[i]);
            if (rc)
            {
                cout<<"Thread Error; return code is "<<rc<<endl;
                return NULL;
            }
        }
        for(int i = 0; i < thread_num; ++i)
        {
            int rc = pthread_join(mThreads[i], NULL);
            if (rc)
            {
                cout<<"Thread Error; return code is "<<rc<<endl;
                return NULL;
            }
        }
        mParallelBitmap->rectifyStringMaskBitmaps();
        for (int i = 0; i < thread_num; ++i) {
            mThreadArgs[i] = i;
            int rc=pthread_create(&mThreads[i], NULL, &ParallelBitmapConstructor::buildLeveledBitmap, &mThreadArgs[i]);
            if (rc)
            {
                cout<<"Thread Error; return code is "<<rc<<endl;
                return NULL;
            }
        }
        for(int i = 0; i < thread_num; ++i)
        {
            int rc = pthread_join(mThreads[i], NULL);
            if (rc)
            {
                cout<<"Thread Error; return code is "<<rc<<endl;
                return NULL;
            }
        }
        mParallelBitmap->mergeBitmaps();
    }
    return mParallelBitmap;
}

Writing /content/parallelSpec/ParallelBitmapConstructor.cpp


In [ ]:
%%writefile /content/parallelSpec/ParallelBitmapConstructor.h

#ifndef PARALLELBITMAPCONSTRUCTOR_H
#define PARALLELBITMAPCONSTRUCTOR_H
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <ctype.h>
#include <pthread.h>
#include <malloc.h>
#include <sys/time.h>
#include <sys/file.h>
#include <unistd.h>
#include <sched.h>
#include "ParallelBitmap.h"
#include "../general/Records.h"
using namespace std;

// ParallelBitmapConstructor is a static class
class ParallelBitmapConstructor {
  private:
    static ParallelBitmap* mParallelBitmap;
    static pthread_t mThreads[MAX_THREAD];
    static int mThreadArgs[MAX_THREAD];
  public:
    static ParallelBitmap* construct(Record* record, int thread_num, int level_num = MAX_LEVEL);
  private:
    // builds bitmap index in non-speculative mode.
    static void* nonSpecIndexConstruction(void* arg) {
        int thread_id = (int)(*((int*)arg));
        // cout<<"thread "<<thread_id<<" starts building structral indexes."<<endl;
        // bind CPU
        cpu_set_t mask;
        cpu_set_t get;
        CPU_ZERO(&mask);
        CPU_SET(thread_id, &mask);
        if(pthread_setaffinity_np(pthread_self(), sizeof(mask), &mask) < 0)
            cout<<"CPU binding failed for thread "<<thread_id<<endl;
        struct timeval begin,end;
        double duration;
        gettimeofday(&begin,NULL);
        mParallelBitmap->mBitmaps[thread_id]->nonSpecIndexConstruction();
        // cout<<thread_id<<"th thread finishes structural index construction."<<endl;
        gettimeofday(&end,NULL);
        duration=1000000*(end.tv_sec-begin.tv_sec)+end.tv_usec-begin.tv_usec;
        // printf("The duration of index construction in %dth thread is %lf\n",thread_id, duration/1000000);
        return NULL;
    }
    // builds bitmap index in speculative mode (Step 1 - 3)
    static void* buildStringMaskBitmap(void* arg) {
        int thread_id = (int)(*((int*)arg));
        // bind CPU
        cpu_set_t mask;
        cpu_set_t get;
        CPU_ZERO(&mask);
        CPU_SET(thread_id, &mask);
        if(pthread_setaffinity_np(pthread_self(), sizeof(mask), &mask)<0)
            cout<<"CPU binding failed for thread "<<thread_id<<endl;
        //cout<<thread_id<<"th thread starts building string mask bitmap."<<endl;
        mParallelBitmap->mBitmaps[thread_id]->buildStringMaskBitmap();
        //cout<<thread_id<<"th thread finishes building string mask bitmap."<<endl;
        return NULL;
    }
    // finish the last two steps to finish strucutral index construction
    static void* buildLeveledBitmap(void* arg) {
        int thread_id = (int)(*((int*)arg));
        // bind CPU
        cpu_set_t mask;
        cpu_set_t get;
        CPU_ZERO(&mask);
        CPU_SET(thread_id, &mask);
        if(pthread_setaffinity_np(pthread_self(), sizeof(mask), &mask)<0)
            cout<<"CPU binding failed for thread "<<thread_id<<endl;
        mParallelBitmap->mBitmaps[thread_id]->buildLeveledBitmap();
        //cout<<thread_id<<"th thread finishes building leveled bitmap."<<endl;
        return NULL;
    }
};
#endif

Writing /content/parallelSpec/ParallelBitmapConstructor.h


In [ ]:
%%writefile /content/parallelSpec/ParallelBitmapIterator.cpp

#include "ParallelBitmapIterator.h"
#include <sys/time.h>
#include <pthread.h>

struct ParallelBitmapMetadata {
    int start_word_id;
    int end_word_id;
    unsigned long* quote_bitmap;
    unsigned long* lev_colon_bitmap[MAX_LEVEL + 1];
    unsigned long* lev_comma_bitmap[MAX_LEVEL + 1];
};

ParallelBitmapMetadata pb_metadata[MAX_THREAD];

typedef struct CommaPosInfo {
    int thread_id;
    int level;
    long start_pos;
    long end_pos;
    long* comma_positions;
    long top_comma_positions;
}CommaPosInfo;

CommaPosInfo comma_pos_info[MAX_THREAD];

int num_of_threads = 12;

void* generateCommaPositionsInThread(void* arg) {
    int thread_id = (int)(*((int*)arg));;
    int level = comma_pos_info[thread_id].level;
    long start_pos = comma_pos_info[thread_id].start_pos;
    long end_pos = comma_pos_info[thread_id].end_pos;
    comma_pos_info[thread_id].comma_positions = new long[MAX_NUM_ELE /num_of_threads + 1];
    comma_pos_info[thread_id].top_comma_positions = -1;
    // cout<<"thread "<<thread_id<<" start generating comma positions."<<endl;
    // bind CPU
    cpu_set_t mask;
    cpu_set_t get;
    CPU_ZERO(&mask);
    CPU_SET(thread_id, &mask);
    if(pthread_setaffinity_np(pthread_self(), sizeof(mask), &mask) < 0)
        cout<<"CPU binding failed for thread "<<thread_id<<endl;
    unsigned long* levels = pb_metadata[thread_id].lev_comma_bitmap[level];
    if (levels == NULL) {
        return NULL;
    }
    unsigned long commabit;
    long cur_start_pos = pb_metadata[thread_id].start_word_id;
    long cur_end_pos = pb_metadata[thread_id].end_word_id;
    long st = cur_start_pos > (start_pos / 64) ? cur_start_pos : (start_pos / 64);
    long ed = cur_end_pos < (ceil(double(end_pos) / 64)) ? cur_end_pos : (ceil(double(end_pos) / 64));
    for (long i = st; i < ed; ++i) {
        unsigned long idx = 0;
        if (thread_id >= 1) idx = i - cur_start_pos;
        else idx = i;
        commabit = levels[idx];
        int cnt = __builtin_popcountl(commabit);
        while (commabit) {
            long offset = i * 64 + __builtin_ctzll(commabit);
            if (start_pos <= offset && offset <= end_pos) {
                comma_pos_info[thread_id].comma_positions[++comma_pos_info[thread_id].top_comma_positions] = offset;
            }
            commabit = commabit & (commabit - 1);
        }
    }
    return NULL;
}

void ParallelBitmapIterator::generateCommaPositionsParallel(long start_pos, long end_pos, int level, long* comma_positions, long& top_comma_positions) {
    // find starting and ending chunks in linked leveled comma bitmaps
    int start_chunk = -1;
    int end_chunk = -1;
    int chunk_num = mParallelBitmap->mThreadNum;
    for (int i = mCurChunkId; i < chunk_num; ++i) {
        if (pb_metadata[i].start_word_id <= (start_pos / 64)) {
            start_chunk = i;
        }
        if (pb_metadata[i].end_word_id >= (ceil(double(end_pos) / 64)) && end_chunk == -1) {
            end_chunk = i;
        }
        if (start_chunk > -1 && end_chunk > -1) break;
    }
    if(start_chunk == 0 && end_chunk == -1) end_chunk = 0;
    mCurChunkId = start_chunk;
    pthread_t thread[MAX_THREAD];
    int thread_args[MAX_THREAD];
    // iterate through corresponding linked leveled comma bitmaps
    for (int i = start_chunk; i <= end_chunk; ++i) {
        thread_args[i] = i;
        comma_pos_info[i].thread_id = i;
        comma_pos_info[i].level = level;
        comma_pos_info[i].start_pos = start_pos;
        comma_pos_info[i].end_pos = end_pos;
        int rc = pthread_create(&thread[i], NULL, generateCommaPositionsInThread, &thread_args[i]);
        if (rc)
        {
            cout<<"Thread Error; return code is "<<rc<<endl;
            return;
        }
    }
    for (int i = start_chunk; i <= end_chunk; ++i) {
        int rc = pthread_join(thread[i], NULL);
        if (rc)
        {
            cout<<"Thread Error; return code is "<<rc<<endl;
            return;
        }
    }
    for (int i = start_chunk; i <= end_chunk; ++i) {
        for (int j = 0; j <= comma_pos_info[i].top_comma_positions; ++j) {
            comma_positions[++top_comma_positions] = comma_pos_info[i].comma_positions[j];
        }
        free(comma_pos_info[i].comma_positions);
    }
}

// Saving metadata of linked leveled bitmap in consecutive order can further improve the performance.
void ParallelBitmapIterator::gatherParallelBitmapInfo() {
    int chunk_num = mParallelBitmap->mThreadNum;
    int depth = mParallelBitmap->mDepth;
    for (int chunk_id = 0; chunk_id < chunk_num; ++chunk_id) {
        pb_metadata[chunk_id].start_word_id = mParallelBitmap->mBitmaps[chunk_id]->mStartWordId;
        pb_metadata[chunk_id].end_word_id = mParallelBitmap->mBitmaps[chunk_id]->mEndWordId;
        pb_metadata[chunk_id].quote_bitmap = mParallelBitmap->mBitmaps[chunk_id]->mQuoteBitmap;
        for (int l = 0; l <= depth; ++l) {
            pb_metadata[chunk_id].lev_colon_bitmap[l] = mParallelBitmap->mBitmaps[chunk_id]->mFinalLevColonBitmap[l];
            pb_metadata[chunk_id].lev_comma_bitmap[l] = mParallelBitmap->mBitmaps[chunk_id]->mFinalLevCommaBitmap[l];
        }
    }
}

void ParallelBitmapIterator::generateColonPositions(long start_pos, long end_pos, int level, long* colon_positions, long& top_colon_positions) {
    // find starting and ending chunks in linked leveled colon bitmaps
    int start_chunk = -1;
    int end_chunk = -1;
    int thread_num = mParallelBitmap->mThreadNum;
    for (int i = mCurChunkId; i < thread_num; ++i) {
        if (pb_metadata[i].start_word_id <= (start_pos / 64)) {
            start_chunk = i;
        }
        if (pb_metadata[i].end_word_id >= (ceil(double(end_pos) / 64)) && end_chunk == -1) {
            end_chunk = i;
        }
        if (start_chunk > -1 && end_chunk > -1) break;
    }
    if(start_chunk == 0 && end_chunk == -1) end_chunk = 0;
    mCurChunkId = start_chunk;
    // iterate through the corresponding linked leveled colon bitmaps
    int cur_chunk = start_chunk;
    while (cur_chunk <= end_chunk) {
        unsigned long* levels = pb_metadata[cur_chunk].lev_colon_bitmap[level];
        if (levels == NULL) {
            ++cur_chunk;
            continue;
        }
        unsigned long colonbit;
        long cur_start_pos = pb_metadata[cur_chunk].start_word_id;
        long cur_end_pos = pb_metadata[cur_chunk].end_word_id;
        long st = cur_start_pos > (start_pos / 64) ? cur_start_pos : (start_pos / 64);
        long ed = cur_end_pos < (ceil(double(end_pos) / 64)) ? cur_end_pos : (ceil(double(end_pos) / 64));
        for (long i = st; i < ed; ++i) {
            unsigned long idx = 0;
            if (cur_chunk >= 1) idx = i - cur_start_pos;
            else idx = i;
            colonbit = levels[idx];
            int cnt = __builtin_popcountl(colonbit);
            while (colonbit) {
                long offset = i * 64 + __builtin_ctzll(colonbit);
                if (start_pos <= offset && offset <= end_pos) {
                    colon_positions[++top_colon_positions] = offset;
                }
                colonbit = colonbit & (colonbit - 1);
            }
        }
        ++cur_chunk;
    }
}

void ParallelBitmapIterator::generateCommaPositions(long start_pos, long end_pos, int level, long* comma_positions, long& top_comma_positions) {
    // find starting and ending chunks in linked leveled comma bitmaps
    int start_chunk = -1;
    int end_chunk = -1;
    int chunk_num = mParallelBitmap->mThreadNum;
    for (int i = mCurChunkId; i < chunk_num; ++i) {
        if (pb_metadata[i].start_word_id <= (start_pos / 64)) {
            start_chunk = i;
        }
        if (pb_metadata[i].end_word_id >= (ceil(double(end_pos) / 64)) && end_chunk == -1) {
            end_chunk = i;
        }
        if (start_chunk > -1 && end_chunk > -1) break;
    }
    if(start_chunk == 0 && end_chunk == -1) end_chunk = 0;
    mCurChunkId = start_chunk;
    // iterate through the corresponding linked leveled comma bitmaps
    int cur_chunk = start_chunk;
    while (cur_chunk <= end_chunk) {
        unsigned long* levels = pb_metadata[cur_chunk].lev_comma_bitmap[level];
        if (levels == NULL) {
            ++cur_chunk;
            continue;
        }
        unsigned long commabit;
        long cur_start_pos = pb_metadata[cur_chunk].start_word_id;
        long cur_end_pos = pb_metadata[cur_chunk].end_word_id;
        long st = cur_start_pos > (start_pos / 64) ? cur_start_pos : (start_pos / 64);
        long ed = cur_end_pos < (ceil(double(end_pos) / 64)) ? cur_end_pos : (ceil(double(end_pos) / 64));
        for (long i = st; i < ed; ++i) {
            unsigned long idx = 0;
            if (cur_chunk >= 1) idx = i - cur_start_pos;
            else idx = i;
            commabit = levels[idx];
            int cnt = __builtin_popcountl(commabit);
            while (commabit) {
                long offset = i * 64 + __builtin_ctzll(commabit);
                if (start_pos <= offset && offset <= end_pos) {
                    comma_positions[++top_comma_positions] = offset;
                }
                commabit = commabit & (commabit - 1);
            }
        }
        ++cur_chunk;
    }
}

bool ParallelBitmapIterator::findFieldQuotePos(long colon_pos, long& start_pos, long& end_pos) {
    long w_id = colon_pos/64;
    long offset = colon_pos%64;
    long start_quote = 0;
    long end_quote = 0;
    start_pos = 0; end_pos = 0;
    int cur_chunk = -1;
    int chunk_num = mParallelBitmap->mThreadNum;
    // find the chunk where the current colon is in
    for (int i = mCurChunkId; i < chunk_num; ++i) {
        if (w_id >= pb_metadata[i].start_word_id && w_id < pb_metadata[i].end_word_id) {
            cur_chunk = i;
            break;
        }
    }
    if (cur_chunk == -1) {
        return false;
    }
    while (w_id >= 0)
    {
        // check whether the current chunk needs to be updated
        if (w_id < pb_metadata[cur_chunk].start_word_id) {
            //cout<<"update chunk id "<<cur_chunk<<endl;
            if ((--cur_chunk) == -1) {
                return false;
            }
        }
        long quote_id = w_id - pb_metadata[cur_chunk].start_word_id;
        unsigned long quotebit = pb_metadata[cur_chunk].quote_bitmap[quote_id];
        unsigned long offset = w_id * 64 + __builtin_ctzll(quotebit);
        while (quotebit && offset < colon_pos)
        {
            if (end_pos != 0)
            {
                start_quote = offset;
            }
            else if(start_quote == 0)
            {
                start_quote = offset;
            }
            else if(end_quote == 0)
            {
                end_quote = offset;
            }
            else
            {
                start_quote = end_quote;
                end_quote = offset;
            }
            quotebit = quotebit & (quotebit - 1);
            offset = w_id * 64 + __builtin_ctzll(quotebit);
        }
        if(start_quote != 0 && end_quote == 0)
        {
            end_quote = start_quote;
            start_quote = 0;
            end_pos = end_quote;
        }
        else if(start_quote != 0 && end_quote != 0)
        {
            start_pos = start_quote;
            end_pos = end_quote;
            return true;
        }
        --w_id;
    }
    return false;
}

ParallelBitmapIterator* ParallelBitmapIterator::getCopy() {
    ParallelBitmapIterator* pbi = new ParallelBitmapIterator();
    pbi->mParallelBitmap = mParallelBitmap;
    pbi->mCurLevel = mCurLevel;
    pbi->mTopLevel = mCurLevel;
    pbi->mCurChunkId = mCurChunkId;
    pbi->mFindDomArray = mFindDomArray;
    if (pbi->mTopLevel >= 0) {
        pbi->mCtxInfo[mCurLevel].type = mCtxInfo[mCurLevel].type;
        pbi->mCtxInfo[mCurLevel].positions = mCtxInfo[mCurLevel].positions;
        pbi->mCtxInfo[mCurLevel].start_idx = mCtxInfo[mCurLevel].start_idx;
        pbi->mCtxInfo[mCurLevel].end_idx = mCtxInfo[mCurLevel].end_idx;
        pbi->mCtxInfo[mCurLevel].cur_idx = -1;
        pbi->mPosArrAlloc[mCurLevel] = mPosArrAlloc[mCurLevel];
        pbi->mCtxInfo[mCurLevel + 1].positions = NULL;
        for (int i = mCurLevel + 1; i < MAX_LEVEL; ++i) {
            pbi->mPosArrAlloc[i] = false;
            pbi->mPosArrAlloc[i] = NULL;
        }
    }
    pbi->mCopiedIterator = true;
    return pbi;
}

bool ParallelBitmapIterator::up() {
    if (mCurLevel == mTopLevel) return false;
    --mCurLevel;
    return true;
}

bool ParallelBitmapIterator::down() {
    if (mCurLevel < mTopLevel || mCurLevel > mParallelBitmap->mDepth) return false;
    ++mCurLevel;
    long  start_pos = -1;
    long end_pos = -1;
    int thread_num = mParallelBitmap->mThreadNum;
    if (mCurLevel == mTopLevel + 1) {
        if (mTopLevel == -1) {
            long text_length = mParallelBitmap->mRecordLength;
            start_pos = 0;
            end_pos = text_length;
            mCtxInfo[mCurLevel].positions = (long*)malloc((text_length / thread_num + 1) * sizeof (long));
            mPosArrAlloc[mCurLevel] = true;
        } else {
            long cur_idx = mCtxInfo[mCurLevel - 1].cur_idx;
            start_pos = mCtxInfo[mCurLevel - 1].positions[cur_idx];
            end_pos = mCtxInfo[mCurLevel - 1].positions[cur_idx + 1];
            if (mCtxInfo[mCurLevel].positions == NULL || mPosArrAlloc[mCurLevel] == false) {
                mCtxInfo[mCurLevel].positions = (long*)malloc((MAX_NUM_ELE / thread_num + 1) * sizeof (long));
                mPosArrAlloc[mCurLevel] = true;
            }
        }
        mCtxInfo[mCurLevel].start_idx = 0;
        mCtxInfo[mCurLevel].cur_idx = -1;
        mCtxInfo[mCurLevel].end_idx = -1;
    } else {
        long cur_idx = mCtxInfo[mCurLevel - 1].cur_idx;
        if (cur_idx > mCtxInfo[mCurLevel - 1].end_idx) {
            --mCurLevel;
            return false;
        }
        start_pos = mCtxInfo[mCurLevel - 1].positions[cur_idx];
        end_pos = mCtxInfo[mCurLevel - 1].positions[cur_idx + 1];
        mCtxInfo[mCurLevel].positions = mCtxInfo[mCurLevel - 1].positions;
        mCtxInfo[mCurLevel].start_idx = mCtxInfo[mCurLevel - 1].end_idx + 1;
        mCtxInfo[mCurLevel].cur_idx = mCtxInfo[mCurLevel - 1].end_idx;
        mCtxInfo[mCurLevel].end_idx = mCtxInfo[mCurLevel - 1].end_idx;
    }
    long i = start_pos;
    if (start_pos > 0 || mCurLevel > 0) ++i;
    char ch = mParallelBitmap->mRecord[i];
    while (i < end_pos && (ch == ' ' || ch == '\n')) {
        ch = mParallelBitmap->mRecord[++i];
    }
    if (mParallelBitmap->mRecord[i] == '{') {
        mCtxInfo[mCurLevel].type = OBJECT;
        generateColonPositions(i, end_pos, mCurLevel, mCtxInfo[mCurLevel].positions, mCtxInfo[mCurLevel].end_idx);
        return true;
    } else if (mParallelBitmap->mRecord[i] == '[') {
        mCtxInfo[mCurLevel].type = ARRAY;
        if (mFindDomArray == false && (end_pos - i + 1) > SINGLE_THREAD_MAX_ARRAY_SIZE) {
            generateCommaPositionsParallel(i, end_pos, mCurLevel, mCtxInfo[mCurLevel].positions, mCtxInfo[mCurLevel].end_idx);
            mFindDomArray = true;
        } else {
            generateCommaPositions(i, end_pos, mCurLevel, mCtxInfo[mCurLevel].positions, mCtxInfo[mCurLevel].end_idx);
        }
        return true;
    }
    --mCurLevel;
    return false;
}

bool ParallelBitmapIterator::isObject() {
    if (mCurLevel >= 0 && mCurLevel <= mParallelBitmap->mDepth && mCtxInfo[mCurLevel].type == OBJECT) {
        return true;
    }
    return false;
}

bool ParallelBitmapIterator::isArray() {
    if (mCurLevel >= 0 && mCurLevel <= mParallelBitmap->mDepth && mCtxInfo[mCurLevel].type == ARRAY) {
        return true;
    }
    return false;
}

bool ParallelBitmapIterator::moveNext() {
    if (mCurLevel < 0 || mCurLevel > mParallelBitmap->mDepth || mCtxInfo[mCurLevel].type != ARRAY) return false;
    long next_idx = mCtxInfo[mCurLevel].cur_idx + 1;
    if (next_idx >= mCtxInfo[mCurLevel].end_idx) return false;
    mCtxInfo[mCurLevel].cur_idx = next_idx;
    return true;
}

bool ParallelBitmapIterator::moveToKey(char* key) {
    if (mCurLevel < 0 || mCurLevel > mParallelBitmap->mDepth || mCtxInfo[mCurLevel].type != OBJECT) return false;
    long cur_idx = mCtxInfo[mCurLevel].cur_idx + 1;
    long end_idx = mCtxInfo[mCurLevel].end_idx;
    while (cur_idx < end_idx) {
        long colon_pos = mCtxInfo[mCurLevel].positions[cur_idx];
        long start_pos = 0, end_pos = 0;
        if (!findFieldQuotePos(colon_pos, start_pos, end_pos)) {
            return false;
        }
        int key_size = end_pos - start_pos - 1;
        if (key_size == strlen(key)) {
            memcpy(mKey, mParallelBitmap->mRecord + start_pos + 1, key_size);
            mKey[end_pos - start_pos - 1] = '\0';
            if (memcmp(mKey, key, key_size) == 0) {
                mCtxInfo[mCurLevel].cur_idx = cur_idx;
                return true;
            }
        }
        ++cur_idx;
    }
    return false;
}

char* ParallelBitmapIterator::moveToKey(unordered_set<char*>& key_set) {
    if (key_set.empty() == true || mCurLevel < 0 || mCurLevel > mParallelBitmap->mDepth || mCtxInfo[mCurLevel].type != OBJECT) return NULL;
    long cur_idx = mCtxInfo[mCurLevel].cur_idx + 1;
    long end_idx = mCtxInfo[mCurLevel].end_idx;
    while (cur_idx < end_idx) {
        long colon_pos = mCtxInfo[mCurLevel].positions[cur_idx];
        long start_pos = 0, end_pos = 0;
        if (!findFieldQuotePos(colon_pos, start_pos, end_pos)) {
            return NULL;
        }
        bool has_m_key = false;
        unordered_set<char*>::iterator iter;
        for (iter = key_set.begin(); iter != key_set.end(); ++iter) {
            char* key = (*iter);
            int key_size = end_pos - start_pos - 1;
            if (key_size == strlen(key)) {
                if (has_m_key == false) {
                    memcpy(mKey, mParallelBitmap->mRecord + start_pos + 1, key_size);
                    mKey[end_pos - start_pos - 1] = '\0';
                    has_m_key = true;
                }
                if (memcmp(mKey, key, key_size) == 0) {
                    mCtxInfo[mCurLevel].cur_idx = cur_idx;
                    key_set.erase(iter);
                    return key;
                }
            }
        }
        ++cur_idx;
    }
    mCtxInfo[mCurLevel].cur_idx = cur_idx;
    return NULL;
}

int ParallelBitmapIterator::numArrayElements() {
    if (mCurLevel >= 0 && mCurLevel <= mParallelBitmap->mDepth && mCtxInfo[mCurLevel].type == ARRAY) {
        return mCtxInfo[mCurLevel].end_idx - mCtxInfo[mCurLevel].start_idx;
    }
    return 0;
}

bool ParallelBitmapIterator::moveToIndex(int index) {
    if (mCurLevel < 0 || mCurLevel > mParallelBitmap->mDepth || mCtxInfo[mCurLevel].type != ARRAY) return false;
    long next_idx = mCtxInfo[mCurLevel].start_idx + index;
    if (next_idx > mCtxInfo[mCurLevel].end_idx) return false;
    mCtxInfo[mCurLevel].cur_idx = next_idx;
    return true;
}

char* ParallelBitmapIterator::getValue() {
    if (mCurLevel < 0 || mCurLevel > mParallelBitmap->mDepth) return NULL;
    long cur_idx = mCtxInfo[mCurLevel].cur_idx;
    long next_idx = cur_idx + 1;
    if (next_idx > mCtxInfo[mCurLevel].end_idx) return NULL;
    // current ':' or ','
    long cur_pos = mCtxInfo[mCurLevel].positions[cur_idx];
    // next ':' or ','
    long next_pos = mCtxInfo[mCurLevel].positions[next_idx];
    int type = mCtxInfo[mCurLevel].type;
    if (type == OBJECT && next_idx < mCtxInfo[mCurLevel].end_idx) {
        long start_pos = 0, end_pos = 0;
        if (findFieldQuotePos(next_pos, start_pos, end_pos) == false) {
            return "";
        }
        // next quote
        next_pos = start_pos;
    }
    long text_length = next_pos - cur_pos - 1;
    if (text_length <= 0) return "";
    char* ret = (char*)malloc(text_length + 1);
    memcpy(ret, mParallelBitmap->mRecord + cur_pos + 1, text_length);
    ret[text_length] = '\0';
    return ret;
}


Writing /content/parallelSpec/ParallelBitmapIterator.cpp


In [ ]:
%%writefile /content/parallelSpec/ParallelBitmapIterator.h

#ifndef PARALLELBITMAPITERATOR_H
#define PARALLELBITMAPITERATOR_H

#include "ParallelBitmapConstructor.h"
#include "../general/BitmapIterator.h"
#include <string.h>
#include <unordered_set>
using namespace std;

#define MAX_NUM_ELE 1000000
#define SINGLE_THREAD_MAX_ARRAY_SIZE 100000

class ParallelBitmapIterator : public BitmapIterator {
  private:
    ParallelBitmap* mParallelBitmap;
    IterCtxInfo mCtxInfo[MAX_LEVEL];
    bool mPosArrAlloc[MAX_LEVEL];
    int mCurLevel;
    int mTopLevel;
    char mKey[MAX_FIELD_SIZE];
    // improve the performance when iterating leveled bitmap generated by multiple threads
    // mCurChunkId: pointer to the chunk which generates the current leveled bitmaps this iterator visits
    int mCurChunkId;
    bool mFindDomArray;
    bool mCopiedIterator;

  public:
    ParallelBitmapIterator() {}

    ParallelBitmapIterator(ParallelBitmap* pbm) {
        mParallelBitmap = pbm;
        mCurLevel = -1;
        mTopLevel = -1;
        mCurChunkId = 0;
        mFindDomArray = false;
        mCopiedIterator = false;
        for (int i = 0; i < MAX_LEVEL; ++i) {
            mPosArrAlloc[i] = false;
        }
        gatherParallelBitmapInfo();
        // initially, iterator points to the first level of the record
        down();
    }

    ~ParallelBitmapIterator() {
        for (int i = mTopLevel + 1; i < MAX_LEVEL; ++i) {
            if (mPosArrAlloc[i] == true) {
                free(mCtxInfo[i].positions);
                mCtxInfo[i].positions = NULL;
            } else {
                break;
            }
        }
    }

    // Creates a copy of iterator. Often used for parallel querying.
    ParallelBitmapIterator* getCopy();
    // Moves back to the object or array which contains the current nested record.
    // Often used when the current nested record has been processed.
    // Valid except for the first level of the record.
    bool up();
    // Moves to the start of the nested object or array.
    // Gets all colon or comma positions from leveled bitmap indexes for current nested record.
    // Valid if we are at { or [.
    bool down();
    // Whether the iterator points to an object.
    bool isObject();
    // Whether the iterator points to an array.
    bool isArray();
    // Moves iterator to the next array item.
    bool moveNext();
    // Returns the size of the current key field.
    int keySize();
    // Gets the content of the current key field.
    char* getKey();
    // Moves to the corresponding key field inside the current object.
    bool moveToKey(char* key);
    // Moves to the corresponding key fields inside the current object, returns the current key name.
    // After this operation, the current key field will be removed from key_set.
    char* moveToKey(unordered_set<char*>& key_set);
    // Returns the number of elements inside current array.
    int numArrayElements();
    // If the current record is an array, moves to an item based on index.
    // Returns false if the index is out of the boundary.
    bool moveToIndex(int index);
    // Gets the content of the current value inside an object or array.
    char* getValue();

  private:
    // this operation can further improve the performance.
    void gatherParallelBitmapInfo();
    // get positions of all colons between start_idx and end_idx from input stream
    // prev_thread_id: thread which generates leveled bitmap indexes for the previous word
    void generateColonPositions(long start_pos, long end_pos, int level, long* colon_positions, long& top_colon_positions);
    // get positions of all commas between start_idx and end_idx from input stream
    // prev_thread_id: thread which generates leveled bitmap indexes for the previous word
    void generateCommaPositions(long start_pos, long end_pos, int level, long* comma_positions, long& top_comma_positions);
    // generate comma positions in parallel (used for generating positions of dominating array)
    void generateCommaPositionsParallel(long start_pos, long end_pos, int level, long* comma_positions, long& top_comma_positions);
    // prev_thread_id: thread which generates leveled bitmap indexes for the previous word
    bool findFieldQuotePos(long colon_pos, long& start_pos, long& end_pos);
};
#endif

Writing /content/parallelSpec/ParallelBitmapIterator.h


In [ ]:
%%writefile /content/serialSpec/SerialBitmap.cpp

#include "SerialBitmap.h"
#include <immintrin.h>

#include <emmintrin.h>
#include <xmmintrin.h>
#include <string.h>

#include <sys/time.h>
#include <stdio.h>
#include <string.h>
#include <stdlib.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <ctype.h>
#include <pthread.h>
#include <cstdint>
#include <sys/time.h>
#include <sys/file.h>
#include <unistd.h>
#include <sched.h>
#include <unordered_map>

SerialBitmap::SerialBitmap() {

}

SerialBitmap::SerialBitmap(char* record, int level_num) {
    this->mRecord = record;
    this->mDepth = level_num - 1;
    this->mQuoteBitmap = NULL;
    for (int i = 0; i <= this->mDepth; ++i) {
        this->mLevColonBitmap[i] = NULL;
        this->mLevCommaBitmap[i] = NULL;
    }
}

void SerialBitmap::freeMemory()
{
    for(int m = 0; m <= mDepth; m++){
        if (mLevColonBitmap[m]) {
            free(mLevColonBitmap[m]);
            mLevColonBitmap[m] = NULL;
        }
        if (mLevCommaBitmap[m]) {
            free(mLevCommaBitmap[m]);
            mLevCommaBitmap[m] = NULL;
        }
    }
    if (mQuoteBitmap) {
        free(mQuoteBitmap);
        mQuoteBitmap = NULL;
    }
}

SerialBitmap::~SerialBitmap()
{
    freeMemory();
}

void SerialBitmap::setRecordLength(long length) {
    this->mRecordLength = length;
    this->mNumTmpWords = length / 32;
    this->mNumWords = length / 64;
    // allocate memory space for saving temporary results
    if (!mQuoteBitmap) {
        mQuoteBitmap = (unsigned long*)malloc((mNumWords) * sizeof(unsigned long));
    }
}

void SerialBitmap::indexConstruction() {
    // vectors for structural characters
    __m256i v_quote = _mm256_set1_epi8(0x22);
    __m256i v_colon = _mm256_set1_epi8(0x3a);
    __m256i v_escape = _mm256_set1_epi8(0x5c);
    __m256i v_lbrace = _mm256_set1_epi8(0x7b);
    __m256i v_rbrace = _mm256_set1_epi8(0x7d);
    __m256i v_comma = _mm256_set1_epi8(0x2c);
    __m256i v_lbracket = _mm256_set1_epi8(0x5b);
    __m256i v_rbracket = _mm256_set1_epi8(0x5d);

    // variables for saving temporary results in the first four steps
    unsigned long colonbit0, quotebit0, escapebit0, stringbit0, lbracebit0, rbracebit0, commabit0, lbracketbit0, rbracketbit0;
    unsigned long colonbit, quotebit, escapebit, stringbit, lbracebit, rbracebit, commabit, lbracketbit, rbracketbit;
    unsigned long str_mask;

    // variables for saving temporary results in the last step
    unsigned long lb_mask, rb_mask, cb_mask;
    unsigned long lb_bit, rb_bit, cb_bit;
    unsigned long first, second;
    int cur_level = -1;
    int max_positive_level = -1;

    // variables for saving context information among different words
    int top_word = -1;
    uint64_t prev_iter_ends_odd_backslash = 0ULL;
    uint64_t prev_iter_inside_quote = 0ULL;
    const uint64_t even_bits = 0x5555555555555555ULL;
    const uint64_t odd_bits = ~even_bits;

    for (int j = 0; j < mNumTmpWords; ++j) {
        colonbit = 0, quotebit = 0, escapebit = 0, stringbit = 0, lbracebit = 0, rbracebit = 0, commabit = 0, lbracketbit = 0, rbracketbit = 0;
        unsigned long i = j * 32;
        // step 1: build structural character bitmaps
        __m256i v_text = _mm256_loadu_si256(reinterpret_cast<const __m256i *>(mRecord + i));
        colonbit = (unsigned)_mm256_movemask_epi8(_mm256_cmpeq_epi8(v_text, v_colon));
        quotebit = (unsigned)_mm256_movemask_epi8(_mm256_cmpeq_epi8(v_text, v_quote));
        escapebit = (unsigned)_mm256_movemask_epi8(_mm256_cmpeq_epi8(v_text, v_escape));
        lbracebit  = (unsigned)_mm256_movemask_epi8(_mm256_cmpeq_epi8(v_text, v_lbrace));
        rbracebit  = (unsigned)_mm256_movemask_epi8(_mm256_cmpeq_epi8(v_text, v_rbrace));
        commabit = (unsigned)_mm256_movemask_epi8(_mm256_cmpeq_epi8(v_text, v_comma));
	    lbracketbit = (unsigned)_mm256_movemask_epi8(_mm256_cmpeq_epi8(v_text, v_lbracket));
	    rbracketbit = (unsigned)_mm256_movemask_epi8(_mm256_cmpeq_epi8(v_text, v_rbracket));
        // first half of the word (lowest 32 bits)
        if(j % 2 == 0) {
            colonbit0 = colonbit;
            quotebit0 = quotebit;
            escapebit0 = escapebit;
            lbracebit0 = lbracebit;
            rbracebit0 = rbracebit;
            commabit0 = commabit;
            lbracketbit0 = lbracketbit;
            rbracketbit0 = rbracketbit;
            continue;
        } else {
            // highest 32 bits inside a word
            colonbit = (colonbit << 32) | colonbit0;
            quotebit = (quotebit << 32) | quotebit0;
            escapebit = (escapebit << 32) | escapebit0;
            lbracebit = (lbracebit << 32) | lbracebit0;
            rbracebit = (rbracebit << 32) | rbracebit0;
            commabit = (commabit << 32) | commabit0;
            lbracketbit = (lbracketbit << 32) | lbracketbit0;
            rbracketbit = (rbracketbit << 32) | rbracketbit0;

            // step 2: update structural quote bitmaps
            uint64_t bs_bits = escapebit;
            uint64_t start_edges = bs_bits & ~(bs_bits << 1);
            int64_t even_start_mask = even_bits ^ prev_iter_ends_odd_backslash;
            uint64_t even_starts = start_edges & even_start_mask;
            uint64_t odd_starts = start_edges & ~even_start_mask;
            uint64_t even_carries = bs_bits + even_starts;
            int64_t odd_carries;
            bool iter_ends_odd_backslash = __builtin_uaddll_overflow(bs_bits, odd_starts,
                (unsigned long long *)(&odd_carries)); // THIS POINTER WAS LONG LONG BEFORE, MEANING IT WAS 128 BITS? ONLY 64 NOW. BUT ALL POINTERS ARE 64 BITS??
            odd_carries |= prev_iter_ends_odd_backslash;
            prev_iter_ends_odd_backslash = iter_ends_odd_backslash ? 0x1ULL : 0x0ULL;
            uint64_t even_carry_ends = even_carries & ~bs_bits;
            uint64_t odd_carry_ends = odd_carries & ~bs_bits;
            uint64_t even_start_odd_end = even_carry_ends & odd_bits;
            uint64_t odd_start_even_end = odd_carry_ends & even_bits;
            uint64_t odd_ends = even_start_odd_end | odd_start_even_end;
            int64_t quote_bits = quotebit & ~odd_ends;
            mQuoteBitmap[++top_word] = quote_bits;

            // step 3: build string mask bitmaps
            __m128i mulProduct = _mm_clmulepi64_si128(_mm_set_epi64x(0ULL, quote_bits), _mm_set1_epi8(0xFFu), 0);

            unsigned long value[2];
            _mm_storeu_si128((__m128i*)value, mulProduct);

            str_mask = value[0];

            str_mask ^= prev_iter_inside_quote;
            prev_iter_inside_quote = static_cast<uint64_t>(static_cast<int64_t>(str_mask) >> 63);

            // step 4: update structural character bitmaps
            unsigned long tmp = (~str_mask);
            colonbit = colonbit & tmp;
            lbracebit = lbracebit & tmp;
            rbracebit = rbracebit & tmp;
            commabit = commabit & tmp;
            lbracketbit = lbracketbit & tmp;
            rbracketbit = rbracketbit & tmp;

            // step 5: generate leveled bitmaps
            lb_mask = lbracebit | lbracketbit;
            rb_mask = rbracebit | rbracketbit;
            cb_mask = lb_mask | rb_mask;
            lb_bit = lb_mask & (-lb_mask);
            rb_bit = rb_mask & (-rb_mask);
            if (!cb_mask) {
                if (cur_level >= 0 && cur_level <= mDepth) {
                    if (!mLevColonBitmap[cur_level]) {
                        mLevColonBitmap[cur_level] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                    }
                    if (!mLevCommaBitmap[cur_level]) {
                        mLevCommaBitmap[cur_level] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                    }
                    if (colonbit) {
                        mLevColonBitmap[cur_level][top_word] = colonbit;
                    } else {
                        mLevCommaBitmap[cur_level][top_word] = commabit;
	            }
	        }
            } else {
                first = 1;
                while (cb_mask || first) {
                    if (!cb_mask) {
                        // COMMA COMES FROM HERE?????
                        second = 1UL<<63;
                    } else {
                        cb_bit = cb_mask & (-cb_mask);
                        second = cb_bit;
                    }
                    if (cur_level >= 0 && cur_level <= mDepth) {
                        if (!mLevColonBitmap[cur_level]) {
                            mLevColonBitmap[cur_level] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                        }
                        if (!mLevCommaBitmap[cur_level]) {
                            mLevCommaBitmap[cur_level] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                        }
                        unsigned long mask = second - first;
                        if (!cb_mask) mask = mask | second;
                        unsigned long colon_mask = mask & colonbit;
                        if (colon_mask) {
                            mLevColonBitmap[cur_level][top_word] |= colon_mask;
                        } else {
                            mLevCommaBitmap[cur_level][top_word] |= (commabit & mask);
                        }
                        if (cb_mask) {
                            if (cb_bit == rb_bit) {
                                mLevColonBitmap[cur_level][top_word] |= cb_bit;
                                mLevCommaBitmap[cur_level][top_word] |= cb_bit;
                            }
                            else if (cb_bit == lb_bit && cur_level + 1 <= mDepth) {
                                if (!mLevCommaBitmap[cur_level + 1]) {
                                     mLevCommaBitmap[cur_level + 1] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                                }
                                mLevCommaBitmap[cur_level + 1][top_word] |= cb_bit;
                            }
                        }
                    }
                    if (cb_mask) {
                        if (cb_bit == lb_bit) {
                            lb_mask = lb_mask & (lb_mask - 1);
                            lb_bit = lb_mask & (-lb_mask);
                            ++cur_level;
                            if (cur_level == 0) {
                                // JSON record could be an array
                                if (!mLevCommaBitmap[cur_level]) {
                                     mLevCommaBitmap[cur_level] = (unsigned long*)calloc(mNumWords, sizeof(unsigned long));
                                }
                                mLevCommaBitmap[cur_level][top_word] |= cb_bit;
                            }
                        } else if (cb_bit == rb_bit) {
                            rb_mask = rb_mask & (rb_mask - 1);
                            rb_bit = rb_mask & (-rb_mask);
                            --cur_level;
                        }
                        first = second;
                        cb_mask = cb_mask & (cb_mask - 1);
                        if (cur_level > max_positive_level) {
                            max_positive_level = cur_level;
                        }
                    } else {
                        first = 0;
                    }
                }
	    }
        }
    }
    if (mDepth == MAX_LEVEL - 1) mDepth = max_positive_level;
    //cout<<"cur level "<<cur_level<<endl;
}


Writing /content/serialSpec/SerialBitmap.cpp


In [ ]:
%%writefile /content/serialSpec/SerialBitmap.h

#ifndef SERIALBITMAP_H
#define SERIALBITMAP_H
#include <string>
#include <iostream>
#include <vector>
#include <bitset>
#include <cassert>
#include <stack>
#include <algorithm>
#include <unordered_map>
#include <functional>
#include <math.h>
#include <immintrin.h>
#include "../general/Bitmap.h"

class SerialBitmap : public Bitmap {
    friend class SerialBitmapIterator;
  private:
    char* mRecord;
    // for a single large record, stream length equals to record length
    long mRecordLength;
    // each temp word has 32 bytes
    long mNumTmpWords;
    // each word has 64 bytes
    long mNumWords;
    // structural quote bitmap, used for key field parsing
    unsigned long *mQuoteBitmap;
    // leveled colon bitmap
    unsigned long *mLevColonBitmap[MAX_LEVEL];
    // leveled comma bitmap
    unsigned long *mLevCommaBitmap[MAX_LEVEL];
    // the deepest level of leveled bitmap indexes (starting from 0)
    int mDepth;

  public:
    SerialBitmap();
    SerialBitmap(char* record, int level_num);
    ~SerialBitmap();
    void indexConstruction();
    void setRecordLength(long length);

  private:
    void freeMemory();
};
#endif


Writing /content/serialSpec/SerialBitmap.h


In [ ]:
%%writefile /content/serialSpec/SerialBitmapConstructor.cpp

#include "SerialBitmapConstructor.h"
#include <string.h>
#include <sys/time.h>
#include <sys/file.h>

SerialBitmap* SerialBitmapConstructor::construct(Record* record, int level_num) {
    char* record_text = NULL;
    long length = 0;
    if (record->rec_start_pos > 0) record_text = record->text + record->rec_start_pos;
    else record_text = record->text;
    if (record->rec_length > 0) length = record->rec_length;
    else length = strlen(record->text);
    SerialBitmap* bitmap = new SerialBitmap(record_text, level_num);
    bitmap->setRecordLength(length);
    bitmap->indexConstruction();
    return bitmap;
}


Writing /content/serialSpec/SerialBitmapConstructor.cpp


In [ ]:
%%writefile /content/serialSpec/SerialBitmapConstructor.h

#ifndef SERIALBITMAPCONSTRUCTOR_H
#define SERIALBITMAPCONSTRUCTOR_H

#include "SerialBitmap.h"
#include "../general/Records.h"

class SerialBitmapConstructor {
  public:
    static SerialBitmap* construct(Record* record, int level_num = MAX_LEVEL);
};
#endif


Writing /content/serialSpec/SerialBitmapConstructor.h


In [ ]:
%%writefile /content/serialSpec/SerialBitmapIterator.cpp

#include "SerialBitmapIterator.h"

void SerialBitmapIterator::generateColonPositions(long start_pos, long end_pos, int level, long* colon_positions, long& top_colon_positions) {
    unsigned long colonbit;
    long st = start_pos / 64;
    long ed = ceil(double(end_pos) / 64);
    for (long i = st; i < ed; ++i) {
        colonbit = mSerialBitmap->mLevColonBitmap[level][i];
        int cnt = __builtin_popcountl(colonbit);
        while (colonbit) {
            long offset = i * 64 + __builtin_ctzll(colonbit);
            if (start_pos <= offset && offset <= end_pos) {
                colon_positions[++top_colon_positions] = offset;
            }
            colonbit = colonbit & (colonbit - 1);
        }
    }
}

void SerialBitmapIterator::generateCommaPositions(long start_pos, long end_pos, int level, long* comma_positions, long& top_comma_positions) {
    unsigned long commabit;
    long st = start_pos / 64;
    long ed = ceil(double(end_pos) / 64);
    for (long i = st; i < ed; ++i) {
        commabit = mSerialBitmap->mLevCommaBitmap[level][i];
        int cnt = __builtin_popcountl(commabit);
        while (commabit) {
            long offset = i * 64 + __builtin_ctzll(commabit);
            if (start_pos <= offset && offset <= end_pos) {
                comma_positions[++top_comma_positions] = offset;
            }
            commabit = commabit & (commabit - 1);
        }
    }
}

bool SerialBitmapIterator::findFieldQuotePos(long colon_pos, long& start_pos, long& end_pos) {
    long w_id = colon_pos/64;
    long offset = colon_pos%64;
    long start_quote = 0;
    long end_quote = 0;
    start_pos = 0; end_pos = 0;
    while (w_id >= 0)
    {
        unsigned long quotebit = mSerialBitmap->mQuoteBitmap[w_id];
        unsigned long offset = w_id * 64 + __builtin_ctzll(quotebit);
        while (quotebit && offset < colon_pos)
        {
            if (end_pos != 0)
            {
                start_quote = offset;
            }
            else if(start_quote == 0)
            {
                start_quote = offset;
            }
            else if(end_quote == 0)
            {
                end_quote = offset;
            }
            else
            {
                start_quote = end_quote;
                end_quote = offset;
            }
            quotebit = quotebit & (quotebit - 1);
            offset = w_id * 64 + __builtin_ctzll(quotebit);
        }
        if(start_quote != 0 && end_quote == 0)
        {
            end_quote = start_quote;
            start_quote = 0;
            end_pos = end_quote;
        }
        else if(start_quote != 0 && end_quote != 0)
        {
            start_pos = start_quote;
            end_pos = end_quote;
            return true;
        }
        --w_id;
    }
    return false;
}

SerialBitmapIterator* SerialBitmapIterator::getCopy() {
    SerialBitmapIterator* sbi = new SerialBitmapIterator();
    sbi->mSerialBitmap = mSerialBitmap;
    sbi->mCurLevel = mCurLevel;
    sbi->mTopLevel = mCurLevel;
    if (sbi->mTopLevel >= 0) {
        sbi->mCtxInfo[mCurLevel].type = mCtxInfo[mCurLevel].type;
        sbi->mCtxInfo[mCurLevel].positions = mCtxInfo[mCurLevel].positions;
        sbi->mCtxInfo[mCurLevel].start_idx = mCtxInfo[mCurLevel].start_idx;
        sbi->mCtxInfo[mCurLevel].end_idx = mCtxInfo[mCurLevel].end_idx;
        sbi->mCtxInfo[mCurLevel].cur_idx = -1;
        sbi->mCtxInfo[mCurLevel + 1].positions = NULL;
    }
    return sbi;
}

bool SerialBitmapIterator::up() {
    if (mCurLevel == mTopLevel) return false;
    --mCurLevel;
    return true;
}

bool SerialBitmapIterator::down() {
    if (mCurLevel < mTopLevel || mCurLevel > mSerialBitmap->mDepth) return false;
    ++mCurLevel;
    //cout<<"down function cur level "<<mCurLevel<<" "<<mTopLevel<<" "<<mSerialBitmap->mDepth<<endl;
    long start_pos = -1;
    long end_pos = -1;
    if (mCurLevel == mTopLevel + 1) {
        if (mTopLevel == -1) {
            long record_length = mSerialBitmap->mRecordLength;
            start_pos = 0;
            end_pos = record_length;
            mCtxInfo[mCurLevel].positions = new long[record_length / 8 + 1];
            mPosArrAlloc[mCurLevel] = true;
        } else {
            long cur_idx = mCtxInfo[mCurLevel - 1].cur_idx;
            start_pos = mCtxInfo[mCurLevel - 1].positions[cur_idx];
            end_pos = mCtxInfo[mCurLevel - 1].positions[cur_idx + 1];
            if (mCtxInfo[mCurLevel].positions == NULL || mPosArrAlloc[mCurLevel] == false) {
                mCtxInfo[mCurLevel].positions = new long[MAX_NUM_ELE / 8 + 1];
                mPosArrAlloc[mCurLevel] = true;
            }
        }
        mCtxInfo[mCurLevel].start_idx = 0;
        mCtxInfo[mCurLevel].cur_idx = -1;
        mCtxInfo[mCurLevel].end_idx = -1;
    } else {
        long cur_idx = mCtxInfo[mCurLevel - 1].cur_idx;
        if (cur_idx > mCtxInfo[mCurLevel - 1].end_idx) {
            --mCurLevel;
  //          cout<<"exception "<<cur_idx<<" "<<mCtxInfo[mCurLevel - 1].end_idx<<endl;
            return false;
        }
        start_pos = mCtxInfo[mCurLevel - 1].positions[cur_idx];
        end_pos = mCtxInfo[mCurLevel - 1].positions[cur_idx + 1];
        mCtxInfo[mCurLevel].positions = mCtxInfo[mCurLevel - 1].positions;
        mCtxInfo[mCurLevel].start_idx = mCtxInfo[mCurLevel - 1].end_idx + 1;
        mCtxInfo[mCurLevel].cur_idx = mCtxInfo[mCurLevel - 1].end_idx;
        mCtxInfo[mCurLevel].end_idx = mCtxInfo[mCurLevel - 1].end_idx;
    }
    long i = start_pos;
    if (start_pos > 0 || mCurLevel > 0) ++i;
    char ch = mSerialBitmap->mRecord[i];
    while (i < end_pos && (ch == ' ' || ch == '\n')) {
        ch = mSerialBitmap->mRecord[++i];
    }
    if (mSerialBitmap->mRecord[i] == '{') {
        mCtxInfo[mCurLevel].type = OBJECT;
        generateColonPositions(i, end_pos, mCurLevel, mCtxInfo[mCurLevel].positions, mCtxInfo[mCurLevel].end_idx);
        return true;
    } else if (mSerialBitmap->mRecord[i] == '[') {
        mCtxInfo[mCurLevel].type = ARRAY;
        generateCommaPositions(i, end_pos, mCurLevel, mCtxInfo[mCurLevel].positions, mCtxInfo[mCurLevel].end_idx);
        return true;
    }
    --mCurLevel;
    return false;
}

bool SerialBitmapIterator::isObject() {
    if (mCurLevel >= 0 && mCurLevel <= mSerialBitmap->mDepth && mCtxInfo[mCurLevel].type == OBJECT) {
        return true;
    }
    return false;
}

bool SerialBitmapIterator::isArray() {
    if (mCurLevel >= 0 && mCurLevel <= mSerialBitmap->mDepth && mCtxInfo[mCurLevel].type == ARRAY) {
        return true;
    }
    return false;
}

bool SerialBitmapIterator::moveNext() {
    if (mCurLevel < 0 || mCurLevel > mSerialBitmap->mDepth || mCtxInfo[mCurLevel].type != ARRAY) return false;
    long next_idx = mCtxInfo[mCurLevel].cur_idx + 1;
    if (next_idx >= mCtxInfo[mCurLevel].end_idx) return false;
    mCtxInfo[mCurLevel].cur_idx = next_idx;
    return true;
}

bool SerialBitmapIterator::moveToKey(char* key) {
    if (mCurLevel < 0 || mCurLevel > mSerialBitmap->mDepth || mCtxInfo[mCurLevel].type != OBJECT) return false;
    long cur_idx = mCtxInfo[mCurLevel].cur_idx + 1;
    long end_idx = mCtxInfo[mCurLevel].end_idx;
    while (cur_idx < end_idx) {
        long colon_pos = mCtxInfo[mCurLevel].positions[cur_idx];
        long start_pos = 0, end_pos = 0;
        if (!findFieldQuotePos(colon_pos, start_pos, end_pos)) {
            return false;
        }
        ++mVisitedFields;
        int key_size = end_pos - start_pos - 1;
        if (key_size == strlen(key)) {
            memcpy(mKey, mSerialBitmap->mRecord + start_pos + 1, key_size);
            mKey[end_pos - start_pos - 1] = '\0';
            if (memcmp(mKey, key, key_size) == 0) {
                mCtxInfo[mCurLevel].cur_idx = cur_idx;
                return true;
            }
        }
        ++cur_idx;
    }
    mCtxInfo[mCurLevel].cur_idx = cur_idx;
    return false;
}

char* SerialBitmapIterator::moveToKey(std::unordered_set<char*>& key_set) {
    if (key_set.empty() == true|| mCurLevel < 0 || mCurLevel > mSerialBitmap->mDepth || mCtxInfo[mCurLevel].type != OBJECT) return NULL;
    long cur_idx = mCtxInfo[mCurLevel].cur_idx + 1;
    long end_idx = mCtxInfo[mCurLevel].end_idx;
    while (cur_idx < end_idx) {
        long colon_pos = mCtxInfo[mCurLevel].positions[cur_idx];
        long start_pos = 0, end_pos = 0;
        if (!findFieldQuotePos(colon_pos, start_pos, end_pos)) {
            return NULL;
        }
        ++mVisitedFields;
        bool has_m_key = false;
        std::unordered_set<char*>::iterator iter;
        for (iter = key_set.begin(); iter != key_set.end(); ++iter) {
            char* key = (*iter);
            int key_size = end_pos - start_pos - 1;
            if (key_size == strlen(key)) {
                if (has_m_key == false) {
                    memcpy(mKey, mSerialBitmap->mRecord + start_pos + 1, key_size);
                    mKey[end_pos - start_pos - 1] = '\0';
                    has_m_key = true;
                }
                if (memcmp(mKey, key, key_size) == 0) {
                    mCtxInfo[mCurLevel].cur_idx = cur_idx;
                    key_set.erase(iter);
                    return key;
                }
            }
        }
        ++cur_idx;
    }
    mCtxInfo[mCurLevel].cur_idx = cur_idx;
    return NULL;
}

int SerialBitmapIterator::numArrayElements() {
    if (mCurLevel >= 0 && mCurLevel <= mSerialBitmap->mDepth && mCtxInfo[mCurLevel].type == ARRAY) {
        return mCtxInfo[mCurLevel].end_idx - mCtxInfo[mCurLevel].start_idx;
    }
    return 0;
}

bool SerialBitmapIterator::moveToIndex(int index) {
    if (mCurLevel < 0 || mCurLevel > mSerialBitmap->mDepth || mCtxInfo[mCurLevel].type != ARRAY) return false;
    long next_idx = mCtxInfo[mCurLevel].start_idx + index;
    if (next_idx > mCtxInfo[mCurLevel].end_idx) return false;
    mCtxInfo[mCurLevel].cur_idx = next_idx;
    return true;
}

char* SerialBitmapIterator::getValue() {
    if (mCurLevel < 0 || mCurLevel > mSerialBitmap->mDepth) return NULL;
    long cur_idx = mCtxInfo[mCurLevel].cur_idx;
    long next_idx = cur_idx + 1;
    if (next_idx > mCtxInfo[mCurLevel].end_idx) return NULL;
    // current ':' or ','
    long cur_pos = mCtxInfo[mCurLevel].positions[cur_idx];
    // next ':' or ','
    long next_pos = mCtxInfo[mCurLevel].positions[next_idx];
    int type = mCtxInfo[mCurLevel].type;
    if (type == OBJECT && next_idx < mCtxInfo[mCurLevel].end_idx) {
        long start_pos = 0, end_pos = 0;
        if (findFieldQuotePos(next_pos, start_pos, end_pos) == false) {
            return (char*)"";
        }
        // next quote
        next_pos = start_pos;
    }
    long text_length = next_pos - cur_pos - 1;
    if (text_length <= 0) return (char*)"";
    char* ret = (char*)malloc(text_length + 1);
///    cout<<"cur pos "<<(cur_pos + 1)<<" length "<<text_length<<endl;
    memcpy(ret, mSerialBitmap->mRecord + cur_pos + 1, text_length);
    ret[text_length] = '\0';
    return ret;
}


Writing /content/serialSpec/SerialBitmapIterator.cpp


In [ ]:
%%writefile /content/serialSpec/SerialBitmapIterator.h

#ifndef SERIALBITMAPITERATOR_H
#define SERIALBITMAPITERATOR_H

#include "SerialBitmap.h"
#include "../general/BitmapIterator.h"
#include <string.h>
#include <unordered_set>

#define MAX_NUM_ELE 1000000

class SerialBitmap;

class SerialBitmapIterator : public BitmapIterator {
  private:
    SerialBitmap* mSerialBitmap;
    IterCtxInfo mCtxInfo[MAX_LEVEL];
    bool mPosArrAlloc[MAX_LEVEL];
    int mCurLevel;
    // the top level of the input record, or each sub-record (for parallel querying)
    int mTopLevel;
    // more efficient than allocating memory during runtime
    char mKey[MAX_FIELD_SIZE];

  public:
    SerialBitmapIterator() {}
    SerialBitmapIterator(SerialBitmap* sbm) {
        mSerialBitmap = sbm;
        mCurLevel = -1;
        mTopLevel = -1;
        mVisitedFields = 0;
        for (int i = 0; i < MAX_LEVEL; ++i) {
            mPosArrAlloc[i] = false;
            mCtxInfo[i].positions = NULL;
        }
        // initially, iterator points to the first level of the record
        down();
    }

    ~SerialBitmapIterator() {
        for (int i = mTopLevel + 1; i < MAX_LEVEL; ++i) {
            if (mPosArrAlloc[i] == true) {
                free(mCtxInfo[i].positions);
                mCtxInfo[i].positions = NULL;
            } else {
                break;
            }
        }
    }

    // Creates a copy of iterator. Often used for parallel querying.
    SerialBitmapIterator* getCopy();
    // Moves back to the object or array which contains the current nested record.
    // Often used when the current nested record has been processed.
    // Valid except for the first level of the record.
    bool up();
    // Moves to the start of the nested object or array.
    // Gets all colon or comma positions from leveled bitmap indexes for current nested record.
    // Valid if we are at { or [.
    bool down();
    // Whether the iterator points to an object.
    bool isObject();
    // Whether the iterator points to an array.
    bool isArray();
    // Moves iterator to the next array item.
    bool moveNext();
    // Moves to the corresponding key field inside the current object.
    bool moveToKey(char* key);
    // Moves to the corresponding key fields inside the current object, returns the current key name.
    // After this operation, the current key field will be removed from key_set.
    char* moveToKey(std::unordered_set<char*>& key_set);
    // Returns the number of elements inside current array.
    int numArrayElements();
    // If the current record is an array, moves to an item based on index.
    // Returns false if the index is out of the boundary.
    bool moveToIndex(int index);
    // Gets the content of the current value inside an object or array.
    char* getValue();

  private:
    // get positions of all colons between start_idx and end_idx from input stream
    void generateColonPositions(long start_pos, long end_pos, int level, long* colon_positions, long& top_colon_positions);
    // get positions of all commas between start_idx and end_idx from input stream
    void generateCommaPositions(long start_pos, long end_pos, int level, long* comma_positions, long& top_comma_positions);
    // get starting and ending positions for a string
    bool findFieldQuotePos(long colon_pos, long& start_pos, long& end_pos);
};
#endif


Writing /content/serialSpec/SerialBitmapIterator.h


In [ ]:
%%writefile /content/testing/parallelMain1.cpp
#include <chrono>
#include "../general/RecordLoader.h"
#include "../general/BitmapIterator.h"
#include "../general/BitmapConstructor.h"
using namespace std::chrono;

// $[*].user.id
std::string query(BitmapIterator* iter) {
    std::string output = "";
    while (iter->isArray() && iter->moveNext() == true) {
        if (iter->down() == false) continue;  /* array element on the top level */
        if (iter->isObject() && iter->moveToKey((char*)"user")) {
            if (iter->down() == false) continue; /* value of "user" */
            if (iter->isObject() && iter->moveToKey((char*)"id")) {
                // value of "id"
                char* value = iter->getValue();
                output.append(value).append(";").append("\n");
                if (value) free(value);
            }
            iter->up();
        }
        iter->up();
    }
    return output;
}

int main() {
    // PATH TO SAMPLE TEST FILE: "dataset/twitter_sample_large_record.json"
    const char* file_path = "/content/drive/MyDrive/pthreads/data/twitter_large_record.json";
    Record* rec = RecordLoader::loadSingleRecord(file_path);
    if (rec == NULL) {
        std::cout << "record loading fails." << std::endl;
        return -1;
    }

    // set the number of threads for parallel bitmap construction
    int thread_num = 12;

    /* set the number of levels of bitmaps to create, either based on the
     * query or the JSON records. E.g., query $[*].user.id needs three levels
     * (level 0, 1, 2), but the record may be of more than three levels
     */
    int level_num = 3;

    cout << "Running on " << thread_num << " threads" << endl;

    // Start clock to measure speeds
    auto start = high_resolution_clock::now();


    /* process the input record: first build bitmap, then perform
     * the query with a bitmap iterator
     */
    Bitmap* bm = BitmapConstructor::construct(rec, thread_num, level_num);
    BitmapIterator* iter = BitmapConstructor::getIterator(bm);
    std::string output = query(iter);

    // End clock
    auto stop = high_resolution_clock::now();
    auto duration = duration_cast<microseconds>(stop - start);
    cout << "Duration: " << duration.count() << endl;

    // Uncomment to see matches
    //std::cout << "matches are: " << output << std::endl;

    delete iter;
    delete bm;


    return 0;
}


Writing /content/testing/parallelMain1.cpp


In [ ]:
%%writefile /content/testing/serialMain1.cpp

#include "../general/RecordLoader.h"
#include "../general/BitmapIterator.h"
#include "../general/BitmapConstructor.h"

// {$.user.id, $.retweet_count}
std::string query(BitmapIterator* iter) {
    std::string output = "";
    if (iter->isObject()) {
        std::unordered_set<char*> set;

        set.insert((char*)"user");
        set.insert((char*)"text");
        char* key = NULL;
        while ((key = iter->moveToKey(set)) != NULL) {
            if (strcmp(key, "text") == 0) {
                // value of "retweet_count"
                char* value = iter->getValue();
                output.append(value).append(";");
                if (value) free(value);
            } else {
                if (iter->down() == false) continue;  /* value of "user" */
                if (iter->isObject() && iter->moveToKey((char*)"id")) {
                    // value of "id"
                    char* value = iter->getValue();
                    output.append(value).append(";");
                    if (value) free(value);
                }
                iter->up();
            }
        }
    }
    return output;
}

int main() {
    // PATH TO LARGE RECORD = "/content/drive/MyDrive/pthreads/data/twitter_small_records.json"
    const char* file_path = "dataset/twitter_sample_small_records.json";
    RecordSet* record_set = RecordLoader::loadRecords(file_path);
    if (record_set->size() == 0) {
        std::cout << "record loading fails." << std::endl;
        return -1;
    }
    std::string output = "";

    // set the number of threads for parallel bitmap construction
    int thread_num = 1;

    /* set the number of levels of bitmaps to create, either based on the
     * query or the JSON records. E.g., query $[*].user.id needs three levels
     * (level 0, 1, 2), but the record may be of more than three levels
     */
    int level_num = 2;

    /* process the records one by one: for each one, first build bitmap, then perform
     * the query with a bitmap iterator
     */
    int num_recs = record_set->size();
    Bitmap* bm = NULL;
    //cout << "Num_recs: " << num_recs << endl;
    //int counter = 0;
    for (int i = 0; i < num_recs; i++) {
        //cout << "Counter:" << counter << endl;
        //++counter;
        bm = BitmapConstructor::construct((*record_set)[i], thread_num, level_num);
        BitmapIterator* iter = BitmapConstructor::getIterator(bm);
        std::string out = query(iter);
        std::cout << "String output: " << out << std::endl;
        output.append(out);
        delete iter;
    }

    //cout << "matches are: " << output << endl;

    delete bm;

    return 0;
}


Writing /content/testing/serialMain1.cpp


In [ ]:
%%writefile Makefile
# Makefile

DIR = bin
EXEC1 = $(DIR)/serialMain1
EXEC2 = $(DIR)/parallelMain1
TARGET = $(EXEC1) ${EXEC2}

all: ${TARGET}

CC = g++
# Additional that could be added: -mpclmul
CC_FLAGS = -O3 -madx -msse4.1 -march=native -pthread
POST_FLAGS = -mcmodel=medium -static-libstdc++ -lpthread

SOURCE1 = general/*.cpp serialSpec/*.cpp parallelSpec/*.cpp testing/serialMain1.cpp
$(DIR)/serialMain1: $(SOURCE1)
	mkdir $(DIR)
	$(CC) $(CC_FLAGS) -o $(DIR)/serialMain1 $(SOURCE1) $(POST_FLAGS)

SOURCE2 = general/*.cpp serialSpec/*.cpp parallelSpec/*.cpp testing/parallelMain1.cpp
$(DIR)/parallelMain1: $(SOURCE2)
	mkdir $(DIR)
	$(CC) $(CC_FLAGS) -o $(DIR)/parallelMain1 $(SOURCE2) $(POST_FLAGS)


clean:
	rm -rf $(DIR)

Writing Makefile


In [ ]:
%%writefile /content/dataset/twitter_sample_large_record.json
[
    {"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985754408484865,"id_str":"787985754408484865","text":"RT @emilycrockett: After Mike Tyson was convicted of rape in 1992, Trump said Tyson was the real victim https:\/\/t.co\/TPjPp9PAqG","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":787518866763067392,"id_str":"787518866763067392","name":"SamazxdKeU3s7lly","screen_name":"u3s7lly","location":null,"url":null,"description":null,"protected":false,"verified":false,"followers_count":0,"friends_count":0,"listed_count":3,"favourites_count":0,"statuses_count":53,"created_at":"Sun Oct 16 05:01:44 +0000 2016","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"ru","contributors_enabled":false,"is_translator":false,"profile_background_color":"F5F8FA","profile_background_image_url":"","profile_background_image_url_https":"","profile_background_tile":false,"profile_link_color":"2B7BB9","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/787680463963947009\/SW_J98FZ_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/787680463963947009\/SW_J98FZ_normal.jpg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Tue Oct 11 20:40:03 +0000 2016","id":785943065047470080,"id_str":"785943065047470080","text":"After Mike Tyson was convicted of rape in 1992, Trump said Tyson was the real victim https:\/\/t.co\/TPjPp9PAqG","source":"\u003ca href=\"http:\/\/www.vox.com\" rel=\"nofollow\"\u003eVox.com\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":29372259,"id_str":"29372259","name":"Emily Crockett","screen_name":"emilycrockett","location":"Washington, DC","url":null,"description":"Covering gender issues at @voxdotcom. Word nerd. Scurrilous wench. Opera singer. The terror that flaps in the night. Views are my own.","protected":false,"verified":true,"followers_count":5116,"friends_count":1988,"listed_count":277,"favourites_count":3485,"statuses_count":27255,"created_at":"Tue Apr 07 03:41:47 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/993458716\/blueshirt_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/993458716\/blueshirt_normal.jpg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":3,"favorite_count":4,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/TPjPp9PAqG","expanded_url":"http:\/\/www.vox.com\/identities\/2016\/10\/11\/13247206\/trump-mike-tyson-howard-stern-rape-recording?utm_campaign=emilycrockett&utm_content=chorus&utm_medium=social&utm_source=twitter","display_url":"vox.com\/identities\/201\u2026","indices":[85,108]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/TPjPp9PAqG","expanded_url":"http:\/\/www.vox.com\/identities\/2016\/10\/11\/13247206\/trump-mike-tyson-howard-stern-rape-recording?utm_campaign=emilycrockett&utm_content=chorus&utm_medium=social&utm_source=twitter","display_url":"vox.com\/identities\/201\u2026","indices":[104,127]}],"user_mentions":[{"screen_name":"emilycrockett","name":"Emily Crockett","id":29372259,"id_str":"29372259","indices":[3,17]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418575"}
    ,
    {"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985754504962048,"id_str":"787985754504962048","text":"RT @youngsinick: Just to correct you, it's a she said, she said, she said, she said, she said, she said, she said, she said, she sai\u2026 ","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":389683297,"id_str":"389683297","name":"Nico Baier","screen_name":"nbbaier","location":"Berkeley, CA","url":null,"description":"linguistics grad student","protected":false,"verified":false,"followers_count":127,"friends_count":614,"listed_count":3,"favourites_count":571,"statuses_count":945,"created_at":"Wed Oct 12 20:47:31 +0000 2011","utc_offset":-25200,"time_zone":"Pacific Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/758523055660347392\/WxZ6i0bD_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/758523055660347392\/WxZ6i0bD_normal.jpg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Sun Oct 16 13:49:50 +0000 2016","id":787651771569172480,"id_str":"787651771569172480","text":"Just to correct you, it's a she said, she said, she said, she said, she said, she said, she said, she said, she sai\u2026 https:\/\/t.co\/LMBMozXzoy","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":22111533,"id_str":"22111533","name":"Michael Arceneaux","screen_name":"youngsinick","location":"H-Town vicious in NYC","url":"http:\/\/www.thecynicalones.com\/about","description":"Writer. Talker. Texas Bamma.","protected":false,"verified":true,"followers_count":17890,"friends_count":1384,"listed_count":404,"favourites_count":7423,"statuses_count":120315,"created_at":"Fri Feb 27 05:26:42 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"ABB8C2","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/114220301\/banner.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/114220301\/banner.jpg","profile_background_tile":true,"profile_link_color":"0084B4","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"DDFFCC","profile_text_color":"333333","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/781684961912979456\/YRO7D00T_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/781684961912979456\/YRO7D00T_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/22111533\/1467895165","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"quoted_status_id":787649841438593028,"quoted_status_id_str":"787649841438593028","quoted_status":{"created_at":"Sun Oct 16 13:42:10 +0000 2016","id":787649841438593028,"id_str":"787649841438593028","text":"Rep. Renee Ellmers calls Trump allegations \"a she said\/he said situation\" https:\/\/t.co\/7eshI3SCbQ #CNNSOTU https:\/\/t.co\/Jy2wjcObg6","source":"\u003ca href=\"https:\/\/about.twitter.com\/products\/tweetdeck\" rel=\"nofollow\"\u003eTweetDeck\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":759251,"id_str":"759251","name":"CNN","screen_name":"CNN","location":null,"url":"http:\/\/www.cnn.com","description":"It\u2019s our job to #GoThere and tell the most difficult stories. Come with us!","protected":false,"verified":true,"followers_count":28712321,"friends_count":1122,"listed_count":119333,"favourites_count":1433,"statuses_count":102189,"created_at":"Fri Feb 09 00:35:02 +0000 2007","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"323232","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/515228058286952449\/zVfUxqPl.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/515228058286952449\/zVfUxqPl.jpeg","profile_background_tile":false,"profile_link_color":"004287","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"EEEEEE","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/508960761826131968\/LnvhR8ED_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/508960761826131968\/LnvhR8ED_normal.png","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/759251\/1476074145","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":618,"favorite_count":1075,"entities":{"hashtags":[{"text":"CNNSOTU","indices":[98,106]}],"urls":[{"url":"https:\/\/t.co\/7eshI3SCbQ","expanded_url":"http:\/\/cnn.com\/politics","display_url":"cnn.com\/politics","indices":[74,97]},{"url":"https:\/\/t.co\/Jy2wjcObg6","expanded_url":"http:\/\/cnn.it\/2e9jVth","display_url":"cnn.it\/2e9jVth","indices":[107,130]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"extended_tweet":{"full_text":"Just to correct you, it's a she said, she said, she said, she said, she said, she said, she said, she said, she said situation. https:\/\/t.co\/6vH1ONl100","display_text_range":[0,127],"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/6vH1ONl100","expanded_url":"https:\/\/twitter.com\/cnn\/status\/787649841438593028","display_url":"twitter.com\/cnn\/status\/787\u2026","indices":[128,151]}],"user_mentions":[],"symbols":[]}},"retweet_count":240,"favorite_count":514,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/LMBMozXzoy","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787651771569172480","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[117,140]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"quoted_status_id":787649841438593028,"quoted_status_id_str":"787649841438593028","quoted_status":{"created_at":"Sun Oct 16 13:42:10 +0000 2016","id":787649841438593028,"id_str":"787649841438593028","text":"Rep. Renee Ellmers calls Trump allegations \"a she said\/he said situation\" https:\/\/t.co\/7eshI3SCbQ #CNNSOTU https:\/\/t.co\/Jy2wjcObg6","source":"\u003ca href=\"https:\/\/about.twitter.com\/products\/tweetdeck\" rel=\"nofollow\"\u003eTweetDeck\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":759251,"id_str":"759251","name":"CNN","screen_name":"CNN","location":null,"url":"http:\/\/www.cnn.com","description":"It\u2019s our job to #GoThere and tell the most difficult stories. Come with us!","protected":false,"verified":true,"followers_count":28712321,"friends_count":1122,"listed_count":119333,"favourites_count":1433,"statuses_count":102189,"created_at":"Fri Feb 09 00:35:02 +0000 2007","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"323232","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/515228058286952449\/zVfUxqPl.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/515228058286952449\/zVfUxqPl.jpeg","profile_background_tile":false,"profile_link_color":"004287","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"EEEEEE","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/508960761826131968\/LnvhR8ED_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/508960761826131968\/LnvhR8ED_normal.png","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/759251\/1476074145","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":618,"favorite_count":1075,"entities":{"hashtags":[{"text":"CNNSOTU","indices":[98,106]}],"urls":[{"url":"https:\/\/t.co\/7eshI3SCbQ","expanded_url":"http:\/\/cnn.com\/politics","display_url":"cnn.com\/politics","indices":[74,97]},{"url":"https:\/\/t.co\/Jy2wjcObg6","expanded_url":"http:\/\/cnn.it\/2e9jVth","display_url":"cnn.it\/2e9jVth","indices":[107,130]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"","expanded_url":null,"indices":[134,134]}],"user_mentions":[{"screen_name":"youngsinick","name":"Michael Arceneaux","id":22111533,"id_str":"22111533","indices":[3,15]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418598"}
    ,
    {"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985755133980672,"id_str":"787985755133980672","text":"Being Muslim and American in the year of Donald Trump: Muslim Americans describe the September 11, 2001, terr... https:\/\/t.co\/SJKsmeBCNv","source":"\u003ca href=\"http:\/\/twitterfeed.com\" rel=\"nofollow\"\u003etwitterfeed\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1702078267,"id_str":"1702078267","name":"aayushi shah","screen_name":"aayushis06","location":null,"url":null,"description":null,"protected":false,"verified":false,"followers_count":40,"friends_count":30,"listed_count":11,"favourites_count":0,"statuses_count":41992,"created_at":"Mon Aug 26 14:17:31 +0000 2013","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_1_normal.png","profile_image_url_https":"https:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_1_normal.png","default_profile":true,"default_profile_image":true,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/SJKsmeBCNv","expanded_url":"http:\/\/cnn.it\/2eleo1w","display_url":"cnn.it\/2eleo1w","indices":[113,136]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418748"}
    ,
    {"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985754379128832,"id_str":"787985754379128832","text":"RT @thinkprogress: Clinton proposes plan that would lift millions out of poverty\nhttps:\/\/t.co\/B6T1jj2hST https:\/\/t.co\/blQy8JxViF","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":630450707,"id_str":"630450707","name":"Bruce M. Foster","screen_name":"roadshowrigolet","location":"NYC, NY United States","url":"http:\/\/www.roadshowrigoletto.com\/","description":"Born. Yet to complete the ride. \u201cDo not fear death so much\u00a0but rather the inadequate life.\u201d\u00a0- Bertolt Brecht\u00a0","protected":false,"verified":false,"followers_count":634,"friends_count":2252,"listed_count":49,"favourites_count":8769,"statuses_count":15057,"created_at":"Sun Jul 08 19:14:53 +0000 2012","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"F02005","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/445991262089248768\/OlwWkYG4.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/445991262089248768\/OlwWkYG4.jpeg","profile_background_tile":true,"profile_link_color":"0E19F0","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"C0DFEC","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/417860381470908416\/iTtCQOwN_normal.jpeg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/417860381470908416\/iTtCQOwN_normal.jpeg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/630450707\/1391378643","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:40:03 +0000 2016","id":787981495390576640,"id_str":"787981495390576640","text":"Clinton proposes plan that would lift millions out of poverty\nhttps:\/\/t.co\/B6T1jj2hST https:\/\/t.co\/blQy8JxViF","display_text_range":[0,85],"source":"\u003ca href=\"http:\/\/meetedgar.com\" rel=\"nofollow\"\u003eMeet Edgar\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":55355654,"id_str":"55355654","name":"ThinkProgress","screen_name":"thinkprogress","location":"Washington, D.C.","url":"http:\/\/www.thinkprogress.org","description":"Moving news forward.","protected":false,"verified":true,"followers_count":610741,"friends_count":976,"listed_count":13459,"favourites_count":211,"statuses_count":84028,"created_at":"Thu Jul 09 20:42:08 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"E6EDED","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/365080126\/tp-twitter.gif","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/365080126\/tp-twitter.gif","profile_background_tile":false,"profile_link_color":"531AAA","profile_sidebar_border_color":"EDEDED","profile_sidebar_fill_color":"D4D4D4","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/762609394265632768\/H_UR_W7r_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/762609394265632768\/H_UR_W7r_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/55355654\/1470649637","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":30,"favorite_count":34,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/B6T1jj2hST","expanded_url":"http:\/\/thkpr.gs\/998b84c3392c","display_url":"thkpr.gs\/998b84c3392c","indices":[62,85]}],"user_mentions":[],"symbols":[],"media":[{"id":787981492932796416,"id_str":"787981492932796416","indices":[86,109],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu94_i_XYAAJ_7Z.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu94_i_XYAAJ_7Z.jpg","url":"https:\/\/t.co\/blQy8JxViF","display_url":"pic.twitter.com\/blQy8JxViF","expanded_url":"https:\/\/twitter.com\/thinkprogress\/status\/787981495390576640\/photo\/1","type":"photo","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":1200,"h":800,"resize":"fit"},"small":{"w":680,"h":453,"resize":"fit"},"large":{"w":2000,"h":1333,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787981492932796416,"id_str":"787981492932796416","indices":[86,109],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu94_i_XYAAJ_7Z.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu94_i_XYAAJ_7Z.jpg","url":"https:\/\/t.co\/blQy8JxViF","display_url":"pic.twitter.com\/blQy8JxViF","expanded_url":"https:\/\/twitter.com\/thinkprogress\/status\/787981495390576640\/photo\/1","type":"photo","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":1200,"h":800,"resize":"fit"},"small":{"w":680,"h":453,"resize":"fit"},"large":{"w":2000,"h":1333,"resize":"fit"}}}]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/B6T1jj2hST","expanded_url":"http:\/\/thkpr.gs\/998b84c3392c","display_url":"thkpr.gs\/998b84c3392c","indices":[81,104]}],"user_mentions":[{"screen_name":"thinkprogress","name":"ThinkProgress","id":55355654,"id_str":"55355654","indices":[3,17]}],"symbols":[],"media":[{"id":787981492932796416,"id_str":"787981492932796416","indices":[105,128],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu94_i_XYAAJ_7Z.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu94_i_XYAAJ_7Z.jpg","url":"https:\/\/t.co\/blQy8JxViF","display_url":"pic.twitter.com\/blQy8JxViF","expanded_url":"https:\/\/twitter.com\/thinkprogress\/status\/787981495390576640\/photo\/1","type":"photo","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":1200,"h":800,"resize":"fit"},"small":{"w":680,"h":453,"resize":"fit"},"large":{"w":2000,"h":1333,"resize":"fit"}},"source_status_id":787981495390576640,"source_status_id_str":"787981495390576640","source_user_id":55355654,"source_user_id_str":"55355654"}]},"extended_entities":{"media":[{"id":787981492932796416,"id_str":"787981492932796416","indices":[105,128],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu94_i_XYAAJ_7Z.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu94_i_XYAAJ_7Z.jpg","url":"https:\/\/t.co\/blQy8JxViF","display_url":"pic.twitter.com\/blQy8JxViF","expanded_url":"https:\/\/twitter.com\/thinkprogress\/status\/787981495390576640\/photo\/1","type":"photo","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":1200,"h":800,"resize":"fit"},"small":{"w":680,"h":453,"resize":"fit"},"large":{"w":2000,"h":1333,"resize":"fit"}},"source_status_id":787981495390576640,"source_status_id_str":"787981495390576640","source_user_id":55355654,"source_user_id_str":"55355654"}]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418568"}
    ,
    {"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985755192889344,"id_str":"787985755192889344","text":"Trump and the Terrible, Horrible, No Good, Very Bad Day #MovieTitleToDescribeElection","source":"\u003ca href=\"http:\/\/twitter.com\/#!\/download\/ipad\" rel=\"nofollow\"\u003eTwitter for iPad\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":715057889,"id_str":"715057889","name":"Barbie","screen_name":"Barbandoll","location":null,"url":null,"description":"Picasso's Muse, Joined Twitter one tweet at a time. I need humor to get by.","protected":false,"verified":false,"followers_count":177,"friends_count":338,"listed_count":3,"favourites_count":1284,"statuses_count":884,"created_at":"Mon Oct 14 20:24:30 +0000 2013","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/752988156346175489\/nFCSUtOV_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/752988156346175489\/nFCSUtOV_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/715057889\/1473374264","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[{"text":"MovieTitleToDescribeElection","indices":[56,85]}],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418762"}
    ,
    {"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985755461214208,"id_str":"787985755461214208","text":"RT @dudleydew: @cher THE \ud83c\udf4e DOESN'T FALL FAR FROM THE \ud83d\udca9 TREE! https:\/\/t.co\/4sfkW7msCa","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":739239346348433408,"id_str":"739239346348433408","name":"Divia Va Darri","screen_name":"VaaVadarri","location":null,"url":null,"description":"I don't believe in fairytales","protected":false,"verified":false,"followers_count":23,"friends_count":100,"listed_count":1,"favourites_count":1328,"statuses_count":1061,"created_at":"Sat Jun 04 23:36:09 +0000 2016","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"F5F8FA","profile_background_image_url":"","profile_background_image_url_https":"","profile_background_tile":false,"profile_link_color":"2B7BB9","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/787830567454797824\/JxRxNnpr_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/787830567454797824\/JxRxNnpr_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/739239346348433408\/1465084368","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 08:01:53 +0000 2016","id":787926593490460672,"id_str":"787926593490460672","text":"@cher THE \ud83c\udf4e DOESN'T FALL FAR FROM THE \ud83d\udca9 TREE! https:\/\/t.co\/4sfkW7msCa","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":124003770,"in_reply_to_user_id_str":"124003770","in_reply_to_screen_name":"cher","user":{"id":284191160,"id_str":"284191160","name":"\u2728DOUGIE\u2728","screen_name":"dudleydew","location":null,"url":null,"description":"They call me Dougie! I'm dancing as fast as I can and it seems I just keep going backwards. And let me tell you, I'm no Ginger Rogers.","protected":false,"verified":false,"followers_count":637,"friends_count":204,"listed_count":24,"favourites_count":10727,"statuses_count":43990,"created_at":"Mon Apr 18 20:20:35 +0000 2011","utc_offset":null,"time_zone":null,"geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"642D8B","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme10\/bg.gif","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme10\/bg.gif","profile_background_tile":true,"profile_link_color":"FF0000","profile_sidebar_border_color":"65B0DA","profile_sidebar_fill_color":"7AC3EE","profile_text_color":"3D1957","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/773637700352040960\/PY_VyGfz_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/773637700352040960\/PY_VyGfz_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/284191160\/1472540244","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":118,"favorite_count":242,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/4sfkW7msCa","expanded_url":"http:\/\/www.huffingtonpost.com\/entry\/donald-trump-jr-women-harrassment-workforce_us_5800b9bbe4b0162c043b532e?ncid=engmodushpmg00000003","display_url":"huffingtonpost.com\/entry\/donald-t\u2026","indices":[46,69]}],"user_mentions":[{"screen_name":"cher","name":"Cher","id":124003770,"id_str":"124003770","indices":[0,5]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/4sfkW7msCa","expanded_url":"http:\/\/www.huffingtonpost.com\/entry\/donald-trump-jr-women-harrassment-workforce_us_5800b9bbe4b0162c043b532e?ncid=engmodushpmg00000003","display_url":"huffingtonpost.com\/entry\/donald-t\u2026","indices":[61,84]}],"user_mentions":[{"screen_name":"dudleydew","name":"\u2728DOUGIE\u2728","id":284191160,"id_str":"284191160","indices":[3,13]},{"screen_name":"cher","name":"Cher","id":124003770,"id_str":"124003770","indices":[15,20]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418826"}
    ,
    {"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985755599544321,"id_str":"787985755599544321","text":"Email hacks reveal Clinton campaign's transparency problems: Clinton campaign chairman John Podesta's hacked ... https:\/\/t.co\/1vz9ph3p4Y","source":"\u003ca href=\"http:\/\/twitterfeed.com\" rel=\"nofollow\"\u003etwitterfeed\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":187595854,"id_str":"187595854","name":"2354","screen_name":"kindelov","location":null,"url":null,"description":null,"protected":false,"verified":false,"followers_count":4488,"friends_count":93,"listed_count":55,"favourites_count":0,"statuses_count":759769,"created_at":"Mon Sep 06 17:21:53 +0000 2010","utc_offset":10800,"time_zone":"Volgograd","geo_enabled":true,"lang":"ru","contributors_enabled":false,"is_translator":false,"profile_background_color":"FFFFFF","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/324255683\/morphia-dl.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/324255683\/morphia-dl.jpg","profile_background_tile":false,"profile_link_color":"2C7BCF","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"FFFFFF","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/1664860392\/ArmadaUnicron02_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/1664860392\/ArmadaUnicron02_normal.jpg","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/1vz9ph3p4Y","expanded_url":"http:\/\/cbsn.ws\/2enbA4W","display_url":"cbsn.ws\/2enbA4W","indices":[113,136]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418859"}
    ,
    {"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985755570331649,"id_str":"787985755570331649","text":"RT @PrisonPlanet: Today it's Assange, next year who else's Internet will be cut for criticizing Hillary Clinton? #FreeJulian","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":291986334,"id_str":"291986334","name":"Adam LeFevre","screen_name":"AdamLeFevre","location":"Fort Recovery, Ohio","url":null,"description":"FRHS alum. State Champ. RHIT Class of 2020. Impossible is only the starting point. life is too short to last long","protected":false,"verified":false,"followers_count":691,"friends_count":2095,"listed_count":5,"favourites_count":7203,"statuses_count":9699,"created_at":"Mon May 02 22:56:03 +0000 2011","utc_offset":null,"time_zone":null,"geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"131516","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/456161319314395137\/vwLwaoMG.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/456161319314395137\/vwLwaoMG.jpeg","profile_background_tile":true,"profile_link_color":"009999","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"EFEFEF","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/679899607632113665\/2Zk3ZLdb_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/679899607632113665\/2Zk3ZLdb_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/291986334\/1467253888","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 09:30:53 +0000 2016","id":787948990411210752,"id_str":"787948990411210752","text":"Today it's Assange, next year who else's Internet will be cut for criticizing Hillary Clinton? #FreeJulian","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":18643437,"id_str":"18643437","name":"Paul Joseph Watson","screen_name":"PrisonPlanet","location":"London","url":"https:\/\/www.youtube.com\/user\/PrisonPlanetLive","description":"Contrarian polemicist, Infowars Editor-at-Large, 'pasty geek' (Louise Mensch)","protected":false,"verified":true,"followers_count":288036,"friends_count":278,"listed_count":3517,"favourites_count":4724,"statuses_count":26826,"created_at":"Mon Jan 05 20:04:23 +0000 2009","utc_offset":-18000,"time_zone":"Central Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"1A1B1F","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/378800000049129232\/f1cb5b3448d8a6d24e287249b765ae2a.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/378800000049129232\/f1cb5b3448d8a6d24e287249b765ae2a.jpeg","profile_background_tile":false,"profile_link_color":"2FC2EF","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"252429","profile_text_color":"666666","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/756825397388533760\/yXYx6CNC_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/756825397388533760\/yXYx6CNC_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/18643437\/1476394531","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":2516,"favorite_count":3029,"entities":{"hashtags":[{"text":"FreeJulian","indices":[95,106]}],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[{"text":"FreeJulian","indices":[113,124]}],"urls":[],"user_mentions":[{"screen_name":"PrisonPlanet","name":"Paul Joseph Watson","id":18643437,"id_str":"18643437","indices":[3,16]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418852"}
    ,
    {"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985755775897600,"id_str":"787985755775897600","text":"RT @tonyschwartz: Trump actually believes he is a victim? Make no mistake: he is a victimizer, exploiting supporters who, like him, want so\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":744646569710989312,"id_str":"744646569710989312","name":"Sam","screen_name":"SamSam48412631","location":"Brighton, NY","url":null,"description":"#Election2016 Government & Politics","protected":false,"verified":false,"followers_count":15,"friends_count":19,"listed_count":0,"favourites_count":1273,"statuses_count":422,"created_at":"Sun Jun 19 21:42:31 +0000 2016","utc_offset":null,"time_zone":null,"geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"F5F8FA","profile_background_image_url":"","profile_background_image_url_https":"","profile_background_tile":false,"profile_link_color":"2B7BB9","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/745226443576180736\/9OuxzC2S_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/745226443576180736\/9OuxzC2S_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/744646569710989312\/1471568591","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Sun Oct 16 16:38:23 +0000 2016","id":787694185423962112,"id_str":"787694185423962112","text":"Trump actually believes he is a victim? Make no mistake: he is a victimizer, exploiting supporters who, like him, want someone to blame.","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":17579312,"id_str":"17579312","name":"Tony Schwartz","screen_name":"tonyschwartz","location":"Riverdale NY (i.e. the Bronx)","url":"http:\/\/TheEnergyProject.com","description":"CEO, The Energy Project. Author. The Art of the Deal ghostwriter now speaking out. Passionate about transforming the way the world works. Craves depth","protected":false,"verified":true,"followers_count":87684,"friends_count":60,"listed_count":2084,"favourites_count":19,"statuses_count":1743,"created_at":"Sun Nov 23 22:08:04 +0000 2008","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/3292198225\/7e454d576a5206f4a9475af269a2485e_normal.jpeg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/3292198225\/7e454d576a5206f4a9475af269a2485e_normal.jpeg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/17579312\/1469723369","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":1133,"favorite_count":2300,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"tonyschwartz","name":"Tony Schwartz","id":17579312,"id_str":"17579312","indices":[3,16]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418901"}
    ,
    {"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985755918307328,"id_str":"787985755918307328","text":"RT @StefanMolyneux: If the state takes down Assange now, they send a clear message that Hillary Clinton must be kept very far away from any\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":312439360,"id_str":"312439360","name":"MURI","screen_name":"govmuri","location":"the greatest country on earth","url":null,"description":"you follow me I follow you.","protected":false,"verified":false,"followers_count":49,"friends_count":142,"listed_count":1,"favourites_count":1493,"statuses_count":1897,"created_at":"Tue Jun 07 03:52:35 +0000 2011","utc_offset":-25200,"time_zone":"Pacific Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"3B94D9","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/787825565118058497\/wn3y5D9I_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/787825565118058497\/wn3y5D9I_normal.jpg","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 06:20:23 +0000 2016","id":787901050925842432,"id_str":"787901050925842432","text":"If the state takes down Assange now, they send a clear message that Hillary Clinton must be kept very far away from any power. #FreeJulian","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":313038011,"id_str":"313038011","name":"Stefan Molyneux","screen_name":"StefanMolyneux","location":null,"url":"http:\/\/freedomainradio.com","description":"Peaceful Parent, Philosopher and Host of Freedomain Radio.\n\nhttp:\/\/youtube.com\/freedomainradio\nhttp:\/\/fdrpodcasts.com\nhttp:\/\/soundcloud.com\/stefan-molyneux","protected":false,"verified":true,"followers_count":77296,"friends_count":6,"listed_count":975,"favourites_count":2,"statuses_count":5822,"created_at":"Wed Jun 08 02:04:43 +0000 2011","utc_offset":-10800,"time_zone":"Atlantic Time (Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"022330","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme15\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme15\/bg.png","profile_background_tile":false,"profile_link_color":"0890C2","profile_sidebar_border_color":"A8C7F7","profile_sidebar_fill_color":"C0DFEC","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/744340099522039809\/7hWYBbgl_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/744340099522039809\/7hWYBbgl_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/313038011\/1398278552","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":2639,"favorite_count":3111,"entities":{"hashtags":[{"text":"FreeJulian","indices":[127,138]}],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"StefanMolyneux","name":"Stefan Molyneux","id":313038011,"id_str":"313038011","indices":[3,18]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418935"}
    ,
    {"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985755192909825,"id_str":"787985755192909825","text":"RT @pink_lady56: Corrupt Clinton Times\n\n#FreeJulian https:\/\/t.co\/hSxk0D8bhr","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":596615669,"id_str":"596615669","name":"DeplorableChesam57\ud83c\udf1e","screen_name":"Chesam57","location":null,"url":null,"description":"#2ndamendment supporter, #patriot      #defundPP #Trump2016 #WethePeople  #Americafirst #TrumpTrain #Nevercruz\nDO NOT PUT ME ON ANY LISTS! You will be blocked.","protected":false,"verified":false,"followers_count":1386,"friends_count":847,"listed_count":82,"favourites_count":37969,"statuses_count":32505,"created_at":"Fri Jun 01 16:49:02 +0000 2012","utc_offset":-25200,"time_zone":"Pacific Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme16\/bg.gif","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme16\/bg.gif","profile_background_tile":false,"profile_link_color":"19CF86","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/718423188728729600\/3nvxNqA4_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/718423188728729600\/3nvxNqA4_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/596615669\/1469812973","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:28:10 +0000 2016","id":787978506940641285,"id_str":"787978506940641285","text":"Corrupt Clinton Times\n\n#FreeJulian https:\/\/t.co\/hSxk0D8bhr","display_text_range":[0,34],"source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":291874773,"id_str":"291874773","name":"\u2665Deplorable in Pink\u2665","screen_name":"pink_lady56","location":"United States","url":null,"description":"I VOTED for #DonaldTrump #MikePence\nI continue to support them. #MAGA\n #BlockTrolls","protected":false,"verified":false,"followers_count":13358,"friends_count":9884,"listed_count":169,"favourites_count":115442,"statuses_count":58788,"created_at":"Mon May 02 19:10:57 +0000 2011","utc_offset":-21600,"time_zone":"America\/Denver","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"022330","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme15\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme15\/bg.png","profile_background_tile":false,"profile_link_color":"DB3474","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/778595608219127812\/q8y5bCJX_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/778595608219127812\/q8y5bCJX_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/291874773\/1474460850","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"quoted_status_id":787949276089516032,"quoted_status_id_str":"787949276089516032","quoted_status":{"created_at":"Mon Oct 17 09:32:01 +0000 2016","id":787949276089516032,"id_str":"787949276089516032","text":"\ud83d\ude82\ud83d\udca8\ud83c\uddfa\ud83c\uddf8\ud83c\uddfa\ud83c\uddf8\ud83c\uddfa\ud83c\uddf8\ud83c\uddfa\ud83c\uddf8\n@realDonaldTrump\n#TrumpPence16\n#MAGA\n#TrumpTrain\n#TrumpArmy \n@EricTrump @IvankaTrump \n@EricTrumpFdn\u2026 https:\/\/t.co\/fqH71vSNXS","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/dashboard.twitter.com\" rel=\"nofollow\"\u003eTwitter Dashboard for iPhone\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":25867588,"id_str":"25867588","name":"Darryl P. Jordan","screen_name":"darryljordan","location":"BUNKER n Athens, Nashville, NC","url":"http:\/\/rnc.org","description":"NATIVE U.S.A. #UGA (9th GERERATON) CEO - JBA, INC. VENT & GET IT OUT! MANAGMENT INFO SYSTEMS, MBA, Finance, Real-Estate. #MAGA #NRA","protected":false,"verified":false,"followers_count":5416,"friends_count":5920,"listed_count":111,"favourites_count":3831,"statuses_count":19551,"created_at":"Sun Mar 22 19:47:35 +0000 2009","utc_offset":-14400,"time_zone":"Indiana (East)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"1A1B1F","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme9\/bg.gif","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme9\/bg.gif","profile_background_tile":false,"profile_link_color":"2FC2EF","profile_sidebar_border_color":"181A1E","profile_sidebar_fill_color":"252429","profile_text_color":"666666","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/756930286227783680\/d4mghwMQ_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/756930286227783680\/d4mghwMQ_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/25867588\/1468080148","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"extended_tweet":{"full_text":"\ud83d\ude82\ud83d\udca8\ud83c\uddfa\ud83c\uddf8\ud83c\uddfa\ud83c\uddf8\ud83c\uddfa\ud83c\uddf8\ud83c\uddfa\ud83c\uddf8\n@realDonaldTrump\n#TrumpPence16\n#MAGA\n#TrumpTrain\n#TrumpArmy \n@EricTrump @IvankaTrump \n@EricTrumpFdn \n@StJudeResearch \n#CROOKED https:\/\/t.co\/vIaFXilOZK","display_text_range":[0,137],"entities":{"hashtags":[{"text":"TrumpPence16","indices":[28,41]},{"text":"MAGA","indices":[42,47]},{"text":"TrumpTrain","indices":[48,59]},{"text":"TrumpArmy","indices":[60,70]},{"text":"CROOKED","indices":[129,137]}],"urls":[],"user_mentions":[{"screen_name":"realDonaldTrump","name":"Donald J. Trump","id":25073877,"id_str":"25073877","indices":[11,27]},{"screen_name":"EricTrump","name":"Eric Trump","id":39349894,"id_str":"39349894","indices":[72,82]},{"screen_name":"IvankaTrump","name":"Ivanka Trump","id":52544275,"id_str":"52544275","indices":[83,95]},{"screen_name":"EricTrumpFdn","name":"The Eric Trump Fdn","id":154567275,"id_str":"154567275","indices":[97,110]},{"screen_name":"StJudeResearch","name":"St. Jude Research","id":413308631,"id_str":"413308631","indices":[112,127]}],"symbols":[],"media":[{"id":787949267684057088,"id_str":"787949267684057088","indices":[138,161],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu9brylWYAAXoZQ.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu9brylWYAAXoZQ.jpg","url":"https:\/\/t.co\/vIaFXilOZK","display_url":"pic.twitter.com\/vIaFXilOZK","expanded_url":"https:\/\/twitter.com\/darryljordan\/status\/787949276089516032\/photo\/1","type":"photo","sizes":{"small":{"w":680,"h":509,"resize":"fit"},"medium":{"w":1200,"h":899,"resize":"fit"},"large":{"w":2048,"h":1534,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"}}}]},"extended_entities":{"media":[{"id":787949267684057088,"id_str":"787949267684057088","indices":[138,161],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu9brylWYAAXoZQ.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu9brylWYAAXoZQ.jpg","url":"https:\/\/t.co\/vIaFXilOZK","display_url":"pic.twitter.com\/vIaFXilOZK","expanded_url":"https:\/\/twitter.com\/darryljordan\/status\/787949276089516032\/photo\/1","type":"photo","sizes":{"small":{"w":680,"h":509,"resize":"fit"},"medium":{"w":1200,"h":899,"resize":"fit"},"large":{"w":2048,"h":1534,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"}}}]}},"retweet_count":40,"favorite_count":32,"entities":{"hashtags":[{"text":"TrumpPence16","indices":[28,41]},{"text":"MAGA","indices":[42,47]},{"text":"TrumpTrain","indices":[48,59]},{"text":"TrumpArmy","indices":[60,70]}],"urls":[{"url":"https:\/\/t.co\/fqH71vSNXS","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787949276089516032","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[112,135]}],"user_mentions":[{"screen_name":"realDonaldTrump","name":"Donald J. Trump","id":25073877,"id_str":"25073877","indices":[11,27]},{"screen_name":"EricTrump","name":"Eric Trump","id":39349894,"id_str":"39349894","indices":[72,82]},{"screen_name":"IvankaTrump","name":"Ivanka Trump","id":52544275,"id_str":"52544275","indices":[83,95]},{"screen_name":"EricTrumpFdn","name":"The Eric Trump Fdn","id":154567275,"id_str":"154567275","indices":[97,110]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"und"},"is_quote_status":true,"retweet_count":14,"favorite_count":12,"entities":{"hashtags":[{"text":"FreeJulian","indices":[23,34]}],"urls":[{"url":"https:\/\/t.co\/hSxk0D8bhr","expanded_url":"https:\/\/twitter.com\/darryljordan\/status\/787949276089516032","display_url":"twitter.com\/darryljordan\/s\u2026","indices":[35,58]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"quoted_status_id":787949276089516032,"quoted_status_id_str":"787949276089516032","quoted_status":{"created_at":"Mon Oct 17 09:32:01 +0000 2016","id":787949276089516032,"id_str":"787949276089516032","text":"\ud83d\ude82\ud83d\udca8\ud83c\uddfa\ud83c\uddf8\ud83c\uddfa\ud83c\uddf8\ud83c\uddfa\ud83c\uddf8\ud83c\uddfa\ud83c\uddf8\n@realDonaldTrump\n#TrumpPence16\n#MAGA\n#TrumpTrain\n#TrumpArmy \n@EricTrump @IvankaTrump \n@EricTrumpFdn\u2026 https:\/\/t.co\/fqH71vSNXS","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/dashboard.twitter.com\" rel=\"nofollow\"\u003eTwitter Dashboard for iPhone\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":25867588,"id_str":"25867588","name":"Darryl P. Jordan","screen_name":"darryljordan","location":"BUNKER n Athens, Nashville, NC","url":"http:\/\/rnc.org","description":"NATIVE U.S.A. #UGA (9th GERERATON) CEO - JBA, INC. VENT & GET IT OUT! MANAGMENT INFO SYSTEMS, MBA, Finance, Real-Estate. #MAGA #NRA","protected":false,"verified":false,"followers_count":5416,"friends_count":5920,"listed_count":111,"favourites_count":3831,"statuses_count":19551,"created_at":"Sun Mar 22 19:47:35 +0000 2009","utc_offset":-14400,"time_zone":"Indiana (East)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"1A1B1F","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme9\/bg.gif","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme9\/bg.gif","profile_background_tile":false,"profile_link_color":"2FC2EF","profile_sidebar_border_color":"181A1E","profile_sidebar_fill_color":"252429","profile_text_color":"666666","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/756930286227783680\/d4mghwMQ_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/756930286227783680\/d4mghwMQ_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/25867588\/1468080148","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"extended_tweet":{"full_text":"\ud83d\ude82\ud83d\udca8\ud83c\uddfa\ud83c\uddf8\ud83c\uddfa\ud83c\uddf8\ud83c\uddfa\ud83c\uddf8\ud83c\uddfa\ud83c\uddf8\n@realDonaldTrump\n#TrumpPence16\n#MAGA\n#TrumpTrain\n#TrumpArmy \n@EricTrump @IvankaTrump \n@EricTrumpFdn \n@StJudeResearch \n#CROOKED https:\/\/t.co\/vIaFXilOZK","display_text_range":[0,137],"entities":{"hashtags":[{"text":"TrumpPence16","indices":[28,41]},{"text":"MAGA","indices":[42,47]},{"text":"TrumpTrain","indices":[48,59]},{"text":"TrumpArmy","indices":[60,70]},{"text":"CROOKED","indices":[129,137]}],"urls":[],"user_mentions":[{"screen_name":"realDonaldTrump","name":"Donald J. Trump","id":25073877,"id_str":"25073877","indices":[11,27]},{"screen_name":"EricTrump","name":"Eric Trump","id":39349894,"id_str":"39349894","indices":[72,82]},{"screen_name":"IvankaTrump","name":"Ivanka Trump","id":52544275,"id_str":"52544275","indices":[83,95]},{"screen_name":"EricTrumpFdn","name":"The Eric Trump Fdn","id":154567275,"id_str":"154567275","indices":[97,110]},{"screen_name":"StJudeResearch","name":"St. Jude Research","id":413308631,"id_str":"413308631","indices":[112,127]}],"symbols":[],"media":[{"id":787949267684057088,"id_str":"787949267684057088","indices":[138,161],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu9brylWYAAXoZQ.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu9brylWYAAXoZQ.jpg","url":"https:\/\/t.co\/vIaFXilOZK","display_url":"pic.twitter.com\/vIaFXilOZK","expanded_url":"https:\/\/twitter.com\/darryljordan\/status\/787949276089516032\/photo\/1","type":"photo","sizes":{"small":{"w":680,"h":509,"resize":"fit"},"medium":{"w":1200,"h":899,"resize":"fit"},"large":{"w":2048,"h":1534,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"}}}]},"extended_entities":{"media":[{"id":787949267684057088,"id_str":"787949267684057088","indices":[138,161],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu9brylWYAAXoZQ.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu9brylWYAAXoZQ.jpg","url":"https:\/\/t.co\/vIaFXilOZK","display_url":"pic.twitter.com\/vIaFXilOZK","expanded_url":"https:\/\/twitter.com\/darryljordan\/status\/787949276089516032\/photo\/1","type":"photo","sizes":{"small":{"w":680,"h":509,"resize":"fit"},"medium":{"w":1200,"h":899,"resize":"fit"},"large":{"w":2048,"h":1534,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"}}}]}},"retweet_count":40,"favorite_count":32,"entities":{"hashtags":[{"text":"TrumpPence16","indices":[28,41]},{"text":"MAGA","indices":[42,47]},{"text":"TrumpTrain","indices":[48,59]},{"text":"TrumpArmy","indices":[60,70]}],"urls":[{"url":"https:\/\/t.co\/fqH71vSNXS","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787949276089516032","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[112,135]}],"user_mentions":[{"screen_name":"realDonaldTrump","name":"Donald J. Trump","id":25073877,"id_str":"25073877","indices":[11,27]},{"screen_name":"EricTrump","name":"Eric Trump","id":39349894,"id_str":"39349894","indices":[72,82]},{"screen_name":"IvankaTrump","name":"Ivanka Trump","id":52544275,"id_str":"52544275","indices":[83,95]},{"screen_name":"EricTrumpFdn","name":"The Eric Trump Fdn","id":154567275,"id_str":"154567275","indices":[97,110]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"und"},"is_quote_status":true,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[{"text":"FreeJulian","indices":[40,51]}],"urls":[{"url":"https:\/\/t.co\/hSxk0D8bhr","expanded_url":"https:\/\/twitter.com\/darryljordan\/status\/787949276089516032","display_url":"twitter.com\/darryljordan\/s\u2026","indices":[52,75]}],"user_mentions":[{"screen_name":"pink_lady56","name":"\u2665Deplorable in Pink\u2665","id":291874773,"id_str":"291874773","indices":[3,15]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418762"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985756228820992,"id_str":"787985756228820992","text":"Yo, @SpeakerRyan, you gonna put up with this shit? That bitch Trump doesn't even have the balls to tag you! https:\/\/t.co\/SvW1JsxJft","display_text_range":[0,107],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":18104752,"id_str":"18104752","name":"Dave Hill","screen_name":"mrdavehill","location":"New York City","url":"http:\/\/www.davehillonline.com","description":"My 2nd book DAVE HILL DOESN'T LIVE HERE ANYMORE out now! Get it!: https:\/\/t.co\/F4XTFUJD6v. The Goddamn Dave Hill Show Mondays 9pm-12am on @WFMU. S-T-A-U-N-C-H.","protected":false,"verified":true,"followers_count":54878,"friends_count":4342,"listed_count":1723,"favourites_count":23946,"statuses_count":56257,"created_at":"Sat Dec 13 20:55:21 +0000 2008","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"1A1B1F","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/580895891445301249\/LGGcqZkm.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/580895891445301249\/LGGcqZkm.jpg","profile_background_tile":true,"profile_link_color":"2FC2EF","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"252429","profile_text_color":"666666","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/719575070033735681\/S_3SjFPP_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/719575070033735681\/S_3SjFPP_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/18104752\/1467752456","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"quoted_status_id":787758668565581824,"quoted_status_id_str":"787758668565581824","quoted_status":{"created_at":"Sun Oct 16 20:54:37 +0000 2016","id":787758668565581824,"id_str":"787758668565581824","text":"Paul Ryan, a man who doesn't know how to win (including failed run four years ago), must start focusing on the budget, military, vets etc.","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":25073877,"id_str":"25073877","name":"Donald J. Trump","screen_name":"realDonaldTrump","location":"New York, NY","url":"http:\/\/www.DonaldJTrump.com","description":null,"protected":false,"verified":true,"followers_count":12517453,"friends_count":41,"listed_count":40274,"favourites_count":44,"statuses_count":33597,"created_at":"Wed Mar 18 13:46:38 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"6D5C18","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/530021613\/trump_scotland__43_of_70_cc.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/530021613\/trump_scotland__43_of_70_cc.jpg","profile_background_tile":true,"profile_link_color":"0D5B73","profile_sidebar_border_color":"BDDCAD","profile_sidebar_fill_color":"C5CEC0","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/1980294624\/DJT_Headshot_V2_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/1980294624\/DJT_Headshot_V2_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/25073877\/1468988952","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":9735,"favorite_count":26385,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/SvW1JsxJft","expanded_url":"https:\/\/twitter.com\/realdonaldtrump\/status\/787758668565581824","display_url":"twitter.com\/realdonaldtrum\u2026","indices":[108,131]}],"user_mentions":[{"screen_name":"SpeakerRyan","name":"Paul Ryan","id":18916432,"id_str":"18916432","indices":[4,16]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419009"}
    ,
    {"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985756136566785,"id_str":"787985756136566785","text":"RT @DiamondandSilk: Only A Fool Fights In A Burning House. Won't nothing change when you vote for the same thing. @DiamondandSilk will\u2026 ","source":"\u003ca href=\"http:\/\/twitter.com\/#!\/download\/ipad\" rel=\"nofollow\"\u003eTwitter for iPad\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":808156722,"id_str":"808156722","name":"Harold Mann","screen_name":"Mann_HR","location":"Southeastern USA","url":null,"description":"Christian, Original principles of our Constitution\/Bill of Rights. #Prolife. #ProIsrael. #NRA, #Auburn Tiger, #MolonLabe, God bless America!","protected":false,"verified":false,"followers_count":1445,"friends_count":1515,"listed_count":23,"favourites_count":3820,"statuses_count":15421,"created_at":"Fri Sep 07 04:21:04 +0000 2012","utc_offset":-18000,"time_zone":"Central Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/478283602728198144\/KsKaup4a_normal.jpeg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/478283602728198144\/KsKaup4a_normal.jpeg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/808156722\/1355780957","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 09:31:04 +0000 2016","id":787949037022425089,"id_str":"787949037022425089","text":"Only A Fool Fights In A Burning House. Won't nothing change when you vote for the same thing. @DiamondandSilk will\u2026 https:\/\/t.co\/3RMF3EvmWw","display_text_range":[0,140],"source":"\u003ca href=\"https:\/\/ads.twitter.com\" rel=\"nofollow\"\u003eTwitter Ads\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":2908170952,"id_str":"2908170952","name":"Diamond and Silk\u00ae","screen_name":"DiamondandSilk","location":"United States","url":"http:\/\/www.DiamondandSilk.com","description":"#STUMP4TRUMPBABY #DITCHANDSWITCHNOW   #STUMP4TRUMPGIRLS  #STOMP4TRUMP   #TheUnitedStatesNotTheDividedStates  http:\/\/www.diamondandsilkinc.com\/about-us\/","protected":false,"verified":false,"followers_count":151104,"friends_count":40,"listed_count":767,"favourites_count":1065,"statuses_count":3287,"created_at":"Sat Dec 06 18:49:06 +0000 2014","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme15\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme15\/bg.png","profile_background_tile":false,"profile_link_color":"9266CC","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/638558385617567744\/mq4EKwOy_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/638558385617567744\/mq4EKwOy_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/2908170952\/1441236155","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"extended_tweet":{"full_text":"Only A Fool Fights In A Burning House. Won't nothing change when you vote for the same thing. @DiamondandSilk will Vote Trump on Nov 8,2016. https:\/\/t.co\/Htg2sy09JS","display_text_range":[0,140],"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"DiamondandSilk","name":"Diamond and Silk\u00ae","id":2908170952,"id_str":"2908170952","indices":[94,109]}],"symbols":[],"media":[{"id":770640547715350528,"id_str":"770640547715350528","indices":[141,164],"media_url":"http:\/\/pbs.twimg.com\/media\/CtEdY5gXYAILf5q.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/CtEdY5gXYAILf5q.jpg","url":"https:\/\/t.co\/Htg2sy09JS","display_url":"pic.twitter.com\/Htg2sy09JS","expanded_url":"https:\/\/twitter.com\/DiamondandSilk\/status\/787949037022425089\/video\/1","type":"video","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":640,"h":360,"resize":"fit"},"large":{"w":640,"h":360,"resize":"fit"},"medium":{"w":640,"h":360,"resize":"fit"}},"video_info":{"aspect_ratio":[16,9],"duration_millis":182917,"variants":[{"content_type":"application\/x-mpegURL","url":"https:\/\/video.twimg.com\/amplify_video\/770640547715350528\/pl\/THTUQ9Gr6JEQ8ori.m3u8"},{"content_type":"application\/dash+xml","url":"https:\/\/video.twimg.com\/amplify_video\/770640547715350528\/pl\/THTUQ9Gr6JEQ8ori.mpd"},{"bitrate":320000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/amplify_video\/770640547715350528\/vid\/320x180\/J1erQ4hJZoqwS9Y0.mp4"},{"bitrate":2176000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/amplify_video\/770640547715350528\/vid\/1280x720\/3sw_1UymMEcu5kWC.mp4"},{"bitrate":832000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/amplify_video\/770640547715350528\/vid\/640x360\/Kby0CaN6O6qLPBlv.mp4"}]}}]},"extended_entities":{"media":[{"id":770640547715350528,"id_str":"770640547715350528","indices":[141,164],"media_url":"http:\/\/pbs.twimg.com\/media\/CtEdY5gXYAILf5q.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/CtEdY5gXYAILf5q.jpg","url":"https:\/\/t.co\/Htg2sy09JS","display_url":"pic.twitter.com\/Htg2sy09JS","expanded_url":"https:\/\/twitter.com\/DiamondandSilk\/status\/787949037022425089\/video\/1","type":"video","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":640,"h":360,"resize":"fit"},"large":{"w":640,"h":360,"resize":"fit"},"medium":{"w":640,"h":360,"resize":"fit"}},"video_info":{"aspect_ratio":[16,9],"duration_millis":182917,"variants":[{"content_type":"application\/x-mpegURL","url":"https:\/\/video.twimg.com\/amplify_video\/770640547715350528\/pl\/THTUQ9Gr6JEQ8ori.m3u8"},{"content_type":"application\/dash+xml","url":"https:\/\/video.twimg.com\/amplify_video\/770640547715350528\/pl\/THTUQ9Gr6JEQ8ori.mpd"},{"bitrate":320000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/amplify_video\/770640547715350528\/vid\/320x180\/J1erQ4hJZoqwS9Y0.mp4"},{"bitrate":2176000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/amplify_video\/770640547715350528\/vid\/1280x720\/3sw_1UymMEcu5kWC.mp4"},{"bitrate":832000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/amplify_video\/770640547715350528\/vid\/640x360\/Kby0CaN6O6qLPBlv.mp4"}]}}]}},"retweet_count":582,"favorite_count":901,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/3RMF3EvmWw","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787949037022425089","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[116,139]}],"user_mentions":[{"screen_name":"DiamondandSilk","name":"Diamond and Silk\u00ae","id":2908170952,"id_str":"2908170952","indices":[94,109]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"","expanded_url":null,"indices":[136,136]}],"user_mentions":[{"screen_name":"DiamondandSilk","name":"Diamond and Silk\u00ae","id":2908170952,"id_str":"2908170952","indices":[3,18]},{"screen_name":"DiamondandSilk","name":"Diamond and Silk\u00ae","id":2908170952,"id_str":"2908170952","indices":[114,129]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418987"}
    ,
    {"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985755612348416,"id_str":"787985755612348416","text":"RT @mtracey: This is disgusting war propaganda and everyone associated with the Clinton campaign should be ashamed of themselves https:\/\/t.\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":893221620,"id_str":"893221620","name":"Satyan Agarwal","screen_name":"SatyanAgarwal","location":"United States","url":null,"description":"Live Life King Size","protected":false,"verified":false,"followers_count":88,"friends_count":190,"listed_count":7,"favourites_count":215,"statuses_count":348,"created_at":"Sat Oct 20 13:15:52 +0000 2012","utc_offset":null,"time_zone":null,"geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/780110454907600896\/cyx9Wt1M_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/780110454907600896\/cyx9Wt1M_normal.jpg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 03:11:12 +0000 2016","id":787853439808245760,"id_str":"787853439808245760","text":"This is disgusting war propaganda and everyone associated with the Clinton campaign should be ashamed of themselves https:\/\/t.co\/OcI5Bj0g8T","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":24127811,"id_str":"24127811","name":"Michael Tracey","screen_name":"mtracey","location":"NYC","url":"http:\/\/mctracey.com","description":"Roving journalist, VICE columnist, friend to all dogs","protected":false,"verified":true,"followers_count":33686,"friends_count":1373,"listed_count":889,"favourites_count":5044,"statuses_count":64398,"created_at":"Fri Mar 13 04:31:49 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/517120721328156672\/7zUx36V__normal.jpeg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/517120721328156672\/7zUx36V__normal.jpeg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/24127811\/1475721783","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"quoted_status_id":787755182792794112,"quoted_status_id_str":"787755182792794112","quoted_status":{"created_at":"Sun Oct 16 20:40:46 +0000 2016","id":787755182792794112,"id_str":"787755182792794112","text":"Wow\u2014 Clinton camp out with 6-minute video highlighting Trump's deep ties to Russia: https:\/\/t.co\/RwJr3eWNnf","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":32871086,"id_str":"32871086","name":"Kyle Griffin","screen_name":"kylegriffin1","location":"Manhattan, NY","url":"http:\/\/www.msnbc.com\/the-last-word","description":"Producer on MSNBC's @TheLastWord. Former Sydneysider. Opinions my own. Please clap.","protected":false,"verified":true,"followers_count":20199,"friends_count":1218,"listed_count":521,"favourites_count":33957,"statuses_count":19336,"created_at":"Sat Apr 18 12:45:48 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/238292858\/Twitter_background.png","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/238292858\/Twitter_background.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/658832379130200064\/YUk99vQB_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/658832379130200064\/YUk99vQB_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/32871086\/1455243585","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":2419,"favorite_count":2305,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[],"media":[{"id":787379765267271680,"id_str":"787379765267271680","indices":[84,107],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu50qLMUMAADynP.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu50qLMUMAADynP.jpg","url":"https:\/\/t.co\/RwJr3eWNnf","display_url":"pic.twitter.com\/RwJr3eWNnf","expanded_url":"https:\/\/twitter.com\/TheBriefing2016\/status\/787695356075188224\/video\/1","type":"photo","sizes":{"small":{"w":680,"h":383,"resize":"fit"},"medium":{"w":1200,"h":675,"resize":"fit"},"large":{"w":1280,"h":720,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"}},"source_status_id":787695356075188224,"source_status_id_str":"787695356075188224","source_user_id":3232054991,"source_user_id_str":"3232054991"}]},"extended_entities":{"media":[{"id":787379765267271680,"id_str":"787379765267271680","indices":[84,107],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu50qLMUMAADynP.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu50qLMUMAADynP.jpg","url":"https:\/\/t.co\/RwJr3eWNnf","display_url":"pic.twitter.com\/RwJr3eWNnf","expanded_url":"https:\/\/twitter.com\/TheBriefing2016\/status\/787695356075188224\/video\/1","type":"video","sizes":{"small":{"w":680,"h":383,"resize":"fit"},"medium":{"w":1200,"h":675,"resize":"fit"},"large":{"w":1280,"h":720,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"}},"source_status_id":787695356075188224,"source_status_id_str":"787695356075188224","source_user_id":3232054991,"source_user_id_str":"3232054991","video_info":{"aspect_ratio":[16,9],"duration_millis":352186,"variants":[{"bitrate":2176000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/amplify_video\/787379765267271680\/vid\/1280x720\/5MScr9_aW2WtB0fX.mp4"},{"content_type":"application\/x-mpegURL","url":"https:\/\/video.twimg.com\/amplify_video\/787379765267271680\/pl\/wW3NUuVigguWbYjQ.m3u8"},{"content_type":"application\/dash+xml","url":"https:\/\/video.twimg.com\/amplify_video\/787379765267271680\/pl\/wW3NUuVigguWbYjQ.mpd"},{"bitrate":832000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/amplify_video\/787379765267271680\/vid\/640x360\/KHglDFoJ36RPJ7Gm.mp4"},{"bitrate":320000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/amplify_video\/787379765267271680\/vid\/320x180\/jaXSzSRNF58IC7GJ.mp4"}]}}]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":842,"favorite_count":1079,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/OcI5Bj0g8T","expanded_url":"https:\/\/twitter.com\/kylegriffin1\/status\/787755182792794112","display_url":"twitter.com\/kylegriffin1\/s\u2026","indices":[116,139]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"quoted_status_id":787755182792794112,"quoted_status_id_str":"787755182792794112","quoted_status":{"created_at":"Sun Oct 16 20:40:46 +0000 2016","id":787755182792794112,"id_str":"787755182792794112","text":"Wow\u2014 Clinton camp out with 6-minute video highlighting Trump's deep ties to Russia: https:\/\/t.co\/RwJr3eWNnf","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":32871086,"id_str":"32871086","name":"Kyle Griffin","screen_name":"kylegriffin1","location":"Manhattan, NY","url":"http:\/\/www.msnbc.com\/the-last-word","description":"Producer on MSNBC's @TheLastWord. Former Sydneysider. Opinions my own. Please clap.","protected":false,"verified":true,"followers_count":20199,"friends_count":1218,"listed_count":521,"favourites_count":33957,"statuses_count":19336,"created_at":"Sat Apr 18 12:45:48 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/238292858\/Twitter_background.png","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/238292858\/Twitter_background.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/658832379130200064\/YUk99vQB_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/658832379130200064\/YUk99vQB_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/32871086\/1455243585","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":2419,"favorite_count":2305,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[],"media":[{"id":787379765267271680,"id_str":"787379765267271680","indices":[84,107],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu50qLMUMAADynP.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu50qLMUMAADynP.jpg","url":"https:\/\/t.co\/RwJr3eWNnf","display_url":"pic.twitter.com\/RwJr3eWNnf","expanded_url":"https:\/\/twitter.com\/TheBriefing2016\/status\/787695356075188224\/video\/1","type":"photo","sizes":{"small":{"w":680,"h":383,"resize":"fit"},"medium":{"w":1200,"h":675,"resize":"fit"},"large":{"w":1280,"h":720,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"}},"source_status_id":787695356075188224,"source_status_id_str":"787695356075188224","source_user_id":3232054991,"source_user_id_str":"3232054991"}]},"extended_entities":{"media":[{"id":787379765267271680,"id_str":"787379765267271680","indices":[84,107],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu50qLMUMAADynP.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu50qLMUMAADynP.jpg","url":"https:\/\/t.co\/RwJr3eWNnf","display_url":"pic.twitter.com\/RwJr3eWNnf","expanded_url":"https:\/\/twitter.com\/TheBriefing2016\/status\/787695356075188224\/video\/1","type":"video","sizes":{"small":{"w":680,"h":383,"resize":"fit"},"medium":{"w":1200,"h":675,"resize":"fit"},"large":{"w":1280,"h":720,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"}},"source_status_id":787695356075188224,"source_status_id_str":"787695356075188224","source_user_id":3232054991,"source_user_id_str":"3232054991","video_info":{"aspect_ratio":[16,9],"duration_millis":352186,"variants":[{"bitrate":2176000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/amplify_video\/787379765267271680\/vid\/1280x720\/5MScr9_aW2WtB0fX.mp4"},{"content_type":"application\/x-mpegURL","url":"https:\/\/video.twimg.com\/amplify_video\/787379765267271680\/pl\/wW3NUuVigguWbYjQ.m3u8"},{"content_type":"application\/dash+xml","url":"https:\/\/video.twimg.com\/amplify_video\/787379765267271680\/pl\/wW3NUuVigguWbYjQ.mpd"},{"bitrate":832000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/amplify_video\/787379765267271680\/vid\/640x360\/KHglDFoJ36RPJ7Gm.mp4"},{"bitrate":320000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/amplify_video\/787379765267271680\/vid\/320x180\/jaXSzSRNF58IC7GJ.mp4"}]}}]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"mtracey","name":"Michael Tracey","id":24127811,"id_str":"24127811","indices":[3,11]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418862"}
    ,
    {"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985755977179136,"id_str":"787985755977179136","text":"Is...is a major party candidate on the verge of depressing his OWN turnout? https:\/\/t.co\/B3S9RxBoFC","display_text_range":[0,75],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":169062735,"id_str":"169062735","name":"Tim Krepp","screen_name":"timkrepp","location":"DC","url":"http:\/\/www.timkrepp.com","description":"just google it...","protected":false,"verified":false,"followers_count":2515,"friends_count":1443,"listed_count":96,"favourites_count":43329,"statuses_count":62393,"created_at":"Wed Jul 21 13:09:12 +0000 2010","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/766328696965894145\/hZ8Sm-YY_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/766328696965894145\/hZ8Sm-YY_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/169062735\/1465924878","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":{"id":"d6819fe60643ebc1","url":"https:\/\/api.twitter.com\/1.1\/geo\/id\/d6819fe60643ebc1.json","place_type":"city","name":"Alexandria","full_name":"Alexandria, VA","country_code":"US","country":"United States","bounding_box":{"type":"Polygon","coordinates":[[[-77.144435,38.789907],[-77.144435,38.844853],[-77.037304,38.844853],[-77.037304,38.789907]]]},"attributes":{}},"contributors":null,"quoted_status_id":787984785792696320,"quoted_status_id_str":"787984785792696320","quoted_status":{"created_at":"Mon Oct 17 11:53:07 +0000 2016","id":787984785792696320,"id_str":"787984785792696320","text":"Trump\u2019s \u201crigged\u201d lie appears to be working with supporters. Only half think their votes will be counted accurately\u2026 https:\/\/t.co\/W3MAskRQUl","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":63004503,"id_str":"63004503","name":"Michael Calderone","screen_name":"mlcalderone","location":"Brooklyn, NY","url":"http:\/\/michaelcalderone.com\/","description":"Senior media reporter, @HuffingtonPost; Adjunct faculty, @nyu_journalism; Ex-Politico\/Yahoo News\/NY Observer\nmichael.calderone@huffingtonpost.com","protected":false,"verified":true,"followers_count":53057,"friends_count":1698,"listed_count":2299,"favourites_count":0,"statuses_count":19689,"created_at":"Wed Aug 05 02:23:01 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"8B542B","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/49995328\/unknown.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/49995328\/unknown.jpg","profile_background_tile":true,"profile_link_color":"9D582E","profile_sidebar_border_color":"D9B17E","profile_sidebar_fill_color":"EADEAA","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/707744338172907520\/sOBnnXo8_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/707744338172907520\/sOBnnXo8_normal.jpg","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"extended_tweet":{"full_text":"Trump\u2019s \u201crigged\u201d lie appears to be working with supporters. Only half think their votes will be counted accurately https:\/\/t.co\/1NcxJ8C5LL https:\/\/t.co\/akfSdqYq6o","display_text_range":[0,138],"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/1NcxJ8C5LL","expanded_url":"http:\/\/bit.ly\/2duOBGv","display_url":"bit.ly\/2duOBGv","indices":[115,138]}],"user_mentions":[],"symbols":[],"media":[{"id":787984587003682816,"id_str":"787984587003682816","indices":[139,162],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu97zpTWYAAc5Td.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu97zpTWYAAc5Td.jpg","url":"https:\/\/t.co\/akfSdqYq6o","display_url":"pic.twitter.com\/akfSdqYq6o","expanded_url":"https:\/\/twitter.com\/mlcalderone\/status\/787984785792696320\/photo\/1","type":"photo","sizes":{"large":{"w":1419,"h":859,"resize":"fit"},"small":{"w":680,"h":412,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":1200,"h":726,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787984587003682816,"id_str":"787984587003682816","indices":[139,162],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu97zpTWYAAc5Td.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu97zpTWYAAc5Td.jpg","url":"https:\/\/t.co\/akfSdqYq6o","display_url":"pic.twitter.com\/akfSdqYq6o","expanded_url":"https:\/\/twitter.com\/mlcalderone\/status\/787984785792696320\/photo\/1","type":"photo","sizes":{"large":{"w":1419,"h":859,"resize":"fit"},"small":{"w":680,"h":412,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":1200,"h":726,"resize":"fit"}}}]}},"retweet_count":5,"favorite_count":4,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/W3MAskRQUl","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787984785792696320","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[116,139]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/B3S9RxBoFC","expanded_url":"https:\/\/twitter.com\/mlcalderone\/status\/787984785792696320","display_url":"twitter.com\/mlcalderone\/st\u2026","indices":[76,99]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418949"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985756346261504,"id_str":"787985756346261504","text":"RT @PrisonPlanet: CNN Host Blames Firebombing of Republican HQ on Donald Trump - https:\/\/t.co\/8yY06ncB8n https:\/\/t.co\/eAFC6RHlhe","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":397585567,"id_str":"397585567","name":"Roger Johnson","screen_name":"RogerJohnson13","location":null,"url":null,"description":null,"protected":false,"verified":false,"followers_count":17,"friends_count":88,"listed_count":3,"favourites_count":184,"statuses_count":2065,"created_at":"Mon Oct 24 21:58:11 +0000 2011","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_1_normal.png","profile_image_url_https":"https:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_1_normal.png","default_profile":true,"default_profile_image":true,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:54:42 +0000 2016","id":787985184297775105,"id_str":"787985184297775105","text":"CNN Host Blames Firebombing of Republican HQ on Donald Trump - https:\/\/t.co\/8yY06ncB8n https:\/\/t.co\/eAFC6RHlhe","display_text_range":[0,86],"source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":18643437,"id_str":"18643437","name":"Paul Joseph Watson","screen_name":"PrisonPlanet","location":"London","url":"https:\/\/www.youtube.com\/user\/PrisonPlanetLive","description":"Contrarian polemicist, Infowars Editor-at-Large, 'pasty geek' (Louise Mensch)","protected":false,"verified":true,"followers_count":288036,"friends_count":278,"listed_count":3517,"favourites_count":4724,"statuses_count":26826,"created_at":"Mon Jan 05 20:04:23 +0000 2009","utc_offset":-18000,"time_zone":"Central Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"1A1B1F","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/378800000049129232\/f1cb5b3448d8a6d24e287249b765ae2a.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/378800000049129232\/f1cb5b3448d8a6d24e287249b765ae2a.jpeg","profile_background_tile":false,"profile_link_color":"2FC2EF","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"252429","profile_text_color":"666666","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/756825397388533760\/yXYx6CNC_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/756825397388533760\/yXYx6CNC_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/18643437\/1476394531","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":45,"favorite_count":45,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/8yY06ncB8n","expanded_url":"http:\/\/www.infowars.com\/cnn-host-blames-firebombing-of-republican-hq-on-donald-trump\/","display_url":"infowars.com\/cnn-host-blame\u2026","indices":[63,86]}],"user_mentions":[],"symbols":[],"media":[{"id":787985177343627264,"id_str":"787985177343627264","indices":[87,110],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu98WAfXEAA5TP4.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu98WAfXEAA5TP4.jpg","url":"https:\/\/t.co\/eAFC6RHlhe","display_url":"pic.twitter.com\/eAFC6RHlhe","expanded_url":"https:\/\/twitter.com\/PrisonPlanet\/status\/787985184297775105\/photo\/1","type":"photo","sizes":{"large":{"w":800,"h":420,"resize":"fit"},"medium":{"w":800,"h":420,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":680,"h":357,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787985177343627264,"id_str":"787985177343627264","indices":[87,110],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu98WAfXEAA5TP4.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu98WAfXEAA5TP4.jpg","url":"https:\/\/t.co\/eAFC6RHlhe","display_url":"pic.twitter.com\/eAFC6RHlhe","expanded_url":"https:\/\/twitter.com\/PrisonPlanet\/status\/787985184297775105\/photo\/1","type":"photo","sizes":{"large":{"w":800,"h":420,"resize":"fit"},"medium":{"w":800,"h":420,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":680,"h":357,"resize":"fit"}}}]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/8yY06ncB8n","expanded_url":"http:\/\/www.infowars.com\/cnn-host-blames-firebombing-of-republican-hq-on-donald-trump\/","display_url":"infowars.com\/cnn-host-blame\u2026","indices":[81,104]}],"user_mentions":[{"screen_name":"PrisonPlanet","name":"Paul Joseph Watson","id":18643437,"id_str":"18643437","indices":[3,16]}],"symbols":[],"media":[{"id":787985177343627264,"id_str":"787985177343627264","indices":[105,128],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu98WAfXEAA5TP4.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu98WAfXEAA5TP4.jpg","url":"https:\/\/t.co\/eAFC6RHlhe","display_url":"pic.twitter.com\/eAFC6RHlhe","expanded_url":"https:\/\/twitter.com\/PrisonPlanet\/status\/787985184297775105\/photo\/1","type":"photo","sizes":{"large":{"w":800,"h":420,"resize":"fit"},"medium":{"w":800,"h":420,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":680,"h":357,"resize":"fit"}},"source_status_id":787985184297775105,"source_status_id_str":"787985184297775105","source_user_id":18643437,"source_user_id_str":"18643437"}]},"extended_entities":{"media":[{"id":787985177343627264,"id_str":"787985177343627264","indices":[105,128],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu98WAfXEAA5TP4.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu98WAfXEAA5TP4.jpg","url":"https:\/\/t.co\/eAFC6RHlhe","display_url":"pic.twitter.com\/eAFC6RHlhe","expanded_url":"https:\/\/twitter.com\/PrisonPlanet\/status\/787985184297775105\/photo\/1","type":"photo","sizes":{"large":{"w":800,"h":420,"resize":"fit"},"medium":{"w":800,"h":420,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":680,"h":357,"resize":"fit"}},"source_status_id":787985184297775105,"source_status_id_str":"787985184297775105","source_user_id":18643437,"source_user_id_str":"18643437"}]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419037"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985756677603329,"id_str":"787985756677603329","text":"RT @ron_fournier: Clinton's supporters at PM USA considered push polling on her behalf. @JoeNBC is right: you're arguing a distinctio\u2026 ","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":85982089,"id_str":"85982089","name":"Javier Barberan","screen_name":"Toyoking","location":"Pembroke Pines, FL","url":null,"description":null,"protected":false,"verified":false,"followers_count":369,"friends_count":860,"listed_count":77,"favourites_count":16864,"statuses_count":16725,"created_at":"Thu Oct 29 03:09:00 +0000 2009","utc_offset":7200,"time_zone":"Copenhagen","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/694132909222002688\/q17HHTnK_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/694132909222002688\/q17HHTnK_normal.jpg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 10:41:03 +0000 2016","id":787966649928392704,"id_str":"787966649928392704","text":"Clinton's supporters at PM USA considered push polling on her behalf. @JoeNBC is right: you're arguing a distinctio\u2026 https:\/\/t.co\/0TjFMEVWKQ","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":14268812,"id_str":"14268812","name":"Ron Fournier","screen_name":"ron_fournier","location":"Detroit, MI","url":"http:\/\/www.theatlantic.com\/author\/ron-fournier\/","description":"Associate publisher Crain's Detroit. Dabble in politics for @TheAtlantic. Author, NYT best-seller #LoveThatBoy http:\/\/lovethatboybook.com\/books\/love-that-boy-hc","protected":false,"verified":true,"followers_count":56903,"friends_count":2110,"listed_count":1818,"favourites_count":53702,"statuses_count":142278,"created_at":"Mon Mar 31 19:35:07 +0000 2008","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/531032491\/NJTwitterBackground.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/531032491\/NJTwitterBackground.jpg","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/777365314094899200\/w59eXkTc_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/777365314094899200\/w59eXkTc_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/14268812\/1473438246","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"quoted_status_id":787961268305100800,"quoted_status_id_str":"787961268305100800","quoted_status":{"created_at":"Mon Oct 17 10:19:40 +0000 2016","id":787961268305100800,"id_str":"787961268305100800","text":"@ron_fournier Also Progressive Media USA conducted those polls, not Clinton campaign.","display_text_range":[14,85],"source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":787961084661596160,"in_reply_to_status_id_str":"787961084661596160","in_reply_to_user_id":16713361,"in_reply_to_user_id_str":"16713361","in_reply_to_screen_name":"SuzanneKelleher","user":{"id":16713361,"id_str":"16713361","name":"Suzanne Kelleher","screen_name":"SuzanneKelleher","location":null,"url":null,"description":"mom of three | family vacations expert | freelance travel writer | content strategy consultant #familytravel","protected":false,"verified":false,"followers_count":15844,"friends_count":5453,"listed_count":923,"favourites_count":13447,"statuses_count":44887,"created_at":"Sun Oct 12 23:17:55 +0000 2008","utc_offset":-7200,"time_zone":"Mid-Atlantic","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"9AE4E8","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/722820902\/35d7996f15723dabd4528fd9b06043a5.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/722820902\/35d7996f15723dabd4528fd9b06043a5.jpeg","profile_background_tile":false,"profile_link_color":"DD2E44","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"DDFFCC","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/418474427173462016\/5wWRhgkB_normal.jpeg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/418474427173462016\/5wWRhgkB_normal.jpeg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/16713361\/1434544459","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"ron_fournier","name":"Ron Fournier","id":14268812,"id_str":"14268812","indices":[0,13]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"extended_tweet":{"full_text":"Clinton's supporters at PM USA considered push polling on her behalf. @JoeNBC is right: you're arguing a distinction with little difference https:\/\/t.co\/BczW8M52yr","display_text_range":[0,139],"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/BczW8M52yr","expanded_url":"https:\/\/twitter.com\/suzannekelleher\/status\/787961268305100800","display_url":"twitter.com\/suzannekellehe\u2026","indices":[140,163]}],"user_mentions":[{"screen_name":"JoeNBC","name":"Joe Scarborough","id":21619519,"id_str":"21619519","indices":[70,77]}],"symbols":[]}},"retweet_count":7,"favorite_count":15,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/0TjFMEVWKQ","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787966649928392704","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[117,140]}],"user_mentions":[{"screen_name":"JoeNBC","name":"Joe Scarborough","id":21619519,"id_str":"21619519","indices":[70,77]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"quoted_status_id":787961268305100800,"quoted_status_id_str":"787961268305100800","quoted_status":{"created_at":"Mon Oct 17 10:19:40 +0000 2016","id":787961268305100800,"id_str":"787961268305100800","text":"@ron_fournier Also Progressive Media USA conducted those polls, not Clinton campaign.","display_text_range":[14,85],"source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":787961084661596160,"in_reply_to_status_id_str":"787961084661596160","in_reply_to_user_id":16713361,"in_reply_to_user_id_str":"16713361","in_reply_to_screen_name":"SuzanneKelleher","user":{"id":16713361,"id_str":"16713361","name":"Suzanne Kelleher","screen_name":"SuzanneKelleher","location":null,"url":null,"description":"mom of three | family vacations expert | freelance travel writer | content strategy consultant #familytravel","protected":false,"verified":false,"followers_count":15844,"friends_count":5453,"listed_count":923,"favourites_count":13447,"statuses_count":44887,"created_at":"Sun Oct 12 23:17:55 +0000 2008","utc_offset":-7200,"time_zone":"Mid-Atlantic","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"9AE4E8","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/722820902\/35d7996f15723dabd4528fd9b06043a5.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/722820902\/35d7996f15723dabd4528fd9b06043a5.jpeg","profile_background_tile":false,"profile_link_color":"DD2E44","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"DDFFCC","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/418474427173462016\/5wWRhgkB_normal.jpeg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/418474427173462016\/5wWRhgkB_normal.jpeg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/16713361\/1434544459","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"ron_fournier","name":"Ron Fournier","id":14268812,"id_str":"14268812","indices":[0,13]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"","expanded_url":null,"indices":[135,135]}],"user_mentions":[{"screen_name":"ron_fournier","name":"Ron Fournier","id":14268812,"id_str":"14268812","indices":[3,16]},{"screen_name":"JoeNBC","name":"Joe Scarborough","id":21619519,"id_str":"21619519","indices":[88,95]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419116"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985757210304512,"id_str":"787985757210304512","text":"RT @CNNEE: Resumen de encuestas: Clinton aventaja a Trump por 8 puntos https:\/\/t.co\/OHAdy3KMxg","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":60212445,"id_str":"60212445","name":"Richard Mill\u00e1n","screen_name":"millanrichard","location":"Hate\u00f1o y Vallecaucano ","url":null,"description":"Periodista de pura sangre #combomadrugador #hinchadelcali y voto SI","protected":false,"verified":false,"followers_count":1259,"friends_count":167,"listed_count":18,"favourites_count":322,"statuses_count":6186,"created_at":"Sun Jul 26 02:40:17 +0000 2009","utc_offset":-18000,"time_zone":"Bogota","geo_enabled":true,"lang":"es","contributors_enabled":false,"is_translator":false,"profile_background_color":"8B542B","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme8\/bg.gif","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme8\/bg.gif","profile_background_tile":true,"profile_link_color":"9D582E","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"EADEAA","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/697961720015364097\/_gLD_iuH_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/697961720015364097\/_gLD_iuH_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/60212445\/1397913978","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:21:21 +0000 2016","id":787976789893283841,"id_str":"787976789893283841","text":"Resumen de encuestas: Clinton aventaja a Trump por 8 puntos https:\/\/t.co\/OHAdy3KMxg","source":"\u003ca href=\"https:\/\/about.twitter.com\/products\/tweetdeck\" rel=\"nofollow\"\u003eTweetDeck\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":33884545,"id_str":"33884545","name":"CNN en Espa\u00f1ol","screen_name":"CNNEE","location":"Atlanta","url":"https:\/\/telegram.me\/cnneebrk","description":"CNN en Espa\u00f1ol es tu principal fuente de informaci\u00f3n y breaking news. Cubrimos las noticias de Am\u00e9rica Latina y el resto del mundo. Vive la noticia.","protected":false,"verified":true,"followers_count":14039334,"friends_count":832,"listed_count":48093,"favourites_count":1269,"statuses_count":88501,"created_at":"Tue Apr 21 12:14:47 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"es","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/526905009\/CNNespanol_TwitterSkin.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/526905009\/CNNespanol_TwitterSkin.jpg","profile_background_tile":false,"profile_link_color":"AB1529","profile_sidebar_border_color":"B5B5B5","profile_sidebar_fill_color":"B5B5B5","profile_text_color":"000000","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/589795319216504832\/6a71ZHkx_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/589795319216504832\/6a71ZHkx_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/33884545\/1411070627","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":57,"favorite_count":43,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/OHAdy3KMxg","expanded_url":"http:\/\/cnn.it\/2dhPUKq","display_url":"cnn.it\/2dhPUKq","indices":[60,83]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"es"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/OHAdy3KMxg","expanded_url":"http:\/\/cnn.it\/2dhPUKq","display_url":"cnn.it\/2dhPUKq","indices":[71,94]}],"user_mentions":[{"screen_name":"CNNEE","name":"CNN en Espa\u00f1ol","id":33884545,"id_str":"33884545","indices":[3,9]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"es","timestamp_ms":"1476705419243"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985756866347009,"id_str":"787985756866347009","text":"@Slate \nAccusers with Zero evidence.\nPaid Hit Men.\nPaid to destroy credibility.\n11 years ago.\n11th hour of Election.\nI was touched.","display_text_range":[8,131],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":787982514669383680,"in_reply_to_status_id_str":"787982514669383680","in_reply_to_user_id":15164565,"in_reply_to_user_id_str":"15164565","in_reply_to_screen_name":"Slate","user":{"id":288973453,"id_str":"288973453","name":"mike james","screen_name":"twthead","location":"EaRTH","url":null,"description":"earth born citizen.  psychological experiment.  transient being.  made of billion year old hydrogen molecules. running on the treadmill of life.","protected":false,"verified":false,"followers_count":331,"friends_count":167,"listed_count":3,"favourites_count":2151,"statuses_count":6631,"created_at":"Wed Apr 27 19:49:32 +0000 2011","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"022330","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme15\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme15\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"A8C7F7","profile_sidebar_fill_color":"C0DFEC","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/1329720707\/images_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/1329720707\/images_normal.jpg","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"Slate","name":"Slate","id":15164565,"id_str":"15164565","indices":[0,6]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419161"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985756715450368,"id_str":"787985756715450368","text":"RT @WesleyRickard: 10-16-2016 Donald Trump Rally 20,000 people ready to vote for Donald Trump on November 8th Retweet and Post to Face\u2026 ","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":346068301,"id_str":"346068301","name":"Jean Watson","screen_name":"jeanloveslife","location":"North Carolina","url":null,"description":"I love our God, my Family and Nascar!","protected":false,"verified":false,"followers_count":504,"friends_count":2351,"listed_count":10,"favourites_count":2522,"statuses_count":3580,"created_at":"Sun Jul 31 17:32:01 +0000 2011","utc_offset":-10800,"time_zone":"Atlantic Time (Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"ACDED6","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme18\/bg.gif","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme18\/bg.gif","profile_background_tile":false,"profile_link_color":"DD2E44","profile_sidebar_border_color":"EEEEEE","profile_sidebar_fill_color":"F6F6F6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/569842177715347456\/pW7IyVB8_normal.jpeg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/569842177715347456\/pW7IyVB8_normal.jpeg","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 03:37:48 +0000 2016","id":787860136635359232,"id_str":"787860136635359232","text":"10-16-2016 Donald Trump Rally 20,000 people ready to vote for Donald Trump on November 8th Retweet and Post to Face\u2026 https:\/\/t.co\/yEiGj7mmbq","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":699344605888782341,"id_str":"699344605888782341","name":"VideoLeaks","screen_name":"WesleyRickard","location":"New York, NY","url":null,"description":"Bringing you the latest videos from across the United States for Election 2016. #Trump #Clinton #NYC (+ download 4) (Google Play Store) - Transfer to FB or IG","protected":false,"verified":false,"followers_count":16381,"friends_count":17906,"listed_count":164,"favourites_count":3546,"statuses_count":4821,"created_at":"Mon Feb 15 21:28:42 +0000 2016","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"F5F8FA","profile_background_image_url":"","profile_background_image_url_https":"","profile_background_tile":false,"profile_link_color":"2B7BB9","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/786029450047713280\/RYz1PuYg_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/786029450047713280\/RYz1PuYg_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/699344605888782341\/1476239103","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"extended_tweet":{"full_text":"10-16-2016 Donald Trump Rally 20,000 people ready to vote for Donald Trump on November 8th Retweet and Post to Facebook #PA #MI #NY #Trump https:\/\/t.co\/lK35z6NX5D","display_text_range":[0,138],"entities":{"hashtags":[{"text":"PA","indices":[120,123]},{"text":"MI","indices":[124,127]},{"text":"NY","indices":[128,131]},{"text":"Trump","indices":[132,138]}],"urls":[],"user_mentions":[],"symbols":[],"media":[{"id":787859892744888320,"id_str":"787859892744888320","indices":[139,162],"media_url":"http:\/\/pbs.twimg.com\/ext_tw_video_thumb\/787859892744888320\/pu\/img\/1qfAep82gJQV-HcA.jpg","media_url_https":"https:\/\/pbs.twimg.com\/ext_tw_video_thumb\/787859892744888320\/pu\/img\/1qfAep82gJQV-HcA.jpg","url":"https:\/\/t.co\/lK35z6NX5D","display_url":"pic.twitter.com\/lK35z6NX5D","expanded_url":"https:\/\/twitter.com\/WesleyRickard\/status\/787860136635359232\/video\/1","type":"video","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":600,"h":338,"resize":"fit"},"small":{"w":340,"h":191,"resize":"fit"},"large":{"w":1024,"h":576,"resize":"fit"}},"video_info":{"aspect_ratio":[16,9],"duration_millis":16343,"variants":[{"content_type":"application\/x-mpegURL","url":"https:\/\/video.twimg.com\/ext_tw_video\/787859892744888320\/pu\/pl\/L7hylf17HkCNL3Sw.m3u8"},{"bitrate":2176000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/ext_tw_video\/787859892744888320\/pu\/vid\/1280x720\/TYilOoYQ2P3CaOia.mp4"},{"content_type":"application\/dash+xml","url":"https:\/\/video.twimg.com\/ext_tw_video\/787859892744888320\/pu\/pl\/L7hylf17HkCNL3Sw.mpd"},{"bitrate":320000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/ext_tw_video\/787859892744888320\/pu\/vid\/320x180\/imrr4u7pCHdyotoy.mp4"},{"bitrate":832000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/ext_tw_video\/787859892744888320\/pu\/vid\/640x360\/0GhEhzEVZfjWG0RE.mp4"}]}}]},"extended_entities":{"media":[{"id":787859892744888320,"id_str":"787859892744888320","indices":[139,162],"media_url":"http:\/\/pbs.twimg.com\/ext_tw_video_thumb\/787859892744888320\/pu\/img\/1qfAep82gJQV-HcA.jpg","media_url_https":"https:\/\/pbs.twimg.com\/ext_tw_video_thumb\/787859892744888320\/pu\/img\/1qfAep82gJQV-HcA.jpg","url":"https:\/\/t.co\/lK35z6NX5D","display_url":"pic.twitter.com\/lK35z6NX5D","expanded_url":"https:\/\/twitter.com\/WesleyRickard\/status\/787860136635359232\/video\/1","type":"video","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":600,"h":338,"resize":"fit"},"small":{"w":340,"h":191,"resize":"fit"},"large":{"w":1024,"h":576,"resize":"fit"}},"video_info":{"aspect_ratio":[16,9],"duration_millis":16343,"variants":[{"content_type":"application\/x-mpegURL","url":"https:\/\/video.twimg.com\/ext_tw_video\/787859892744888320\/pu\/pl\/L7hylf17HkCNL3Sw.m3u8"},{"bitrate":2176000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/ext_tw_video\/787859892744888320\/pu\/vid\/1280x720\/TYilOoYQ2P3CaOia.mp4"},{"content_type":"application\/dash+xml","url":"https:\/\/video.twimg.com\/ext_tw_video\/787859892744888320\/pu\/pl\/L7hylf17HkCNL3Sw.mpd"},{"bitrate":320000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/ext_tw_video\/787859892744888320\/pu\/vid\/320x180\/imrr4u7pCHdyotoy.mp4"},{"bitrate":832000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/ext_tw_video\/787859892744888320\/pu\/vid\/640x360\/0GhEhzEVZfjWG0RE.mp4"}]}}]}},"retweet_count":199,"favorite_count":187,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/yEiGj7mmbq","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787860136635359232","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[117,140]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"","expanded_url":null,"indices":[136,136]}],"user_mentions":[{"screen_name":"WesleyRickard","name":"VideoLeaks","id":699344605888782341,"id_str":"699344605888782341","indices":[3,17]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419125"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985757365415936,"id_str":"787985757365415936","text":"RT @lukeoneil47: In his book 'Trump 101' \"When things go wrong look at yourself first. Don't instinctively blame others or the circu\u2026 ","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1167701558,"id_str":"1167701558","name":"Brendan Little","screen_name":"blittle86","location":null,"url":null,"description":"Policy Director at Mayor's Office of Recovery Services, City of Boston (@ORSBoston) | Singer-songwriter: http:\/\/brendanlittle.bandcamp.com","protected":false,"verified":false,"followers_count":834,"friends_count":1063,"listed_count":22,"favourites_count":1235,"statuses_count":1156,"created_at":"Mon Feb 11 03:20:15 +0000 2013","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/677150669057482752\/0KlvyoLI_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/677150669057482752\/0KlvyoLI_normal.jpg","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Sun Oct 16 14:05:52 +0000 2016","id":787655804761542656,"id_str":"787655804761542656","text":"In his book 'Trump 101' \"When things go wrong look at yourself first. Don't instinctively blame others or the circu\u2026 https:\/\/t.co\/j2hJpbTQRI","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":108338399,"id_str":"108338399","name":"luke o'neil","screen_name":"lukeoneil47","location":"Boston, MA","url":null,"description":"Writer-at-large @esquire. tips: loneil@hearst.com","protected":false,"verified":true,"followers_count":14871,"friends_count":1747,"listed_count":471,"favourites_count":38575,"statuses_count":71859,"created_at":"Mon Jan 25 16:40:01 +0000 2010","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"131516","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme14\/bg.gif","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme14\/bg.gif","profile_background_tile":true,"profile_link_color":"009999","profile_sidebar_border_color":"EEEEEE","profile_sidebar_fill_color":"EFEFEF","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/783710778260160512\/D1IWNLpj_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/783710778260160512\/D1IWNLpj_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/108338399\/1452280698","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"extended_tweet":{"full_text":"In his book 'Trump 101' \"When things go wrong look at yourself first. Don't instinctively blame others or the circumstances....\" https:\/\/t.co\/Hkx6ysx8Ct","display_text_range":[0,128],"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[],"media":[{"id":787655418898059264,"id_str":"787655418898059264","indices":[129,152],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu5QbijUkAAeM-O.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu5QbijUkAAeM-O.jpg","url":"https:\/\/t.co\/Hkx6ysx8Ct","display_url":"pic.twitter.com\/Hkx6ysx8Ct","expanded_url":"https:\/\/twitter.com\/lukeoneil47\/status\/787655804761542656\/photo\/1","type":"photo","sizes":{"large":{"w":664,"h":578,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":664,"h":578,"resize":"fit"},"small":{"w":664,"h":578,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787655418898059264,"id_str":"787655418898059264","indices":[129,152],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu5QbijUkAAeM-O.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu5QbijUkAAeM-O.jpg","url":"https:\/\/t.co\/Hkx6ysx8Ct","display_url":"pic.twitter.com\/Hkx6ysx8Ct","expanded_url":"https:\/\/twitter.com\/lukeoneil47\/status\/787655804761542656\/photo\/1","type":"photo","sizes":{"large":{"w":664,"h":578,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":664,"h":578,"resize":"fit"},"small":{"w":664,"h":578,"resize":"fit"}}}]}},"retweet_count":157,"favorite_count":354,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/j2hJpbTQRI","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787655804761542656","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[117,140]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"","expanded_url":null,"indices":[134,134]}],"user_mentions":[{"screen_name":"lukeoneil47","name":"luke o'neil","id":108338399,"id_str":"108338399","indices":[3,15]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419280"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985757419999236,"id_str":"787985757419999236","text":"RT @GeorgeTakei: Trump was too quick to blame Hillary for GOP office arson in NC. She strongly condemned the attack, and had zero logically\u2026","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":18770638,"id_str":"18770638","name":"(((Courtney Fay)))","screen_name":"misscrf","location":"Rochester, NY","url":null,"description":"Developer @ a law firm. Love my life, especially when there's coffee in the morning.","protected":false,"verified":false,"followers_count":217,"friends_count":847,"listed_count":32,"favourites_count":621,"statuses_count":8371,"created_at":"Thu Jan 08 17:20:49 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme7\/bg.gif","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme7\/bg.gif","profile_background_tile":false,"profile_link_color":"ABB8C2","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/735182941748400128\/TTkzMXAX_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/735182941748400128\/TTkzMXAX_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/18770638\/1476456847","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 02:40:15 +0000 2016","id":787845653074481153,"id_str":"787845653074481153","text":"Trump was too quick to blame Hillary for GOP office arson in NC. She strongly condemned the attack, and had zero logically to gain from it.","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":237845487,"id_str":"237845487","name":"George Takei","screen_name":"GeorgeTakei","location":"Broadway - New York, NY","url":"http:\/\/ohmyyy.gt\/GeorgeTakei","description":"Some know me as Mr. Sulu from Star Trek but I hope all know me as a believer in, and a fighter for, the equality and dignity of all human beings.","protected":false,"verified":true,"followers_count":1859686,"friends_count":485,"listed_count":20176,"favourites_count":1144,"statuses_count":23482,"created_at":"Thu Jan 13 19:33:56 +0000 2011","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"022330","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/479993169363218432\/YRp7xuMf.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/479993169363218432\/YRp7xuMf.jpeg","profile_background_tile":false,"profile_link_color":"ABB8C2","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/700355030117937152\/3uW9IKHD_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/700355030117937152\/3uW9IKHD_normal.png","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/237845487\/1474948080","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":691,"favorite_count":2629,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"GeorgeTakei","name":"George Takei","id":237845487,"id_str":"237845487","indices":[3,15]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419293"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985757550116864,"id_str":"787985757550116864","text":"RT @TrumpFever2016: DON'T BLAME TRUMP\n4= Civil UN-Rest\n4= EVERYTHING Wrong with USA\nTRUMP is OUR Last Hope\n@realDonaldTrump @JoeGruters htt\u2026","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":4895096885,"id_str":"4895096885","name":"Kim Meese","screen_name":"intheknow1798","location":"United States","url":null,"description":"Vote for Trump\/Pence!  Conservative Christian. Married. Supporter of NRA. Fight for America, Freedom, Security & Truth.  Porn will be blocked.","protected":false,"verified":false,"followers_count":4592,"friends_count":4483,"listed_count":40,"favourites_count":20010,"statuses_count":35732,"created_at":"Wed Feb 10 14:37:19 +0000 2016","utc_offset":-25200,"time_zone":"Pacific Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"F5F8FA","profile_background_image_url":"","profile_background_image_url_https":"","profile_background_tile":false,"profile_link_color":"2B7BB9","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/781819144287191040\/0EXdOtlw_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/781819144287191040\/0EXdOtlw_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/4895096885\/1455225517","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Thu Mar 17 21:19:43 +0000 2016","id":710576369638494209,"id_str":"710576369638494209","text":"DON'T BLAME TRUMP\n4= Civil UN-Rest\n4= EVERYTHING Wrong with USA\nTRUMP is OUR Last Hope\n@realDonaldTrump @JoeGruters https:\/\/t.co\/YNbuY0yYCX","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":2990789335,"id_str":"2990789335","name":"Trump 4 Women","screen_name":"TrumpFever2016","location":" DEPLORABLE America","url":"http:\/\/TrumpFever.US","description":"SEE @TheTRUMPetts #1 OFFICIAL TRUMP TRAIN Vid   #TeamTRUMP  \n HIT SONGWRITERS =USMC \/ LEO VETS","protected":false,"verified":false,"followers_count":12588,"friends_count":4055,"listed_count":144,"favourites_count":15116,"statuses_count":45186,"created_at":"Tue Jan 20 06:18:34 +0000 2015","utc_offset":-25200,"time_zone":"Pacific Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/695492228727201792\/svlvnAFE.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/695492228727201792\/svlvnAFE.jpg","profile_background_tile":true,"profile_link_color":"1B95E0","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/753333346357354497\/wHrXsp5x_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/753333346357354497\/wHrXsp5x_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/2990789335\/1455430399","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":{"id":"4ec01c9dbc693497","url":"https:\/\/api.twitter.com\/1.1\/geo\/id\/4ec01c9dbc693497.json","place_type":"admin","name":"Florida","full_name":"Florida, USA","country_code":"US","country":"United States","bounding_box":{"type":"Polygon","coordinates":[[[-87.634643,24.396308],[-87.634643,31.001056],[-79.974307,31.001056],[-79.974307,24.396308]]]},"attributes":{}},"contributors":null,"is_quote_status":false,"retweet_count":215,"favorite_count":215,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"realDonaldTrump","name":"Donald J. Trump","id":25073877,"id_str":"25073877","indices":[87,103]},{"screen_name":"JoeGruters","name":"Joe Gruters","id":40045792,"id_str":"40045792","indices":[104,115]}],"symbols":[],"media":[{"id":710576368921223169,"id_str":"710576368921223169","indices":[116,139],"media_url":"http:\/\/pbs.twimg.com\/media\/Cdx5cVdWIAEkqT3.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cdx5cVdWIAEkqT3.jpg","url":"https:\/\/t.co\/YNbuY0yYCX","display_url":"pic.twitter.com\/YNbuY0yYCX","expanded_url":"http:\/\/twitter.com\/chriskyleband\/status\/710576369638494209\/photo\/1","type":"photo","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":480,"h":372,"resize":"fit"},"large":{"w":480,"h":372,"resize":"fit"},"small":{"w":340,"h":264,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":710576368921223169,"id_str":"710576368921223169","indices":[116,139],"media_url":"http:\/\/pbs.twimg.com\/media\/Cdx5cVdWIAEkqT3.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cdx5cVdWIAEkqT3.jpg","url":"https:\/\/t.co\/YNbuY0yYCX","display_url":"pic.twitter.com\/YNbuY0yYCX","expanded_url":"http:\/\/twitter.com\/chriskyleband\/status\/710576369638494209\/photo\/1","type":"photo","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":480,"h":372,"resize":"fit"},"large":{"w":480,"h":372,"resize":"fit"},"small":{"w":340,"h":264,"resize":"fit"}}}]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"TrumpFever2016","name":"Trump 4 Women","id":2990789335,"id_str":"2990789335","indices":[3,18]},{"screen_name":"realDonaldTrump","name":"Donald J. Trump","id":25073877,"id_str":"25073877","indices":[107,123]},{"screen_name":"JoeGruters","name":"Joe Gruters","id":40045792,"id_str":"40045792","indices":[124,135]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419324"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985757742915584,"id_str":"787985757742915584","text":"RT @truthmashup: Trump TV, all Trump all day. What could go wrong? Btw, I'm sure Rupert Murdoch is loving this. #uspoli https:\/\/t.co\/9V3boc\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":2315874241,"id_str":"2315874241","name":"Robin","screen_name":"RobinBall1961","location":null,"url":null,"description":"Never Forget the #DecadeOfDarkness under #HarperGovt & #CPC! Don't let history repeat itself-keep #Tories out of Gov't. The Truth is Out There!","protected":false,"verified":false,"followers_count":1500,"friends_count":1051,"listed_count":761,"favourites_count":129514,"statuses_count":165753,"created_at":"Tue Jan 28 17:59:08 +0000 2014","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"9266CC","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/656304598198579200\/r62aVwpQ_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/656304598198579200\/r62aVwpQ_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/2315874241\/1453175663","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:25:08 +0000 2016","id":787977742432206848,"id_str":"787977742432206848","text":"Trump TV, all Trump all day. What could go wrong? Btw, I'm sure Rupert Murdoch is loving this. #uspoli https:\/\/t.co\/9V3bocNiXh","source":"\u003ca href=\"https:\/\/about.twitter.com\/products\/tweetdeck\" rel=\"nofollow\"\u003eTweetDeck\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":557449649,"id_str":"557449649","name":"Truth Mashup","screen_name":"truthmashup","location":"Toronto","url":"http:\/\/www.harperhistory.com","description":"Producer @VinceKes tweeting for @TYTNetwork's comedy channel. Hosted By @DanSpeerin! Tweets on #cdnpoli #uspoli and Donald Trump.","protected":false,"verified":false,"followers_count":2666,"friends_count":1990,"listed_count":166,"favourites_count":4069,"statuses_count":40505,"created_at":"Thu Apr 19 05:29:22 +0000 2012","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"131516","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme14\/bg.gif","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme14\/bg.gif","profile_background_tile":true,"profile_link_color":"009999","profile_sidebar_border_color":"EEEEEE","profile_sidebar_fill_color":"EFEFEF","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/730153519836094464\/S3EvKi1T_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/730153519836094464\/S3EvKi1T_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/557449649\/1446078916","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"quoted_status_id":787977369881575425,"quoted_status_id_str":"787977369881575425","quoted_status":{"created_at":"Mon Oct 17 11:23:39 +0000 2016","id":787977369881575425,"id_str":"787977369881575425","text":"Trump\u2019s son-in-law already working to set up Trump TV ahead of looming election defeat: report\u2026 https:\/\/t.co\/lL06xR8Is0","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/www.hootsuite.com\" rel=\"nofollow\"\u003eHootsuite\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":16041234,"id_str":"16041234","name":"Raw Story","screen_name":"RawStory","location":"Washington, DC","url":"http:\/\/www.rawstory.com","description":"The official Twitter of http:\/\/RawStory.com","protected":false,"verified":true,"followers_count":103869,"friends_count":2311,"listed_count":4126,"favourites_count":199,"statuses_count":122382,"created_at":"Fri Aug 29 12:29:21 +0000 2008","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/204237561\/rawreplaytwitterbackground.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/204237561\/rawreplaytwitterbackground.jpg","profile_background_tile":false,"profile_link_color":"000000","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"7D0B0B","profile_text_color":"000000","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/511012709270183936\/wEVYV7Lf_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/511012709270183936\/wEVYV7Lf_normal.png","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/16041234\/1472223971","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"extended_tweet":{"full_text":"Trump\u2019s son-in-law already working to set up Trump TV ahead of looming election defeat: report https:\/\/t.co\/D6vS0EpDGo https:\/\/t.co\/EVNCJbuiTy","display_text_range":[0,118],"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/D6vS0EpDGo","expanded_url":"http:\/\/ow.ly\/kOoA305fphQ","display_url":"ow.ly\/kOoA305fphQ","indices":[95,118]}],"user_mentions":[],"symbols":[],"media":[{"id":787977367276904448,"id_str":"787977367276904448","indices":[119,142],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu91PZuWYAAUFvU.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu91PZuWYAAUFvU.jpg","url":"https:\/\/t.co\/EVNCJbuiTy","display_url":"pic.twitter.com\/EVNCJbuiTy","expanded_url":"https:\/\/twitter.com\/RawStory\/status\/787977369881575425\/photo\/1","type":"photo","sizes":{"large":{"w":800,"h":430,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":680,"h":366,"resize":"fit"},"medium":{"w":800,"h":430,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787977367276904448,"id_str":"787977367276904448","indices":[119,142],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu91PZuWYAAUFvU.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu91PZuWYAAUFvU.jpg","url":"https:\/\/t.co\/EVNCJbuiTy","display_url":"pic.twitter.com\/EVNCJbuiTy","expanded_url":"https:\/\/twitter.com\/RawStory\/status\/787977369881575425\/photo\/1","type":"photo","sizes":{"large":{"w":800,"h":430,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":680,"h":366,"resize":"fit"},"medium":{"w":800,"h":430,"resize":"fit"}}}]}},"retweet_count":22,"favorite_count":10,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/lL06xR8Is0","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787977369881575425","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[96,119]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":2,"favorite_count":1,"entities":{"hashtags":[{"text":"uspoli","indices":[95,102]}],"urls":[{"url":"https:\/\/t.co\/9V3bocNiXh","expanded_url":"https:\/\/twitter.com\/RawStory\/status\/787977369881575425","display_url":"twitter.com\/RawStory\/statu\u2026","indices":[103,126]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"quoted_status_id":787977369881575425,"quoted_status_id_str":"787977369881575425","quoted_status":{"created_at":"Mon Oct 17 11:23:39 +0000 2016","id":787977369881575425,"id_str":"787977369881575425","text":"Trump\u2019s son-in-law already working to set up Trump TV ahead of looming election defeat: report\u2026 https:\/\/t.co\/lL06xR8Is0","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/www.hootsuite.com\" rel=\"nofollow\"\u003eHootsuite\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":16041234,"id_str":"16041234","name":"Raw Story","screen_name":"RawStory","location":"Washington, DC","url":"http:\/\/www.rawstory.com","description":"The official Twitter of http:\/\/RawStory.com","protected":false,"verified":true,"followers_count":103869,"friends_count":2311,"listed_count":4126,"favourites_count":199,"statuses_count":122382,"created_at":"Fri Aug 29 12:29:21 +0000 2008","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/204237561\/rawreplaytwitterbackground.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/204237561\/rawreplaytwitterbackground.jpg","profile_background_tile":false,"profile_link_color":"000000","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"7D0B0B","profile_text_color":"000000","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/511012709270183936\/wEVYV7Lf_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/511012709270183936\/wEVYV7Lf_normal.png","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/16041234\/1472223971","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"extended_tweet":{"full_text":"Trump\u2019s son-in-law already working to set up Trump TV ahead of looming election defeat: report https:\/\/t.co\/D6vS0EpDGo https:\/\/t.co\/EVNCJbuiTy","display_text_range":[0,118],"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/D6vS0EpDGo","expanded_url":"http:\/\/ow.ly\/kOoA305fphQ","display_url":"ow.ly\/kOoA305fphQ","indices":[95,118]}],"user_mentions":[],"symbols":[],"media":[{"id":787977367276904448,"id_str":"787977367276904448","indices":[119,142],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu91PZuWYAAUFvU.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu91PZuWYAAUFvU.jpg","url":"https:\/\/t.co\/EVNCJbuiTy","display_url":"pic.twitter.com\/EVNCJbuiTy","expanded_url":"https:\/\/twitter.com\/RawStory\/status\/787977369881575425\/photo\/1","type":"photo","sizes":{"large":{"w":800,"h":430,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":680,"h":366,"resize":"fit"},"medium":{"w":800,"h":430,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787977367276904448,"id_str":"787977367276904448","indices":[119,142],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu91PZuWYAAUFvU.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu91PZuWYAAUFvU.jpg","url":"https:\/\/t.co\/EVNCJbuiTy","display_url":"pic.twitter.com\/EVNCJbuiTy","expanded_url":"https:\/\/twitter.com\/RawStory\/status\/787977369881575425\/photo\/1","type":"photo","sizes":{"large":{"w":800,"h":430,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":680,"h":366,"resize":"fit"},"medium":{"w":800,"h":430,"resize":"fit"}}}]}},"retweet_count":22,"favorite_count":10,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/lL06xR8Is0","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787977369881575425","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[96,119]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[{"text":"uspoli","indices":[112,119]}],"urls":[],"user_mentions":[{"screen_name":"truthmashup","name":"Truth Mashup","id":557449649,"id_str":"557449649","indices":[3,15]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419370"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985757180866560,"id_str":"787985757180866560","text":"RT @JackPosobiec: \"The US is creating an enemy out of Russsia to distract from domestic problems in an election season\" - Putin","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":226565178,"id_str":"226565178","name":"DeplorablyIndivisibl","screen_name":"Stormenight","location":"USA ","url":null,"description":"I'm about life liberty & happiness.Sick o party politics&MSM BS.I understand freewill and self-responsibility so no victim mentality here & no fear. #MAGA","protected":false,"verified":false,"followers_count":629,"friends_count":806,"listed_count":10,"favourites_count":4816,"statuses_count":9959,"created_at":"Tue Dec 14 14:01:14 +0000 2010","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"022330","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/407303694\/snglobe.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/407303694\/snglobe.jpg","profile_background_tile":true,"profile_link_color":"0084B4","profile_sidebar_border_color":"A8C7F7","profile_sidebar_fill_color":"C0DFEC","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/675173584315260934\/o-X20FBv_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/675173584315260934\/o-X20FBv_normal.jpg","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:56:12 +0000 2016","id":787985562049273857,"id_str":"787985562049273857","text":"\"The US is creating an enemy out of Russsia to distract from domestic problems in an election season\" - Putin","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":592730371,"id_str":"592730371","name":"Jack Posobiec","screen_name":"JackPosobiec","location":"Washington, DC","url":"http:\/\/CitizensForTrump.com","description":"Special Projects Director of @Citizens4Trump. Recovering political operative. Veteran. \u4e2d\u6587","protected":false,"verified":false,"followers_count":29406,"friends_count":17056,"listed_count":349,"favourites_count":31085,"statuses_count":64988,"created_at":"Mon May 28 13:31:22 +0000 2012","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/862971572\/fb7aadcfd66ea5e4cad749bf1e4412de.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/862971572\/fb7aadcfd66ea5e4cad749bf1e4412de.jpeg","profile_background_tile":true,"profile_link_color":"1B95E0","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/779488773126193156\/3uVx_qBy_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/779488773126193156\/3uVx_qBy_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/592730371\/1474771459","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":12,"favorite_count":7,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"JackPosobiec","name":"Jack Posobiec","id":592730371,"id_str":"592730371","indices":[3,16]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419236"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985757747093504,"id_str":"787985757747093504","text":"Help me to win #galaxynote7 and by the way poop on #trump or #clinton ;) https:\/\/t.co\/7Nmef5Ce5E","source":"\u003ca href=\"https:\/\/gleam.io\" rel=\"nofollow\"\u003eGleam Competition App\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":4887143782,"id_str":"4887143782","name":"RKgem","screen_name":"CrispTechRK","location":"Mumbai, India","url":null,"description":null,"protected":false,"verified":false,"followers_count":59,"friends_count":577,"listed_count":20,"favourites_count":2276,"statuses_count":4496,"created_at":"Mon Feb 08 06:39:55 +0000 2016","utc_offset":19800,"time_zone":"Mumbai","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"F5F8FA","profile_background_image_url":"","profile_background_image_url_https":"","profile_background_tile":false,"profile_link_color":"2B7BB9","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/697981705244676096\/iGJ5USn1_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/697981705244676096\/iGJ5USn1_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/4887143782\/1455097584","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[{"text":"galaxynote7","indices":[15,27]},{"text":"trump","indices":[51,57]},{"text":"clinton","indices":[61,69]}],"urls":[{"url":"https:\/\/t.co\/7Nmef5Ce5E","expanded_url":"https:\/\/play.google.com\/store\/apps\/details?id=com.ZeroSenseGaming.PoopOfJustice","display_url":"play.google.com\/store\/apps\/det\u2026","indices":[73,96]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419371"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985757491388420,"id_str":"787985757491388420","text":"RT @MrWillRitter: And there it is, the long con. \nThis national nightmare was a promo campaign to sell us red hats &amp; Trump TV. https:\/\/t.co\u2026","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":21167820,"id_str":"21167820","name":"pgh-Tos","screen_name":"pghtos","location":"Pittsburgh","url":null,"description":null,"protected":false,"verified":false,"followers_count":46,"friends_count":281,"listed_count":1,"favourites_count":321,"statuses_count":1988,"created_at":"Wed Feb 18 04:12:57 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/786401302448312320\/F-OVkagb_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/786401302448312320\/F-OVkagb_normal.jpg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:17:41 +0000 2016","id":787975868106833924,"id_str":"787975868106833924","text":"And there it is, the long con. \nThis national nightmare was a promo campaign to sell us red hats &amp; Trump TV. https:\/\/t.co\/rJMNiYxMRd","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":247421834,"id_str":"247421834","name":"Will Ritter","screen_name":"MrWillRitter","location":"RVA by way of Boston","url":"http:\/\/poolhouse.co","description":"Co-Founder, @PoolhouseAgency. Political 'professional'. Romney\/Ryan, Rubio, etc alum. RTs are accidents.","protected":false,"verified":false,"followers_count":3337,"friends_count":998,"listed_count":107,"favourites_count":8673,"statuses_count":6606,"created_at":"Fri Feb 04 19:24:05 +0000 2011","utc_offset":-10800,"time_zone":"Atlantic Time (Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/641702567534137344\/7_d10VsZ_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/641702567534137344\/7_d10VsZ_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/247421834\/1470729681","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"quoted_status_id":787968740927369216,"quoted_status_id_str":"787968740927369216","quoted_status":{"created_at":"Mon Oct 17 10:49:22 +0000 2016","id":787968740927369216,"id_str":"787968740927369216","text":"EXCLUSIVE: Donald Trump's son-in-law has held talks to set up Trump TV after the presidential election\u2026 https:\/\/t.co\/rkmJmBBUIN","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/www.socialflow.com\" rel=\"nofollow\"\u003eSocialFlow\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":18949452,"id_str":"18949452","name":"Financial Times","screen_name":"FT","location":"London","url":"http:\/\/www.ft.com\/","description":"News stories, features and updates from the FT. For headlines follow @financialtimes. Our customer service team is @FTcare.","protected":false,"verified":true,"followers_count":2508217,"friends_count":714,"listed_count":30109,"favourites_count":18,"statuses_count":138311,"created_at":"Tue Jan 13 19:28:24 +0000 2009","utc_offset":3600,"time_zone":"London","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"FFF1E0","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/234732899\/ft-twitter-back.gif","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/234732899\/ft-twitter-back.gif","profile_background_tile":false,"profile_link_color":"9E2F50","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"E9DECF","profile_text_color":"000000","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/466972537704824832\/eflEColL_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/466972537704824832\/eflEColL_normal.png","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/18949452\/1462794252","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"extended_tweet":{"full_text":"EXCLUSIVE: Donald Trump's son-in-law has held talks to set up Trump TV after the presidential election https:\/\/t.co\/fTiqGNFF9q https:\/\/t.co\/xtbWPpBqsd","display_text_range":[0,126],"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/fTiqGNFF9q","expanded_url":"http:\/\/on.ft.com\/2dhOAqV","display_url":"on.ft.com\/2dhOAqV","indices":[103,126]}],"user_mentions":[],"symbols":[],"media":[{"id":787968738414956544,"id_str":"787968738414956544","indices":[127,150],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu9tZItWEAA-aTy.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu9tZItWEAA-aTy.jpg","url":"https:\/\/t.co\/xtbWPpBqsd","display_url":"pic.twitter.com\/xtbWPpBqsd","expanded_url":"https:\/\/twitter.com\/FT\/status\/787968740927369216\/photo\/1","type":"photo","sizes":{"small":{"w":680,"h":383,"resize":"fit"},"large":{"w":700,"h":394,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":700,"h":394,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787968738414956544,"id_str":"787968738414956544","indices":[127,150],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu9tZItWEAA-aTy.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu9tZItWEAA-aTy.jpg","url":"https:\/\/t.co\/xtbWPpBqsd","display_url":"pic.twitter.com\/xtbWPpBqsd","expanded_url":"https:\/\/twitter.com\/FT\/status\/787968740927369216\/photo\/1","type":"photo","sizes":{"small":{"w":680,"h":383,"resize":"fit"},"large":{"w":700,"h":394,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":700,"h":394,"resize":"fit"}}}]}},"retweet_count":316,"favorite_count":118,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/rkmJmBBUIN","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787968740927369216","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[104,127]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":14,"favorite_count":24,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/rJMNiYxMRd","expanded_url":"https:\/\/twitter.com\/ft\/status\/787968740927369216","display_url":"twitter.com\/ft\/status\/7879\u2026","indices":[113,136]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"quoted_status_id":787968740927369216,"quoted_status_id_str":"787968740927369216","quoted_status":{"created_at":"Mon Oct 17 10:49:22 +0000 2016","id":787968740927369216,"id_str":"787968740927369216","text":"EXCLUSIVE: Donald Trump's son-in-law has held talks to set up Trump TV after the presidential election\u2026 https:\/\/t.co\/rkmJmBBUIN","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/www.socialflow.com\" rel=\"nofollow\"\u003eSocialFlow\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":18949452,"id_str":"18949452","name":"Financial Times","screen_name":"FT","location":"London","url":"http:\/\/www.ft.com\/","description":"News stories, features and updates from the FT. For headlines follow @financialtimes. Our customer service team is @FTcare.","protected":false,"verified":true,"followers_count":2508217,"friends_count":714,"listed_count":30109,"favourites_count":18,"statuses_count":138311,"created_at":"Tue Jan 13 19:28:24 +0000 2009","utc_offset":3600,"time_zone":"London","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"FFF1E0","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/234732899\/ft-twitter-back.gif","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/234732899\/ft-twitter-back.gif","profile_background_tile":false,"profile_link_color":"9E2F50","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"E9DECF","profile_text_color":"000000","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/466972537704824832\/eflEColL_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/466972537704824832\/eflEColL_normal.png","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/18949452\/1462794252","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"extended_tweet":{"full_text":"EXCLUSIVE: Donald Trump's son-in-law has held talks to set up Trump TV after the presidential election https:\/\/t.co\/fTiqGNFF9q https:\/\/t.co\/xtbWPpBqsd","display_text_range":[0,126],"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/fTiqGNFF9q","expanded_url":"http:\/\/on.ft.com\/2dhOAqV","display_url":"on.ft.com\/2dhOAqV","indices":[103,126]}],"user_mentions":[],"symbols":[],"media":[{"id":787968738414956544,"id_str":"787968738414956544","indices":[127,150],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu9tZItWEAA-aTy.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu9tZItWEAA-aTy.jpg","url":"https:\/\/t.co\/xtbWPpBqsd","display_url":"pic.twitter.com\/xtbWPpBqsd","expanded_url":"https:\/\/twitter.com\/FT\/status\/787968740927369216\/photo\/1","type":"photo","sizes":{"small":{"w":680,"h":383,"resize":"fit"},"large":{"w":700,"h":394,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":700,"h":394,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787968738414956544,"id_str":"787968738414956544","indices":[127,150],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu9tZItWEAA-aTy.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu9tZItWEAA-aTy.jpg","url":"https:\/\/t.co\/xtbWPpBqsd","display_url":"pic.twitter.com\/xtbWPpBqsd","expanded_url":"https:\/\/twitter.com\/FT\/status\/787968740927369216\/photo\/1","type":"photo","sizes":{"small":{"w":680,"h":383,"resize":"fit"},"large":{"w":700,"h":394,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":700,"h":394,"resize":"fit"}}}]}},"retweet_count":316,"favorite_count":118,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/rkmJmBBUIN","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787968740927369216","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[104,127]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"MrWillRitter","name":"Will Ritter","id":247421834,"id_str":"247421834","indices":[3,16]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419310"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985758074368000,"id_str":"787985758074368000","text":"RT @Democrat4LifeTX: Democrats Put Trump To Shame By Fundraising To Help GOP Reopen Firebombed NC County Office via @politicususa https:\/\/t\u2026","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":2387648143,"id_str":"2387648143","name":"Patrick","screen_name":"Pat1036","location":null,"url":null,"description":null,"protected":false,"verified":false,"followers_count":443,"friends_count":911,"listed_count":22,"favourites_count":2272,"statuses_count":12786,"created_at":"Thu Mar 13 18:30:10 +0000 2014","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"1B95E0","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/754630699014119425\/_9dBOPYX_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/754630699014119425\/_9dBOPYX_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/2387648143\/1457200330","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 03:43:24 +0000 2016","id":787861542406529024,"id_str":"787861542406529024","text":"Democrats Put Trump To Shame By Fundraising To Help GOP Reopen Firebombed NC County Office via @politicususa https:\/\/t.co\/TYdkfAUv2J","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":4800796580,"id_str":"4800796580","name":"Real Demos 4 Hillary","screen_name":"Democrat4LifeTX","location":"Austin, TX","url":"http:\/\/www.hillaryclinton.com","description":"Teacher\/Admin for 30 Years, Writer and Owner of an Educational Resource Brand, Wife of a US Army Colonel, Democrat for LIfe!","protected":false,"verified":false,"followers_count":2611,"friends_count":3078,"listed_count":55,"favourites_count":12740,"statuses_count":21901,"created_at":"Sat Jan 23 02:13:50 +0000 2016","utc_offset":-25200,"time_zone":"Pacific Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"981CEB","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/766683659889881088\/Mi2RUeLA_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/766683659889881088\/Mi2RUeLA_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/4800796580\/1472730617","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":29,"favorite_count":32,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/TYdkfAUv2J","expanded_url":"http:\/\/www.politicususa.com\/2016\/10\/16\/democrats-fundraise-gop-reopen-firebombed-north-carolina-county-office.html","display_url":"politicususa.com\/2016\/10\/16\/dem\u2026","indices":[109,132]}],"user_mentions":[{"screen_name":"politicususa","name":"PoliticusUSA","id":14792049,"id_str":"14792049","indices":[95,108]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"Democrat4LifeTX","name":"Real Demos 4 Hillary","id":4800796580,"id_str":"4800796580","indices":[3,19]},{"screen_name":"politicususa","name":"PoliticusUSA","id":14792049,"id_str":"14792049","indices":[116,129]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419449"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985758015467520,"id_str":"787985758015467520","text":"Being Muslim and American in the year of Donald Trump: Muslim Americans describe the September 11, 2001, terr... https:\/\/t.co\/208YVcnLBB","source":"\u003ca href=\"http:\/\/twitterfeed.com\" rel=\"nofollow\"\u003etwitterfeed\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1705270981,"id_str":"1705270981","name":"aalap metha","screen_name":"aalapm06","location":null,"url":null,"description":null,"protected":false,"verified":false,"followers_count":28,"friends_count":31,"listed_count":4,"favourites_count":0,"statuses_count":20543,"created_at":"Tue Aug 27 17:51:08 +0000 2013","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_1_normal.png","profile_image_url_https":"https:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_1_normal.png","default_profile":true,"default_profile_image":true,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/208YVcnLBB","expanded_url":"http:\/\/cnn.it\/2eleo1w","display_url":"cnn.it\/2eleo1w","indices":[113,136]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419435"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985758485372928,"id_str":"787985758485372928","text":"RT @LouDobbs: Silence of GOP in the Face of Clinton Cartel's Corruption is Acquiescence to Greater Corruption  #MAGA #TrumpPence16 #TrumpTR\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":4766281718,"id_str":"4766281718","name":"scott","screen_name":"scott006648","location":null,"url":null,"description":null,"protected":false,"verified":false,"followers_count":130,"friends_count":116,"listed_count":22,"favourites_count":256,"statuses_count":7187,"created_at":"Sat Jan 16 03:36:54 +0000 2016","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"F5F8FA","profile_background_image_url":"","profile_background_image_url_https":"","profile_background_tile":false,"profile_link_color":"2B7BB9","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_2_normal.png","profile_image_url_https":"https:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_2_normal.png","default_profile":true,"default_profile_image":true,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:40:16 +0000 2016","id":787981551518806017,"id_str":"787981551518806017","text":"Silence of GOP in the Face of Clinton Cartel's Corruption is Acquiescence to Greater Corruption  #MAGA #TrumpPence16 #TrumpTRain #Dobbs","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":26487169,"id_str":"26487169","name":"Lou Dobbs","screen_name":"LouDobbs","location":"New York, NY","url":"http:\/\/loudobbs.com","description":"Lou Dobbs Tonight, Fox Business Network, 7 & 11 pm    \nIG: http:\/\/instagram.com\/loudobbstonight\/","protected":false,"verified":true,"followers_count":809353,"friends_count":2477,"listed_count":3121,"favourites_count":3574,"statuses_count":23721,"created_at":"Wed Mar 25 12:39:59 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/575734940228190209\/iXHsvsM7.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/575734940228190209\/iXHsvsM7.jpeg","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/663851941571641344\/OqqkE56l_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/663851941571641344\/OqqkE56l_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/26487169\/1426100831","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":223,"favorite_count":278,"entities":{"hashtags":[{"text":"MAGA","indices":[97,102]},{"text":"TrumpPence16","indices":[103,116]},{"text":"TrumpTRain","indices":[117,128]},{"text":"Dobbs","indices":[129,135]}],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[{"text":"MAGA","indices":[111,116]},{"text":"TrumpPence16","indices":[117,130]}],"urls":[],"user_mentions":[{"screen_name":"LouDobbs","name":"Lou Dobbs","id":26487169,"id_str":"26487169","indices":[3,12]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419547"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985758846005249,"id_str":"787985758846005249","text":"RT @activist360: IT'S TIME TO TELL THE TRUTH: Trump isn\u2019t running a presidential campaign, he\u2019s leading a white nationalist hate cult https\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":772053764022820864,"id_str":"772053764022820864","name":"Butterflycu2","screen_name":"butterflyn2u","location":"Oxford, GA","url":null,"description":"Southern educated, witted and politically savvy female residing in GA. Interested in intelligent conversation and feedback to advance differing points of views.","protected":false,"verified":false,"followers_count":73,"friends_count":119,"listed_count":3,"favourites_count":1628,"statuses_count":1659,"created_at":"Sat Sep 03 12:48:56 +0000 2016","utc_offset":null,"time_zone":null,"geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"F5F8FA","profile_background_image_url":"","profile_background_image_url_https":"","profile_background_tile":false,"profile_link_color":"2B7BB9","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/772448314679844865\/6Ks4ReGc_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/772448314679844865\/6Ks4ReGc_normal.jpg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 02:25:06 +0000 2016","id":787841838954778625,"id_str":"787841838954778625","text":"IT'S TIME TO TELL THE TRUTH: Trump isn\u2019t running a presidential campaign, he\u2019s leading a white nationalist hate cult https:\/\/t.co\/MlrYlOE1ld","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":349803020,"id_str":"349803020","name":"Bill Madden","screen_name":"activist360","location":"Lower East Side, New York City","url":"http:\/\/about.me\/bill.madden","description":"Singer-songwriter, musician, activist, poet, yogi, fierce paladin for social justice and the environment. NEW RELIGION is available at https:\/\/t.co\/2h5mwsMcV7","protected":false,"verified":false,"followers_count":51933,"friends_count":51025,"listed_count":727,"favourites_count":2686,"statuses_count":23688,"created_at":"Sat Aug 06 18:34:39 +0000 2011","utc_offset":-18000,"time_zone":"Quito","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"CCCC99","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"B32025","profile_sidebar_border_color":"999966","profile_sidebar_fill_color":"CCCC99","profile_text_color":"333333","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/659427646540001280\/1sIxdKlD_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/659427646540001280\/1sIxdKlD_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/349803020\/1446148252","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":452,"favorite_count":750,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/MlrYlOE1ld","expanded_url":"http:\/\/bit.ly\/2eHgddb","display_url":"bit.ly\/2eHgddb","indices":[117,140]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"activist360","name":"Bill Madden","id":349803020,"id_str":"349803020","indices":[3,15]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419633"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985758577627137,"id_str":"787985758577627137","text":"RT @ThePlumLineGS: Our polling has found that half of Trump supporters are inclined to agree that the election will be rigged: https:\/\/t.co\u2026","source":"\u003ca href=\"https:\/\/about.twitter.com\/products\/tweetdeck\" rel=\"nofollow\"\u003eTweetDeck\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":19087651,"id_str":"19087651","name":"Greg Dworkin","screen_name":"DemFromCT","location":"Newtown, CT","url":"http:\/\/dailykos.com","description":"Contributing editor, Daily Kos. I'm a doctor who tweets about politics, and sometimes, being a doctor. Personal account, reflecting only my own view.","protected":false,"verified":false,"followers_count":9418,"friends_count":1108,"listed_count":494,"favourites_count":27184,"statuses_count":171658,"created_at":"Fri Jan 16 21:59:39 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"9AE4E8","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/6518208\/KosTwitterBG.png","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/6518208\/KosTwitterBG.png","profile_background_tile":true,"profile_link_color":"0084B4","profile_sidebar_border_color":"BDDCAD","profile_sidebar_fill_color":"DDFFCC","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/412221142040985600\/rfwvOE-u_normal.jpeg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/412221142040985600\/rfwvOE-u_normal.jpeg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/19087651\/1398375050","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:56:46 +0000 2016","id":787985704571793408,"id_str":"787985704571793408","text":"Our polling has found that half of Trump supporters are inclined to agree that the election will be rigged: https:\/\/t.co\/fEmIa046WJ","source":"\u003ca href=\"https:\/\/about.twitter.com\/products\/tweetdeck\" rel=\"nofollow\"\u003eTweetDeck\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":20508720,"id_str":"20508720","name":"Greg Sargent","screen_name":"ThePlumLineGS","location":null,"url":"https:\/\/www.washingtonpost.com\/blogs\/plum-line\/","description":"A blog about politics, politics, and politics","protected":false,"verified":true,"followers_count":107797,"friends_count":3405,"listed_count":4196,"favourites_count":1420,"statuses_count":103655,"created_at":"Tue Feb 10 12:04:17 +0000 2009","utc_offset":-18000,"time_zone":"Central Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/77712921\/greg_spic_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/77712921\/greg_spic_normal.jpg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":1,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/fEmIa046WJ","expanded_url":"https:\/\/www.washingtonpost.com\/blogs\/plum-line\/wp\/2016\/09\/15\/donald-trump-says-the-election-will-be-rigged-half-of-his-supporters-seem-inclined-to-agree\/","display_url":"washingtonpost.com\/blogs\/plum-lin\u2026","indices":[108,131]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"ThePlumLineGS","name":"Greg Sargent","id":20508720,"id_str":"20508720","indices":[3,17]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419569"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985759785529344,"id_str":"787985759785529344","text":"RT @mitchellvii: CBS knows Hillary leads with degreed voters while Trump has a huge lead with blue collar.  So they oversample degreed by 2\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/#!\/download\/ipad\" rel=\"nofollow\"\u003eTwitter for iPad\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":3308721355,"id_str":"3308721355","name":"Nikki","screen_name":"nikki20152014","location":"United States of America","url":null,"description":"PATRIOTS VOTE NOV 8! I UNFOLLOW those WHO Do Not Follow ME BACK. #ClintonScandals #PENCE #veterans #WakeUpAmerica  \ud83d\udd34MAGA","protected":false,"verified":false,"followers_count":94,"friends_count":158,"listed_count":0,"favourites_count":559,"statuses_count":2675,"created_at":"Fri Aug 07 10:59:32 +0000 2015","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/784021553490964482\/mfLoegGK_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/784021553490964482\/mfLoegGK_normal.jpg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:56:46 +0000 2016","id":787985705259634688,"id_str":"787985705259634688","text":"CBS knows Hillary leads with degreed voters while Trump has a huge lead with blue collar.  So they oversample degreed by 20%.","source":"\u003ca href=\"https:\/\/about.twitter.com\/products\/tweetdeck\" rel=\"nofollow\"\u003eTweetDeck\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":17980523,"id_str":"17980523","name":"Bill Mitchell","screen_name":"mitchellvii","location":"Charlotte, NC","url":"https:\/\/www.yourvoiceradio.com","description":"Tune in to YourVoice\u2122 Radio, Spreaker's #1 Political Talk Show at https:\/\/yourvoiceradio.com\/,  Sun-Thu at 8pm EST!  24\/7 Hotline 704-490-4520.","protected":false,"verified":true,"followers_count":96175,"friends_count":8186,"listed_count":1066,"favourites_count":13917,"statuses_count":69860,"created_at":"Tue Dec 09 01:54:21 +0000 2008","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"ABB8C2","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/754052443491696640\/4wI9t0qV_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/754052443491696640\/4wI9t0qV_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/17980523\/1470932562","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":1,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"mitchellvii","name":"Bill Mitchell","id":17980523,"id_str":"17980523","indices":[3,15]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419857"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985759521402880,"id_str":"787985759521402880","text":"RT @WeNeedTrump: GOOD MORNING TRUMP TRAIN. Start this week off strong. Keep fighting to fix this country.\n\n#MakeAmericaGreatAgain https:\/\/t\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":134126795,"id_str":"134126795","name":"Bob Scheiner","screen_name":"fleetbob1","location":"Union, NJ","url":null,"description":"only one choice for President\nDonald J Trump","protected":false,"verified":false,"followers_count":249,"friends_count":194,"listed_count":26,"favourites_count":8445,"statuses_count":6580,"created_at":"Sat Apr 17 14:33:35 +0000 2010","utc_offset":-14400,"time_zone":"America\/New_York","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/653210346921005057\/PeqauN_0_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/653210346921005057\/PeqauN_0_normal.jpg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:06:00 +0000 2016","id":787972927383801856,"id_str":"787972927383801856","text":"GOOD MORNING TRUMP TRAIN. Start this week off strong. Keep fighting to fix this country.\n\n#MakeAmericaGreatAgain https:\/\/t.co\/zeXnkl6qtd","display_text_range":[0,112],"source":"\u003ca href=\"https:\/\/about.twitter.com\/products\/tweetdeck\" rel=\"nofollow\"\u003eTweetDeck\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":3284112337,"id_str":"3284112337","name":"We Need Trump","screen_name":"WeNeedTrump","location":"United States of America ","url":"http:\/\/donaldjtrump.com","description":"The best TRUMP account on Twitter! America has enormous potential under a real leader. Following the campaign up to the minute.","protected":false,"verified":false,"followers_count":78673,"friends_count":57729,"listed_count":298,"favourites_count":3075,"statuses_count":5217,"created_at":"Sun Jul 19 06:38:28 +0000 2015","utc_offset":-18000,"time_zone":"Central Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"253885","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/622657650635898880\/sRnv9Ilr_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/622657650635898880\/sRnv9Ilr_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/3284112337\/1468134358","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":117,"favorite_count":127,"entities":{"hashtags":[{"text":"MakeAmericaGreatAgain","indices":[90,112]}],"urls":[],"user_mentions":[],"symbols":[],"media":[{"id":787816950185373696,"id_str":"787816950185373696","indices":[113,136],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu7jV5fUsAAIm5-.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu7jV5fUsAAIm5-.jpg","url":"https:\/\/t.co\/zeXnkl6qtd","display_url":"pic.twitter.com\/zeXnkl6qtd","expanded_url":"https:\/\/twitter.com\/WeNeedTrump\/status\/787972927383801856\/photo\/1","type":"photo","sizes":{"medium":{"w":960,"h":540,"resize":"fit"},"small":{"w":680,"h":383,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"large":{"w":960,"h":540,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787816950185373696,"id_str":"787816950185373696","indices":[113,136],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu7jV5fUsAAIm5-.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu7jV5fUsAAIm5-.jpg","url":"https:\/\/t.co\/zeXnkl6qtd","display_url":"pic.twitter.com\/zeXnkl6qtd","expanded_url":"https:\/\/twitter.com\/WeNeedTrump\/status\/787972927383801856\/photo\/1","type":"photo","sizes":{"medium":{"w":960,"h":540,"resize":"fit"},"small":{"w":680,"h":383,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"large":{"w":960,"h":540,"resize":"fit"}}}]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[{"text":"MakeAmericaGreatAgain","indices":[107,129]}],"urls":[],"user_mentions":[{"screen_name":"WeNeedTrump","name":"We Need Trump","id":3284112337,"id_str":"3284112337","indices":[3,15]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419794"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985759810617345,"id_str":"787985759810617345","text":"RT @TweetBrettMac: Who blackmails a decorated veteran and forces her to resign? Hillary Clinton.\n\nBut she's not a gold star Muslim so\u2026 ","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":778918696718774272,"id_str":"778918696718774272","name":"writeonright1\ud83d\udc38","screen_name":"DeplorableDJT","location":"NC","url":null,"description":"American Nationalist. Free Speech Activist. 82 AirBorne. Author #FightForTheWest #MAGA Googles are tools of Skypes","protected":false,"verified":false,"followers_count":638,"friends_count":895,"listed_count":12,"favourites_count":12619,"statuses_count":6164,"created_at":"Thu Sep 22 11:27:43 +0000 2016","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"F5F8FA","profile_background_image_url":"","profile_background_image_url_https":"","profile_background_tile":false,"profile_link_color":"2B7BB9","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/784133781749899264\/-mJ9S3rD_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/784133781749899264\/-mJ9S3rD_normal.jpg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Sat Oct 15 23:10:06 +0000 2016","id":787430379020029953,"id_str":"787430379020029953","text":"Who blackmails a decorated veteran and forces her to resign? Hillary Clinton.\n\nBut she's not a gold star Muslim so\u2026 https:\/\/t.co\/urJILUlPfj","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":305211380,"id_str":"305211380","name":"Brett Mac","screen_name":"TweetBrettMac","location":"Liberty's Cradle and Grave","url":"http:\/\/www.politicalwave.org","description":"Writes for AFF and RWT. \n\nTips? EMAIL: tweetbrettmac[at]Gmail[.]com\n\n#ReplaceTheMainstream","protected":false,"verified":false,"followers_count":13630,"friends_count":8025,"listed_count":115,"favourites_count":11586,"statuses_count":24739,"created_at":"Wed May 25 20:48:01 +0000 2011","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"3B5980","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/761023527696814080\/T0-WPOqz_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/761023527696814080\/T0-WPOqz_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/305211380\/1454875589","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"extended_tweet":{"full_text":"Who blackmails a decorated veteran and forces her to resign? Hillary Clinton.\n\nBut she's not a gold star Muslim so you won't hear about it. https:\/\/t.co\/5JjaFZfm4z","display_text_range":[0,139],"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[],"media":[{"id":787430368681005056,"id_str":"787430368681005056","indices":[140,163],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu2Dv5KWEAAFhou.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu2Dv5KWEAAFhou.jpg","url":"https:\/\/t.co\/5JjaFZfm4z","display_url":"pic.twitter.com\/5JjaFZfm4z","expanded_url":"https:\/\/twitter.com\/TweetBrettMac\/status\/787430379020029953\/photo\/1","type":"photo","sizes":{"small":{"w":494,"h":680,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":871,"h":1200,"resize":"fit"},"large":{"w":1440,"h":1984,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787430368681005056,"id_str":"787430368681005056","indices":[140,163],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu2Dv5KWEAAFhou.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu2Dv5KWEAAFhou.jpg","url":"https:\/\/t.co\/5JjaFZfm4z","display_url":"pic.twitter.com\/5JjaFZfm4z","expanded_url":"https:\/\/twitter.com\/TweetBrettMac\/status\/787430379020029953\/photo\/1","type":"photo","sizes":{"small":{"w":494,"h":680,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":871,"h":1200,"resize":"fit"},"large":{"w":1440,"h":1984,"resize":"fit"}}}]}},"retweet_count":895,"favorite_count":769,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/urJILUlPfj","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787430379020029953","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[116,139]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"","expanded_url":null,"indices":[135,135]}],"user_mentions":[{"screen_name":"TweetBrettMac","name":"Brett Mac","id":305211380,"id_str":"305211380","indices":[3,17]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419863"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985759919677440,"id_str":"787985759919677440","text":"RT @johnpender123: Fox News 10\/15\/16 | LATEST WIKILEAKS Revelations DEADLY For Hillary Clinton https:\/\/t.co\/3xLRKU7xfc via @YouTube","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":50831585,"id_str":"50831585","name":"john pender","screen_name":"johnpender123","location":"PDX-Vancouver","url":"https:\/\/www.roaddogfilms.com\/","description":"Bazooka Media, LLC  ~ Writer\/Producer ~ http:\/\/RoadDogFilms.com","protected":false,"verified":false,"followers_count":1557,"friends_count":1540,"listed_count":22,"favourites_count":11,"statuses_count":4689,"created_at":"Fri Jun 26 00:10:44 +0000 2009","utc_offset":-25200,"time_zone":"Pacific Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"000000","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/642416103537250304\/ycl_-BwU_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/642416103537250304\/ycl_-BwU_normal.jpg","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Sun Oct 16 19:07:11 +0000 2016","id":787731631775838208,"id_str":"787731631775838208","text":"Fox News 10\/15\/16 | LATEST WIKILEAKS Revelations DEADLY For Hillary Clinton https:\/\/t.co\/3xLRKU7xfc via @YouTube","source":"\u003ca href=\"http:\/\/linkis.com\" rel=\"nofollow\"\u003ePut your button on any page! \u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":50831585,"id_str":"50831585","name":"john pender","screen_name":"johnpender123","location":"PDX-Vancouver","url":"https:\/\/www.roaddogfilms.com\/","description":"Bazooka Media, LLC  ~ Writer\/Producer ~ http:\/\/RoadDogFilms.com","protected":false,"verified":false,"followers_count":1557,"friends_count":1540,"listed_count":22,"favourites_count":11,"statuses_count":4688,"created_at":"Fri Jun 26 00:10:44 +0000 2009","utc_offset":-25200,"time_zone":"Pacific Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"000000","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/642416103537250304\/ycl_-BwU_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/642416103537250304\/ycl_-BwU_normal.jpg","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":2,"favorite_count":1,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/3xLRKU7xfc","expanded_url":"http:\/\/ln.is\/www.youtube.com\/1XkCA","display_url":"ln.is\/www.youtube.co\u2026","indices":[76,99]}],"user_mentions":[{"screen_name":"YouTube","name":"YouTube","id":10228272,"id_str":"10228272","indices":[104,112]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/3xLRKU7xfc","expanded_url":"http:\/\/ln.is\/www.youtube.com\/1XkCA","display_url":"ln.is\/www.youtube.co\u2026","indices":[95,118]}],"user_mentions":[{"screen_name":"johnpender123","name":"john pender","id":50831585,"id_str":"50831585","indices":[3,17]},{"screen_name":"YouTube","name":"YouTube","id":10228272,"id_str":"10228272","indices":[123,131]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419889"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985760070709249,"id_str":"787985760070709249","text":"Email hacks reveal Clinton campaign's transparency problems: Clinton campaign chairman John Podesta's hacked ... https:\/\/t.co\/hXoWKvRXUD","source":"\u003ca href=\"http:\/\/twitterfeed.com\" rel=\"nofollow\"\u003etwitterfeed\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1727261088,"id_str":"1727261088","name":"US National News","screen_name":"USANationalNews","location":"United States","url":null,"description":"News for the United States. Android app available at http:\/\/bit.ly\/usnationalnews","protected":false,"verified":false,"followers_count":399,"friends_count":26,"listed_count":53,"favourites_count":5,"statuses_count":124927,"created_at":"Wed Sep 04 02:18:21 +0000 2013","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C6E2EE","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme2\/bg.gif","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme2\/bg.gif","profile_background_tile":false,"profile_link_color":"1F98C7","profile_sidebar_border_color":"C6E2EE","profile_sidebar_fill_color":"DAECF4","profile_text_color":"663B12","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/378800000412005745\/5ebdff1181a3f1b9a5877cd44bb8dba4_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/378800000412005745\/5ebdff1181a3f1b9a5877cd44bb8dba4_normal.png","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/hXoWKvRXUD","expanded_url":"http:\/\/cbsn.ws\/2dYAYzQ","display_url":"cbsn.ws\/2dYAYzQ","indices":[113,136]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419925"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985760003600384,"id_str":"787985760003600384","text":"@HfxAnna I have begun to hate everyday Americans - Hillary Clinton  (guess that is you)","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":2863785729,"in_reply_to_user_id_str":"2863785729","in_reply_to_screen_name":"HfxAnna","user":{"id":35735966,"id_str":"35735966","name":"Ken Hoag","screen_name":"35start","location":null,"url":null,"description":null,"protected":false,"verified":false,"followers_count":372,"friends_count":385,"listed_count":24,"favourites_count":301,"statuses_count":17713,"created_at":"Mon Apr 27 13:06:17 +0000 2009","utc_offset":null,"time_zone":null,"geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/2947900299\/5d2895ccfe93c37f436e4547c7654b6f_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/2947900299\/5d2895ccfe93c37f436e4547c7654b6f_normal.png","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"HfxAnna","name":"Anna","id":2863785729,"id_str":"2863785729","indices":[0,8]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419909"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985759936454656,"id_str":"787985759936454656","text":"RT @StephenKing: If you're a man whose wife or daughters have been sexually harassed, and you're still planning to vote for Donald Trump, s\u2026","source":"\u003ca href=\"http:\/\/www.tweetcaster.com\" rel=\"nofollow\"\u003eTweetCaster for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":17375008,"id_str":"17375008","name":"MJ","screen_name":"ItsMJ254","location":null,"url":null,"description":"Life lessons \u2022 Sports \u2022 Humor \u2022 Black Lives Matter.","protected":false,"verified":false,"followers_count":4686,"friends_count":413,"listed_count":87,"favourites_count":1374,"statuses_count":304804,"created_at":"Thu Nov 13 23:02:19 +0000 2008","utc_offset":-18000,"time_zone":"America\/Chicago","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/783631593776648192\/oPXthem4_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/783631593776648192\/oPXthem4_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/17375008\/1439383857","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:26:06 +0000 2016","id":787977986486263808,"id_str":"787977986486263808","text":"If you're a man whose wife or daughters have been sexually harassed, and you're still planning to vote for Donald Trump, shame on you.","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":2233154425,"id_str":"2233154425","name":"Stephen King","screen_name":"StephenKing","location":null,"url":"http:\/\/stephenking.com","description":"Author","protected":false,"verified":true,"followers_count":2441166,"friends_count":47,"listed_count":9736,"favourites_count":4,"statuses_count":1344,"created_at":"Fri Dec 06 15:26:35 +0000 2013","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/378800000836981162\/b683f7509ec792c3e481ead332940cdc_normal.jpeg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/378800000836981162\/b683f7509ec792c3e481ead332940cdc_normal.jpeg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":1038,"favorite_count":2416,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"StephenKing","name":"Stephen King","id":2233154425,"id_str":"2233154425","indices":[3,15]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419893"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985759810756608,"id_str":"787985759810756608","text":"RT @Robbonixon: Thankless job Trumpsplaining. https:\/\/t.co\/RiFGAznXL0","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":760080250617335808,"id_str":"760080250617335808","name":"The Independent","screen_name":"thinkerdamous","location":"Multiple Citizenship","url":null,"description":"Healthy 2 party political system is important for democracy. People before technology. When we change spiritually, success follows. DONT ADD TO LISTS","protected":false,"verified":false,"followers_count":828,"friends_count":2526,"listed_count":8,"favourites_count":3766,"statuses_count":5734,"created_at":"Mon Aug 01 11:50:28 +0000 2016","utc_offset":null,"time_zone":null,"geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"F5F8FA","profile_background_image_url":"","profile_background_image_url_https":"","profile_background_tile":false,"profile_link_color":"2B7BB9","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/787302806290063360\/bSXGtB5o_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/787302806290063360\/bSXGtB5o_normal.jpg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:34:31 +0000 2016","id":787980103200366592,"id_str":"787980103200366592","text":"Thankless job Trumpsplaining. https:\/\/t.co\/RiFGAznXL0","display_text_range":[0,29],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":23892889,"id_str":"23892889","name":"Laura Peterson","screen_name":"Robbonixon","location":"Farmington, MN","url":null,"description":"dropped out, dropped acid, drops f-bombs, drops the ball...raining on parades since birth i remain!","protected":false,"verified":false,"followers_count":1744,"friends_count":2632,"listed_count":90,"favourites_count":23835,"statuses_count":45112,"created_at":"Thu Mar 12 04:06:24 +0000 2009","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"ABB8C2","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/775195866709626880\/H93-Hq48_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/775195866709626880\/H93-Hq48_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/23892889\/1473153954","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"quoted_status_id":787974213764251648,"quoted_status_id_str":"787974213764251648","quoted_status":{"created_at":"Mon Oct 17 11:11:07 +0000 2016","id":787974213764251648,"id_str":"787974213764251648","text":"Trump stomps all over Mike Pence's efforts to clean up \"rigged election\" comments https:\/\/t.co\/1nPxSNamWw https:\/\/t.co\/vj1TnZQNeo","display_text_range":[0,105],"source":"\u003ca href=\"http:\/\/www.socialflow.com\" rel=\"nofollow\"\u003eSocialFlow\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":15458694,"id_str":"15458694","name":"HuffPost Politics","screen_name":"HuffPostPol","location":"Washington, D.C.","url":"http:\/\/www.huffingtonpost.com\/politics","description":"The latest political news from The Huffington Post's politics team.","protected":false,"verified":true,"followers_count":941734,"friends_count":9052,"listed_count":15990,"favourites_count":110,"statuses_count":259490,"created_at":"Wed Jul 16 19:45:46 +0000 2008","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"04508E","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/565873048\/huffpostpol.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/565873048\/huffpostpol.jpg","profile_background_tile":false,"profile_link_color":"0000FF","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"FFFFFF","profile_text_color":"000000","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/696718733721542656\/2lsNdbhH_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/696718733721542656\/2lsNdbhH_normal.png","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/15458694\/1426905134","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":31,"favorite_count":31,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/1nPxSNamWw","expanded_url":"http:\/\/huff.to\/2dWWFQN","display_url":"huff.to\/2dWWFQN","indices":[82,105]}],"user_mentions":[],"symbols":[],"media":[{"id":787974211411279872,"id_str":"787974211411279872","indices":[106,129],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu9yXtNWgAAczFQ.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu9yXtNWgAAczFQ.jpg","url":"https:\/\/t.co\/vj1TnZQNeo","display_url":"pic.twitter.com\/vj1TnZQNeo","expanded_url":"https:\/\/twitter.com\/HuffPostPol\/status\/787974213764251648\/photo\/1","type":"photo","sizes":{"large":{"w":2000,"h":1000,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":680,"h":340,"resize":"fit"},"medium":{"w":1200,"h":600,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787974211411279872,"id_str":"787974211411279872","indices":[106,129],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu9yXtNWgAAczFQ.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu9yXtNWgAAczFQ.jpg","url":"https:\/\/t.co\/vj1TnZQNeo","display_url":"pic.twitter.com\/vj1TnZQNeo","expanded_url":"https:\/\/twitter.com\/HuffPostPol\/status\/787974213764251648\/photo\/1","type":"photo","sizes":{"large":{"w":2000,"h":1000,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":680,"h":340,"resize":"fit"},"medium":{"w":1200,"h":600,"resize":"fit"}}}]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":2,"favorite_count":2,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/RiFGAznXL0","expanded_url":"https:\/\/twitter.com\/huffpostpol\/status\/787974213764251648","display_url":"twitter.com\/huffpostpol\/st\u2026","indices":[30,53]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"quoted_status_id":787974213764251648,"quoted_status_id_str":"787974213764251648","quoted_status":{"created_at":"Mon Oct 17 11:11:07 +0000 2016","id":787974213764251648,"id_str":"787974213764251648","text":"Trump stomps all over Mike Pence's efforts to clean up \"rigged election\" comments https:\/\/t.co\/1nPxSNamWw https:\/\/t.co\/vj1TnZQNeo","display_text_range":[0,105],"source":"\u003ca href=\"http:\/\/www.socialflow.com\" rel=\"nofollow\"\u003eSocialFlow\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":15458694,"id_str":"15458694","name":"HuffPost Politics","screen_name":"HuffPostPol","location":"Washington, D.C.","url":"http:\/\/www.huffingtonpost.com\/politics","description":"The latest political news from The Huffington Post's politics team.","protected":false,"verified":true,"followers_count":941734,"friends_count":9052,"listed_count":15990,"favourites_count":110,"statuses_count":259490,"created_at":"Wed Jul 16 19:45:46 +0000 2008","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"04508E","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/565873048\/huffpostpol.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/565873048\/huffpostpol.jpg","profile_background_tile":false,"profile_link_color":"0000FF","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"FFFFFF","profile_text_color":"000000","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/696718733721542656\/2lsNdbhH_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/696718733721542656\/2lsNdbhH_normal.png","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/15458694\/1426905134","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":31,"favorite_count":31,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/1nPxSNamWw","expanded_url":"http:\/\/huff.to\/2dWWFQN","display_url":"huff.to\/2dWWFQN","indices":[82,105]}],"user_mentions":[],"symbols":[],"media":[{"id":787974211411279872,"id_str":"787974211411279872","indices":[106,129],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu9yXtNWgAAczFQ.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu9yXtNWgAAczFQ.jpg","url":"https:\/\/t.co\/vj1TnZQNeo","display_url":"pic.twitter.com\/vj1TnZQNeo","expanded_url":"https:\/\/twitter.com\/HuffPostPol\/status\/787974213764251648\/photo\/1","type":"photo","sizes":{"large":{"w":2000,"h":1000,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":680,"h":340,"resize":"fit"},"medium":{"w":1200,"h":600,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787974211411279872,"id_str":"787974211411279872","indices":[106,129],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu9yXtNWgAAczFQ.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu9yXtNWgAAczFQ.jpg","url":"https:\/\/t.co\/vj1TnZQNeo","display_url":"pic.twitter.com\/vj1TnZQNeo","expanded_url":"https:\/\/twitter.com\/HuffPostPol\/status\/787974213764251648\/photo\/1","type":"photo","sizes":{"large":{"w":2000,"h":1000,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":680,"h":340,"resize":"fit"},"medium":{"w":1200,"h":600,"resize":"fit"}}}]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/RiFGAznXL0","expanded_url":"https:\/\/twitter.com\/huffpostpol\/status\/787974213764251648","display_url":"twitter.com\/huffpostpol\/st\u2026","indices":[46,69]}],"user_mentions":[{"screen_name":"Robbonixon","name":"Laura Peterson","id":23892889,"id_str":"23892889","indices":[3,14]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419863"}
    ,
    {"created_at":"Mon Oct 17 11:57:00 +0000 2016","id":787985760536309764,"id_str":"787985760536309764","text":"RT @Sherman_Alexie: Trump &amp; the Dakota Pipeline have turned my previously apolitical Native relatives into leftist activists! Awesome!","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":469034848,"id_str":"469034848","name":"Steve Langhorst","screen_name":"SteveLanghorst","location":"StL","url":null,"description":null,"protected":false,"verified":false,"followers_count":51,"friends_count":70,"listed_count":27,"favourites_count":2033,"statuses_count":1399,"created_at":"Fri Jan 20 04:59:11 +0000 2012","utc_offset":null,"time_zone":null,"geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_2_normal.png","profile_image_url_https":"https:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_2_normal.png","default_profile":true,"default_profile_image":true,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Sun Oct 16 21:09:46 +0000 2016","id":787762483998003200,"id_str":"787762483998003200","text":"Trump &amp; the Dakota Pipeline have turned my previously apolitical Native relatives into leftist activists! Awesome!","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":321003525,"id_str":"321003525","name":"Sherman Alexie","screen_name":"Sherman_Alexie","location":null,"url":"http:\/\/fallsapart.com","description":"National Book Award winning author of The Absolutely True Diary of a Part-Time Indian and upcoming picture book, Thunder Boy Jr.","protected":false,"verified":true,"followers_count":109359,"friends_count":86,"listed_count":2353,"favourites_count":0,"statuses_count":3583,"created_at":"Mon Jun 20 21:36:31 +0000 2011","utc_offset":-25200,"time_zone":"Pacific Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"1A1B1F","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/725737413163646977\/VRoyviRM.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/725737413163646977\/VRoyviRM.jpg","profile_background_tile":false,"profile_link_color":"FF691F","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"252429","profile_text_color":"666666","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/378800000530919364\/4eee066b0af8f853dca75dfcc6bc7178_normal.jpeg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/378800000530919364\/4eee066b0af8f853dca75dfcc6bc7178_normal.jpeg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/321003525\/1462195961","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":137,"favorite_count":898,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"Sherman_Alexie","name":"Sherman Alexie","id":321003525,"id_str":"321003525","indices":[3,18]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705420036"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985760238592000,"id_str":"787985760238592000","text":"@RapefugeeWatch  ..because identifying others illegal and criminal activities does not serve Clinton well.","display_text_range":[17,106],"source":"\u003ca href=\"https:\/\/mobile.twitter.com\" rel=\"nofollow\"\u003eMobile Web (M5)\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":787978063057334273,"in_reply_to_status_id_str":"787978063057334273","in_reply_to_user_id":1116292664,"in_reply_to_user_id_str":"1116292664","in_reply_to_screen_name":"RapefugeeWatch","user":{"id":2865679515,"id_str":"2865679515","name":"Deplorable Dumas","screen_name":"PebblesDunham","location":"  .  .  #TrumpPence2016  .  .   #TrumpTrain  .  .  #Rubio4Senate2016  .  .  '2017 The End of The Obama Error'   . .  #DeleteHillary","url":"http:\/\/Deplorablelivesmatter.org","description":"ObamaNation:  abom\u00b7i\u00b7na\u00b7tion  noun \\\u0259-\u02ccb\u00e4-m\u0259-\u02c8n\u0101-sh\u0259n\\    : something that causes disgust or hatred.","protected":false,"verified":false,"followers_count":616,"friends_count":609,"listed_count":3,"favourites_count":11073,"statuses_count":4977,"created_at":"Fri Nov 07 14:38:26 +0000 2014","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"1B95E0","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/777338838209429504\/LwwVYO2h_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/777338838209429504\/LwwVYO2h_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/2865679515\/1462661833","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"RapefugeeWatch","name":"Rapefugee Watch","id":1116292664,"id_str":"1116292664","indices":[0,15]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419965"}
    ,
    {"created_at":"Mon Oct 17 11:57:00 +0000 2016","id":787985760909725696,"id_str":"787985760909725696","text":"RT @maytennis: .@morning_joe:@Markhalperin:@morningmika:M\/J have ignored that Trump said he couldn't have sexually assaulted these women b\/\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":707626622900371456,"id_str":"707626622900371456","name":"Susie16","screen_name":"susieq201688","location":"Winston-Salem, NC","url":null,"description":null,"protected":false,"verified":false,"followers_count":362,"friends_count":72,"listed_count":54,"favourites_count":16189,"statuses_count":28213,"created_at":"Wed Mar 09 17:58:28 +0000 2016","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"F5F8FA","profile_background_image_url":"","profile_background_image_url_https":"","profile_background_tile":false,"profile_link_color":"2B7BB9","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_2_normal.png","profile_image_url_https":"https:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_2_normal.png","default_profile":true,"default_profile_image":true,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:30:14 +0000 2016","id":787979024387739648,"id_str":"787979024387739648","text":".@morning_joe:@Markhalperin:@morningmika:M\/J have ignored that Trump said he couldn't have sexually assaulted these women b\/c they're ugly!","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":16906056,"id_str":"16906056","name":"maytennis","screen_name":"maytennis","location":"NJ","url":null,"description":"Politics junkie. Despise untruth in politics because it destroys our Democracy. Love to help people.","protected":false,"verified":false,"followers_count":148,"friends_count":6,"listed_count":13,"favourites_count":214,"statuses_count":61485,"created_at":"Wed Oct 22 14:27:53 +0000 2008","utc_offset":-18000,"time_zone":"Central Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_6_normal.png","profile_image_url_https":"https:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_6_normal.png","default_profile":true,"default_profile_image":true,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":1,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"Morning_Joe","name":"Morning Joe","id":254117355,"id_str":"254117355","indices":[1,13]},{"screen_name":"MarkHalperin","name":"Mark Halperin","id":184136149,"id_str":"184136149","indices":[14,27]},{"screen_name":"morningmika","name":"Mika Brzezinski","id":18227519,"id_str":"18227519","indices":[28,40]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"maytennis","name":"maytennis","id":16906056,"id_str":"16906056","indices":[3,13]},{"screen_name":"Morning_Joe","name":"Morning Joe","id":254117355,"id_str":"254117355","indices":[16,28]},{"screen_name":"MarkHalperin","name":"Mark Halperin","id":184136149,"id_str":"184136149","indices":[29,42]},{"screen_name":"morningmika","name":"Mika Brzezinski","id":18227519,"id_str":"18227519","indices":[43,55]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705420125"}
    ,
    {"created_at":"Mon Oct 17 11:57:00 +0000 2016","id":787985761027002368,"id_str":"787985761027002368","text":"RT @DeanSchanzel: BREAKING DOJ - Obama Cover Up Mass Hillary Clinton Arms Sales that went to our enemies in Syria &amp; to Libya\n https:\/\/t.co\/\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":4270517292,"id_str":"4270517292","name":"Scooter\ud83d\udc38","screen_name":"BobbyAnnF","location":"Minnesota","url":null,"description":"Activist, Proud American, Independent Thinker.  Love God, my husband, my kids and my country. On the Trump Train. Porn-blocked.       I can't stand stupid!\ud83d\ude15","protected":false,"verified":false,"followers_count":1472,"friends_count":1622,"listed_count":7,"favourites_count":26537,"statuses_count":29223,"created_at":"Tue Nov 17 18:53:39 +0000 2015","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/764882489873530880\/mcTBQbZ5_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/764882489873530880\/mcTBQbZ5_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/4270517292\/1470653869","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Sun Oct 09 10:00:01 +0000 2016","id":785057219771502592,"id_str":"785057219771502592","text":"BREAKING DOJ - Obama Cover Up Mass Hillary Clinton Arms Sales that went to our enemies in Syria &amp; to Libya\n https:\/\/t.co\/Ops6ZFhLgS","source":"\u003ca href=\"http:\/\/twitter.com\/#!\/download\/ipad\" rel=\"nofollow\"\u003eTwitter for iPad\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":705333567543504896,"id_str":"705333567543504896","name":"Dean","screen_name":"DeanSchanzel","location":"Chardon, Ohio","url":"https:\/\/www.donaldjtrump.com\/policies\/","description":"Retired: NCOIC, Officer Personnel, Ft.Bliss,Tx; Personnel Director, City of Cleveland; Republican County Chairman; Network Management, Ohio Lottery. VOTED TRUMP","protected":false,"verified":false,"followers_count":5719,"friends_count":5006,"listed_count":15,"favourites_count":8694,"statuses_count":40742,"created_at":"Thu Mar 03 10:06:41 +0000 2016","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"F5F8FA","profile_background_image_url":"","profile_background_image_url_https":"","profile_background_tile":false,"profile_link_color":"2B7BB9","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/781901284995637248\/xEwdd9CK_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/781901284995637248\/xEwdd9CK_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/705333567543504896\/1473087423","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":1653,"favorite_count":1105,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[],"media":[{"id":783801259996217344,"id_str":"783801259996217344","indices":[112,135],"media_url":"http:\/\/pbs.twimg.com\/ext_tw_video_thumb\/783801259996217344\/pu\/img\/ED2zgBvioze0fGfr.jpg","media_url_https":"https:\/\/pbs.twimg.com\/ext_tw_video_thumb\/783801259996217344\/pu\/img\/ED2zgBvioze0fGfr.jpg","url":"https:\/\/t.co\/Ops6ZFhLgS","display_url":"pic.twitter.com\/Ops6ZFhLgS","expanded_url":"https:\/\/twitter.com\/Darren32895836\/status\/783801430394085376\/video\/1","type":"photo","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":600,"h":338,"resize":"fit"},"small":{"w":340,"h":191,"resize":"fit"},"large":{"w":1024,"h":576,"resize":"fit"}},"source_status_id":783801430394085376,"source_status_id_str":"783801430394085376","source_user_id":4761064634,"source_user_id_str":"4761064634"}]},"extended_entities":{"media":[{"id":783801259996217344,"id_str":"783801259996217344","indices":[112,135],"media_url":"http:\/\/pbs.twimg.com\/ext_tw_video_thumb\/783801259996217344\/pu\/img\/ED2zgBvioze0fGfr.jpg","media_url_https":"https:\/\/pbs.twimg.com\/ext_tw_video_thumb\/783801259996217344\/pu\/img\/ED2zgBvioze0fGfr.jpg","url":"https:\/\/t.co\/Ops6ZFhLgS","display_url":"pic.twitter.com\/Ops6ZFhLgS","expanded_url":"https:\/\/twitter.com\/Darren32895836\/status\/783801430394085376\/video\/1","type":"video","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":600,"h":338,"resize":"fit"},"small":{"w":340,"h":191,"resize":"fit"},"large":{"w":1024,"h":576,"resize":"fit"}},"source_status_id":783801430394085376,"source_status_id_str":"783801430394085376","source_user_id":4761064634,"source_user_id_str":"4761064634","video_info":{"aspect_ratio":[16,9],"duration_millis":64688,"variants":[{"bitrate":832000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/ext_tw_video\/783801259996217344\/pu\/vid\/640x360\/394OJr0hl1IN6Scf.mp4"},{"bitrate":320000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/ext_tw_video\/783801259996217344\/pu\/vid\/320x180\/5Sq8tS8R7K3cEr_c.mp4"},{"content_type":"application\/dash+xml","url":"https:\/\/video.twimg.com\/ext_tw_video\/783801259996217344\/pu\/pl\/6fuVuXsk998MyiRY.mpd"},{"content_type":"application\/x-mpegURL","url":"https:\/\/video.twimg.com\/ext_tw_video\/783801259996217344\/pu\/pl\/6fuVuXsk998MyiRY.m3u8"},{"bitrate":2176000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/ext_tw_video\/783801259996217344\/pu\/vid\/1280x720\/aF2061M4ffSS8wSk.mp4"}]}}]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"DeanSchanzel","name":"Dean","id":705333567543504896,"id_str":"705333567543504896","indices":[3,16]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705420153"}
    ,
    {"created_at":"Mon Oct 17 11:57:00 +0000 2016","id":787985761102471168,"id_str":"787985761102471168","text":"@mitchellvii This is correct. Now is the time to turn up the heat If your undecided ask yourself who can create a better\/safer economy.Trump","display_text_range":[13,140],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":787984232945750016,"in_reply_to_status_id_str":"787984232945750016","in_reply_to_user_id":17980523,"in_reply_to_user_id_str":"17980523","in_reply_to_screen_name":"mitchellvii","user":{"id":3434919779,"id_str":"3434919779","name":"Stewart M. Scott","screen_name":"StewartMScott52","location":"Georgia, USA","url":null,"description":"Love my Family and Friends. Enjoy fishing and enjoy things that God made in our outdoors. Firefighter \/ Business owner\/ Sports Trump For President 2016","protected":false,"verified":false,"followers_count":154,"friends_count":178,"listed_count":2,"favourites_count":6,"statuses_count":1564,"created_at":"Sat Aug 22 01:15:03 +0000 2015","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/655945345055223808\/03ILUo7h_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/655945345055223808\/03ILUo7h_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/3434919779\/1445224530","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"mitchellvii","name":"Bill Mitchell","id":17980523,"id_str":"17980523","indices":[0,12]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705420171"}
    ,
    {"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985759668170753,"id_str":"787985759668170753","text":"Poll: Trump supporters like communist Russia like their candidate does https:\/\/t.co\/kmAkwE7Fyz https:\/\/t.co\/uuyQUJ8Nhd","display_text_range":[0,94],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":15172502,"id_str":"15172502","name":"Eric","screen_name":"ramseur","location":"New York, NY","url":"http:\/\/www.anant.us","description":"Web Startup Founder & CTO at @AnantCorp. These views are my own! I proudly have New York City Moderate Republican Values. #RepublicansForHillary","protected":false,"verified":false,"followers_count":2757,"friends_count":1675,"listed_count":145,"favourites_count":3258,"statuses_count":134272,"created_at":"Thu Jun 19 19:33:09 +0000 2008","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"352726","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/149855144\/bk.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/149855144\/bk.jpeg","profile_background_tile":true,"profile_link_color":"D02B55","profile_sidebar_border_color":"829D5E","profile_sidebar_fill_color":"99CC33","profile_text_color":"3E4415","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/2797636677\/449246ccb93f127720162730f8b2a136_normal.jpeg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/2797636677\/449246ccb93f127720162730f8b2a136_normal.jpeg","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/kmAkwE7Fyz","expanded_url":"http:\/\/politi.co\/2dlvyei","display_url":"politi.co\/2dlvyei","indices":[71,94]}],"user_mentions":[],"symbols":[],"media":[{"id":787985756644073472,"id_str":"787985756644073472","indices":[95,118],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu983ujWYAA346x.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu983ujWYAA346x.jpg","url":"https:\/\/t.co\/uuyQUJ8Nhd","display_url":"pic.twitter.com\/uuyQUJ8Nhd","expanded_url":"https:\/\/twitter.com\/ramseur\/status\/787985759668170753\/photo\/1","type":"photo","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":1024,"h":555,"resize":"fit"},"large":{"w":1024,"h":555,"resize":"fit"},"small":{"w":680,"h":369,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787985756644073472,"id_str":"787985756644073472","indices":[95,118],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu983ujWYAA346x.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu983ujWYAA346x.jpg","url":"https:\/\/t.co\/uuyQUJ8Nhd","display_url":"pic.twitter.com\/uuyQUJ8Nhd","expanded_url":"https:\/\/twitter.com\/ramseur\/status\/787985759668170753\/photo\/1","type":"photo","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":1024,"h":555,"resize":"fit"},"large":{"w":1024,"h":555,"resize":"fit"},"small":{"w":680,"h":369,"resize":"fit"}}}]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419829"}
    ,
    {"created_at":"Mon Oct 17 11:57:00 +0000 2016","id":787985761232646144,"id_str":"787985761232646144","text":"RT @theryangeorge: Wait so\nTRUMP: i just kiss women, i don't even ask\n12 WOMEN: Trump kissed me without asking\nTRUMP: that's insane, you're\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":331208078,"id_str":"331208078","name":"Mary H","screen_name":"MARYlaand_","location":"md de","url":"http:\/\/instagram.com\/maryxhealey","description":"Bernie Sanders","protected":false,"verified":false,"followers_count":23836,"friends_count":499,"listed_count":44,"favourites_count":8257,"statuses_count":25832,"created_at":"Thu Jul 07 20:23:57 +0000 2011","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/553717814217818112\/lDf3jQLn.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/553717814217818112\/lDf3jQLn.jpeg","profile_background_tile":true,"profile_link_color":"000000","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/783475833902170112\/rrKCDWEm_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/783475833902170112\/rrKCDWEm_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/331208078\/1474422862","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Thu Oct 13 22:29:27 +0000 2016","id":786695373062569985,"id_str":"786695373062569985","text":"Wait so\nTRUMP: i just kiss women, i don't even ask\n12 WOMEN: Trump kissed me without asking\nTRUMP: that's insane, you're insane","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":17923412,"id_str":"17923412","name":"Ryan George","screen_name":"theryangeorge","location":"Montreal","url":"http:\/\/youtube.com\/movingmindstudio","description":"Host\/Producer at @screenrant, Comedian at @MovingMind. It's pronounced Rrr (the yan is silent)","protected":false,"verified":false,"followers_count":1124,"friends_count":568,"listed_count":20,"favourites_count":3212,"statuses_count":2881,"created_at":"Sat Dec 06 15:51:08 +0000 2008","utc_offset":-18000,"time_zone":"Quito","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"FFFFFF","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/378800000025064783\/406257ae1005f5b36e20525276b3e072.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/378800000025064783\/406257ae1005f5b36e20525276b3e072.jpeg","profile_background_tile":true,"profile_link_color":"1B95E0","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"C0DFEC","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/777925537050361857\/3kisVslj_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/777925537050361857\/3kisVslj_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/17923412\/1460463698","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":1140,"favorite_count":1753,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"theryangeorge","name":"Ryan George","id":17923412,"id_str":"17923412","indices":[3,17]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705420202"}
    ,
    {"created_at":"Mon Oct 17 11:57:00 +0000 2016","id":787985760850935808,"id_str":"787985760850935808","text":"I liked a @YouTube video from @mattconyt https:\/\/t.co\/N2jkmDkb9L #22 | Blonde Doxed, Trump Accusers, Podesta Emails | Beauty &amp; the","source":"\u003ca href=\"http:\/\/www.google.com\/\" rel=\"nofollow\"\u003eGoogle\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1907604540,"id_str":"1907604540","name":"\ud83c\udf85IRIS\ud83c\udf85","screen_name":"IRISnGEN","location":"New York","url":"http:\/\/youtube.com\/c\/IRISnGEN","description":"I\u2019m actually not funny. I\u2019m just really mean and people think I am joking. Also here to avoid Facebook friends.","protected":false,"verified":false,"followers_count":240,"friends_count":82,"listed_count":15,"favourites_count":375,"statuses_count":13586,"created_at":"Thu Sep 26 11:48:10 +0000 2013","utc_offset":-14400,"time_zone":"America\/New_York","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/378800000082752810\/76df28cbc0cc69be06e79a1901552bed.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/378800000082752810\/76df28cbc0cc69be06e79a1901552bed.jpeg","profile_background_tile":true,"profile_link_color":"0084B4","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/737637584839528448\/9KAhqc9K_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/737637584839528448\/9KAhqc9K_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/1907604540\/1430838410","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/N2jkmDkb9L","expanded_url":"http:\/\/youtu.be\/S7oUL7-sqrE?a","display_url":"youtu.be\/S7oUL7-sqrE?a","indices":[41,64]}],"user_mentions":[{"screen_name":"YouTube","name":"YouTube","id":10228272,"id_str":"10228272","indices":[10,18]},{"screen_name":"MattConYT","name":"Matt Christiansen","id":451746511,"id_str":"451746511","indices":[30,40]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705420111"}
    ,
    {"created_at":"Mon Oct 17 11:57:00 +0000 2016","id":787985760511111168,"id_str":"787985760511111168","text":"RT @Twiggy164: Welcome to the Banana Republic of America. https:\/\/t.co\/fDoSlarnOc","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":750766596797247488,"id_str":"750766596797247488","name":"Jen","screen_name":"beforethedawn16","location":"Canada","url":null,"description":"38. Fannibal. World-weary INTJ. Not a fan of bullshit. Sweary as fuck. Canadian. Longtime writing partner to @destinyawakened, and animal enthusiast.","protected":false,"verified":false,"followers_count":122,"friends_count":131,"listed_count":5,"favourites_count":4087,"statuses_count":4319,"created_at":"Wed Jul 06 19:01:20 +0000 2016","utc_offset":-25200,"time_zone":"Pacific Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"E81C4F","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/750769070262226944\/RO6qnE3D_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/750769070262226944\/RO6qnE3D_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/750766596797247488\/1470783560","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 03:31:09 +0000 2016","id":787858462948532224,"id_str":"787858462948532224","text":"Welcome to the Banana Republic of America. https:\/\/t.co\/fDoSlarnOc","display_text_range":[0,42],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":910854372,"id_str":"910854372","name":"Twiggy164","screen_name":"Twiggy164","location":"The Loon Star State","url":null,"description":"1960s SDS activist in Daley's Chicago, marched w\/Black Panthers. Grew up in PR, fluent Spanish\/teach it\/love it! Love all creatures & this good earth.#UniteBlue","protected":false,"verified":false,"followers_count":1837,"friends_count":1804,"listed_count":54,"favourites_count":18595,"statuses_count":23032,"created_at":"Sun Oct 28 18:35:06 +0000 2012","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/624427544767393792\/sCMLBp5j_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/624427544767393792\/sCMLBp5j_normal.png","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/910854372\/1433987199","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"quoted_status_id":787817046218199040,"quoted_status_id_str":"787817046218199040","quoted_status":{"created_at":"Mon Oct 17 00:46:35 +0000 2016","id":787817046218199040,"id_str":"787817046218199040","text":"Trump acts like Hillary's undermining our legitimate democratic system (\"its rigged\") while threatening to jail his\u2026 https:\/\/t.co\/Q2eoNlJTKK","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":435564722,"id_str":"435564722","name":"Jon Riley","screen_name":"JonRiley7","location":"Austin, TX","url":"http:\/\/politicsisfunandgames.blogspot.com","description":"If you like political cartoons flip through my media tab. I always #FollowBack on principle. RT\u2260endorsement. #ImWithHer","protected":false,"verified":false,"followers_count":15153,"friends_count":16021,"listed_count":143,"favourites_count":12766,"statuses_count":14005,"created_at":"Tue Dec 13 06:04:01 +0000 2011","utc_offset":-21600,"time_zone":"Mountain Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"9AE4E8","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme16\/bg.gif","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme16\/bg.gif","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"BDDCAD","profile_sidebar_fill_color":"DDFFCC","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/782074979445673984\/c2afRbbY_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/782074979445673984\/c2afRbbY_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/435564722\/1464120764","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"quoted_status_id":787267564405653505,"quoted_status_id_str":"787267564405653505","is_quote_status":true,"extended_tweet":{"full_text":"Trump acts like Hillary's undermining our legitimate democratic system (\"its rigged\") while threatening to jail his opponent like a dictator https:\/\/t.co\/MMi8A0gi2a","display_text_range":[0,140],"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/MMi8A0gi2a","expanded_url":"https:\/\/twitter.com\/realdonaldtrump\/status\/787267564405653505","display_url":"twitter.com\/realdonaldtrum\u2026","indices":[141,164]}],"user_mentions":[],"symbols":[]}},"retweet_count":43,"favorite_count":66,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/Q2eoNlJTKK","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787817046218199040","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[117,140]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":5,"favorite_count":9,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/fDoSlarnOc","expanded_url":"https:\/\/twitter.com\/jonriley7\/status\/787817046218199040","display_url":"twitter.com\/jonriley7\/stat\u2026","indices":[43,66]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"quoted_status_id":787817046218199040,"quoted_status_id_str":"787817046218199040","quoted_status":{"created_at":"Mon Oct 17 00:46:35 +0000 2016","id":787817046218199040,"id_str":"787817046218199040","text":"Trump acts like Hillary's undermining our legitimate democratic system (\"its rigged\") while threatening to jail his\u2026 https:\/\/t.co\/Q2eoNlJTKK","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":435564722,"id_str":"435564722","name":"Jon Riley","screen_name":"JonRiley7","location":"Austin, TX","url":"http:\/\/politicsisfunandgames.blogspot.com","description":"If you like political cartoons flip through my media tab. I always #FollowBack on principle. RT\u2260endorsement. #ImWithHer","protected":false,"verified":false,"followers_count":15153,"friends_count":16021,"listed_count":143,"favourites_count":12766,"statuses_count":14005,"created_at":"Tue Dec 13 06:04:01 +0000 2011","utc_offset":-21600,"time_zone":"Mountain Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"9AE4E8","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme16\/bg.gif","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme16\/bg.gif","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"BDDCAD","profile_sidebar_fill_color":"DDFFCC","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/782074979445673984\/c2afRbbY_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/782074979445673984\/c2afRbbY_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/435564722\/1464120764","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"quoted_status_id":787267564405653505,"quoted_status_id_str":"787267564405653505","is_quote_status":true,"extended_tweet":{"full_text":"Trump acts like Hillary's undermining our legitimate democratic system (\"its rigged\") while threatening to jail his opponent like a dictator https:\/\/t.co\/MMi8A0gi2a","display_text_range":[0,140],"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/MMi8A0gi2a","expanded_url":"https:\/\/twitter.com\/realdonaldtrump\/status\/787267564405653505","display_url":"twitter.com\/realdonaldtrum\u2026","indices":[141,164]}],"user_mentions":[],"symbols":[]}},"retweet_count":43,"favorite_count":66,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/Q2eoNlJTKK","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787817046218199040","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[117,140]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/fDoSlarnOc","expanded_url":"https:\/\/twitter.com\/jonriley7\/status\/787817046218199040","display_url":"twitter.com\/jonriley7\/stat\u2026","indices":[58,81]}],"user_mentions":[{"screen_name":"Twiggy164","name":"Twiggy164","id":910854372,"id_str":"910854372","indices":[3,13]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705420030"}
    ,
    {"created_at":"Mon Oct 17 11:57:00 +0000 2016","id":787985761488343041,"id_str":"787985761488343041","text":"RT @LouDobbs: Wikileaks docs have revealed and proved the vast corruption of the  Clinton Cartel: Media blackout, now this\u2026 ","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":3306160258,"id_str":"3306160258","name":"SailfishMD","screen_name":"sailfishmd","location":"Indiana, USA","url":null,"description":"Fiesty Conservative. Physician. Lover of chess, travel, gardening, wine, & fishing. Political junkie. #AllLivesMatter #NeverHillary #EndMOC","protected":false,"verified":false,"followers_count":369,"friends_count":1240,"listed_count":22,"favourites_count":8278,"statuses_count":9493,"created_at":"Mon Jun 01 23:10:55 +0000 2015","utc_offset":null,"time_zone":null,"geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/772111433232445440\/hDX8Wj1W_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/772111433232445440\/hDX8Wj1W_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/3306160258\/1452009102","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:19:50 +0000 2016","id":787976407972450304,"id_str":"787976407972450304","text":"Wikileaks docs have revealed and proved the vast corruption of the  Clinton Cartel: Media blackout, now this\u2026 https:\/\/t.co\/AiPdPMpAfu","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":26487169,"id_str":"26487169","name":"Lou Dobbs","screen_name":"LouDobbs","location":"New York, NY","url":"http:\/\/loudobbs.com","description":"Lou Dobbs Tonight, Fox Business Network, 7 & 11 pm    \nIG: http:\/\/instagram.com\/loudobbstonight\/","protected":false,"verified":true,"followers_count":809360,"friends_count":2477,"listed_count":3116,"favourites_count":3574,"statuses_count":23721,"created_at":"Wed Mar 25 12:39:59 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/575734940228190209\/iXHsvsM7.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/575734940228190209\/iXHsvsM7.jpeg","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/663851941571641344\/OqqkE56l_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/663851941571641344\/OqqkE56l_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/26487169\/1426100831","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"quoted_status_id":787889195507417088,"quoted_status_id_str":"787889195507417088","quoted_status":{"created_at":"Mon Oct 17 05:33:17 +0000 2016","id":787889195507417088,"id_str":"787889195507417088","text":"Julian Assange's internet link has been intentionally severed by a state party. We have activated the appropriate contingency plans.","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":16589206,"id_str":"16589206","name":"WikiLeaks","screen_name":"wikileaks","location":"Everywhere","url":"http:\/\/wikileaks.org","description":"We open governments. \nContact: https:\/\/wikileaks.org\/#submit\n\nPGP: A04C 5E09 ED02 B328 03EB 6116 93ED 732E 9231 8DBA","protected":false,"verified":true,"followers_count":3708046,"friends_count":7597,"listed_count":53987,"favourites_count":4,"statuses_count":44147,"created_at":"Sat Oct 04 06:41:05 +0000 2008","utc_offset":3600,"time_zone":"Dublin","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"9AE4E8","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/3147857\/WL_Hour_Glass.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/3147857\/WL_Hour_Glass.jpg","profile_background_tile":true,"profile_link_color":"0084B4","profile_sidebar_border_color":"BDDCAD","profile_sidebar_fill_color":"DDFFCC","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/512138307870785536\/Fe00yVS2_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/512138307870785536\/Fe00yVS2_normal.png","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/16589206\/1402313050","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":20573,"favorite_count":19890,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"extended_tweet":{"full_text":"Wikileaks docs have revealed and proved the vast corruption of the  Clinton Cartel: Media blackout, now this    #MAGA #TrumpPence 16 #Dobbs https:\/\/t.co\/CNLzNXFezN","display_text_range":[0,139],"entities":{"hashtags":[{"text":"MAGA","indices":[112,117]},{"text":"TrumpPence","indices":[118,129]},{"text":"Dobbs","indices":[133,139]}],"urls":[{"url":"https:\/\/t.co\/CNLzNXFezN","expanded_url":"https:\/\/twitter.com\/wikileaks\/status\/787889195507417088","display_url":"twitter.com\/wikileaks\/stat\u2026","indices":[140,163]}],"user_mentions":[],"symbols":[]}},"retweet_count":802,"favorite_count":839,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/AiPdPMpAfu","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787976407972450304","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[110,133]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"quoted_status_id":787889195507417088,"quoted_status_id_str":"787889195507417088","quoted_status":{"created_at":"Mon Oct 17 05:33:17 +0000 2016","id":787889195507417088,"id_str":"787889195507417088","text":"Julian Assange's internet link has been intentionally severed by a state party. We have activated the appropriate contingency plans.","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":16589206,"id_str":"16589206","name":"WikiLeaks","screen_name":"wikileaks","location":"Everywhere","url":"http:\/\/wikileaks.org","description":"We open governments. \nContact: https:\/\/wikileaks.org\/#submit\n\nPGP: A04C 5E09 ED02 B328 03EB 6116 93ED 732E 9231 8DBA","protected":false,"verified":true,"followers_count":3708046,"friends_count":7597,"listed_count":53987,"favourites_count":4,"statuses_count":44147,"created_at":"Sat Oct 04 06:41:05 +0000 2008","utc_offset":3600,"time_zone":"Dublin","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"9AE4E8","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/3147857\/WL_Hour_Glass.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/3147857\/WL_Hour_Glass.jpg","profile_background_tile":true,"profile_link_color":"0084B4","profile_sidebar_border_color":"BDDCAD","profile_sidebar_fill_color":"DDFFCC","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/512138307870785536\/Fe00yVS2_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/512138307870785536\/Fe00yVS2_normal.png","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/16589206\/1402313050","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":20573,"favorite_count":19890,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"","expanded_url":null,"indices":[124,124]}],"user_mentions":[{"screen_name":"LouDobbs","name":"Lou Dobbs","id":26487169,"id_str":"26487169","indices":[3,12]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705420263"}
    ,
    {"created_at":"Mon Oct 17 11:57:00 +0000 2016","id":787985761320767492,"id_str":"787985761320767492","text":"Wie k\u00e4uflich ist Hillary #Clinton? | Was dieser Mann f\u00fcr 10 Millionen Dollar will https:\/\/t.co\/cD5WIoKR9u - Haim Saban","source":"\u003ca href=\"http:\/\/bufferapp.com\" rel=\"nofollow\"\u003eBuffer\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":40532269,"id_str":"40532269","name":"Flensburg online","screen_name":"FlensburgOnline","location":"Flensburg, Germany","url":"http:\/\/www.flensburg-online.de\/blog\/","description":"Zeitung war gestern. Internet ist heute. Bibel ist immer.\r\nhttp:\/\/www.flensburg-online.de\/impressum.html","protected":false,"verified":false,"followers_count":6209,"friends_count":6817,"listed_count":109,"favourites_count":0,"statuses_count":223113,"created_at":"Sat May 16 20:07:58 +0000 2009","utc_offset":7200,"time_zone":"Berlin","geo_enabled":false,"lang":"de","contributors_enabled":false,"is_translator":false,"profile_background_color":"FFFFFF","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/23289230\/flensburg.gif","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/23289230\/flensburg.gif","profile_background_tile":false,"profile_link_color":"0099B9","profile_sidebar_border_color":"F5F5F5","profile_sidebar_fill_color":"F5F5F5","profile_text_color":"3C3940","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/1760237451\/flo-twitter-buchstaben_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/1760237451\/flo-twitter-buchstaben_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/40532269\/1465766815","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[{"text":"Clinton","indices":[25,33]}],"urls":[{"url":"https:\/\/t.co\/cD5WIoKR9u","expanded_url":"http:\/\/buff.ly\/2dVvnJv","display_url":"buff.ly\/2dVvnJv","indices":[82,105]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"de","timestamp_ms":"1476705420223"}
    ]

Writing /content/dataset/twitter_sample_large_record.json


In [ ]:
%%writefile /content/dataset/twitter_sample_small_records.json
{"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985754408484865,"id_str":"787985754408484865","text":"RT @emilycrockett: After Mike Tyson was convicted of rape in 1992, Trump said Tyson was the real victim https:\/\/t.co\/TPjPp9PAqG","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":787518866763067392,"id_str":"787518866763067392","name":"SamazxdKeU3s7lly","screen_name":"u3s7lly","location":null,"url":null,"description":null,"protected":false,"verified":false,"followers_count":0,"friends_count":0,"listed_count":3,"favourites_count":0,"statuses_count":53,"created_at":"Sun Oct 16 05:01:44 +0000 2016","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"ru","contributors_enabled":false,"is_translator":false,"profile_background_color":"F5F8FA","profile_background_image_url":"","profile_background_image_url_https":"","profile_background_tile":false,"profile_link_color":"2B7BB9","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/787680463963947009\/SW_J98FZ_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/787680463963947009\/SW_J98FZ_normal.jpg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Tue Oct 11 20:40:03 +0000 2016","id":785943065047470080,"id_str":"785943065047470080","text":"After Mike Tyson was convicted of rape in 1992, Trump said Tyson was the real victim https:\/\/t.co\/TPjPp9PAqG","source":"\u003ca href=\"http:\/\/www.vox.com\" rel=\"nofollow\"\u003eVox.com\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":29372259,"id_str":"29372259","name":"Emily Crockett","screen_name":"emilycrockett","location":"Washington, DC","url":null,"description":"Covering gender issues at @voxdotcom. Word nerd. Scurrilous wench. Opera singer. The terror that flaps in the night. Views are my own.","protected":false,"verified":true,"followers_count":5116,"friends_count":1988,"listed_count":277,"favourites_count":3485,"statuses_count":27255,"created_at":"Tue Apr 07 03:41:47 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/993458716\/blueshirt_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/993458716\/blueshirt_normal.jpg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":3,"favorite_count":4,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/TPjPp9PAqG","expanded_url":"http:\/\/www.vox.com\/identities\/2016\/10\/11\/13247206\/trump-mike-tyson-howard-stern-rape-recording?utm_campaign=emilycrockett&utm_content=chorus&utm_medium=social&utm_source=twitter","display_url":"vox.com\/identities\/201\u2026","indices":[85,108]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/TPjPp9PAqG","expanded_url":"http:\/\/www.vox.com\/identities\/2016\/10\/11\/13247206\/trump-mike-tyson-howard-stern-rape-recording?utm_campaign=emilycrockett&utm_content=chorus&utm_medium=social&utm_source=twitter","display_url":"vox.com\/identities\/201\u2026","indices":[104,127]}],"user_mentions":[{"screen_name":"emilycrockett","name":"Emily Crockett","id":29372259,"id_str":"29372259","indices":[3,17]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418575"}

{"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985754504962048,"id_str":"787985754504962048","text":"RT @youngsinick: Just to correct you, it's a she said, she said, she said, she said, she said, she said, she said, she said, she sai\u2026 ","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":389683297,"id_str":"389683297","name":"Nico Baier","screen_name":"nbbaier","location":"Berkeley, CA","url":null,"description":"linguistics grad student","protected":false,"verified":false,"followers_count":127,"friends_count":614,"listed_count":3,"favourites_count":571,"statuses_count":945,"created_at":"Wed Oct 12 20:47:31 +0000 2011","utc_offset":-25200,"time_zone":"Pacific Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/758523055660347392\/WxZ6i0bD_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/758523055660347392\/WxZ6i0bD_normal.jpg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Sun Oct 16 13:49:50 +0000 2016","id":787651771569172480,"id_str":"787651771569172480","text":"Just to correct you, it's a she said, she said, she said, she said, she said, she said, she said, she said, she sai\u2026 https:\/\/t.co\/LMBMozXzoy","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":22111533,"id_str":"22111533","name":"Michael Arceneaux","screen_name":"youngsinick","location":"H-Town vicious in NYC","url":"http:\/\/www.thecynicalones.com\/about","description":"Writer. Talker. Texas Bamma.","protected":false,"verified":true,"followers_count":17890,"friends_count":1384,"listed_count":404,"favourites_count":7423,"statuses_count":120315,"created_at":"Fri Feb 27 05:26:42 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"ABB8C2","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/114220301\/banner.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/114220301\/banner.jpg","profile_background_tile":true,"profile_link_color":"0084B4","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"DDFFCC","profile_text_color":"333333","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/781684961912979456\/YRO7D00T_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/781684961912979456\/YRO7D00T_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/22111533\/1467895165","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"quoted_status_id":787649841438593028,"quoted_status_id_str":"787649841438593028","quoted_status":{"created_at":"Sun Oct 16 13:42:10 +0000 2016","id":787649841438593028,"id_str":"787649841438593028","text":"Rep. Renee Ellmers calls Trump allegations \"a she said\/he said situation\" https:\/\/t.co\/7eshI3SCbQ #CNNSOTU https:\/\/t.co\/Jy2wjcObg6","source":"\u003ca href=\"https:\/\/about.twitter.com\/products\/tweetdeck\" rel=\"nofollow\"\u003eTweetDeck\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":759251,"id_str":"759251","name":"CNN","screen_name":"CNN","location":null,"url":"http:\/\/www.cnn.com","description":"It\u2019s our job to #GoThere and tell the most difficult stories. Come with us!","protected":false,"verified":true,"followers_count":28712321,"friends_count":1122,"listed_count":119333,"favourites_count":1433,"statuses_count":102189,"created_at":"Fri Feb 09 00:35:02 +0000 2007","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"323232","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/515228058286952449\/zVfUxqPl.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/515228058286952449\/zVfUxqPl.jpeg","profile_background_tile":false,"profile_link_color":"004287","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"EEEEEE","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/508960761826131968\/LnvhR8ED_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/508960761826131968\/LnvhR8ED_normal.png","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/759251\/1476074145","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":618,"favorite_count":1075,"entities":{"hashtags":[{"text":"CNNSOTU","indices":[98,106]}],"urls":[{"url":"https:\/\/t.co\/7eshI3SCbQ","expanded_url":"http:\/\/cnn.com\/politics","display_url":"cnn.com\/politics","indices":[74,97]},{"url":"https:\/\/t.co\/Jy2wjcObg6","expanded_url":"http:\/\/cnn.it\/2e9jVth","display_url":"cnn.it\/2e9jVth","indices":[107,130]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"extended_tweet":{"full_text":"Just to correct you, it's a she said, she said, she said, she said, she said, she said, she said, she said, she said situation. https:\/\/t.co\/6vH1ONl100","display_text_range":[0,127],"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/6vH1ONl100","expanded_url":"https:\/\/twitter.com\/cnn\/status\/787649841438593028","display_url":"twitter.com\/cnn\/status\/787\u2026","indices":[128,151]}],"user_mentions":[],"symbols":[]}},"retweet_count":240,"favorite_count":514,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/LMBMozXzoy","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787651771569172480","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[117,140]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"quoted_status_id":787649841438593028,"quoted_status_id_str":"787649841438593028","quoted_status":{"created_at":"Sun Oct 16 13:42:10 +0000 2016","id":787649841438593028,"id_str":"787649841438593028","text":"Rep. Renee Ellmers calls Trump allegations \"a she said\/he said situation\" https:\/\/t.co\/7eshI3SCbQ #CNNSOTU https:\/\/t.co\/Jy2wjcObg6","source":"\u003ca href=\"https:\/\/about.twitter.com\/products\/tweetdeck\" rel=\"nofollow\"\u003eTweetDeck\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":759251,"id_str":"759251","name":"CNN","screen_name":"CNN","location":null,"url":"http:\/\/www.cnn.com","description":"It\u2019s our job to #GoThere and tell the most difficult stories. Come with us!","protected":false,"verified":true,"followers_count":28712321,"friends_count":1122,"listed_count":119333,"favourites_count":1433,"statuses_count":102189,"created_at":"Fri Feb 09 00:35:02 +0000 2007","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"323232","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/515228058286952449\/zVfUxqPl.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/515228058286952449\/zVfUxqPl.jpeg","profile_background_tile":false,"profile_link_color":"004287","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"EEEEEE","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/508960761826131968\/LnvhR8ED_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/508960761826131968\/LnvhR8ED_normal.png","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/759251\/1476074145","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":618,"favorite_count":1075,"entities":{"hashtags":[{"text":"CNNSOTU","indices":[98,106]}],"urls":[{"url":"https:\/\/t.co\/7eshI3SCbQ","expanded_url":"http:\/\/cnn.com\/politics","display_url":"cnn.com\/politics","indices":[74,97]},{"url":"https:\/\/t.co\/Jy2wjcObg6","expanded_url":"http:\/\/cnn.it\/2e9jVth","display_url":"cnn.it\/2e9jVth","indices":[107,130]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"","expanded_url":null,"indices":[134,134]}],"user_mentions":[{"screen_name":"youngsinick","name":"Michael Arceneaux","id":22111533,"id_str":"22111533","indices":[3,15]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418598"}

{"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985755133980672,"id_str":"787985755133980672","text":"Being Muslim and American in the year of Donald Trump: Muslim Americans describe the September 11, 2001, terr... https:\/\/t.co\/SJKsmeBCNv","source":"\u003ca href=\"http:\/\/twitterfeed.com\" rel=\"nofollow\"\u003etwitterfeed\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1702078267,"id_str":"1702078267","name":"aayushi shah","screen_name":"aayushis06","location":null,"url":null,"description":null,"protected":false,"verified":false,"followers_count":40,"friends_count":30,"listed_count":11,"favourites_count":0,"statuses_count":41992,"created_at":"Mon Aug 26 14:17:31 +0000 2013","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_1_normal.png","profile_image_url_https":"https:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_1_normal.png","default_profile":true,"default_profile_image":true,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/SJKsmeBCNv","expanded_url":"http:\/\/cnn.it\/2eleo1w","display_url":"cnn.it\/2eleo1w","indices":[113,136]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418748"}

{"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985754379128832,"id_str":"787985754379128832","text":"RT @thinkprogress: Clinton proposes plan that would lift millions out of poverty\nhttps:\/\/t.co\/B6T1jj2hST https:\/\/t.co\/blQy8JxViF","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":630450707,"id_str":"630450707","name":"Bruce M. Foster","screen_name":"roadshowrigolet","location":"NYC, NY United States","url":"http:\/\/www.roadshowrigoletto.com\/","description":"Born. Yet to complete the ride. \u201cDo not fear death so much\u00a0but rather the inadequate life.\u201d\u00a0- Bertolt Brecht\u00a0","protected":false,"verified":false,"followers_count":634,"friends_count":2252,"listed_count":49,"favourites_count":8769,"statuses_count":15057,"created_at":"Sun Jul 08 19:14:53 +0000 2012","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"F02005","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/445991262089248768\/OlwWkYG4.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/445991262089248768\/OlwWkYG4.jpeg","profile_background_tile":true,"profile_link_color":"0E19F0","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"C0DFEC","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/417860381470908416\/iTtCQOwN_normal.jpeg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/417860381470908416\/iTtCQOwN_normal.jpeg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/630450707\/1391378643","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:40:03 +0000 2016","id":787981495390576640,"id_str":"787981495390576640","text":"Clinton proposes plan that would lift millions out of poverty\nhttps:\/\/t.co\/B6T1jj2hST https:\/\/t.co\/blQy8JxViF","display_text_range":[0,85],"source":"\u003ca href=\"http:\/\/meetedgar.com\" rel=\"nofollow\"\u003eMeet Edgar\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":55355654,"id_str":"55355654","name":"ThinkProgress","screen_name":"thinkprogress","location":"Washington, D.C.","url":"http:\/\/www.thinkprogress.org","description":"Moving news forward.","protected":false,"verified":true,"followers_count":610741,"friends_count":976,"listed_count":13459,"favourites_count":211,"statuses_count":84028,"created_at":"Thu Jul 09 20:42:08 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"E6EDED","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/365080126\/tp-twitter.gif","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/365080126\/tp-twitter.gif","profile_background_tile":false,"profile_link_color":"531AAA","profile_sidebar_border_color":"EDEDED","profile_sidebar_fill_color":"D4D4D4","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/762609394265632768\/H_UR_W7r_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/762609394265632768\/H_UR_W7r_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/55355654\/1470649637","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":30,"favorite_count":34,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/B6T1jj2hST","expanded_url":"http:\/\/thkpr.gs\/998b84c3392c","display_url":"thkpr.gs\/998b84c3392c","indices":[62,85]}],"user_mentions":[],"symbols":[],"media":[{"id":787981492932796416,"id_str":"787981492932796416","indices":[86,109],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu94_i_XYAAJ_7Z.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu94_i_XYAAJ_7Z.jpg","url":"https:\/\/t.co\/blQy8JxViF","display_url":"pic.twitter.com\/blQy8JxViF","expanded_url":"https:\/\/twitter.com\/thinkprogress\/status\/787981495390576640\/photo\/1","type":"photo","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":1200,"h":800,"resize":"fit"},"small":{"w":680,"h":453,"resize":"fit"},"large":{"w":2000,"h":1333,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787981492932796416,"id_str":"787981492932796416","indices":[86,109],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu94_i_XYAAJ_7Z.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu94_i_XYAAJ_7Z.jpg","url":"https:\/\/t.co\/blQy8JxViF","display_url":"pic.twitter.com\/blQy8JxViF","expanded_url":"https:\/\/twitter.com\/thinkprogress\/status\/787981495390576640\/photo\/1","type":"photo","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":1200,"h":800,"resize":"fit"},"small":{"w":680,"h":453,"resize":"fit"},"large":{"w":2000,"h":1333,"resize":"fit"}}}]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/B6T1jj2hST","expanded_url":"http:\/\/thkpr.gs\/998b84c3392c","display_url":"thkpr.gs\/998b84c3392c","indices":[81,104]}],"user_mentions":[{"screen_name":"thinkprogress","name":"ThinkProgress","id":55355654,"id_str":"55355654","indices":[3,17]}],"symbols":[],"media":[{"id":787981492932796416,"id_str":"787981492932796416","indices":[105,128],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu94_i_XYAAJ_7Z.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu94_i_XYAAJ_7Z.jpg","url":"https:\/\/t.co\/blQy8JxViF","display_url":"pic.twitter.com\/blQy8JxViF","expanded_url":"https:\/\/twitter.com\/thinkprogress\/status\/787981495390576640\/photo\/1","type":"photo","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":1200,"h":800,"resize":"fit"},"small":{"w":680,"h":453,"resize":"fit"},"large":{"w":2000,"h":1333,"resize":"fit"}},"source_status_id":787981495390576640,"source_status_id_str":"787981495390576640","source_user_id":55355654,"source_user_id_str":"55355654"}]},"extended_entities":{"media":[{"id":787981492932796416,"id_str":"787981492932796416","indices":[105,128],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu94_i_XYAAJ_7Z.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu94_i_XYAAJ_7Z.jpg","url":"https:\/\/t.co\/blQy8JxViF","display_url":"pic.twitter.com\/blQy8JxViF","expanded_url":"https:\/\/twitter.com\/thinkprogress\/status\/787981495390576640\/photo\/1","type":"photo","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":1200,"h":800,"resize":"fit"},"small":{"w":680,"h":453,"resize":"fit"},"large":{"w":2000,"h":1333,"resize":"fit"}},"source_status_id":787981495390576640,"source_status_id_str":"787981495390576640","source_user_id":55355654,"source_user_id_str":"55355654"}]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418568"}

{"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985755192889344,"id_str":"787985755192889344","text":"Trump and the Terrible, Horrible, No Good, Very Bad Day #MovieTitleToDescribeElection","source":"\u003ca href=\"http:\/\/twitter.com\/#!\/download\/ipad\" rel=\"nofollow\"\u003eTwitter for iPad\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":715057889,"id_str":"715057889","name":"Barbie","screen_name":"Barbandoll","location":null,"url":null,"description":"Picasso's Muse, Joined Twitter one tweet at a time. I need humor to get by.","protected":false,"verified":false,"followers_count":177,"friends_count":338,"listed_count":3,"favourites_count":1284,"statuses_count":884,"created_at":"Mon Oct 14 20:24:30 +0000 2013","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/752988156346175489\/nFCSUtOV_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/752988156346175489\/nFCSUtOV_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/715057889\/1473374264","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[{"text":"MovieTitleToDescribeElection","indices":[56,85]}],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418762"}

{"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985755461214208,"id_str":"787985755461214208","text":"RT @dudleydew: @cher THE \ud83c\udf4e DOESN'T FALL FAR FROM THE \ud83d\udca9 TREE! https:\/\/t.co\/4sfkW7msCa","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":739239346348433408,"id_str":"739239346348433408","name":"Divia Va Darri","screen_name":"VaaVadarri","location":null,"url":null,"description":"I don't believe in fairytales","protected":false,"verified":false,"followers_count":23,"friends_count":100,"listed_count":1,"favourites_count":1328,"statuses_count":1061,"created_at":"Sat Jun 04 23:36:09 +0000 2016","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"F5F8FA","profile_background_image_url":"","profile_background_image_url_https":"","profile_background_tile":false,"profile_link_color":"2B7BB9","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/787830567454797824\/JxRxNnpr_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/787830567454797824\/JxRxNnpr_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/739239346348433408\/1465084368","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 08:01:53 +0000 2016","id":787926593490460672,"id_str":"787926593490460672","text":"@cher THE \ud83c\udf4e DOESN'T FALL FAR FROM THE \ud83d\udca9 TREE! https:\/\/t.co\/4sfkW7msCa","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":124003770,"in_reply_to_user_id_str":"124003770","in_reply_to_screen_name":"cher","user":{"id":284191160,"id_str":"284191160","name":"\u2728DOUGIE\u2728","screen_name":"dudleydew","location":null,"url":null,"description":"They call me Dougie! I'm dancing as fast as I can and it seems I just keep going backwards. And let me tell you, I'm no Ginger Rogers.","protected":false,"verified":false,"followers_count":637,"friends_count":204,"listed_count":24,"favourites_count":10727,"statuses_count":43990,"created_at":"Mon Apr 18 20:20:35 +0000 2011","utc_offset":null,"time_zone":null,"geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"642D8B","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme10\/bg.gif","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme10\/bg.gif","profile_background_tile":true,"profile_link_color":"FF0000","profile_sidebar_border_color":"65B0DA","profile_sidebar_fill_color":"7AC3EE","profile_text_color":"3D1957","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/773637700352040960\/PY_VyGfz_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/773637700352040960\/PY_VyGfz_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/284191160\/1472540244","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":118,"favorite_count":242,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/4sfkW7msCa","expanded_url":"http:\/\/www.huffingtonpost.com\/entry\/donald-trump-jr-women-harrassment-workforce_us_5800b9bbe4b0162c043b532e?ncid=engmodushpmg00000003","display_url":"huffingtonpost.com\/entry\/donald-t\u2026","indices":[46,69]}],"user_mentions":[{"screen_name":"cher","name":"Cher","id":124003770,"id_str":"124003770","indices":[0,5]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/4sfkW7msCa","expanded_url":"http:\/\/www.huffingtonpost.com\/entry\/donald-trump-jr-women-harrassment-workforce_us_5800b9bbe4b0162c043b532e?ncid=engmodushpmg00000003","display_url":"huffingtonpost.com\/entry\/donald-t\u2026","indices":[61,84]}],"user_mentions":[{"screen_name":"dudleydew","name":"\u2728DOUGIE\u2728","id":284191160,"id_str":"284191160","indices":[3,13]},{"screen_name":"cher","name":"Cher","id":124003770,"id_str":"124003770","indices":[15,20]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418826"}

{"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985755599544321,"id_str":"787985755599544321","text":"Email hacks reveal Clinton campaign's transparency problems: Clinton campaign chairman John Podesta's hacked ... https:\/\/t.co\/1vz9ph3p4Y","source":"\u003ca href=\"http:\/\/twitterfeed.com\" rel=\"nofollow\"\u003etwitterfeed\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":187595854,"id_str":"187595854","name":"2354","screen_name":"kindelov","location":null,"url":null,"description":null,"protected":false,"verified":false,"followers_count":4488,"friends_count":93,"listed_count":55,"favourites_count":0,"statuses_count":759769,"created_at":"Mon Sep 06 17:21:53 +0000 2010","utc_offset":10800,"time_zone":"Volgograd","geo_enabled":true,"lang":"ru","contributors_enabled":false,"is_translator":false,"profile_background_color":"FFFFFF","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/324255683\/morphia-dl.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/324255683\/morphia-dl.jpg","profile_background_tile":false,"profile_link_color":"2C7BCF","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"FFFFFF","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/1664860392\/ArmadaUnicron02_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/1664860392\/ArmadaUnicron02_normal.jpg","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/1vz9ph3p4Y","expanded_url":"http:\/\/cbsn.ws\/2enbA4W","display_url":"cbsn.ws\/2enbA4W","indices":[113,136]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418859"}

{"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985755570331649,"id_str":"787985755570331649","text":"RT @PrisonPlanet: Today it's Assange, next year who else's Internet will be cut for criticizing Hillary Clinton? #FreeJulian","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":291986334,"id_str":"291986334","name":"Adam LeFevre","screen_name":"AdamLeFevre","location":"Fort Recovery, Ohio","url":null,"description":"FRHS alum. State Champ. RHIT Class of 2020. Impossible is only the starting point. life is too short to last long","protected":false,"verified":false,"followers_count":691,"friends_count":2095,"listed_count":5,"favourites_count":7203,"statuses_count":9699,"created_at":"Mon May 02 22:56:03 +0000 2011","utc_offset":null,"time_zone":null,"geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"131516","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/456161319314395137\/vwLwaoMG.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/456161319314395137\/vwLwaoMG.jpeg","profile_background_tile":true,"profile_link_color":"009999","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"EFEFEF","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/679899607632113665\/2Zk3ZLdb_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/679899607632113665\/2Zk3ZLdb_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/291986334\/1467253888","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 09:30:53 +0000 2016","id":787948990411210752,"id_str":"787948990411210752","text":"Today it's Assange, next year who else's Internet will be cut for criticizing Hillary Clinton? #FreeJulian","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":18643437,"id_str":"18643437","name":"Paul Joseph Watson","screen_name":"PrisonPlanet","location":"London","url":"https:\/\/www.youtube.com\/user\/PrisonPlanetLive","description":"Contrarian polemicist, Infowars Editor-at-Large, 'pasty geek' (Louise Mensch)","protected":false,"verified":true,"followers_count":288036,"friends_count":278,"listed_count":3517,"favourites_count":4724,"statuses_count":26826,"created_at":"Mon Jan 05 20:04:23 +0000 2009","utc_offset":-18000,"time_zone":"Central Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"1A1B1F","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/378800000049129232\/f1cb5b3448d8a6d24e287249b765ae2a.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/378800000049129232\/f1cb5b3448d8a6d24e287249b765ae2a.jpeg","profile_background_tile":false,"profile_link_color":"2FC2EF","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"252429","profile_text_color":"666666","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/756825397388533760\/yXYx6CNC_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/756825397388533760\/yXYx6CNC_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/18643437\/1476394531","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":2516,"favorite_count":3029,"entities":{"hashtags":[{"text":"FreeJulian","indices":[95,106]}],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[{"text":"FreeJulian","indices":[113,124]}],"urls":[],"user_mentions":[{"screen_name":"PrisonPlanet","name":"Paul Joseph Watson","id":18643437,"id_str":"18643437","indices":[3,16]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418852"}

{"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985755775897600,"id_str":"787985755775897600","text":"RT @tonyschwartz: Trump actually believes he is a victim? Make no mistake: he is a victimizer, exploiting supporters who, like him, want so\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":744646569710989312,"id_str":"744646569710989312","name":"Sam","screen_name":"SamSam48412631","location":"Brighton, NY","url":null,"description":"#Election2016 Government & Politics","protected":false,"verified":false,"followers_count":15,"friends_count":19,"listed_count":0,"favourites_count":1273,"statuses_count":422,"created_at":"Sun Jun 19 21:42:31 +0000 2016","utc_offset":null,"time_zone":null,"geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"F5F8FA","profile_background_image_url":"","profile_background_image_url_https":"","profile_background_tile":false,"profile_link_color":"2B7BB9","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/745226443576180736\/9OuxzC2S_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/745226443576180736\/9OuxzC2S_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/744646569710989312\/1471568591","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Sun Oct 16 16:38:23 +0000 2016","id":787694185423962112,"id_str":"787694185423962112","text":"Trump actually believes he is a victim? Make no mistake: he is a victimizer, exploiting supporters who, like him, want someone to blame.","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":17579312,"id_str":"17579312","name":"Tony Schwartz","screen_name":"tonyschwartz","location":"Riverdale NY (i.e. the Bronx)","url":"http:\/\/TheEnergyProject.com","description":"CEO, The Energy Project. Author. The Art of the Deal ghostwriter now speaking out. Passionate about transforming the way the world works. Craves depth","protected":false,"verified":true,"followers_count":87684,"friends_count":60,"listed_count":2084,"favourites_count":19,"statuses_count":1743,"created_at":"Sun Nov 23 22:08:04 +0000 2008","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/3292198225\/7e454d576a5206f4a9475af269a2485e_normal.jpeg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/3292198225\/7e454d576a5206f4a9475af269a2485e_normal.jpeg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/17579312\/1469723369","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":1133,"favorite_count":2300,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"tonyschwartz","name":"Tony Schwartz","id":17579312,"id_str":"17579312","indices":[3,16]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418901"}

{"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985755918307328,"id_str":"787985755918307328","text":"RT @StefanMolyneux: If the state takes down Assange now, they send a clear message that Hillary Clinton must be kept very far away from any\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":312439360,"id_str":"312439360","name":"MURI","screen_name":"govmuri","location":"the greatest country on earth","url":null,"description":"you follow me I follow you.","protected":false,"verified":false,"followers_count":49,"friends_count":142,"listed_count":1,"favourites_count":1493,"statuses_count":1897,"created_at":"Tue Jun 07 03:52:35 +0000 2011","utc_offset":-25200,"time_zone":"Pacific Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"3B94D9","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/787825565118058497\/wn3y5D9I_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/787825565118058497\/wn3y5D9I_normal.jpg","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 06:20:23 +0000 2016","id":787901050925842432,"id_str":"787901050925842432","text":"If the state takes down Assange now, they send a clear message that Hillary Clinton must be kept very far away from any power. #FreeJulian","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":313038011,"id_str":"313038011","name":"Stefan Molyneux","screen_name":"StefanMolyneux","location":null,"url":"http:\/\/freedomainradio.com","description":"Peaceful Parent, Philosopher and Host of Freedomain Radio.\n\nhttp:\/\/youtube.com\/freedomainradio\nhttp:\/\/fdrpodcasts.com\nhttp:\/\/soundcloud.com\/stefan-molyneux","protected":false,"verified":true,"followers_count":77296,"friends_count":6,"listed_count":975,"favourites_count":2,"statuses_count":5822,"created_at":"Wed Jun 08 02:04:43 +0000 2011","utc_offset":-10800,"time_zone":"Atlantic Time (Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"022330","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme15\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme15\/bg.png","profile_background_tile":false,"profile_link_color":"0890C2","profile_sidebar_border_color":"A8C7F7","profile_sidebar_fill_color":"C0DFEC","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/744340099522039809\/7hWYBbgl_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/744340099522039809\/7hWYBbgl_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/313038011\/1398278552","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":2639,"favorite_count":3111,"entities":{"hashtags":[{"text":"FreeJulian","indices":[127,138]}],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"StefanMolyneux","name":"Stefan Molyneux","id":313038011,"id_str":"313038011","indices":[3,18]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418935"}

{"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985755192909825,"id_str":"787985755192909825","text":"RT @pink_lady56: Corrupt Clinton Times\n\n#FreeJulian https:\/\/t.co\/hSxk0D8bhr","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":596615669,"id_str":"596615669","name":"DeplorableChesam57\ud83c\udf1e","screen_name":"Chesam57","location":null,"url":null,"description":"#2ndamendment supporter, #patriot      #defundPP #Trump2016 #WethePeople  #Americafirst #TrumpTrain #Nevercruz\nDO NOT PUT ME ON ANY LISTS! You will be blocked.","protected":false,"verified":false,"followers_count":1386,"friends_count":847,"listed_count":82,"favourites_count":37969,"statuses_count":32505,"created_at":"Fri Jun 01 16:49:02 +0000 2012","utc_offset":-25200,"time_zone":"Pacific Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme16\/bg.gif","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme16\/bg.gif","profile_background_tile":false,"profile_link_color":"19CF86","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/718423188728729600\/3nvxNqA4_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/718423188728729600\/3nvxNqA4_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/596615669\/1469812973","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:28:10 +0000 2016","id":787978506940641285,"id_str":"787978506940641285","text":"Corrupt Clinton Times\n\n#FreeJulian https:\/\/t.co\/hSxk0D8bhr","display_text_range":[0,34],"source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":291874773,"id_str":"291874773","name":"\u2665Deplorable in Pink\u2665","screen_name":"pink_lady56","location":"United States","url":null,"description":"I VOTED for #DonaldTrump #MikePence\nI continue to support them. #MAGA\n #BlockTrolls","protected":false,"verified":false,"followers_count":13358,"friends_count":9884,"listed_count":169,"favourites_count":115442,"statuses_count":58788,"created_at":"Mon May 02 19:10:57 +0000 2011","utc_offset":-21600,"time_zone":"America\/Denver","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"022330","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme15\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme15\/bg.png","profile_background_tile":false,"profile_link_color":"DB3474","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/778595608219127812\/q8y5bCJX_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/778595608219127812\/q8y5bCJX_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/291874773\/1474460850","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"quoted_status_id":787949276089516032,"quoted_status_id_str":"787949276089516032","quoted_status":{"created_at":"Mon Oct 17 09:32:01 +0000 2016","id":787949276089516032,"id_str":"787949276089516032","text":"\ud83d\ude82\ud83d\udca8\ud83c\uddfa\ud83c\uddf8\ud83c\uddfa\ud83c\uddf8\ud83c\uddfa\ud83c\uddf8\ud83c\uddfa\ud83c\uddf8\n@realDonaldTrump\n#TrumpPence16\n#MAGA\n#TrumpTrain\n#TrumpArmy \n@EricTrump @IvankaTrump \n@EricTrumpFdn\u2026 https:\/\/t.co\/fqH71vSNXS","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/dashboard.twitter.com\" rel=\"nofollow\"\u003eTwitter Dashboard for iPhone\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":25867588,"id_str":"25867588","name":"Darryl P. Jordan","screen_name":"darryljordan","location":"BUNKER n Athens, Nashville, NC","url":"http:\/\/rnc.org","description":"NATIVE U.S.A. #UGA (9th GERERATON) CEO - JBA, INC. VENT & GET IT OUT! MANAGMENT INFO SYSTEMS, MBA, Finance, Real-Estate. #MAGA #NRA","protected":false,"verified":false,"followers_count":5416,"friends_count":5920,"listed_count":111,"favourites_count":3831,"statuses_count":19551,"created_at":"Sun Mar 22 19:47:35 +0000 2009","utc_offset":-14400,"time_zone":"Indiana (East)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"1A1B1F","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme9\/bg.gif","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme9\/bg.gif","profile_background_tile":false,"profile_link_color":"2FC2EF","profile_sidebar_border_color":"181A1E","profile_sidebar_fill_color":"252429","profile_text_color":"666666","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/756930286227783680\/d4mghwMQ_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/756930286227783680\/d4mghwMQ_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/25867588\/1468080148","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"extended_tweet":{"full_text":"\ud83d\ude82\ud83d\udca8\ud83c\uddfa\ud83c\uddf8\ud83c\uddfa\ud83c\uddf8\ud83c\uddfa\ud83c\uddf8\ud83c\uddfa\ud83c\uddf8\n@realDonaldTrump\n#TrumpPence16\n#MAGA\n#TrumpTrain\n#TrumpArmy \n@EricTrump @IvankaTrump \n@EricTrumpFdn \n@StJudeResearch \n#CROOKED https:\/\/t.co\/vIaFXilOZK","display_text_range":[0,137],"entities":{"hashtags":[{"text":"TrumpPence16","indices":[28,41]},{"text":"MAGA","indices":[42,47]},{"text":"TrumpTrain","indices":[48,59]},{"text":"TrumpArmy","indices":[60,70]},{"text":"CROOKED","indices":[129,137]}],"urls":[],"user_mentions":[{"screen_name":"realDonaldTrump","name":"Donald J. Trump","id":25073877,"id_str":"25073877","indices":[11,27]},{"screen_name":"EricTrump","name":"Eric Trump","id":39349894,"id_str":"39349894","indices":[72,82]},{"screen_name":"IvankaTrump","name":"Ivanka Trump","id":52544275,"id_str":"52544275","indices":[83,95]},{"screen_name":"EricTrumpFdn","name":"The Eric Trump Fdn","id":154567275,"id_str":"154567275","indices":[97,110]},{"screen_name":"StJudeResearch","name":"St. Jude Research","id":413308631,"id_str":"413308631","indices":[112,127]}],"symbols":[],"media":[{"id":787949267684057088,"id_str":"787949267684057088","indices":[138,161],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu9brylWYAAXoZQ.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu9brylWYAAXoZQ.jpg","url":"https:\/\/t.co\/vIaFXilOZK","display_url":"pic.twitter.com\/vIaFXilOZK","expanded_url":"https:\/\/twitter.com\/darryljordan\/status\/787949276089516032\/photo\/1","type":"photo","sizes":{"small":{"w":680,"h":509,"resize":"fit"},"medium":{"w":1200,"h":899,"resize":"fit"},"large":{"w":2048,"h":1534,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"}}}]},"extended_entities":{"media":[{"id":787949267684057088,"id_str":"787949267684057088","indices":[138,161],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu9brylWYAAXoZQ.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu9brylWYAAXoZQ.jpg","url":"https:\/\/t.co\/vIaFXilOZK","display_url":"pic.twitter.com\/vIaFXilOZK","expanded_url":"https:\/\/twitter.com\/darryljordan\/status\/787949276089516032\/photo\/1","type":"photo","sizes":{"small":{"w":680,"h":509,"resize":"fit"},"medium":{"w":1200,"h":899,"resize":"fit"},"large":{"w":2048,"h":1534,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"}}}]}},"retweet_count":40,"favorite_count":32,"entities":{"hashtags":[{"text":"TrumpPence16","indices":[28,41]},{"text":"MAGA","indices":[42,47]},{"text":"TrumpTrain","indices":[48,59]},{"text":"TrumpArmy","indices":[60,70]}],"urls":[{"url":"https:\/\/t.co\/fqH71vSNXS","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787949276089516032","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[112,135]}],"user_mentions":[{"screen_name":"realDonaldTrump","name":"Donald J. Trump","id":25073877,"id_str":"25073877","indices":[11,27]},{"screen_name":"EricTrump","name":"Eric Trump","id":39349894,"id_str":"39349894","indices":[72,82]},{"screen_name":"IvankaTrump","name":"Ivanka Trump","id":52544275,"id_str":"52544275","indices":[83,95]},{"screen_name":"EricTrumpFdn","name":"The Eric Trump Fdn","id":154567275,"id_str":"154567275","indices":[97,110]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"und"},"is_quote_status":true,"retweet_count":14,"favorite_count":12,"entities":{"hashtags":[{"text":"FreeJulian","indices":[23,34]}],"urls":[{"url":"https:\/\/t.co\/hSxk0D8bhr","expanded_url":"https:\/\/twitter.com\/darryljordan\/status\/787949276089516032","display_url":"twitter.com\/darryljordan\/s\u2026","indices":[35,58]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"quoted_status_id":787949276089516032,"quoted_status_id_str":"787949276089516032","quoted_status":{"created_at":"Mon Oct 17 09:32:01 +0000 2016","id":787949276089516032,"id_str":"787949276089516032","text":"\ud83d\ude82\ud83d\udca8\ud83c\uddfa\ud83c\uddf8\ud83c\uddfa\ud83c\uddf8\ud83c\uddfa\ud83c\uddf8\ud83c\uddfa\ud83c\uddf8\n@realDonaldTrump\n#TrumpPence16\n#MAGA\n#TrumpTrain\n#TrumpArmy \n@EricTrump @IvankaTrump \n@EricTrumpFdn\u2026 https:\/\/t.co\/fqH71vSNXS","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/dashboard.twitter.com\" rel=\"nofollow\"\u003eTwitter Dashboard for iPhone\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":25867588,"id_str":"25867588","name":"Darryl P. Jordan","screen_name":"darryljordan","location":"BUNKER n Athens, Nashville, NC","url":"http:\/\/rnc.org","description":"NATIVE U.S.A. #UGA (9th GERERATON) CEO - JBA, INC. VENT & GET IT OUT! MANAGMENT INFO SYSTEMS, MBA, Finance, Real-Estate. #MAGA #NRA","protected":false,"verified":false,"followers_count":5416,"friends_count":5920,"listed_count":111,"favourites_count":3831,"statuses_count":19551,"created_at":"Sun Mar 22 19:47:35 +0000 2009","utc_offset":-14400,"time_zone":"Indiana (East)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"1A1B1F","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme9\/bg.gif","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme9\/bg.gif","profile_background_tile":false,"profile_link_color":"2FC2EF","profile_sidebar_border_color":"181A1E","profile_sidebar_fill_color":"252429","profile_text_color":"666666","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/756930286227783680\/d4mghwMQ_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/756930286227783680\/d4mghwMQ_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/25867588\/1468080148","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"extended_tweet":{"full_text":"\ud83d\ude82\ud83d\udca8\ud83c\uddfa\ud83c\uddf8\ud83c\uddfa\ud83c\uddf8\ud83c\uddfa\ud83c\uddf8\ud83c\uddfa\ud83c\uddf8\n@realDonaldTrump\n#TrumpPence16\n#MAGA\n#TrumpTrain\n#TrumpArmy \n@EricTrump @IvankaTrump \n@EricTrumpFdn \n@StJudeResearch \n#CROOKED https:\/\/t.co\/vIaFXilOZK","display_text_range":[0,137],"entities":{"hashtags":[{"text":"TrumpPence16","indices":[28,41]},{"text":"MAGA","indices":[42,47]},{"text":"TrumpTrain","indices":[48,59]},{"text":"TrumpArmy","indices":[60,70]},{"text":"CROOKED","indices":[129,137]}],"urls":[],"user_mentions":[{"screen_name":"realDonaldTrump","name":"Donald J. Trump","id":25073877,"id_str":"25073877","indices":[11,27]},{"screen_name":"EricTrump","name":"Eric Trump","id":39349894,"id_str":"39349894","indices":[72,82]},{"screen_name":"IvankaTrump","name":"Ivanka Trump","id":52544275,"id_str":"52544275","indices":[83,95]},{"screen_name":"EricTrumpFdn","name":"The Eric Trump Fdn","id":154567275,"id_str":"154567275","indices":[97,110]},{"screen_name":"StJudeResearch","name":"St. Jude Research","id":413308631,"id_str":"413308631","indices":[112,127]}],"symbols":[],"media":[{"id":787949267684057088,"id_str":"787949267684057088","indices":[138,161],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu9brylWYAAXoZQ.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu9brylWYAAXoZQ.jpg","url":"https:\/\/t.co\/vIaFXilOZK","display_url":"pic.twitter.com\/vIaFXilOZK","expanded_url":"https:\/\/twitter.com\/darryljordan\/status\/787949276089516032\/photo\/1","type":"photo","sizes":{"small":{"w":680,"h":509,"resize":"fit"},"medium":{"w":1200,"h":899,"resize":"fit"},"large":{"w":2048,"h":1534,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"}}}]},"extended_entities":{"media":[{"id":787949267684057088,"id_str":"787949267684057088","indices":[138,161],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu9brylWYAAXoZQ.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu9brylWYAAXoZQ.jpg","url":"https:\/\/t.co\/vIaFXilOZK","display_url":"pic.twitter.com\/vIaFXilOZK","expanded_url":"https:\/\/twitter.com\/darryljordan\/status\/787949276089516032\/photo\/1","type":"photo","sizes":{"small":{"w":680,"h":509,"resize":"fit"},"medium":{"w":1200,"h":899,"resize":"fit"},"large":{"w":2048,"h":1534,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"}}}]}},"retweet_count":40,"favorite_count":32,"entities":{"hashtags":[{"text":"TrumpPence16","indices":[28,41]},{"text":"MAGA","indices":[42,47]},{"text":"TrumpTrain","indices":[48,59]},{"text":"TrumpArmy","indices":[60,70]}],"urls":[{"url":"https:\/\/t.co\/fqH71vSNXS","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787949276089516032","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[112,135]}],"user_mentions":[{"screen_name":"realDonaldTrump","name":"Donald J. Trump","id":25073877,"id_str":"25073877","indices":[11,27]},{"screen_name":"EricTrump","name":"Eric Trump","id":39349894,"id_str":"39349894","indices":[72,82]},{"screen_name":"IvankaTrump","name":"Ivanka Trump","id":52544275,"id_str":"52544275","indices":[83,95]},{"screen_name":"EricTrumpFdn","name":"The Eric Trump Fdn","id":154567275,"id_str":"154567275","indices":[97,110]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"und"},"is_quote_status":true,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[{"text":"FreeJulian","indices":[40,51]}],"urls":[{"url":"https:\/\/t.co\/hSxk0D8bhr","expanded_url":"https:\/\/twitter.com\/darryljordan\/status\/787949276089516032","display_url":"twitter.com\/darryljordan\/s\u2026","indices":[52,75]}],"user_mentions":[{"screen_name":"pink_lady56","name":"\u2665Deplorable in Pink\u2665","id":291874773,"id_str":"291874773","indices":[3,15]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418762"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985756228820992,"id_str":"787985756228820992","text":"Yo, @SpeakerRyan, you gonna put up with this shit? That bitch Trump doesn't even have the balls to tag you! https:\/\/t.co\/SvW1JsxJft","display_text_range":[0,107],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":18104752,"id_str":"18104752","name":"Dave Hill","screen_name":"mrdavehill","location":"New York City","url":"http:\/\/www.davehillonline.com","description":"My 2nd book DAVE HILL DOESN'T LIVE HERE ANYMORE out now! Get it!: https:\/\/t.co\/F4XTFUJD6v. The Goddamn Dave Hill Show Mondays 9pm-12am on @WFMU. S-T-A-U-N-C-H.","protected":false,"verified":true,"followers_count":54878,"friends_count":4342,"listed_count":1723,"favourites_count":23946,"statuses_count":56257,"created_at":"Sat Dec 13 20:55:21 +0000 2008","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"1A1B1F","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/580895891445301249\/LGGcqZkm.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/580895891445301249\/LGGcqZkm.jpg","profile_background_tile":true,"profile_link_color":"2FC2EF","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"252429","profile_text_color":"666666","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/719575070033735681\/S_3SjFPP_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/719575070033735681\/S_3SjFPP_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/18104752\/1467752456","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"quoted_status_id":787758668565581824,"quoted_status_id_str":"787758668565581824","quoted_status":{"created_at":"Sun Oct 16 20:54:37 +0000 2016","id":787758668565581824,"id_str":"787758668565581824","text":"Paul Ryan, a man who doesn't know how to win (including failed run four years ago), must start focusing on the budget, military, vets etc.","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":25073877,"id_str":"25073877","name":"Donald J. Trump","screen_name":"realDonaldTrump","location":"New York, NY","url":"http:\/\/www.DonaldJTrump.com","description":null,"protected":false,"verified":true,"followers_count":12517453,"friends_count":41,"listed_count":40274,"favourites_count":44,"statuses_count":33597,"created_at":"Wed Mar 18 13:46:38 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"6D5C18","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/530021613\/trump_scotland__43_of_70_cc.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/530021613\/trump_scotland__43_of_70_cc.jpg","profile_background_tile":true,"profile_link_color":"0D5B73","profile_sidebar_border_color":"BDDCAD","profile_sidebar_fill_color":"C5CEC0","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/1980294624\/DJT_Headshot_V2_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/1980294624\/DJT_Headshot_V2_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/25073877\/1468988952","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":9735,"favorite_count":26385,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/SvW1JsxJft","expanded_url":"https:\/\/twitter.com\/realdonaldtrump\/status\/787758668565581824","display_url":"twitter.com\/realdonaldtrum\u2026","indices":[108,131]}],"user_mentions":[{"screen_name":"SpeakerRyan","name":"Paul Ryan","id":18916432,"id_str":"18916432","indices":[4,16]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419009"}

{"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985756136566785,"id_str":"787985756136566785","text":"RT @DiamondandSilk: Only A Fool Fights In A Burning House. Won't nothing change when you vote for the same thing. @DiamondandSilk will\u2026 ","source":"\u003ca href=\"http:\/\/twitter.com\/#!\/download\/ipad\" rel=\"nofollow\"\u003eTwitter for iPad\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":808156722,"id_str":"808156722","name":"Harold Mann","screen_name":"Mann_HR","location":"Southeastern USA","url":null,"description":"Christian, Original principles of our Constitution\/Bill of Rights. #Prolife. #ProIsrael. #NRA, #Auburn Tiger, #MolonLabe, God bless America!","protected":false,"verified":false,"followers_count":1445,"friends_count":1515,"listed_count":23,"favourites_count":3820,"statuses_count":15421,"created_at":"Fri Sep 07 04:21:04 +0000 2012","utc_offset":-18000,"time_zone":"Central Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/478283602728198144\/KsKaup4a_normal.jpeg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/478283602728198144\/KsKaup4a_normal.jpeg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/808156722\/1355780957","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 09:31:04 +0000 2016","id":787949037022425089,"id_str":"787949037022425089","text":"Only A Fool Fights In A Burning House. Won't nothing change when you vote for the same thing. @DiamondandSilk will\u2026 https:\/\/t.co\/3RMF3EvmWw","display_text_range":[0,140],"source":"\u003ca href=\"https:\/\/ads.twitter.com\" rel=\"nofollow\"\u003eTwitter Ads\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":2908170952,"id_str":"2908170952","name":"Diamond and Silk\u00ae","screen_name":"DiamondandSilk","location":"United States","url":"http:\/\/www.DiamondandSilk.com","description":"#STUMP4TRUMPBABY #DITCHANDSWITCHNOW   #STUMP4TRUMPGIRLS  #STOMP4TRUMP   #TheUnitedStatesNotTheDividedStates  http:\/\/www.diamondandsilkinc.com\/about-us\/","protected":false,"verified":false,"followers_count":151104,"friends_count":40,"listed_count":767,"favourites_count":1065,"statuses_count":3287,"created_at":"Sat Dec 06 18:49:06 +0000 2014","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme15\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme15\/bg.png","profile_background_tile":false,"profile_link_color":"9266CC","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/638558385617567744\/mq4EKwOy_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/638558385617567744\/mq4EKwOy_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/2908170952\/1441236155","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"extended_tweet":{"full_text":"Only A Fool Fights In A Burning House. Won't nothing change when you vote for the same thing. @DiamondandSilk will Vote Trump on Nov 8,2016. https:\/\/t.co\/Htg2sy09JS","display_text_range":[0,140],"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"DiamondandSilk","name":"Diamond and Silk\u00ae","id":2908170952,"id_str":"2908170952","indices":[94,109]}],"symbols":[],"media":[{"id":770640547715350528,"id_str":"770640547715350528","indices":[141,164],"media_url":"http:\/\/pbs.twimg.com\/media\/CtEdY5gXYAILf5q.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/CtEdY5gXYAILf5q.jpg","url":"https:\/\/t.co\/Htg2sy09JS","display_url":"pic.twitter.com\/Htg2sy09JS","expanded_url":"https:\/\/twitter.com\/DiamondandSilk\/status\/787949037022425089\/video\/1","type":"video","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":640,"h":360,"resize":"fit"},"large":{"w":640,"h":360,"resize":"fit"},"medium":{"w":640,"h":360,"resize":"fit"}},"video_info":{"aspect_ratio":[16,9],"duration_millis":182917,"variants":[{"content_type":"application\/x-mpegURL","url":"https:\/\/video.twimg.com\/amplify_video\/770640547715350528\/pl\/THTUQ9Gr6JEQ8ori.m3u8"},{"content_type":"application\/dash+xml","url":"https:\/\/video.twimg.com\/amplify_video\/770640547715350528\/pl\/THTUQ9Gr6JEQ8ori.mpd"},{"bitrate":320000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/amplify_video\/770640547715350528\/vid\/320x180\/J1erQ4hJZoqwS9Y0.mp4"},{"bitrate":2176000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/amplify_video\/770640547715350528\/vid\/1280x720\/3sw_1UymMEcu5kWC.mp4"},{"bitrate":832000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/amplify_video\/770640547715350528\/vid\/640x360\/Kby0CaN6O6qLPBlv.mp4"}]}}]},"extended_entities":{"media":[{"id":770640547715350528,"id_str":"770640547715350528","indices":[141,164],"media_url":"http:\/\/pbs.twimg.com\/media\/CtEdY5gXYAILf5q.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/CtEdY5gXYAILf5q.jpg","url":"https:\/\/t.co\/Htg2sy09JS","display_url":"pic.twitter.com\/Htg2sy09JS","expanded_url":"https:\/\/twitter.com\/DiamondandSilk\/status\/787949037022425089\/video\/1","type":"video","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":640,"h":360,"resize":"fit"},"large":{"w":640,"h":360,"resize":"fit"},"medium":{"w":640,"h":360,"resize":"fit"}},"video_info":{"aspect_ratio":[16,9],"duration_millis":182917,"variants":[{"content_type":"application\/x-mpegURL","url":"https:\/\/video.twimg.com\/amplify_video\/770640547715350528\/pl\/THTUQ9Gr6JEQ8ori.m3u8"},{"content_type":"application\/dash+xml","url":"https:\/\/video.twimg.com\/amplify_video\/770640547715350528\/pl\/THTUQ9Gr6JEQ8ori.mpd"},{"bitrate":320000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/amplify_video\/770640547715350528\/vid\/320x180\/J1erQ4hJZoqwS9Y0.mp4"},{"bitrate":2176000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/amplify_video\/770640547715350528\/vid\/1280x720\/3sw_1UymMEcu5kWC.mp4"},{"bitrate":832000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/amplify_video\/770640547715350528\/vid\/640x360\/Kby0CaN6O6qLPBlv.mp4"}]}}]}},"retweet_count":582,"favorite_count":901,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/3RMF3EvmWw","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787949037022425089","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[116,139]}],"user_mentions":[{"screen_name":"DiamondandSilk","name":"Diamond and Silk\u00ae","id":2908170952,"id_str":"2908170952","indices":[94,109]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"","expanded_url":null,"indices":[136,136]}],"user_mentions":[{"screen_name":"DiamondandSilk","name":"Diamond and Silk\u00ae","id":2908170952,"id_str":"2908170952","indices":[3,18]},{"screen_name":"DiamondandSilk","name":"Diamond and Silk\u00ae","id":2908170952,"id_str":"2908170952","indices":[114,129]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418987"}

{"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985755612348416,"id_str":"787985755612348416","text":"RT @mtracey: This is disgusting war propaganda and everyone associated with the Clinton campaign should be ashamed of themselves https:\/\/t.\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":893221620,"id_str":"893221620","name":"Satyan Agarwal","screen_name":"SatyanAgarwal","location":"United States","url":null,"description":"Live Life King Size","protected":false,"verified":false,"followers_count":88,"friends_count":190,"listed_count":7,"favourites_count":215,"statuses_count":348,"created_at":"Sat Oct 20 13:15:52 +0000 2012","utc_offset":null,"time_zone":null,"geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/780110454907600896\/cyx9Wt1M_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/780110454907600896\/cyx9Wt1M_normal.jpg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 03:11:12 +0000 2016","id":787853439808245760,"id_str":"787853439808245760","text":"This is disgusting war propaganda and everyone associated with the Clinton campaign should be ashamed of themselves https:\/\/t.co\/OcI5Bj0g8T","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":24127811,"id_str":"24127811","name":"Michael Tracey","screen_name":"mtracey","location":"NYC","url":"http:\/\/mctracey.com","description":"Roving journalist, VICE columnist, friend to all dogs","protected":false,"verified":true,"followers_count":33686,"friends_count":1373,"listed_count":889,"favourites_count":5044,"statuses_count":64398,"created_at":"Fri Mar 13 04:31:49 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/517120721328156672\/7zUx36V__normal.jpeg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/517120721328156672\/7zUx36V__normal.jpeg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/24127811\/1475721783","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"quoted_status_id":787755182792794112,"quoted_status_id_str":"787755182792794112","quoted_status":{"created_at":"Sun Oct 16 20:40:46 +0000 2016","id":787755182792794112,"id_str":"787755182792794112","text":"Wow\u2014 Clinton camp out with 6-minute video highlighting Trump's deep ties to Russia: https:\/\/t.co\/RwJr3eWNnf","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":32871086,"id_str":"32871086","name":"Kyle Griffin","screen_name":"kylegriffin1","location":"Manhattan, NY","url":"http:\/\/www.msnbc.com\/the-last-word","description":"Producer on MSNBC's @TheLastWord. Former Sydneysider. Opinions my own. Please clap.","protected":false,"verified":true,"followers_count":20199,"friends_count":1218,"listed_count":521,"favourites_count":33957,"statuses_count":19336,"created_at":"Sat Apr 18 12:45:48 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/238292858\/Twitter_background.png","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/238292858\/Twitter_background.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/658832379130200064\/YUk99vQB_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/658832379130200064\/YUk99vQB_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/32871086\/1455243585","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":2419,"favorite_count":2305,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[],"media":[{"id":787379765267271680,"id_str":"787379765267271680","indices":[84,107],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu50qLMUMAADynP.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu50qLMUMAADynP.jpg","url":"https:\/\/t.co\/RwJr3eWNnf","display_url":"pic.twitter.com\/RwJr3eWNnf","expanded_url":"https:\/\/twitter.com\/TheBriefing2016\/status\/787695356075188224\/video\/1","type":"photo","sizes":{"small":{"w":680,"h":383,"resize":"fit"},"medium":{"w":1200,"h":675,"resize":"fit"},"large":{"w":1280,"h":720,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"}},"source_status_id":787695356075188224,"source_status_id_str":"787695356075188224","source_user_id":3232054991,"source_user_id_str":"3232054991"}]},"extended_entities":{"media":[{"id":787379765267271680,"id_str":"787379765267271680","indices":[84,107],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu50qLMUMAADynP.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu50qLMUMAADynP.jpg","url":"https:\/\/t.co\/RwJr3eWNnf","display_url":"pic.twitter.com\/RwJr3eWNnf","expanded_url":"https:\/\/twitter.com\/TheBriefing2016\/status\/787695356075188224\/video\/1","type":"video","sizes":{"small":{"w":680,"h":383,"resize":"fit"},"medium":{"w":1200,"h":675,"resize":"fit"},"large":{"w":1280,"h":720,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"}},"source_status_id":787695356075188224,"source_status_id_str":"787695356075188224","source_user_id":3232054991,"source_user_id_str":"3232054991","video_info":{"aspect_ratio":[16,9],"duration_millis":352186,"variants":[{"bitrate":2176000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/amplify_video\/787379765267271680\/vid\/1280x720\/5MScr9_aW2WtB0fX.mp4"},{"content_type":"application\/x-mpegURL","url":"https:\/\/video.twimg.com\/amplify_video\/787379765267271680\/pl\/wW3NUuVigguWbYjQ.m3u8"},{"content_type":"application\/dash+xml","url":"https:\/\/video.twimg.com\/amplify_video\/787379765267271680\/pl\/wW3NUuVigguWbYjQ.mpd"},{"bitrate":832000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/amplify_video\/787379765267271680\/vid\/640x360\/KHglDFoJ36RPJ7Gm.mp4"},{"bitrate":320000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/amplify_video\/787379765267271680\/vid\/320x180\/jaXSzSRNF58IC7GJ.mp4"}]}}]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":842,"favorite_count":1079,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/OcI5Bj0g8T","expanded_url":"https:\/\/twitter.com\/kylegriffin1\/status\/787755182792794112","display_url":"twitter.com\/kylegriffin1\/s\u2026","indices":[116,139]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"quoted_status_id":787755182792794112,"quoted_status_id_str":"787755182792794112","quoted_status":{"created_at":"Sun Oct 16 20:40:46 +0000 2016","id":787755182792794112,"id_str":"787755182792794112","text":"Wow\u2014 Clinton camp out with 6-minute video highlighting Trump's deep ties to Russia: https:\/\/t.co\/RwJr3eWNnf","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":32871086,"id_str":"32871086","name":"Kyle Griffin","screen_name":"kylegriffin1","location":"Manhattan, NY","url":"http:\/\/www.msnbc.com\/the-last-word","description":"Producer on MSNBC's @TheLastWord. Former Sydneysider. Opinions my own. Please clap.","protected":false,"verified":true,"followers_count":20199,"friends_count":1218,"listed_count":521,"favourites_count":33957,"statuses_count":19336,"created_at":"Sat Apr 18 12:45:48 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/238292858\/Twitter_background.png","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/238292858\/Twitter_background.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/658832379130200064\/YUk99vQB_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/658832379130200064\/YUk99vQB_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/32871086\/1455243585","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":2419,"favorite_count":2305,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[],"media":[{"id":787379765267271680,"id_str":"787379765267271680","indices":[84,107],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu50qLMUMAADynP.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu50qLMUMAADynP.jpg","url":"https:\/\/t.co\/RwJr3eWNnf","display_url":"pic.twitter.com\/RwJr3eWNnf","expanded_url":"https:\/\/twitter.com\/TheBriefing2016\/status\/787695356075188224\/video\/1","type":"photo","sizes":{"small":{"w":680,"h":383,"resize":"fit"},"medium":{"w":1200,"h":675,"resize":"fit"},"large":{"w":1280,"h":720,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"}},"source_status_id":787695356075188224,"source_status_id_str":"787695356075188224","source_user_id":3232054991,"source_user_id_str":"3232054991"}]},"extended_entities":{"media":[{"id":787379765267271680,"id_str":"787379765267271680","indices":[84,107],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu50qLMUMAADynP.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu50qLMUMAADynP.jpg","url":"https:\/\/t.co\/RwJr3eWNnf","display_url":"pic.twitter.com\/RwJr3eWNnf","expanded_url":"https:\/\/twitter.com\/TheBriefing2016\/status\/787695356075188224\/video\/1","type":"video","sizes":{"small":{"w":680,"h":383,"resize":"fit"},"medium":{"w":1200,"h":675,"resize":"fit"},"large":{"w":1280,"h":720,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"}},"source_status_id":787695356075188224,"source_status_id_str":"787695356075188224","source_user_id":3232054991,"source_user_id_str":"3232054991","video_info":{"aspect_ratio":[16,9],"duration_millis":352186,"variants":[{"bitrate":2176000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/amplify_video\/787379765267271680\/vid\/1280x720\/5MScr9_aW2WtB0fX.mp4"},{"content_type":"application\/x-mpegURL","url":"https:\/\/video.twimg.com\/amplify_video\/787379765267271680\/pl\/wW3NUuVigguWbYjQ.m3u8"},{"content_type":"application\/dash+xml","url":"https:\/\/video.twimg.com\/amplify_video\/787379765267271680\/pl\/wW3NUuVigguWbYjQ.mpd"},{"bitrate":832000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/amplify_video\/787379765267271680\/vid\/640x360\/KHglDFoJ36RPJ7Gm.mp4"},{"bitrate":320000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/amplify_video\/787379765267271680\/vid\/320x180\/jaXSzSRNF58IC7GJ.mp4"}]}}]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"mtracey","name":"Michael Tracey","id":24127811,"id_str":"24127811","indices":[3,11]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418862"}

{"created_at":"Mon Oct 17 11:56:58 +0000 2016","id":787985755977179136,"id_str":"787985755977179136","text":"Is...is a major party candidate on the verge of depressing his OWN turnout? https:\/\/t.co\/B3S9RxBoFC","display_text_range":[0,75],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":169062735,"id_str":"169062735","name":"Tim Krepp","screen_name":"timkrepp","location":"DC","url":"http:\/\/www.timkrepp.com","description":"just google it...","protected":false,"verified":false,"followers_count":2515,"friends_count":1443,"listed_count":96,"favourites_count":43329,"statuses_count":62393,"created_at":"Wed Jul 21 13:09:12 +0000 2010","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/766328696965894145\/hZ8Sm-YY_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/766328696965894145\/hZ8Sm-YY_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/169062735\/1465924878","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":{"id":"d6819fe60643ebc1","url":"https:\/\/api.twitter.com\/1.1\/geo\/id\/d6819fe60643ebc1.json","place_type":"city","name":"Alexandria","full_name":"Alexandria, VA","country_code":"US","country":"United States","bounding_box":{"type":"Polygon","coordinates":[[[-77.144435,38.789907],[-77.144435,38.844853],[-77.037304,38.844853],[-77.037304,38.789907]]]},"attributes":{}},"contributors":null,"quoted_status_id":787984785792696320,"quoted_status_id_str":"787984785792696320","quoted_status":{"created_at":"Mon Oct 17 11:53:07 +0000 2016","id":787984785792696320,"id_str":"787984785792696320","text":"Trump\u2019s \u201crigged\u201d lie appears to be working with supporters. Only half think their votes will be counted accurately\u2026 https:\/\/t.co\/W3MAskRQUl","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":63004503,"id_str":"63004503","name":"Michael Calderone","screen_name":"mlcalderone","location":"Brooklyn, NY","url":"http:\/\/michaelcalderone.com\/","description":"Senior media reporter, @HuffingtonPost; Adjunct faculty, @nyu_journalism; Ex-Politico\/Yahoo News\/NY Observer\nmichael.calderone@huffingtonpost.com","protected":false,"verified":true,"followers_count":53057,"friends_count":1698,"listed_count":2299,"favourites_count":0,"statuses_count":19689,"created_at":"Wed Aug 05 02:23:01 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"8B542B","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/49995328\/unknown.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/49995328\/unknown.jpg","profile_background_tile":true,"profile_link_color":"9D582E","profile_sidebar_border_color":"D9B17E","profile_sidebar_fill_color":"EADEAA","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/707744338172907520\/sOBnnXo8_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/707744338172907520\/sOBnnXo8_normal.jpg","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"extended_tweet":{"full_text":"Trump\u2019s \u201crigged\u201d lie appears to be working with supporters. Only half think their votes will be counted accurately https:\/\/t.co\/1NcxJ8C5LL https:\/\/t.co\/akfSdqYq6o","display_text_range":[0,138],"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/1NcxJ8C5LL","expanded_url":"http:\/\/bit.ly\/2duOBGv","display_url":"bit.ly\/2duOBGv","indices":[115,138]}],"user_mentions":[],"symbols":[],"media":[{"id":787984587003682816,"id_str":"787984587003682816","indices":[139,162],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu97zpTWYAAc5Td.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu97zpTWYAAc5Td.jpg","url":"https:\/\/t.co\/akfSdqYq6o","display_url":"pic.twitter.com\/akfSdqYq6o","expanded_url":"https:\/\/twitter.com\/mlcalderone\/status\/787984785792696320\/photo\/1","type":"photo","sizes":{"large":{"w":1419,"h":859,"resize":"fit"},"small":{"w":680,"h":412,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":1200,"h":726,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787984587003682816,"id_str":"787984587003682816","indices":[139,162],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu97zpTWYAAc5Td.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu97zpTWYAAc5Td.jpg","url":"https:\/\/t.co\/akfSdqYq6o","display_url":"pic.twitter.com\/akfSdqYq6o","expanded_url":"https:\/\/twitter.com\/mlcalderone\/status\/787984785792696320\/photo\/1","type":"photo","sizes":{"large":{"w":1419,"h":859,"resize":"fit"},"small":{"w":680,"h":412,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":1200,"h":726,"resize":"fit"}}}]}},"retweet_count":5,"favorite_count":4,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/W3MAskRQUl","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787984785792696320","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[116,139]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/B3S9RxBoFC","expanded_url":"https:\/\/twitter.com\/mlcalderone\/status\/787984785792696320","display_url":"twitter.com\/mlcalderone\/st\u2026","indices":[76,99]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705418949"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985756346261504,"id_str":"787985756346261504","text":"RT @PrisonPlanet: CNN Host Blames Firebombing of Republican HQ on Donald Trump - https:\/\/t.co\/8yY06ncB8n https:\/\/t.co\/eAFC6RHlhe","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":397585567,"id_str":"397585567","name":"Roger Johnson","screen_name":"RogerJohnson13","location":null,"url":null,"description":null,"protected":false,"verified":false,"followers_count":17,"friends_count":88,"listed_count":3,"favourites_count":184,"statuses_count":2065,"created_at":"Mon Oct 24 21:58:11 +0000 2011","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_1_normal.png","profile_image_url_https":"https:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_1_normal.png","default_profile":true,"default_profile_image":true,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:54:42 +0000 2016","id":787985184297775105,"id_str":"787985184297775105","text":"CNN Host Blames Firebombing of Republican HQ on Donald Trump - https:\/\/t.co\/8yY06ncB8n https:\/\/t.co\/eAFC6RHlhe","display_text_range":[0,86],"source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":18643437,"id_str":"18643437","name":"Paul Joseph Watson","screen_name":"PrisonPlanet","location":"London","url":"https:\/\/www.youtube.com\/user\/PrisonPlanetLive","description":"Contrarian polemicist, Infowars Editor-at-Large, 'pasty geek' (Louise Mensch)","protected":false,"verified":true,"followers_count":288036,"friends_count":278,"listed_count":3517,"favourites_count":4724,"statuses_count":26826,"created_at":"Mon Jan 05 20:04:23 +0000 2009","utc_offset":-18000,"time_zone":"Central Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"1A1B1F","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/378800000049129232\/f1cb5b3448d8a6d24e287249b765ae2a.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/378800000049129232\/f1cb5b3448d8a6d24e287249b765ae2a.jpeg","profile_background_tile":false,"profile_link_color":"2FC2EF","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"252429","profile_text_color":"666666","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/756825397388533760\/yXYx6CNC_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/756825397388533760\/yXYx6CNC_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/18643437\/1476394531","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":45,"favorite_count":45,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/8yY06ncB8n","expanded_url":"http:\/\/www.infowars.com\/cnn-host-blames-firebombing-of-republican-hq-on-donald-trump\/","display_url":"infowars.com\/cnn-host-blame\u2026","indices":[63,86]}],"user_mentions":[],"symbols":[],"media":[{"id":787985177343627264,"id_str":"787985177343627264","indices":[87,110],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu98WAfXEAA5TP4.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu98WAfXEAA5TP4.jpg","url":"https:\/\/t.co\/eAFC6RHlhe","display_url":"pic.twitter.com\/eAFC6RHlhe","expanded_url":"https:\/\/twitter.com\/PrisonPlanet\/status\/787985184297775105\/photo\/1","type":"photo","sizes":{"large":{"w":800,"h":420,"resize":"fit"},"medium":{"w":800,"h":420,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":680,"h":357,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787985177343627264,"id_str":"787985177343627264","indices":[87,110],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu98WAfXEAA5TP4.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu98WAfXEAA5TP4.jpg","url":"https:\/\/t.co\/eAFC6RHlhe","display_url":"pic.twitter.com\/eAFC6RHlhe","expanded_url":"https:\/\/twitter.com\/PrisonPlanet\/status\/787985184297775105\/photo\/1","type":"photo","sizes":{"large":{"w":800,"h":420,"resize":"fit"},"medium":{"w":800,"h":420,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":680,"h":357,"resize":"fit"}}}]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/8yY06ncB8n","expanded_url":"http:\/\/www.infowars.com\/cnn-host-blames-firebombing-of-republican-hq-on-donald-trump\/","display_url":"infowars.com\/cnn-host-blame\u2026","indices":[81,104]}],"user_mentions":[{"screen_name":"PrisonPlanet","name":"Paul Joseph Watson","id":18643437,"id_str":"18643437","indices":[3,16]}],"symbols":[],"media":[{"id":787985177343627264,"id_str":"787985177343627264","indices":[105,128],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu98WAfXEAA5TP4.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu98WAfXEAA5TP4.jpg","url":"https:\/\/t.co\/eAFC6RHlhe","display_url":"pic.twitter.com\/eAFC6RHlhe","expanded_url":"https:\/\/twitter.com\/PrisonPlanet\/status\/787985184297775105\/photo\/1","type":"photo","sizes":{"large":{"w":800,"h":420,"resize":"fit"},"medium":{"w":800,"h":420,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":680,"h":357,"resize":"fit"}},"source_status_id":787985184297775105,"source_status_id_str":"787985184297775105","source_user_id":18643437,"source_user_id_str":"18643437"}]},"extended_entities":{"media":[{"id":787985177343627264,"id_str":"787985177343627264","indices":[105,128],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu98WAfXEAA5TP4.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu98WAfXEAA5TP4.jpg","url":"https:\/\/t.co\/eAFC6RHlhe","display_url":"pic.twitter.com\/eAFC6RHlhe","expanded_url":"https:\/\/twitter.com\/PrisonPlanet\/status\/787985184297775105\/photo\/1","type":"photo","sizes":{"large":{"w":800,"h":420,"resize":"fit"},"medium":{"w":800,"h":420,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":680,"h":357,"resize":"fit"}},"source_status_id":787985184297775105,"source_status_id_str":"787985184297775105","source_user_id":18643437,"source_user_id_str":"18643437"}]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419037"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985756677603329,"id_str":"787985756677603329","text":"RT @ron_fournier: Clinton's supporters at PM USA considered push polling on her behalf. @JoeNBC is right: you're arguing a distinctio\u2026 ","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":85982089,"id_str":"85982089","name":"Javier Barberan","screen_name":"Toyoking","location":"Pembroke Pines, FL","url":null,"description":null,"protected":false,"verified":false,"followers_count":369,"friends_count":860,"listed_count":77,"favourites_count":16864,"statuses_count":16725,"created_at":"Thu Oct 29 03:09:00 +0000 2009","utc_offset":7200,"time_zone":"Copenhagen","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/694132909222002688\/q17HHTnK_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/694132909222002688\/q17HHTnK_normal.jpg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 10:41:03 +0000 2016","id":787966649928392704,"id_str":"787966649928392704","text":"Clinton's supporters at PM USA considered push polling on her behalf. @JoeNBC is right: you're arguing a distinctio\u2026 https:\/\/t.co\/0TjFMEVWKQ","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":14268812,"id_str":"14268812","name":"Ron Fournier","screen_name":"ron_fournier","location":"Detroit, MI","url":"http:\/\/www.theatlantic.com\/author\/ron-fournier\/","description":"Associate publisher Crain's Detroit. Dabble in politics for @TheAtlantic. Author, NYT best-seller #LoveThatBoy http:\/\/lovethatboybook.com\/books\/love-that-boy-hc","protected":false,"verified":true,"followers_count":56903,"friends_count":2110,"listed_count":1818,"favourites_count":53702,"statuses_count":142278,"created_at":"Mon Mar 31 19:35:07 +0000 2008","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/531032491\/NJTwitterBackground.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/531032491\/NJTwitterBackground.jpg","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/777365314094899200\/w59eXkTc_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/777365314094899200\/w59eXkTc_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/14268812\/1473438246","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"quoted_status_id":787961268305100800,"quoted_status_id_str":"787961268305100800","quoted_status":{"created_at":"Mon Oct 17 10:19:40 +0000 2016","id":787961268305100800,"id_str":"787961268305100800","text":"@ron_fournier Also Progressive Media USA conducted those polls, not Clinton campaign.","display_text_range":[14,85],"source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":787961084661596160,"in_reply_to_status_id_str":"787961084661596160","in_reply_to_user_id":16713361,"in_reply_to_user_id_str":"16713361","in_reply_to_screen_name":"SuzanneKelleher","user":{"id":16713361,"id_str":"16713361","name":"Suzanne Kelleher","screen_name":"SuzanneKelleher","location":null,"url":null,"description":"mom of three | family vacations expert | freelance travel writer | content strategy consultant #familytravel","protected":false,"verified":false,"followers_count":15844,"friends_count":5453,"listed_count":923,"favourites_count":13447,"statuses_count":44887,"created_at":"Sun Oct 12 23:17:55 +0000 2008","utc_offset":-7200,"time_zone":"Mid-Atlantic","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"9AE4E8","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/722820902\/35d7996f15723dabd4528fd9b06043a5.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/722820902\/35d7996f15723dabd4528fd9b06043a5.jpeg","profile_background_tile":false,"profile_link_color":"DD2E44","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"DDFFCC","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/418474427173462016\/5wWRhgkB_normal.jpeg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/418474427173462016\/5wWRhgkB_normal.jpeg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/16713361\/1434544459","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"ron_fournier","name":"Ron Fournier","id":14268812,"id_str":"14268812","indices":[0,13]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"extended_tweet":{"full_text":"Clinton's supporters at PM USA considered push polling on her behalf. @JoeNBC is right: you're arguing a distinction with little difference https:\/\/t.co\/BczW8M52yr","display_text_range":[0,139],"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/BczW8M52yr","expanded_url":"https:\/\/twitter.com\/suzannekelleher\/status\/787961268305100800","display_url":"twitter.com\/suzannekellehe\u2026","indices":[140,163]}],"user_mentions":[{"screen_name":"JoeNBC","name":"Joe Scarborough","id":21619519,"id_str":"21619519","indices":[70,77]}],"symbols":[]}},"retweet_count":7,"favorite_count":15,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/0TjFMEVWKQ","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787966649928392704","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[117,140]}],"user_mentions":[{"screen_name":"JoeNBC","name":"Joe Scarborough","id":21619519,"id_str":"21619519","indices":[70,77]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"quoted_status_id":787961268305100800,"quoted_status_id_str":"787961268305100800","quoted_status":{"created_at":"Mon Oct 17 10:19:40 +0000 2016","id":787961268305100800,"id_str":"787961268305100800","text":"@ron_fournier Also Progressive Media USA conducted those polls, not Clinton campaign.","display_text_range":[14,85],"source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":787961084661596160,"in_reply_to_status_id_str":"787961084661596160","in_reply_to_user_id":16713361,"in_reply_to_user_id_str":"16713361","in_reply_to_screen_name":"SuzanneKelleher","user":{"id":16713361,"id_str":"16713361","name":"Suzanne Kelleher","screen_name":"SuzanneKelleher","location":null,"url":null,"description":"mom of three | family vacations expert | freelance travel writer | content strategy consultant #familytravel","protected":false,"verified":false,"followers_count":15844,"friends_count":5453,"listed_count":923,"favourites_count":13447,"statuses_count":44887,"created_at":"Sun Oct 12 23:17:55 +0000 2008","utc_offset":-7200,"time_zone":"Mid-Atlantic","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"9AE4E8","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/722820902\/35d7996f15723dabd4528fd9b06043a5.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/722820902\/35d7996f15723dabd4528fd9b06043a5.jpeg","profile_background_tile":false,"profile_link_color":"DD2E44","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"DDFFCC","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/418474427173462016\/5wWRhgkB_normal.jpeg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/418474427173462016\/5wWRhgkB_normal.jpeg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/16713361\/1434544459","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"ron_fournier","name":"Ron Fournier","id":14268812,"id_str":"14268812","indices":[0,13]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"","expanded_url":null,"indices":[135,135]}],"user_mentions":[{"screen_name":"ron_fournier","name":"Ron Fournier","id":14268812,"id_str":"14268812","indices":[3,16]},{"screen_name":"JoeNBC","name":"Joe Scarborough","id":21619519,"id_str":"21619519","indices":[88,95]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419116"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985757210304512,"id_str":"787985757210304512","text":"RT @CNNEE: Resumen de encuestas: Clinton aventaja a Trump por 8 puntos https:\/\/t.co\/OHAdy3KMxg","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":60212445,"id_str":"60212445","name":"Richard Mill\u00e1n","screen_name":"millanrichard","location":"Hate\u00f1o y Vallecaucano ","url":null,"description":"Periodista de pura sangre #combomadrugador #hinchadelcali y voto SI","protected":false,"verified":false,"followers_count":1259,"friends_count":167,"listed_count":18,"favourites_count":322,"statuses_count":6186,"created_at":"Sun Jul 26 02:40:17 +0000 2009","utc_offset":-18000,"time_zone":"Bogota","geo_enabled":true,"lang":"es","contributors_enabled":false,"is_translator":false,"profile_background_color":"8B542B","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme8\/bg.gif","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme8\/bg.gif","profile_background_tile":true,"profile_link_color":"9D582E","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"EADEAA","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/697961720015364097\/_gLD_iuH_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/697961720015364097\/_gLD_iuH_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/60212445\/1397913978","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:21:21 +0000 2016","id":787976789893283841,"id_str":"787976789893283841","text":"Resumen de encuestas: Clinton aventaja a Trump por 8 puntos https:\/\/t.co\/OHAdy3KMxg","source":"\u003ca href=\"https:\/\/about.twitter.com\/products\/tweetdeck\" rel=\"nofollow\"\u003eTweetDeck\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":33884545,"id_str":"33884545","name":"CNN en Espa\u00f1ol","screen_name":"CNNEE","location":"Atlanta","url":"https:\/\/telegram.me\/cnneebrk","description":"CNN en Espa\u00f1ol es tu principal fuente de informaci\u00f3n y breaking news. Cubrimos las noticias de Am\u00e9rica Latina y el resto del mundo. Vive la noticia.","protected":false,"verified":true,"followers_count":14039334,"friends_count":832,"listed_count":48093,"favourites_count":1269,"statuses_count":88501,"created_at":"Tue Apr 21 12:14:47 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"es","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/526905009\/CNNespanol_TwitterSkin.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/526905009\/CNNespanol_TwitterSkin.jpg","profile_background_tile":false,"profile_link_color":"AB1529","profile_sidebar_border_color":"B5B5B5","profile_sidebar_fill_color":"B5B5B5","profile_text_color":"000000","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/589795319216504832\/6a71ZHkx_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/589795319216504832\/6a71ZHkx_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/33884545\/1411070627","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":57,"favorite_count":43,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/OHAdy3KMxg","expanded_url":"http:\/\/cnn.it\/2dhPUKq","display_url":"cnn.it\/2dhPUKq","indices":[60,83]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"es"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/OHAdy3KMxg","expanded_url":"http:\/\/cnn.it\/2dhPUKq","display_url":"cnn.it\/2dhPUKq","indices":[71,94]}],"user_mentions":[{"screen_name":"CNNEE","name":"CNN en Espa\u00f1ol","id":33884545,"id_str":"33884545","indices":[3,9]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"es","timestamp_ms":"1476705419243"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985756866347009,"id_str":"787985756866347009","text":"@Slate \nAccusers with Zero evidence.\nPaid Hit Men.\nPaid to destroy credibility.\n11 years ago.\n11th hour of Election.\nI was touched.","display_text_range":[8,131],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":787982514669383680,"in_reply_to_status_id_str":"787982514669383680","in_reply_to_user_id":15164565,"in_reply_to_user_id_str":"15164565","in_reply_to_screen_name":"Slate","user":{"id":288973453,"id_str":"288973453","name":"mike james","screen_name":"twthead","location":"EaRTH","url":null,"description":"earth born citizen.  psychological experiment.  transient being.  made of billion year old hydrogen molecules. running on the treadmill of life.","protected":false,"verified":false,"followers_count":331,"friends_count":167,"listed_count":3,"favourites_count":2151,"statuses_count":6631,"created_at":"Wed Apr 27 19:49:32 +0000 2011","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"022330","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme15\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme15\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"A8C7F7","profile_sidebar_fill_color":"C0DFEC","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/1329720707\/images_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/1329720707\/images_normal.jpg","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"Slate","name":"Slate","id":15164565,"id_str":"15164565","indices":[0,6]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419161"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985756715450368,"id_str":"787985756715450368","text":"RT @WesleyRickard: 10-16-2016 Donald Trump Rally 20,000 people ready to vote for Donald Trump on November 8th Retweet and Post to Face\u2026 ","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":346068301,"id_str":"346068301","name":"Jean Watson","screen_name":"jeanloveslife","location":"North Carolina","url":null,"description":"I love our God, my Family and Nascar!","protected":false,"verified":false,"followers_count":504,"friends_count":2351,"listed_count":10,"favourites_count":2522,"statuses_count":3580,"created_at":"Sun Jul 31 17:32:01 +0000 2011","utc_offset":-10800,"time_zone":"Atlantic Time (Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"ACDED6","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme18\/bg.gif","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme18\/bg.gif","profile_background_tile":false,"profile_link_color":"DD2E44","profile_sidebar_border_color":"EEEEEE","profile_sidebar_fill_color":"F6F6F6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/569842177715347456\/pW7IyVB8_normal.jpeg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/569842177715347456\/pW7IyVB8_normal.jpeg","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 03:37:48 +0000 2016","id":787860136635359232,"id_str":"787860136635359232","text":"10-16-2016 Donald Trump Rally 20,000 people ready to vote for Donald Trump on November 8th Retweet and Post to Face\u2026 https:\/\/t.co\/yEiGj7mmbq","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":699344605888782341,"id_str":"699344605888782341","name":"VideoLeaks","screen_name":"WesleyRickard","location":"New York, NY","url":null,"description":"Bringing you the latest videos from across the United States for Election 2016. #Trump #Clinton #NYC (+ download 4) (Google Play Store) - Transfer to FB or IG","protected":false,"verified":false,"followers_count":16381,"friends_count":17906,"listed_count":164,"favourites_count":3546,"statuses_count":4821,"created_at":"Mon Feb 15 21:28:42 +0000 2016","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"F5F8FA","profile_background_image_url":"","profile_background_image_url_https":"","profile_background_tile":false,"profile_link_color":"2B7BB9","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/786029450047713280\/RYz1PuYg_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/786029450047713280\/RYz1PuYg_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/699344605888782341\/1476239103","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"extended_tweet":{"full_text":"10-16-2016 Donald Trump Rally 20,000 people ready to vote for Donald Trump on November 8th Retweet and Post to Facebook #PA #MI #NY #Trump https:\/\/t.co\/lK35z6NX5D","display_text_range":[0,138],"entities":{"hashtags":[{"text":"PA","indices":[120,123]},{"text":"MI","indices":[124,127]},{"text":"NY","indices":[128,131]},{"text":"Trump","indices":[132,138]}],"urls":[],"user_mentions":[],"symbols":[],"media":[{"id":787859892744888320,"id_str":"787859892744888320","indices":[139,162],"media_url":"http:\/\/pbs.twimg.com\/ext_tw_video_thumb\/787859892744888320\/pu\/img\/1qfAep82gJQV-HcA.jpg","media_url_https":"https:\/\/pbs.twimg.com\/ext_tw_video_thumb\/787859892744888320\/pu\/img\/1qfAep82gJQV-HcA.jpg","url":"https:\/\/t.co\/lK35z6NX5D","display_url":"pic.twitter.com\/lK35z6NX5D","expanded_url":"https:\/\/twitter.com\/WesleyRickard\/status\/787860136635359232\/video\/1","type":"video","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":600,"h":338,"resize":"fit"},"small":{"w":340,"h":191,"resize":"fit"},"large":{"w":1024,"h":576,"resize":"fit"}},"video_info":{"aspect_ratio":[16,9],"duration_millis":16343,"variants":[{"content_type":"application\/x-mpegURL","url":"https:\/\/video.twimg.com\/ext_tw_video\/787859892744888320\/pu\/pl\/L7hylf17HkCNL3Sw.m3u8"},{"bitrate":2176000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/ext_tw_video\/787859892744888320\/pu\/vid\/1280x720\/TYilOoYQ2P3CaOia.mp4"},{"content_type":"application\/dash+xml","url":"https:\/\/video.twimg.com\/ext_tw_video\/787859892744888320\/pu\/pl\/L7hylf17HkCNL3Sw.mpd"},{"bitrate":320000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/ext_tw_video\/787859892744888320\/pu\/vid\/320x180\/imrr4u7pCHdyotoy.mp4"},{"bitrate":832000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/ext_tw_video\/787859892744888320\/pu\/vid\/640x360\/0GhEhzEVZfjWG0RE.mp4"}]}}]},"extended_entities":{"media":[{"id":787859892744888320,"id_str":"787859892744888320","indices":[139,162],"media_url":"http:\/\/pbs.twimg.com\/ext_tw_video_thumb\/787859892744888320\/pu\/img\/1qfAep82gJQV-HcA.jpg","media_url_https":"https:\/\/pbs.twimg.com\/ext_tw_video_thumb\/787859892744888320\/pu\/img\/1qfAep82gJQV-HcA.jpg","url":"https:\/\/t.co\/lK35z6NX5D","display_url":"pic.twitter.com\/lK35z6NX5D","expanded_url":"https:\/\/twitter.com\/WesleyRickard\/status\/787860136635359232\/video\/1","type":"video","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":600,"h":338,"resize":"fit"},"small":{"w":340,"h":191,"resize":"fit"},"large":{"w":1024,"h":576,"resize":"fit"}},"video_info":{"aspect_ratio":[16,9],"duration_millis":16343,"variants":[{"content_type":"application\/x-mpegURL","url":"https:\/\/video.twimg.com\/ext_tw_video\/787859892744888320\/pu\/pl\/L7hylf17HkCNL3Sw.m3u8"},{"bitrate":2176000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/ext_tw_video\/787859892744888320\/pu\/vid\/1280x720\/TYilOoYQ2P3CaOia.mp4"},{"content_type":"application\/dash+xml","url":"https:\/\/video.twimg.com\/ext_tw_video\/787859892744888320\/pu\/pl\/L7hylf17HkCNL3Sw.mpd"},{"bitrate":320000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/ext_tw_video\/787859892744888320\/pu\/vid\/320x180\/imrr4u7pCHdyotoy.mp4"},{"bitrate":832000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/ext_tw_video\/787859892744888320\/pu\/vid\/640x360\/0GhEhzEVZfjWG0RE.mp4"}]}}]}},"retweet_count":199,"favorite_count":187,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/yEiGj7mmbq","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787860136635359232","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[117,140]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"","expanded_url":null,"indices":[136,136]}],"user_mentions":[{"screen_name":"WesleyRickard","name":"VideoLeaks","id":699344605888782341,"id_str":"699344605888782341","indices":[3,17]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419125"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985757365415936,"id_str":"787985757365415936","text":"RT @lukeoneil47: In his book 'Trump 101' \"When things go wrong look at yourself first. Don't instinctively blame others or the circu\u2026 ","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1167701558,"id_str":"1167701558","name":"Brendan Little","screen_name":"blittle86","location":null,"url":null,"description":"Policy Director at Mayor's Office of Recovery Services, City of Boston (@ORSBoston) | Singer-songwriter: http:\/\/brendanlittle.bandcamp.com","protected":false,"verified":false,"followers_count":834,"friends_count":1063,"listed_count":22,"favourites_count":1235,"statuses_count":1156,"created_at":"Mon Feb 11 03:20:15 +0000 2013","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/677150669057482752\/0KlvyoLI_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/677150669057482752\/0KlvyoLI_normal.jpg","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Sun Oct 16 14:05:52 +0000 2016","id":787655804761542656,"id_str":"787655804761542656","text":"In his book 'Trump 101' \"When things go wrong look at yourself first. Don't instinctively blame others or the circu\u2026 https:\/\/t.co\/j2hJpbTQRI","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":108338399,"id_str":"108338399","name":"luke o'neil","screen_name":"lukeoneil47","location":"Boston, MA","url":null,"description":"Writer-at-large @esquire. tips: loneil@hearst.com","protected":false,"verified":true,"followers_count":14871,"friends_count":1747,"listed_count":471,"favourites_count":38575,"statuses_count":71859,"created_at":"Mon Jan 25 16:40:01 +0000 2010","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"131516","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme14\/bg.gif","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme14\/bg.gif","profile_background_tile":true,"profile_link_color":"009999","profile_sidebar_border_color":"EEEEEE","profile_sidebar_fill_color":"EFEFEF","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/783710778260160512\/D1IWNLpj_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/783710778260160512\/D1IWNLpj_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/108338399\/1452280698","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"extended_tweet":{"full_text":"In his book 'Trump 101' \"When things go wrong look at yourself first. Don't instinctively blame others or the circumstances....\" https:\/\/t.co\/Hkx6ysx8Ct","display_text_range":[0,128],"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[],"media":[{"id":787655418898059264,"id_str":"787655418898059264","indices":[129,152],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu5QbijUkAAeM-O.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu5QbijUkAAeM-O.jpg","url":"https:\/\/t.co\/Hkx6ysx8Ct","display_url":"pic.twitter.com\/Hkx6ysx8Ct","expanded_url":"https:\/\/twitter.com\/lukeoneil47\/status\/787655804761542656\/photo\/1","type":"photo","sizes":{"large":{"w":664,"h":578,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":664,"h":578,"resize":"fit"},"small":{"w":664,"h":578,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787655418898059264,"id_str":"787655418898059264","indices":[129,152],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu5QbijUkAAeM-O.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu5QbijUkAAeM-O.jpg","url":"https:\/\/t.co\/Hkx6ysx8Ct","display_url":"pic.twitter.com\/Hkx6ysx8Ct","expanded_url":"https:\/\/twitter.com\/lukeoneil47\/status\/787655804761542656\/photo\/1","type":"photo","sizes":{"large":{"w":664,"h":578,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":664,"h":578,"resize":"fit"},"small":{"w":664,"h":578,"resize":"fit"}}}]}},"retweet_count":157,"favorite_count":354,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/j2hJpbTQRI","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787655804761542656","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[117,140]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"","expanded_url":null,"indices":[134,134]}],"user_mentions":[{"screen_name":"lukeoneil47","name":"luke o'neil","id":108338399,"id_str":"108338399","indices":[3,15]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419280"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985757419999236,"id_str":"787985757419999236","text":"RT @GeorgeTakei: Trump was too quick to blame Hillary for GOP office arson in NC. She strongly condemned the attack, and had zero logically\u2026","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":18770638,"id_str":"18770638","name":"(((Courtney Fay)))","screen_name":"misscrf","location":"Rochester, NY","url":null,"description":"Developer @ a law firm. Love my life, especially when there's coffee in the morning.","protected":false,"verified":false,"followers_count":217,"friends_count":847,"listed_count":32,"favourites_count":621,"statuses_count":8371,"created_at":"Thu Jan 08 17:20:49 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme7\/bg.gif","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme7\/bg.gif","profile_background_tile":false,"profile_link_color":"ABB8C2","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/735182941748400128\/TTkzMXAX_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/735182941748400128\/TTkzMXAX_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/18770638\/1476456847","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 02:40:15 +0000 2016","id":787845653074481153,"id_str":"787845653074481153","text":"Trump was too quick to blame Hillary for GOP office arson in NC. She strongly condemned the attack, and had zero logically to gain from it.","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":237845487,"id_str":"237845487","name":"George Takei","screen_name":"GeorgeTakei","location":"Broadway - New York, NY","url":"http:\/\/ohmyyy.gt\/GeorgeTakei","description":"Some know me as Mr. Sulu from Star Trek but I hope all know me as a believer in, and a fighter for, the equality and dignity of all human beings.","protected":false,"verified":true,"followers_count":1859686,"friends_count":485,"listed_count":20176,"favourites_count":1144,"statuses_count":23482,"created_at":"Thu Jan 13 19:33:56 +0000 2011","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"022330","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/479993169363218432\/YRp7xuMf.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/479993169363218432\/YRp7xuMf.jpeg","profile_background_tile":false,"profile_link_color":"ABB8C2","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/700355030117937152\/3uW9IKHD_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/700355030117937152\/3uW9IKHD_normal.png","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/237845487\/1474948080","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":691,"favorite_count":2629,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"GeorgeTakei","name":"George Takei","id":237845487,"id_str":"237845487","indices":[3,15]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419293"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985757550116864,"id_str":"787985757550116864","text":"RT @TrumpFever2016: DON'T BLAME TRUMP\n4= Civil UN-Rest\n4= EVERYTHING Wrong with USA\nTRUMP is OUR Last Hope\n@realDonaldTrump @JoeGruters htt\u2026","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":4895096885,"id_str":"4895096885","name":"Kim Meese","screen_name":"intheknow1798","location":"United States","url":null,"description":"Vote for Trump\/Pence!  Conservative Christian. Married. Supporter of NRA. Fight for America, Freedom, Security & Truth.  Porn will be blocked.","protected":false,"verified":false,"followers_count":4592,"friends_count":4483,"listed_count":40,"favourites_count":20010,"statuses_count":35732,"created_at":"Wed Feb 10 14:37:19 +0000 2016","utc_offset":-25200,"time_zone":"Pacific Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"F5F8FA","profile_background_image_url":"","profile_background_image_url_https":"","profile_background_tile":false,"profile_link_color":"2B7BB9","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/781819144287191040\/0EXdOtlw_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/781819144287191040\/0EXdOtlw_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/4895096885\/1455225517","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Thu Mar 17 21:19:43 +0000 2016","id":710576369638494209,"id_str":"710576369638494209","text":"DON'T BLAME TRUMP\n4= Civil UN-Rest\n4= EVERYTHING Wrong with USA\nTRUMP is OUR Last Hope\n@realDonaldTrump @JoeGruters https:\/\/t.co\/YNbuY0yYCX","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":2990789335,"id_str":"2990789335","name":"Trump 4 Women","screen_name":"TrumpFever2016","location":" DEPLORABLE America","url":"http:\/\/TrumpFever.US","description":"SEE @TheTRUMPetts #1 OFFICIAL TRUMP TRAIN Vid   #TeamTRUMP  \n HIT SONGWRITERS =USMC \/ LEO VETS","protected":false,"verified":false,"followers_count":12588,"friends_count":4055,"listed_count":144,"favourites_count":15116,"statuses_count":45186,"created_at":"Tue Jan 20 06:18:34 +0000 2015","utc_offset":-25200,"time_zone":"Pacific Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/695492228727201792\/svlvnAFE.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/695492228727201792\/svlvnAFE.jpg","profile_background_tile":true,"profile_link_color":"1B95E0","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/753333346357354497\/wHrXsp5x_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/753333346357354497\/wHrXsp5x_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/2990789335\/1455430399","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":{"id":"4ec01c9dbc693497","url":"https:\/\/api.twitter.com\/1.1\/geo\/id\/4ec01c9dbc693497.json","place_type":"admin","name":"Florida","full_name":"Florida, USA","country_code":"US","country":"United States","bounding_box":{"type":"Polygon","coordinates":[[[-87.634643,24.396308],[-87.634643,31.001056],[-79.974307,31.001056],[-79.974307,24.396308]]]},"attributes":{}},"contributors":null,"is_quote_status":false,"retweet_count":215,"favorite_count":215,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"realDonaldTrump","name":"Donald J. Trump","id":25073877,"id_str":"25073877","indices":[87,103]},{"screen_name":"JoeGruters","name":"Joe Gruters","id":40045792,"id_str":"40045792","indices":[104,115]}],"symbols":[],"media":[{"id":710576368921223169,"id_str":"710576368921223169","indices":[116,139],"media_url":"http:\/\/pbs.twimg.com\/media\/Cdx5cVdWIAEkqT3.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cdx5cVdWIAEkqT3.jpg","url":"https:\/\/t.co\/YNbuY0yYCX","display_url":"pic.twitter.com\/YNbuY0yYCX","expanded_url":"http:\/\/twitter.com\/chriskyleband\/status\/710576369638494209\/photo\/1","type":"photo","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":480,"h":372,"resize":"fit"},"large":{"w":480,"h":372,"resize":"fit"},"small":{"w":340,"h":264,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":710576368921223169,"id_str":"710576368921223169","indices":[116,139],"media_url":"http:\/\/pbs.twimg.com\/media\/Cdx5cVdWIAEkqT3.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cdx5cVdWIAEkqT3.jpg","url":"https:\/\/t.co\/YNbuY0yYCX","display_url":"pic.twitter.com\/YNbuY0yYCX","expanded_url":"http:\/\/twitter.com\/chriskyleband\/status\/710576369638494209\/photo\/1","type":"photo","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":480,"h":372,"resize":"fit"},"large":{"w":480,"h":372,"resize":"fit"},"small":{"w":340,"h":264,"resize":"fit"}}}]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"TrumpFever2016","name":"Trump 4 Women","id":2990789335,"id_str":"2990789335","indices":[3,18]},{"screen_name":"realDonaldTrump","name":"Donald J. Trump","id":25073877,"id_str":"25073877","indices":[107,123]},{"screen_name":"JoeGruters","name":"Joe Gruters","id":40045792,"id_str":"40045792","indices":[124,135]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419324"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985757742915584,"id_str":"787985757742915584","text":"RT @truthmashup: Trump TV, all Trump all day. What could go wrong? Btw, I'm sure Rupert Murdoch is loving this. #uspoli https:\/\/t.co\/9V3boc\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":2315874241,"id_str":"2315874241","name":"Robin","screen_name":"RobinBall1961","location":null,"url":null,"description":"Never Forget the #DecadeOfDarkness under #HarperGovt & #CPC! Don't let history repeat itself-keep #Tories out of Gov't. The Truth is Out There!","protected":false,"verified":false,"followers_count":1500,"friends_count":1051,"listed_count":761,"favourites_count":129514,"statuses_count":165753,"created_at":"Tue Jan 28 17:59:08 +0000 2014","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"9266CC","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/656304598198579200\/r62aVwpQ_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/656304598198579200\/r62aVwpQ_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/2315874241\/1453175663","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:25:08 +0000 2016","id":787977742432206848,"id_str":"787977742432206848","text":"Trump TV, all Trump all day. What could go wrong? Btw, I'm sure Rupert Murdoch is loving this. #uspoli https:\/\/t.co\/9V3bocNiXh","source":"\u003ca href=\"https:\/\/about.twitter.com\/products\/tweetdeck\" rel=\"nofollow\"\u003eTweetDeck\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":557449649,"id_str":"557449649","name":"Truth Mashup","screen_name":"truthmashup","location":"Toronto","url":"http:\/\/www.harperhistory.com","description":"Producer @VinceKes tweeting for @TYTNetwork's comedy channel. Hosted By @DanSpeerin! Tweets on #cdnpoli #uspoli and Donald Trump.","protected":false,"verified":false,"followers_count":2666,"friends_count":1990,"listed_count":166,"favourites_count":4069,"statuses_count":40505,"created_at":"Thu Apr 19 05:29:22 +0000 2012","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"131516","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme14\/bg.gif","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme14\/bg.gif","profile_background_tile":true,"profile_link_color":"009999","profile_sidebar_border_color":"EEEEEE","profile_sidebar_fill_color":"EFEFEF","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/730153519836094464\/S3EvKi1T_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/730153519836094464\/S3EvKi1T_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/557449649\/1446078916","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"quoted_status_id":787977369881575425,"quoted_status_id_str":"787977369881575425","quoted_status":{"created_at":"Mon Oct 17 11:23:39 +0000 2016","id":787977369881575425,"id_str":"787977369881575425","text":"Trump\u2019s son-in-law already working to set up Trump TV ahead of looming election defeat: report\u2026 https:\/\/t.co\/lL06xR8Is0","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/www.hootsuite.com\" rel=\"nofollow\"\u003eHootsuite\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":16041234,"id_str":"16041234","name":"Raw Story","screen_name":"RawStory","location":"Washington, DC","url":"http:\/\/www.rawstory.com","description":"The official Twitter of http:\/\/RawStory.com","protected":false,"verified":true,"followers_count":103869,"friends_count":2311,"listed_count":4126,"favourites_count":199,"statuses_count":122382,"created_at":"Fri Aug 29 12:29:21 +0000 2008","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/204237561\/rawreplaytwitterbackground.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/204237561\/rawreplaytwitterbackground.jpg","profile_background_tile":false,"profile_link_color":"000000","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"7D0B0B","profile_text_color":"000000","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/511012709270183936\/wEVYV7Lf_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/511012709270183936\/wEVYV7Lf_normal.png","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/16041234\/1472223971","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"extended_tweet":{"full_text":"Trump\u2019s son-in-law already working to set up Trump TV ahead of looming election defeat: report https:\/\/t.co\/D6vS0EpDGo https:\/\/t.co\/EVNCJbuiTy","display_text_range":[0,118],"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/D6vS0EpDGo","expanded_url":"http:\/\/ow.ly\/kOoA305fphQ","display_url":"ow.ly\/kOoA305fphQ","indices":[95,118]}],"user_mentions":[],"symbols":[],"media":[{"id":787977367276904448,"id_str":"787977367276904448","indices":[119,142],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu91PZuWYAAUFvU.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu91PZuWYAAUFvU.jpg","url":"https:\/\/t.co\/EVNCJbuiTy","display_url":"pic.twitter.com\/EVNCJbuiTy","expanded_url":"https:\/\/twitter.com\/RawStory\/status\/787977369881575425\/photo\/1","type":"photo","sizes":{"large":{"w":800,"h":430,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":680,"h":366,"resize":"fit"},"medium":{"w":800,"h":430,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787977367276904448,"id_str":"787977367276904448","indices":[119,142],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu91PZuWYAAUFvU.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu91PZuWYAAUFvU.jpg","url":"https:\/\/t.co\/EVNCJbuiTy","display_url":"pic.twitter.com\/EVNCJbuiTy","expanded_url":"https:\/\/twitter.com\/RawStory\/status\/787977369881575425\/photo\/1","type":"photo","sizes":{"large":{"w":800,"h":430,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":680,"h":366,"resize":"fit"},"medium":{"w":800,"h":430,"resize":"fit"}}}]}},"retweet_count":22,"favorite_count":10,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/lL06xR8Is0","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787977369881575425","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[96,119]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":2,"favorite_count":1,"entities":{"hashtags":[{"text":"uspoli","indices":[95,102]}],"urls":[{"url":"https:\/\/t.co\/9V3bocNiXh","expanded_url":"https:\/\/twitter.com\/RawStory\/status\/787977369881575425","display_url":"twitter.com\/RawStory\/statu\u2026","indices":[103,126]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"quoted_status_id":787977369881575425,"quoted_status_id_str":"787977369881575425","quoted_status":{"created_at":"Mon Oct 17 11:23:39 +0000 2016","id":787977369881575425,"id_str":"787977369881575425","text":"Trump\u2019s son-in-law already working to set up Trump TV ahead of looming election defeat: report\u2026 https:\/\/t.co\/lL06xR8Is0","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/www.hootsuite.com\" rel=\"nofollow\"\u003eHootsuite\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":16041234,"id_str":"16041234","name":"Raw Story","screen_name":"RawStory","location":"Washington, DC","url":"http:\/\/www.rawstory.com","description":"The official Twitter of http:\/\/RawStory.com","protected":false,"verified":true,"followers_count":103869,"friends_count":2311,"listed_count":4126,"favourites_count":199,"statuses_count":122382,"created_at":"Fri Aug 29 12:29:21 +0000 2008","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/204237561\/rawreplaytwitterbackground.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/204237561\/rawreplaytwitterbackground.jpg","profile_background_tile":false,"profile_link_color":"000000","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"7D0B0B","profile_text_color":"000000","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/511012709270183936\/wEVYV7Lf_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/511012709270183936\/wEVYV7Lf_normal.png","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/16041234\/1472223971","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"extended_tweet":{"full_text":"Trump\u2019s son-in-law already working to set up Trump TV ahead of looming election defeat: report https:\/\/t.co\/D6vS0EpDGo https:\/\/t.co\/EVNCJbuiTy","display_text_range":[0,118],"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/D6vS0EpDGo","expanded_url":"http:\/\/ow.ly\/kOoA305fphQ","display_url":"ow.ly\/kOoA305fphQ","indices":[95,118]}],"user_mentions":[],"symbols":[],"media":[{"id":787977367276904448,"id_str":"787977367276904448","indices":[119,142],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu91PZuWYAAUFvU.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu91PZuWYAAUFvU.jpg","url":"https:\/\/t.co\/EVNCJbuiTy","display_url":"pic.twitter.com\/EVNCJbuiTy","expanded_url":"https:\/\/twitter.com\/RawStory\/status\/787977369881575425\/photo\/1","type":"photo","sizes":{"large":{"w":800,"h":430,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":680,"h":366,"resize":"fit"},"medium":{"w":800,"h":430,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787977367276904448,"id_str":"787977367276904448","indices":[119,142],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu91PZuWYAAUFvU.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu91PZuWYAAUFvU.jpg","url":"https:\/\/t.co\/EVNCJbuiTy","display_url":"pic.twitter.com\/EVNCJbuiTy","expanded_url":"https:\/\/twitter.com\/RawStory\/status\/787977369881575425\/photo\/1","type":"photo","sizes":{"large":{"w":800,"h":430,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":680,"h":366,"resize":"fit"},"medium":{"w":800,"h":430,"resize":"fit"}}}]}},"retweet_count":22,"favorite_count":10,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/lL06xR8Is0","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787977369881575425","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[96,119]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[{"text":"uspoli","indices":[112,119]}],"urls":[],"user_mentions":[{"screen_name":"truthmashup","name":"Truth Mashup","id":557449649,"id_str":"557449649","indices":[3,15]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419370"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985757180866560,"id_str":"787985757180866560","text":"RT @JackPosobiec: \"The US is creating an enemy out of Russsia to distract from domestic problems in an election season\" - Putin","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":226565178,"id_str":"226565178","name":"DeplorablyIndivisibl","screen_name":"Stormenight","location":"USA ","url":null,"description":"I'm about life liberty & happiness.Sick o party politics&MSM BS.I understand freewill and self-responsibility so no victim mentality here & no fear. #MAGA","protected":false,"verified":false,"followers_count":629,"friends_count":806,"listed_count":10,"favourites_count":4816,"statuses_count":9959,"created_at":"Tue Dec 14 14:01:14 +0000 2010","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"022330","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/407303694\/snglobe.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/407303694\/snglobe.jpg","profile_background_tile":true,"profile_link_color":"0084B4","profile_sidebar_border_color":"A8C7F7","profile_sidebar_fill_color":"C0DFEC","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/675173584315260934\/o-X20FBv_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/675173584315260934\/o-X20FBv_normal.jpg","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:56:12 +0000 2016","id":787985562049273857,"id_str":"787985562049273857","text":"\"The US is creating an enemy out of Russsia to distract from domestic problems in an election season\" - Putin","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":592730371,"id_str":"592730371","name":"Jack Posobiec","screen_name":"JackPosobiec","location":"Washington, DC","url":"http:\/\/CitizensForTrump.com","description":"Special Projects Director of @Citizens4Trump. Recovering political operative. Veteran. \u4e2d\u6587","protected":false,"verified":false,"followers_count":29406,"friends_count":17056,"listed_count":349,"favourites_count":31085,"statuses_count":64988,"created_at":"Mon May 28 13:31:22 +0000 2012","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/862971572\/fb7aadcfd66ea5e4cad749bf1e4412de.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/862971572\/fb7aadcfd66ea5e4cad749bf1e4412de.jpeg","profile_background_tile":true,"profile_link_color":"1B95E0","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/779488773126193156\/3uVx_qBy_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/779488773126193156\/3uVx_qBy_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/592730371\/1474771459","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":12,"favorite_count":7,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"JackPosobiec","name":"Jack Posobiec","id":592730371,"id_str":"592730371","indices":[3,16]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419236"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985757747093504,"id_str":"787985757747093504","text":"Help me to win #galaxynote7 and by the way poop on #trump or #clinton ;) https:\/\/t.co\/7Nmef5Ce5E","source":"\u003ca href=\"https:\/\/gleam.io\" rel=\"nofollow\"\u003eGleam Competition App\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":4887143782,"id_str":"4887143782","name":"RKgem","screen_name":"CrispTechRK","location":"Mumbai, India","url":null,"description":null,"protected":false,"verified":false,"followers_count":59,"friends_count":577,"listed_count":20,"favourites_count":2276,"statuses_count":4496,"created_at":"Mon Feb 08 06:39:55 +0000 2016","utc_offset":19800,"time_zone":"Mumbai","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"F5F8FA","profile_background_image_url":"","profile_background_image_url_https":"","profile_background_tile":false,"profile_link_color":"2B7BB9","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/697981705244676096\/iGJ5USn1_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/697981705244676096\/iGJ5USn1_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/4887143782\/1455097584","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[{"text":"galaxynote7","indices":[15,27]},{"text":"trump","indices":[51,57]},{"text":"clinton","indices":[61,69]}],"urls":[{"url":"https:\/\/t.co\/7Nmef5Ce5E","expanded_url":"https:\/\/play.google.com\/store\/apps\/details?id=com.ZeroSenseGaming.PoopOfJustice","display_url":"play.google.com\/store\/apps\/det\u2026","indices":[73,96]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419371"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985757491388420,"id_str":"787985757491388420","text":"RT @MrWillRitter: And there it is, the long con. \nThis national nightmare was a promo campaign to sell us red hats &amp; Trump TV. https:\/\/t.co\u2026","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":21167820,"id_str":"21167820","name":"pgh-Tos","screen_name":"pghtos","location":"Pittsburgh","url":null,"description":null,"protected":false,"verified":false,"followers_count":46,"friends_count":281,"listed_count":1,"favourites_count":321,"statuses_count":1988,"created_at":"Wed Feb 18 04:12:57 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/786401302448312320\/F-OVkagb_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/786401302448312320\/F-OVkagb_normal.jpg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:17:41 +0000 2016","id":787975868106833924,"id_str":"787975868106833924","text":"And there it is, the long con. \nThis national nightmare was a promo campaign to sell us red hats &amp; Trump TV. https:\/\/t.co\/rJMNiYxMRd","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":247421834,"id_str":"247421834","name":"Will Ritter","screen_name":"MrWillRitter","location":"RVA by way of Boston","url":"http:\/\/poolhouse.co","description":"Co-Founder, @PoolhouseAgency. Political 'professional'. Romney\/Ryan, Rubio, etc alum. RTs are accidents.","protected":false,"verified":false,"followers_count":3337,"friends_count":998,"listed_count":107,"favourites_count":8673,"statuses_count":6606,"created_at":"Fri Feb 04 19:24:05 +0000 2011","utc_offset":-10800,"time_zone":"Atlantic Time (Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/641702567534137344\/7_d10VsZ_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/641702567534137344\/7_d10VsZ_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/247421834\/1470729681","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"quoted_status_id":787968740927369216,"quoted_status_id_str":"787968740927369216","quoted_status":{"created_at":"Mon Oct 17 10:49:22 +0000 2016","id":787968740927369216,"id_str":"787968740927369216","text":"EXCLUSIVE: Donald Trump's son-in-law has held talks to set up Trump TV after the presidential election\u2026 https:\/\/t.co\/rkmJmBBUIN","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/www.socialflow.com\" rel=\"nofollow\"\u003eSocialFlow\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":18949452,"id_str":"18949452","name":"Financial Times","screen_name":"FT","location":"London","url":"http:\/\/www.ft.com\/","description":"News stories, features and updates from the FT. For headlines follow @financialtimes. Our customer service team is @FTcare.","protected":false,"verified":true,"followers_count":2508217,"friends_count":714,"listed_count":30109,"favourites_count":18,"statuses_count":138311,"created_at":"Tue Jan 13 19:28:24 +0000 2009","utc_offset":3600,"time_zone":"London","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"FFF1E0","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/234732899\/ft-twitter-back.gif","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/234732899\/ft-twitter-back.gif","profile_background_tile":false,"profile_link_color":"9E2F50","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"E9DECF","profile_text_color":"000000","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/466972537704824832\/eflEColL_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/466972537704824832\/eflEColL_normal.png","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/18949452\/1462794252","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"extended_tweet":{"full_text":"EXCLUSIVE: Donald Trump's son-in-law has held talks to set up Trump TV after the presidential election https:\/\/t.co\/fTiqGNFF9q https:\/\/t.co\/xtbWPpBqsd","display_text_range":[0,126],"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/fTiqGNFF9q","expanded_url":"http:\/\/on.ft.com\/2dhOAqV","display_url":"on.ft.com\/2dhOAqV","indices":[103,126]}],"user_mentions":[],"symbols":[],"media":[{"id":787968738414956544,"id_str":"787968738414956544","indices":[127,150],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu9tZItWEAA-aTy.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu9tZItWEAA-aTy.jpg","url":"https:\/\/t.co\/xtbWPpBqsd","display_url":"pic.twitter.com\/xtbWPpBqsd","expanded_url":"https:\/\/twitter.com\/FT\/status\/787968740927369216\/photo\/1","type":"photo","sizes":{"small":{"w":680,"h":383,"resize":"fit"},"large":{"w":700,"h":394,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":700,"h":394,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787968738414956544,"id_str":"787968738414956544","indices":[127,150],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu9tZItWEAA-aTy.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu9tZItWEAA-aTy.jpg","url":"https:\/\/t.co\/xtbWPpBqsd","display_url":"pic.twitter.com\/xtbWPpBqsd","expanded_url":"https:\/\/twitter.com\/FT\/status\/787968740927369216\/photo\/1","type":"photo","sizes":{"small":{"w":680,"h":383,"resize":"fit"},"large":{"w":700,"h":394,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":700,"h":394,"resize":"fit"}}}]}},"retweet_count":316,"favorite_count":118,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/rkmJmBBUIN","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787968740927369216","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[104,127]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":14,"favorite_count":24,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/rJMNiYxMRd","expanded_url":"https:\/\/twitter.com\/ft\/status\/787968740927369216","display_url":"twitter.com\/ft\/status\/7879\u2026","indices":[113,136]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"quoted_status_id":787968740927369216,"quoted_status_id_str":"787968740927369216","quoted_status":{"created_at":"Mon Oct 17 10:49:22 +0000 2016","id":787968740927369216,"id_str":"787968740927369216","text":"EXCLUSIVE: Donald Trump's son-in-law has held talks to set up Trump TV after the presidential election\u2026 https:\/\/t.co\/rkmJmBBUIN","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/www.socialflow.com\" rel=\"nofollow\"\u003eSocialFlow\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":18949452,"id_str":"18949452","name":"Financial Times","screen_name":"FT","location":"London","url":"http:\/\/www.ft.com\/","description":"News stories, features and updates from the FT. For headlines follow @financialtimes. Our customer service team is @FTcare.","protected":false,"verified":true,"followers_count":2508217,"friends_count":714,"listed_count":30109,"favourites_count":18,"statuses_count":138311,"created_at":"Tue Jan 13 19:28:24 +0000 2009","utc_offset":3600,"time_zone":"London","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"FFF1E0","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/234732899\/ft-twitter-back.gif","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/234732899\/ft-twitter-back.gif","profile_background_tile":false,"profile_link_color":"9E2F50","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"E9DECF","profile_text_color":"000000","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/466972537704824832\/eflEColL_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/466972537704824832\/eflEColL_normal.png","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/18949452\/1462794252","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"extended_tweet":{"full_text":"EXCLUSIVE: Donald Trump's son-in-law has held talks to set up Trump TV after the presidential election https:\/\/t.co\/fTiqGNFF9q https:\/\/t.co\/xtbWPpBqsd","display_text_range":[0,126],"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/fTiqGNFF9q","expanded_url":"http:\/\/on.ft.com\/2dhOAqV","display_url":"on.ft.com\/2dhOAqV","indices":[103,126]}],"user_mentions":[],"symbols":[],"media":[{"id":787968738414956544,"id_str":"787968738414956544","indices":[127,150],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu9tZItWEAA-aTy.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu9tZItWEAA-aTy.jpg","url":"https:\/\/t.co\/xtbWPpBqsd","display_url":"pic.twitter.com\/xtbWPpBqsd","expanded_url":"https:\/\/twitter.com\/FT\/status\/787968740927369216\/photo\/1","type":"photo","sizes":{"small":{"w":680,"h":383,"resize":"fit"},"large":{"w":700,"h":394,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":700,"h":394,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787968738414956544,"id_str":"787968738414956544","indices":[127,150],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu9tZItWEAA-aTy.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu9tZItWEAA-aTy.jpg","url":"https:\/\/t.co\/xtbWPpBqsd","display_url":"pic.twitter.com\/xtbWPpBqsd","expanded_url":"https:\/\/twitter.com\/FT\/status\/787968740927369216\/photo\/1","type":"photo","sizes":{"small":{"w":680,"h":383,"resize":"fit"},"large":{"w":700,"h":394,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":700,"h":394,"resize":"fit"}}}]}},"retweet_count":316,"favorite_count":118,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/rkmJmBBUIN","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787968740927369216","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[104,127]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"MrWillRitter","name":"Will Ritter","id":247421834,"id_str":"247421834","indices":[3,16]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419310"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985758074368000,"id_str":"787985758074368000","text":"RT @Democrat4LifeTX: Democrats Put Trump To Shame By Fundraising To Help GOP Reopen Firebombed NC County Office via @politicususa https:\/\/t\u2026","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":2387648143,"id_str":"2387648143","name":"Patrick","screen_name":"Pat1036","location":null,"url":null,"description":null,"protected":false,"verified":false,"followers_count":443,"friends_count":911,"listed_count":22,"favourites_count":2272,"statuses_count":12786,"created_at":"Thu Mar 13 18:30:10 +0000 2014","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"1B95E0","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/754630699014119425\/_9dBOPYX_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/754630699014119425\/_9dBOPYX_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/2387648143\/1457200330","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 03:43:24 +0000 2016","id":787861542406529024,"id_str":"787861542406529024","text":"Democrats Put Trump To Shame By Fundraising To Help GOP Reopen Firebombed NC County Office via @politicususa https:\/\/t.co\/TYdkfAUv2J","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":4800796580,"id_str":"4800796580","name":"Real Demos 4 Hillary","screen_name":"Democrat4LifeTX","location":"Austin, TX","url":"http:\/\/www.hillaryclinton.com","description":"Teacher\/Admin for 30 Years, Writer and Owner of an Educational Resource Brand, Wife of a US Army Colonel, Democrat for LIfe!","protected":false,"verified":false,"followers_count":2611,"friends_count":3078,"listed_count":55,"favourites_count":12740,"statuses_count":21901,"created_at":"Sat Jan 23 02:13:50 +0000 2016","utc_offset":-25200,"time_zone":"Pacific Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"981CEB","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/766683659889881088\/Mi2RUeLA_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/766683659889881088\/Mi2RUeLA_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/4800796580\/1472730617","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":29,"favorite_count":32,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/TYdkfAUv2J","expanded_url":"http:\/\/www.politicususa.com\/2016\/10\/16\/democrats-fundraise-gop-reopen-firebombed-north-carolina-county-office.html","display_url":"politicususa.com\/2016\/10\/16\/dem\u2026","indices":[109,132]}],"user_mentions":[{"screen_name":"politicususa","name":"PoliticusUSA","id":14792049,"id_str":"14792049","indices":[95,108]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"Democrat4LifeTX","name":"Real Demos 4 Hillary","id":4800796580,"id_str":"4800796580","indices":[3,19]},{"screen_name":"politicususa","name":"PoliticusUSA","id":14792049,"id_str":"14792049","indices":[116,129]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419449"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985758015467520,"id_str":"787985758015467520","text":"Being Muslim and American in the year of Donald Trump: Muslim Americans describe the September 11, 2001, terr... https:\/\/t.co\/208YVcnLBB","source":"\u003ca href=\"http:\/\/twitterfeed.com\" rel=\"nofollow\"\u003etwitterfeed\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1705270981,"id_str":"1705270981","name":"aalap metha","screen_name":"aalapm06","location":null,"url":null,"description":null,"protected":false,"verified":false,"followers_count":28,"friends_count":31,"listed_count":4,"favourites_count":0,"statuses_count":20543,"created_at":"Tue Aug 27 17:51:08 +0000 2013","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_1_normal.png","profile_image_url_https":"https:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_1_normal.png","default_profile":true,"default_profile_image":true,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/208YVcnLBB","expanded_url":"http:\/\/cnn.it\/2eleo1w","display_url":"cnn.it\/2eleo1w","indices":[113,136]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419435"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985758485372928,"id_str":"787985758485372928","text":"RT @LouDobbs: Silence of GOP in the Face of Clinton Cartel's Corruption is Acquiescence to Greater Corruption  #MAGA #TrumpPence16 #TrumpTR\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":4766281718,"id_str":"4766281718","name":"scott","screen_name":"scott006648","location":null,"url":null,"description":null,"protected":false,"verified":false,"followers_count":130,"friends_count":116,"listed_count":22,"favourites_count":256,"statuses_count":7187,"created_at":"Sat Jan 16 03:36:54 +0000 2016","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"F5F8FA","profile_background_image_url":"","profile_background_image_url_https":"","profile_background_tile":false,"profile_link_color":"2B7BB9","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_2_normal.png","profile_image_url_https":"https:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_2_normal.png","default_profile":true,"default_profile_image":true,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:40:16 +0000 2016","id":787981551518806017,"id_str":"787981551518806017","text":"Silence of GOP in the Face of Clinton Cartel's Corruption is Acquiescence to Greater Corruption  #MAGA #TrumpPence16 #TrumpTRain #Dobbs","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":26487169,"id_str":"26487169","name":"Lou Dobbs","screen_name":"LouDobbs","location":"New York, NY","url":"http:\/\/loudobbs.com","description":"Lou Dobbs Tonight, Fox Business Network, 7 & 11 pm    \nIG: http:\/\/instagram.com\/loudobbstonight\/","protected":false,"verified":true,"followers_count":809353,"friends_count":2477,"listed_count":3121,"favourites_count":3574,"statuses_count":23721,"created_at":"Wed Mar 25 12:39:59 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/575734940228190209\/iXHsvsM7.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/575734940228190209\/iXHsvsM7.jpeg","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/663851941571641344\/OqqkE56l_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/663851941571641344\/OqqkE56l_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/26487169\/1426100831","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":223,"favorite_count":278,"entities":{"hashtags":[{"text":"MAGA","indices":[97,102]},{"text":"TrumpPence16","indices":[103,116]},{"text":"TrumpTRain","indices":[117,128]},{"text":"Dobbs","indices":[129,135]}],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[{"text":"MAGA","indices":[111,116]},{"text":"TrumpPence16","indices":[117,130]}],"urls":[],"user_mentions":[{"screen_name":"LouDobbs","name":"Lou Dobbs","id":26487169,"id_str":"26487169","indices":[3,12]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419547"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985758846005249,"id_str":"787985758846005249","text":"RT @activist360: IT'S TIME TO TELL THE TRUTH: Trump isn\u2019t running a presidential campaign, he\u2019s leading a white nationalist hate cult https\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":772053764022820864,"id_str":"772053764022820864","name":"Butterflycu2","screen_name":"butterflyn2u","location":"Oxford, GA","url":null,"description":"Southern educated, witted and politically savvy female residing in GA. Interested in intelligent conversation and feedback to advance differing points of views.","protected":false,"verified":false,"followers_count":73,"friends_count":119,"listed_count":3,"favourites_count":1628,"statuses_count":1659,"created_at":"Sat Sep 03 12:48:56 +0000 2016","utc_offset":null,"time_zone":null,"geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"F5F8FA","profile_background_image_url":"","profile_background_image_url_https":"","profile_background_tile":false,"profile_link_color":"2B7BB9","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/772448314679844865\/6Ks4ReGc_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/772448314679844865\/6Ks4ReGc_normal.jpg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 02:25:06 +0000 2016","id":787841838954778625,"id_str":"787841838954778625","text":"IT'S TIME TO TELL THE TRUTH: Trump isn\u2019t running a presidential campaign, he\u2019s leading a white nationalist hate cult https:\/\/t.co\/MlrYlOE1ld","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":349803020,"id_str":"349803020","name":"Bill Madden","screen_name":"activist360","location":"Lower East Side, New York City","url":"http:\/\/about.me\/bill.madden","description":"Singer-songwriter, musician, activist, poet, yogi, fierce paladin for social justice and the environment. NEW RELIGION is available at https:\/\/t.co\/2h5mwsMcV7","protected":false,"verified":false,"followers_count":51933,"friends_count":51025,"listed_count":727,"favourites_count":2686,"statuses_count":23688,"created_at":"Sat Aug 06 18:34:39 +0000 2011","utc_offset":-18000,"time_zone":"Quito","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"CCCC99","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"B32025","profile_sidebar_border_color":"999966","profile_sidebar_fill_color":"CCCC99","profile_text_color":"333333","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/659427646540001280\/1sIxdKlD_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/659427646540001280\/1sIxdKlD_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/349803020\/1446148252","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":452,"favorite_count":750,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/MlrYlOE1ld","expanded_url":"http:\/\/bit.ly\/2eHgddb","display_url":"bit.ly\/2eHgddb","indices":[117,140]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"activist360","name":"Bill Madden","id":349803020,"id_str":"349803020","indices":[3,15]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419633"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985758577627137,"id_str":"787985758577627137","text":"RT @ThePlumLineGS: Our polling has found that half of Trump supporters are inclined to agree that the election will be rigged: https:\/\/t.co\u2026","source":"\u003ca href=\"https:\/\/about.twitter.com\/products\/tweetdeck\" rel=\"nofollow\"\u003eTweetDeck\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":19087651,"id_str":"19087651","name":"Greg Dworkin","screen_name":"DemFromCT","location":"Newtown, CT","url":"http:\/\/dailykos.com","description":"Contributing editor, Daily Kos. I'm a doctor who tweets about politics, and sometimes, being a doctor. Personal account, reflecting only my own view.","protected":false,"verified":false,"followers_count":9418,"friends_count":1108,"listed_count":494,"favourites_count":27184,"statuses_count":171658,"created_at":"Fri Jan 16 21:59:39 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"9AE4E8","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/6518208\/KosTwitterBG.png","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/6518208\/KosTwitterBG.png","profile_background_tile":true,"profile_link_color":"0084B4","profile_sidebar_border_color":"BDDCAD","profile_sidebar_fill_color":"DDFFCC","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/412221142040985600\/rfwvOE-u_normal.jpeg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/412221142040985600\/rfwvOE-u_normal.jpeg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/19087651\/1398375050","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:56:46 +0000 2016","id":787985704571793408,"id_str":"787985704571793408","text":"Our polling has found that half of Trump supporters are inclined to agree that the election will be rigged: https:\/\/t.co\/fEmIa046WJ","source":"\u003ca href=\"https:\/\/about.twitter.com\/products\/tweetdeck\" rel=\"nofollow\"\u003eTweetDeck\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":20508720,"id_str":"20508720","name":"Greg Sargent","screen_name":"ThePlumLineGS","location":null,"url":"https:\/\/www.washingtonpost.com\/blogs\/plum-line\/","description":"A blog about politics, politics, and politics","protected":false,"verified":true,"followers_count":107797,"friends_count":3405,"listed_count":4196,"favourites_count":1420,"statuses_count":103655,"created_at":"Tue Feb 10 12:04:17 +0000 2009","utc_offset":-18000,"time_zone":"Central Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/77712921\/greg_spic_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/77712921\/greg_spic_normal.jpg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":1,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/fEmIa046WJ","expanded_url":"https:\/\/www.washingtonpost.com\/blogs\/plum-line\/wp\/2016\/09\/15\/donald-trump-says-the-election-will-be-rigged-half-of-his-supporters-seem-inclined-to-agree\/","display_url":"washingtonpost.com\/blogs\/plum-lin\u2026","indices":[108,131]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"ThePlumLineGS","name":"Greg Sargent","id":20508720,"id_str":"20508720","indices":[3,17]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419569"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985759785529344,"id_str":"787985759785529344","text":"RT @mitchellvii: CBS knows Hillary leads with degreed voters while Trump has a huge lead with blue collar.  So they oversample degreed by 2\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/#!\/download\/ipad\" rel=\"nofollow\"\u003eTwitter for iPad\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":3308721355,"id_str":"3308721355","name":"Nikki","screen_name":"nikki20152014","location":"United States of America","url":null,"description":"PATRIOTS VOTE NOV 8! I UNFOLLOW those WHO Do Not Follow ME BACK. #ClintonScandals #PENCE #veterans #WakeUpAmerica  \ud83d\udd34MAGA","protected":false,"verified":false,"followers_count":94,"friends_count":158,"listed_count":0,"favourites_count":559,"statuses_count":2675,"created_at":"Fri Aug 07 10:59:32 +0000 2015","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/784021553490964482\/mfLoegGK_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/784021553490964482\/mfLoegGK_normal.jpg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:56:46 +0000 2016","id":787985705259634688,"id_str":"787985705259634688","text":"CBS knows Hillary leads with degreed voters while Trump has a huge lead with blue collar.  So they oversample degreed by 20%.","source":"\u003ca href=\"https:\/\/about.twitter.com\/products\/tweetdeck\" rel=\"nofollow\"\u003eTweetDeck\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":17980523,"id_str":"17980523","name":"Bill Mitchell","screen_name":"mitchellvii","location":"Charlotte, NC","url":"https:\/\/www.yourvoiceradio.com","description":"Tune in to YourVoice\u2122 Radio, Spreaker's #1 Political Talk Show at https:\/\/yourvoiceradio.com\/,  Sun-Thu at 8pm EST!  24\/7 Hotline 704-490-4520.","protected":false,"verified":true,"followers_count":96175,"friends_count":8186,"listed_count":1066,"favourites_count":13917,"statuses_count":69860,"created_at":"Tue Dec 09 01:54:21 +0000 2008","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"ABB8C2","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/754052443491696640\/4wI9t0qV_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/754052443491696640\/4wI9t0qV_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/17980523\/1470932562","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":1,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"mitchellvii","name":"Bill Mitchell","id":17980523,"id_str":"17980523","indices":[3,15]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419857"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985759521402880,"id_str":"787985759521402880","text":"RT @WeNeedTrump: GOOD MORNING TRUMP TRAIN. Start this week off strong. Keep fighting to fix this country.\n\n#MakeAmericaGreatAgain https:\/\/t\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":134126795,"id_str":"134126795","name":"Bob Scheiner","screen_name":"fleetbob1","location":"Union, NJ","url":null,"description":"only one choice for President\nDonald J Trump","protected":false,"verified":false,"followers_count":249,"friends_count":194,"listed_count":26,"favourites_count":8445,"statuses_count":6580,"created_at":"Sat Apr 17 14:33:35 +0000 2010","utc_offset":-14400,"time_zone":"America\/New_York","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/653210346921005057\/PeqauN_0_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/653210346921005057\/PeqauN_0_normal.jpg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:06:00 +0000 2016","id":787972927383801856,"id_str":"787972927383801856","text":"GOOD MORNING TRUMP TRAIN. Start this week off strong. Keep fighting to fix this country.\n\n#MakeAmericaGreatAgain https:\/\/t.co\/zeXnkl6qtd","display_text_range":[0,112],"source":"\u003ca href=\"https:\/\/about.twitter.com\/products\/tweetdeck\" rel=\"nofollow\"\u003eTweetDeck\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":3284112337,"id_str":"3284112337","name":"We Need Trump","screen_name":"WeNeedTrump","location":"United States of America ","url":"http:\/\/donaldjtrump.com","description":"The best TRUMP account on Twitter! America has enormous potential under a real leader. Following the campaign up to the minute.","protected":false,"verified":false,"followers_count":78673,"friends_count":57729,"listed_count":298,"favourites_count":3075,"statuses_count":5217,"created_at":"Sun Jul 19 06:38:28 +0000 2015","utc_offset":-18000,"time_zone":"Central Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"253885","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/622657650635898880\/sRnv9Ilr_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/622657650635898880\/sRnv9Ilr_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/3284112337\/1468134358","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":117,"favorite_count":127,"entities":{"hashtags":[{"text":"MakeAmericaGreatAgain","indices":[90,112]}],"urls":[],"user_mentions":[],"symbols":[],"media":[{"id":787816950185373696,"id_str":"787816950185373696","indices":[113,136],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu7jV5fUsAAIm5-.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu7jV5fUsAAIm5-.jpg","url":"https:\/\/t.co\/zeXnkl6qtd","display_url":"pic.twitter.com\/zeXnkl6qtd","expanded_url":"https:\/\/twitter.com\/WeNeedTrump\/status\/787972927383801856\/photo\/1","type":"photo","sizes":{"medium":{"w":960,"h":540,"resize":"fit"},"small":{"w":680,"h":383,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"large":{"w":960,"h":540,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787816950185373696,"id_str":"787816950185373696","indices":[113,136],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu7jV5fUsAAIm5-.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu7jV5fUsAAIm5-.jpg","url":"https:\/\/t.co\/zeXnkl6qtd","display_url":"pic.twitter.com\/zeXnkl6qtd","expanded_url":"https:\/\/twitter.com\/WeNeedTrump\/status\/787972927383801856\/photo\/1","type":"photo","sizes":{"medium":{"w":960,"h":540,"resize":"fit"},"small":{"w":680,"h":383,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"large":{"w":960,"h":540,"resize":"fit"}}}]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[{"text":"MakeAmericaGreatAgain","indices":[107,129]}],"urls":[],"user_mentions":[{"screen_name":"WeNeedTrump","name":"We Need Trump","id":3284112337,"id_str":"3284112337","indices":[3,15]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419794"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985759810617345,"id_str":"787985759810617345","text":"RT @TweetBrettMac: Who blackmails a decorated veteran and forces her to resign? Hillary Clinton.\n\nBut she's not a gold star Muslim so\u2026 ","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":778918696718774272,"id_str":"778918696718774272","name":"writeonright1\ud83d\udc38","screen_name":"DeplorableDJT","location":"NC","url":null,"description":"American Nationalist. Free Speech Activist. 82 AirBorne. Author #FightForTheWest #MAGA Googles are tools of Skypes","protected":false,"verified":false,"followers_count":638,"friends_count":895,"listed_count":12,"favourites_count":12619,"statuses_count":6164,"created_at":"Thu Sep 22 11:27:43 +0000 2016","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"F5F8FA","profile_background_image_url":"","profile_background_image_url_https":"","profile_background_tile":false,"profile_link_color":"2B7BB9","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/784133781749899264\/-mJ9S3rD_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/784133781749899264\/-mJ9S3rD_normal.jpg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Sat Oct 15 23:10:06 +0000 2016","id":787430379020029953,"id_str":"787430379020029953","text":"Who blackmails a decorated veteran and forces her to resign? Hillary Clinton.\n\nBut she's not a gold star Muslim so\u2026 https:\/\/t.co\/urJILUlPfj","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":305211380,"id_str":"305211380","name":"Brett Mac","screen_name":"TweetBrettMac","location":"Liberty's Cradle and Grave","url":"http:\/\/www.politicalwave.org","description":"Writes for AFF and RWT. \n\nTips? EMAIL: tweetbrettmac[at]Gmail[.]com\n\n#ReplaceTheMainstream","protected":false,"verified":false,"followers_count":13630,"friends_count":8025,"listed_count":115,"favourites_count":11586,"statuses_count":24739,"created_at":"Wed May 25 20:48:01 +0000 2011","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"3B5980","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/761023527696814080\/T0-WPOqz_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/761023527696814080\/T0-WPOqz_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/305211380\/1454875589","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"extended_tweet":{"full_text":"Who blackmails a decorated veteran and forces her to resign? Hillary Clinton.\n\nBut she's not a gold star Muslim so you won't hear about it. https:\/\/t.co\/5JjaFZfm4z","display_text_range":[0,139],"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[],"media":[{"id":787430368681005056,"id_str":"787430368681005056","indices":[140,163],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu2Dv5KWEAAFhou.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu2Dv5KWEAAFhou.jpg","url":"https:\/\/t.co\/5JjaFZfm4z","display_url":"pic.twitter.com\/5JjaFZfm4z","expanded_url":"https:\/\/twitter.com\/TweetBrettMac\/status\/787430379020029953\/photo\/1","type":"photo","sizes":{"small":{"w":494,"h":680,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":871,"h":1200,"resize":"fit"},"large":{"w":1440,"h":1984,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787430368681005056,"id_str":"787430368681005056","indices":[140,163],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu2Dv5KWEAAFhou.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu2Dv5KWEAAFhou.jpg","url":"https:\/\/t.co\/5JjaFZfm4z","display_url":"pic.twitter.com\/5JjaFZfm4z","expanded_url":"https:\/\/twitter.com\/TweetBrettMac\/status\/787430379020029953\/photo\/1","type":"photo","sizes":{"small":{"w":494,"h":680,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":871,"h":1200,"resize":"fit"},"large":{"w":1440,"h":1984,"resize":"fit"}}}]}},"retweet_count":895,"favorite_count":769,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/urJILUlPfj","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787430379020029953","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[116,139]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"","expanded_url":null,"indices":[135,135]}],"user_mentions":[{"screen_name":"TweetBrettMac","name":"Brett Mac","id":305211380,"id_str":"305211380","indices":[3,17]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419863"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985759919677440,"id_str":"787985759919677440","text":"RT @johnpender123: Fox News 10\/15\/16 | LATEST WIKILEAKS Revelations DEADLY For Hillary Clinton https:\/\/t.co\/3xLRKU7xfc via @YouTube","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":50831585,"id_str":"50831585","name":"john pender","screen_name":"johnpender123","location":"PDX-Vancouver","url":"https:\/\/www.roaddogfilms.com\/","description":"Bazooka Media, LLC  ~ Writer\/Producer ~ http:\/\/RoadDogFilms.com","protected":false,"verified":false,"followers_count":1557,"friends_count":1540,"listed_count":22,"favourites_count":11,"statuses_count":4689,"created_at":"Fri Jun 26 00:10:44 +0000 2009","utc_offset":-25200,"time_zone":"Pacific Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"000000","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/642416103537250304\/ycl_-BwU_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/642416103537250304\/ycl_-BwU_normal.jpg","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Sun Oct 16 19:07:11 +0000 2016","id":787731631775838208,"id_str":"787731631775838208","text":"Fox News 10\/15\/16 | LATEST WIKILEAKS Revelations DEADLY For Hillary Clinton https:\/\/t.co\/3xLRKU7xfc via @YouTube","source":"\u003ca href=\"http:\/\/linkis.com\" rel=\"nofollow\"\u003ePut your button on any page! \u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":50831585,"id_str":"50831585","name":"john pender","screen_name":"johnpender123","location":"PDX-Vancouver","url":"https:\/\/www.roaddogfilms.com\/","description":"Bazooka Media, LLC  ~ Writer\/Producer ~ http:\/\/RoadDogFilms.com","protected":false,"verified":false,"followers_count":1557,"friends_count":1540,"listed_count":22,"favourites_count":11,"statuses_count":4688,"created_at":"Fri Jun 26 00:10:44 +0000 2009","utc_offset":-25200,"time_zone":"Pacific Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"000000","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/642416103537250304\/ycl_-BwU_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/642416103537250304\/ycl_-BwU_normal.jpg","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":2,"favorite_count":1,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/3xLRKU7xfc","expanded_url":"http:\/\/ln.is\/www.youtube.com\/1XkCA","display_url":"ln.is\/www.youtube.co\u2026","indices":[76,99]}],"user_mentions":[{"screen_name":"YouTube","name":"YouTube","id":10228272,"id_str":"10228272","indices":[104,112]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/3xLRKU7xfc","expanded_url":"http:\/\/ln.is\/www.youtube.com\/1XkCA","display_url":"ln.is\/www.youtube.co\u2026","indices":[95,118]}],"user_mentions":[{"screen_name":"johnpender123","name":"john pender","id":50831585,"id_str":"50831585","indices":[3,17]},{"screen_name":"YouTube","name":"YouTube","id":10228272,"id_str":"10228272","indices":[123,131]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419889"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985760070709249,"id_str":"787985760070709249","text":"Email hacks reveal Clinton campaign's transparency problems: Clinton campaign chairman John Podesta's hacked ... https:\/\/t.co\/hXoWKvRXUD","source":"\u003ca href=\"http:\/\/twitterfeed.com\" rel=\"nofollow\"\u003etwitterfeed\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1727261088,"id_str":"1727261088","name":"US National News","screen_name":"USANationalNews","location":"United States","url":null,"description":"News for the United States. Android app available at http:\/\/bit.ly\/usnationalnews","protected":false,"verified":false,"followers_count":399,"friends_count":26,"listed_count":53,"favourites_count":5,"statuses_count":124927,"created_at":"Wed Sep 04 02:18:21 +0000 2013","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C6E2EE","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme2\/bg.gif","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme2\/bg.gif","profile_background_tile":false,"profile_link_color":"1F98C7","profile_sidebar_border_color":"C6E2EE","profile_sidebar_fill_color":"DAECF4","profile_text_color":"663B12","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/378800000412005745\/5ebdff1181a3f1b9a5877cd44bb8dba4_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/378800000412005745\/5ebdff1181a3f1b9a5877cd44bb8dba4_normal.png","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/hXoWKvRXUD","expanded_url":"http:\/\/cbsn.ws\/2dYAYzQ","display_url":"cbsn.ws\/2dYAYzQ","indices":[113,136]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419925"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985760003600384,"id_str":"787985760003600384","text":"@HfxAnna I have begun to hate everyday Americans - Hillary Clinton  (guess that is you)","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":2863785729,"in_reply_to_user_id_str":"2863785729","in_reply_to_screen_name":"HfxAnna","user":{"id":35735966,"id_str":"35735966","name":"Ken Hoag","screen_name":"35start","location":null,"url":null,"description":null,"protected":false,"verified":false,"followers_count":372,"friends_count":385,"listed_count":24,"favourites_count":301,"statuses_count":17713,"created_at":"Mon Apr 27 13:06:17 +0000 2009","utc_offset":null,"time_zone":null,"geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/2947900299\/5d2895ccfe93c37f436e4547c7654b6f_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/2947900299\/5d2895ccfe93c37f436e4547c7654b6f_normal.png","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"HfxAnna","name":"Anna","id":2863785729,"id_str":"2863785729","indices":[0,8]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419909"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985759936454656,"id_str":"787985759936454656","text":"RT @StephenKing: If you're a man whose wife or daughters have been sexually harassed, and you're still planning to vote for Donald Trump, s\u2026","source":"\u003ca href=\"http:\/\/www.tweetcaster.com\" rel=\"nofollow\"\u003eTweetCaster for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":17375008,"id_str":"17375008","name":"MJ","screen_name":"ItsMJ254","location":null,"url":null,"description":"Life lessons \u2022 Sports \u2022 Humor \u2022 Black Lives Matter.","protected":false,"verified":false,"followers_count":4686,"friends_count":413,"listed_count":87,"favourites_count":1374,"statuses_count":304804,"created_at":"Thu Nov 13 23:02:19 +0000 2008","utc_offset":-18000,"time_zone":"America\/Chicago","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/783631593776648192\/oPXthem4_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/783631593776648192\/oPXthem4_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/17375008\/1439383857","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:26:06 +0000 2016","id":787977986486263808,"id_str":"787977986486263808","text":"If you're a man whose wife or daughters have been sexually harassed, and you're still planning to vote for Donald Trump, shame on you.","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":2233154425,"id_str":"2233154425","name":"Stephen King","screen_name":"StephenKing","location":null,"url":"http:\/\/stephenking.com","description":"Author","protected":false,"verified":true,"followers_count":2441166,"friends_count":47,"listed_count":9736,"favourites_count":4,"statuses_count":1344,"created_at":"Fri Dec 06 15:26:35 +0000 2013","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/378800000836981162\/b683f7509ec792c3e481ead332940cdc_normal.jpeg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/378800000836981162\/b683f7509ec792c3e481ead332940cdc_normal.jpeg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":1038,"favorite_count":2416,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"StephenKing","name":"Stephen King","id":2233154425,"id_str":"2233154425","indices":[3,15]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419893"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985759810756608,"id_str":"787985759810756608","text":"RT @Robbonixon: Thankless job Trumpsplaining. https:\/\/t.co\/RiFGAznXL0","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":760080250617335808,"id_str":"760080250617335808","name":"The Independent","screen_name":"thinkerdamous","location":"Multiple Citizenship","url":null,"description":"Healthy 2 party political system is important for democracy. People before technology. When we change spiritually, success follows. DONT ADD TO LISTS","protected":false,"verified":false,"followers_count":828,"friends_count":2526,"listed_count":8,"favourites_count":3766,"statuses_count":5734,"created_at":"Mon Aug 01 11:50:28 +0000 2016","utc_offset":null,"time_zone":null,"geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"F5F8FA","profile_background_image_url":"","profile_background_image_url_https":"","profile_background_tile":false,"profile_link_color":"2B7BB9","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/787302806290063360\/bSXGtB5o_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/787302806290063360\/bSXGtB5o_normal.jpg","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:34:31 +0000 2016","id":787980103200366592,"id_str":"787980103200366592","text":"Thankless job Trumpsplaining. https:\/\/t.co\/RiFGAznXL0","display_text_range":[0,29],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":23892889,"id_str":"23892889","name":"Laura Peterson","screen_name":"Robbonixon","location":"Farmington, MN","url":null,"description":"dropped out, dropped acid, drops f-bombs, drops the ball...raining on parades since birth i remain!","protected":false,"verified":false,"followers_count":1744,"friends_count":2632,"listed_count":90,"favourites_count":23835,"statuses_count":45112,"created_at":"Thu Mar 12 04:06:24 +0000 2009","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"ABB8C2","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/775195866709626880\/H93-Hq48_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/775195866709626880\/H93-Hq48_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/23892889\/1473153954","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"quoted_status_id":787974213764251648,"quoted_status_id_str":"787974213764251648","quoted_status":{"created_at":"Mon Oct 17 11:11:07 +0000 2016","id":787974213764251648,"id_str":"787974213764251648","text":"Trump stomps all over Mike Pence's efforts to clean up \"rigged election\" comments https:\/\/t.co\/1nPxSNamWw https:\/\/t.co\/vj1TnZQNeo","display_text_range":[0,105],"source":"\u003ca href=\"http:\/\/www.socialflow.com\" rel=\"nofollow\"\u003eSocialFlow\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":15458694,"id_str":"15458694","name":"HuffPost Politics","screen_name":"HuffPostPol","location":"Washington, D.C.","url":"http:\/\/www.huffingtonpost.com\/politics","description":"The latest political news from The Huffington Post's politics team.","protected":false,"verified":true,"followers_count":941734,"friends_count":9052,"listed_count":15990,"favourites_count":110,"statuses_count":259490,"created_at":"Wed Jul 16 19:45:46 +0000 2008","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"04508E","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/565873048\/huffpostpol.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/565873048\/huffpostpol.jpg","profile_background_tile":false,"profile_link_color":"0000FF","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"FFFFFF","profile_text_color":"000000","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/696718733721542656\/2lsNdbhH_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/696718733721542656\/2lsNdbhH_normal.png","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/15458694\/1426905134","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":31,"favorite_count":31,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/1nPxSNamWw","expanded_url":"http:\/\/huff.to\/2dWWFQN","display_url":"huff.to\/2dWWFQN","indices":[82,105]}],"user_mentions":[],"symbols":[],"media":[{"id":787974211411279872,"id_str":"787974211411279872","indices":[106,129],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu9yXtNWgAAczFQ.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu9yXtNWgAAczFQ.jpg","url":"https:\/\/t.co\/vj1TnZQNeo","display_url":"pic.twitter.com\/vj1TnZQNeo","expanded_url":"https:\/\/twitter.com\/HuffPostPol\/status\/787974213764251648\/photo\/1","type":"photo","sizes":{"large":{"w":2000,"h":1000,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":680,"h":340,"resize":"fit"},"medium":{"w":1200,"h":600,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787974211411279872,"id_str":"787974211411279872","indices":[106,129],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu9yXtNWgAAczFQ.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu9yXtNWgAAczFQ.jpg","url":"https:\/\/t.co\/vj1TnZQNeo","display_url":"pic.twitter.com\/vj1TnZQNeo","expanded_url":"https:\/\/twitter.com\/HuffPostPol\/status\/787974213764251648\/photo\/1","type":"photo","sizes":{"large":{"w":2000,"h":1000,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":680,"h":340,"resize":"fit"},"medium":{"w":1200,"h":600,"resize":"fit"}}}]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":2,"favorite_count":2,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/RiFGAznXL0","expanded_url":"https:\/\/twitter.com\/huffpostpol\/status\/787974213764251648","display_url":"twitter.com\/huffpostpol\/st\u2026","indices":[30,53]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"quoted_status_id":787974213764251648,"quoted_status_id_str":"787974213764251648","quoted_status":{"created_at":"Mon Oct 17 11:11:07 +0000 2016","id":787974213764251648,"id_str":"787974213764251648","text":"Trump stomps all over Mike Pence's efforts to clean up \"rigged election\" comments https:\/\/t.co\/1nPxSNamWw https:\/\/t.co\/vj1TnZQNeo","display_text_range":[0,105],"source":"\u003ca href=\"http:\/\/www.socialflow.com\" rel=\"nofollow\"\u003eSocialFlow\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":15458694,"id_str":"15458694","name":"HuffPost Politics","screen_name":"HuffPostPol","location":"Washington, D.C.","url":"http:\/\/www.huffingtonpost.com\/politics","description":"The latest political news from The Huffington Post's politics team.","protected":false,"verified":true,"followers_count":941734,"friends_count":9052,"listed_count":15990,"favourites_count":110,"statuses_count":259490,"created_at":"Wed Jul 16 19:45:46 +0000 2008","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"04508E","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/565873048\/huffpostpol.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/565873048\/huffpostpol.jpg","profile_background_tile":false,"profile_link_color":"0000FF","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"FFFFFF","profile_text_color":"000000","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/696718733721542656\/2lsNdbhH_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/696718733721542656\/2lsNdbhH_normal.png","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/15458694\/1426905134","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":31,"favorite_count":31,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/1nPxSNamWw","expanded_url":"http:\/\/huff.to\/2dWWFQN","display_url":"huff.to\/2dWWFQN","indices":[82,105]}],"user_mentions":[],"symbols":[],"media":[{"id":787974211411279872,"id_str":"787974211411279872","indices":[106,129],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu9yXtNWgAAczFQ.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu9yXtNWgAAczFQ.jpg","url":"https:\/\/t.co\/vj1TnZQNeo","display_url":"pic.twitter.com\/vj1TnZQNeo","expanded_url":"https:\/\/twitter.com\/HuffPostPol\/status\/787974213764251648\/photo\/1","type":"photo","sizes":{"large":{"w":2000,"h":1000,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":680,"h":340,"resize":"fit"},"medium":{"w":1200,"h":600,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787974211411279872,"id_str":"787974211411279872","indices":[106,129],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu9yXtNWgAAczFQ.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu9yXtNWgAAczFQ.jpg","url":"https:\/\/t.co\/vj1TnZQNeo","display_url":"pic.twitter.com\/vj1TnZQNeo","expanded_url":"https:\/\/twitter.com\/HuffPostPol\/status\/787974213764251648\/photo\/1","type":"photo","sizes":{"large":{"w":2000,"h":1000,"resize":"fit"},"thumb":{"w":150,"h":150,"resize":"crop"},"small":{"w":680,"h":340,"resize":"fit"},"medium":{"w":1200,"h":600,"resize":"fit"}}}]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/RiFGAznXL0","expanded_url":"https:\/\/twitter.com\/huffpostpol\/status\/787974213764251648","display_url":"twitter.com\/huffpostpol\/st\u2026","indices":[46,69]}],"user_mentions":[{"screen_name":"Robbonixon","name":"Laura Peterson","id":23892889,"id_str":"23892889","indices":[3,14]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419863"}

{"created_at":"Mon Oct 17 11:57:00 +0000 2016","id":787985760536309764,"id_str":"787985760536309764","text":"RT @Sherman_Alexie: Trump &amp; the Dakota Pipeline have turned my previously apolitical Native relatives into leftist activists! Awesome!","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":469034848,"id_str":"469034848","name":"Steve Langhorst","screen_name":"SteveLanghorst","location":"StL","url":null,"description":null,"protected":false,"verified":false,"followers_count":51,"friends_count":70,"listed_count":27,"favourites_count":2033,"statuses_count":1399,"created_at":"Fri Jan 20 04:59:11 +0000 2012","utc_offset":null,"time_zone":null,"geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_2_normal.png","profile_image_url_https":"https:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_2_normal.png","default_profile":true,"default_profile_image":true,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Sun Oct 16 21:09:46 +0000 2016","id":787762483998003200,"id_str":"787762483998003200","text":"Trump &amp; the Dakota Pipeline have turned my previously apolitical Native relatives into leftist activists! Awesome!","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":321003525,"id_str":"321003525","name":"Sherman Alexie","screen_name":"Sherman_Alexie","location":null,"url":"http:\/\/fallsapart.com","description":"National Book Award winning author of The Absolutely True Diary of a Part-Time Indian and upcoming picture book, Thunder Boy Jr.","protected":false,"verified":true,"followers_count":109359,"friends_count":86,"listed_count":2353,"favourites_count":0,"statuses_count":3583,"created_at":"Mon Jun 20 21:36:31 +0000 2011","utc_offset":-25200,"time_zone":"Pacific Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"1A1B1F","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/725737413163646977\/VRoyviRM.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/725737413163646977\/VRoyviRM.jpg","profile_background_tile":false,"profile_link_color":"FF691F","profile_sidebar_border_color":"FFFFFF","profile_sidebar_fill_color":"252429","profile_text_color":"666666","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/378800000530919364\/4eee066b0af8f853dca75dfcc6bc7178_normal.jpeg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/378800000530919364\/4eee066b0af8f853dca75dfcc6bc7178_normal.jpeg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/321003525\/1462195961","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":137,"favorite_count":898,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"Sherman_Alexie","name":"Sherman Alexie","id":321003525,"id_str":"321003525","indices":[3,18]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705420036"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985760238592000,"id_str":"787985760238592000","text":"@RapefugeeWatch  ..because identifying others illegal and criminal activities does not serve Clinton well.","display_text_range":[17,106],"source":"\u003ca href=\"https:\/\/mobile.twitter.com\" rel=\"nofollow\"\u003eMobile Web (M5)\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":787978063057334273,"in_reply_to_status_id_str":"787978063057334273","in_reply_to_user_id":1116292664,"in_reply_to_user_id_str":"1116292664","in_reply_to_screen_name":"RapefugeeWatch","user":{"id":2865679515,"id_str":"2865679515","name":"Deplorable Dumas","screen_name":"PebblesDunham","location":"  .  .  #TrumpPence2016  .  .   #TrumpTrain  .  .  #Rubio4Senate2016  .  .  '2017 The End of The Obama Error'   . .  #DeleteHillary","url":"http:\/\/Deplorablelivesmatter.org","description":"ObamaNation:  abom\u00b7i\u00b7na\u00b7tion  noun \\\u0259-\u02ccb\u00e4-m\u0259-\u02c8n\u0101-sh\u0259n\\    : something that causes disgust or hatred.","protected":false,"verified":false,"followers_count":616,"friends_count":609,"listed_count":3,"favourites_count":11073,"statuses_count":4977,"created_at":"Fri Nov 07 14:38:26 +0000 2014","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"1B95E0","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/777338838209429504\/LwwVYO2h_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/777338838209429504\/LwwVYO2h_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/2865679515\/1462661833","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"RapefugeeWatch","name":"Rapefugee Watch","id":1116292664,"id_str":"1116292664","indices":[0,15]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419965"}

{"created_at":"Mon Oct 17 11:57:00 +0000 2016","id":787985760909725696,"id_str":"787985760909725696","text":"RT @maytennis: .@morning_joe:@Markhalperin:@morningmika:M\/J have ignored that Trump said he couldn't have sexually assaulted these women b\/\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":707626622900371456,"id_str":"707626622900371456","name":"Susie16","screen_name":"susieq201688","location":"Winston-Salem, NC","url":null,"description":null,"protected":false,"verified":false,"followers_count":362,"friends_count":72,"listed_count":54,"favourites_count":16189,"statuses_count":28213,"created_at":"Wed Mar 09 17:58:28 +0000 2016","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"F5F8FA","profile_background_image_url":"","profile_background_image_url_https":"","profile_background_tile":false,"profile_link_color":"2B7BB9","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_2_normal.png","profile_image_url_https":"https:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_2_normal.png","default_profile":true,"default_profile_image":true,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:30:14 +0000 2016","id":787979024387739648,"id_str":"787979024387739648","text":".@morning_joe:@Markhalperin:@morningmika:M\/J have ignored that Trump said he couldn't have sexually assaulted these women b\/c they're ugly!","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":16906056,"id_str":"16906056","name":"maytennis","screen_name":"maytennis","location":"NJ","url":null,"description":"Politics junkie. Despise untruth in politics because it destroys our Democracy. Love to help people.","protected":false,"verified":false,"followers_count":148,"friends_count":6,"listed_count":13,"favourites_count":214,"statuses_count":61485,"created_at":"Wed Oct 22 14:27:53 +0000 2008","utc_offset":-18000,"time_zone":"Central Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_6_normal.png","profile_image_url_https":"https:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_6_normal.png","default_profile":true,"default_profile_image":true,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":1,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"Morning_Joe","name":"Morning Joe","id":254117355,"id_str":"254117355","indices":[1,13]},{"screen_name":"MarkHalperin","name":"Mark Halperin","id":184136149,"id_str":"184136149","indices":[14,27]},{"screen_name":"morningmika","name":"Mika Brzezinski","id":18227519,"id_str":"18227519","indices":[28,40]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"maytennis","name":"maytennis","id":16906056,"id_str":"16906056","indices":[3,13]},{"screen_name":"Morning_Joe","name":"Morning Joe","id":254117355,"id_str":"254117355","indices":[16,28]},{"screen_name":"MarkHalperin","name":"Mark Halperin","id":184136149,"id_str":"184136149","indices":[29,42]},{"screen_name":"morningmika","name":"Mika Brzezinski","id":18227519,"id_str":"18227519","indices":[43,55]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705420125"}

{"created_at":"Mon Oct 17 11:57:00 +0000 2016","id":787985761027002368,"id_str":"787985761027002368","text":"RT @DeanSchanzel: BREAKING DOJ - Obama Cover Up Mass Hillary Clinton Arms Sales that went to our enemies in Syria &amp; to Libya\n https:\/\/t.co\/\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":4270517292,"id_str":"4270517292","name":"Scooter\ud83d\udc38","screen_name":"BobbyAnnF","location":"Minnesota","url":null,"description":"Activist, Proud American, Independent Thinker.  Love God, my husband, my kids and my country. On the Trump Train. Porn-blocked.       I can't stand stupid!\ud83d\ude15","protected":false,"verified":false,"followers_count":1472,"friends_count":1622,"listed_count":7,"favourites_count":26537,"statuses_count":29223,"created_at":"Tue Nov 17 18:53:39 +0000 2015","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/764882489873530880\/mcTBQbZ5_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/764882489873530880\/mcTBQbZ5_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/4270517292\/1470653869","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Sun Oct 09 10:00:01 +0000 2016","id":785057219771502592,"id_str":"785057219771502592","text":"BREAKING DOJ - Obama Cover Up Mass Hillary Clinton Arms Sales that went to our enemies in Syria &amp; to Libya\n https:\/\/t.co\/Ops6ZFhLgS","source":"\u003ca href=\"http:\/\/twitter.com\/#!\/download\/ipad\" rel=\"nofollow\"\u003eTwitter for iPad\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":705333567543504896,"id_str":"705333567543504896","name":"Dean","screen_name":"DeanSchanzel","location":"Chardon, Ohio","url":"https:\/\/www.donaldjtrump.com\/policies\/","description":"Retired: NCOIC, Officer Personnel, Ft.Bliss,Tx; Personnel Director, City of Cleveland; Republican County Chairman; Network Management, Ohio Lottery. VOTED TRUMP","protected":false,"verified":false,"followers_count":5719,"friends_count":5006,"listed_count":15,"favourites_count":8694,"statuses_count":40742,"created_at":"Thu Mar 03 10:06:41 +0000 2016","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"F5F8FA","profile_background_image_url":"","profile_background_image_url_https":"","profile_background_tile":false,"profile_link_color":"2B7BB9","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/781901284995637248\/xEwdd9CK_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/781901284995637248\/xEwdd9CK_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/705333567543504896\/1473087423","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":1653,"favorite_count":1105,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[],"media":[{"id":783801259996217344,"id_str":"783801259996217344","indices":[112,135],"media_url":"http:\/\/pbs.twimg.com\/ext_tw_video_thumb\/783801259996217344\/pu\/img\/ED2zgBvioze0fGfr.jpg","media_url_https":"https:\/\/pbs.twimg.com\/ext_tw_video_thumb\/783801259996217344\/pu\/img\/ED2zgBvioze0fGfr.jpg","url":"https:\/\/t.co\/Ops6ZFhLgS","display_url":"pic.twitter.com\/Ops6ZFhLgS","expanded_url":"https:\/\/twitter.com\/Darren32895836\/status\/783801430394085376\/video\/1","type":"photo","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":600,"h":338,"resize":"fit"},"small":{"w":340,"h":191,"resize":"fit"},"large":{"w":1024,"h":576,"resize":"fit"}},"source_status_id":783801430394085376,"source_status_id_str":"783801430394085376","source_user_id":4761064634,"source_user_id_str":"4761064634"}]},"extended_entities":{"media":[{"id":783801259996217344,"id_str":"783801259996217344","indices":[112,135],"media_url":"http:\/\/pbs.twimg.com\/ext_tw_video_thumb\/783801259996217344\/pu\/img\/ED2zgBvioze0fGfr.jpg","media_url_https":"https:\/\/pbs.twimg.com\/ext_tw_video_thumb\/783801259996217344\/pu\/img\/ED2zgBvioze0fGfr.jpg","url":"https:\/\/t.co\/Ops6ZFhLgS","display_url":"pic.twitter.com\/Ops6ZFhLgS","expanded_url":"https:\/\/twitter.com\/Darren32895836\/status\/783801430394085376\/video\/1","type":"video","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":600,"h":338,"resize":"fit"},"small":{"w":340,"h":191,"resize":"fit"},"large":{"w":1024,"h":576,"resize":"fit"}},"source_status_id":783801430394085376,"source_status_id_str":"783801430394085376","source_user_id":4761064634,"source_user_id_str":"4761064634","video_info":{"aspect_ratio":[16,9],"duration_millis":64688,"variants":[{"bitrate":832000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/ext_tw_video\/783801259996217344\/pu\/vid\/640x360\/394OJr0hl1IN6Scf.mp4"},{"bitrate":320000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/ext_tw_video\/783801259996217344\/pu\/vid\/320x180\/5Sq8tS8R7K3cEr_c.mp4"},{"content_type":"application\/dash+xml","url":"https:\/\/video.twimg.com\/ext_tw_video\/783801259996217344\/pu\/pl\/6fuVuXsk998MyiRY.mpd"},{"content_type":"application\/x-mpegURL","url":"https:\/\/video.twimg.com\/ext_tw_video\/783801259996217344\/pu\/pl\/6fuVuXsk998MyiRY.m3u8"},{"bitrate":2176000,"content_type":"video\/mp4","url":"https:\/\/video.twimg.com\/ext_tw_video\/783801259996217344\/pu\/vid\/1280x720\/aF2061M4ffSS8wSk.mp4"}]}}]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"DeanSchanzel","name":"Dean","id":705333567543504896,"id_str":"705333567543504896","indices":[3,16]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705420153"}

{"created_at":"Mon Oct 17 11:57:00 +0000 2016","id":787985761102471168,"id_str":"787985761102471168","text":"@mitchellvii This is correct. Now is the time to turn up the heat If your undecided ask yourself who can create a better\/safer economy.Trump","display_text_range":[13,140],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":787984232945750016,"in_reply_to_status_id_str":"787984232945750016","in_reply_to_user_id":17980523,"in_reply_to_user_id_str":"17980523","in_reply_to_screen_name":"mitchellvii","user":{"id":3434919779,"id_str":"3434919779","name":"Stewart M. Scott","screen_name":"StewartMScott52","location":"Georgia, USA","url":null,"description":"Love my Family and Friends. Enjoy fishing and enjoy things that God made in our outdoors. Firefighter \/ Business owner\/ Sports Trump For President 2016","protected":false,"verified":false,"followers_count":154,"friends_count":178,"listed_count":2,"favourites_count":6,"statuses_count":1564,"created_at":"Sat Aug 22 01:15:03 +0000 2015","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/655945345055223808\/03ILUo7h_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/655945345055223808\/03ILUo7h_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/3434919779\/1445224530","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"mitchellvii","name":"Bill Mitchell","id":17980523,"id_str":"17980523","indices":[0,12]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705420171"}

{"created_at":"Mon Oct 17 11:56:59 +0000 2016","id":787985759668170753,"id_str":"787985759668170753","text":"Poll: Trump supporters like communist Russia like their candidate does https:\/\/t.co\/kmAkwE7Fyz https:\/\/t.co\/uuyQUJ8Nhd","display_text_range":[0,94],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":15172502,"id_str":"15172502","name":"Eric","screen_name":"ramseur","location":"New York, NY","url":"http:\/\/www.anant.us","description":"Web Startup Founder & CTO at @AnantCorp. These views are my own! I proudly have New York City Moderate Republican Values. #RepublicansForHillary","protected":false,"verified":false,"followers_count":2757,"friends_count":1675,"listed_count":145,"favourites_count":3258,"statuses_count":134272,"created_at":"Thu Jun 19 19:33:09 +0000 2008","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"352726","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/149855144\/bk.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/149855144\/bk.jpeg","profile_background_tile":true,"profile_link_color":"D02B55","profile_sidebar_border_color":"829D5E","profile_sidebar_fill_color":"99CC33","profile_text_color":"3E4415","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/2797636677\/449246ccb93f127720162730f8b2a136_normal.jpeg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/2797636677\/449246ccb93f127720162730f8b2a136_normal.jpeg","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/kmAkwE7Fyz","expanded_url":"http:\/\/politi.co\/2dlvyei","display_url":"politi.co\/2dlvyei","indices":[71,94]}],"user_mentions":[],"symbols":[],"media":[{"id":787985756644073472,"id_str":"787985756644073472","indices":[95,118],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu983ujWYAA346x.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu983ujWYAA346x.jpg","url":"https:\/\/t.co\/uuyQUJ8Nhd","display_url":"pic.twitter.com\/uuyQUJ8Nhd","expanded_url":"https:\/\/twitter.com\/ramseur\/status\/787985759668170753\/photo\/1","type":"photo","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":1024,"h":555,"resize":"fit"},"large":{"w":1024,"h":555,"resize":"fit"},"small":{"w":680,"h":369,"resize":"fit"}}}]},"extended_entities":{"media":[{"id":787985756644073472,"id_str":"787985756644073472","indices":[95,118],"media_url":"http:\/\/pbs.twimg.com\/media\/Cu983ujWYAA346x.jpg","media_url_https":"https:\/\/pbs.twimg.com\/media\/Cu983ujWYAA346x.jpg","url":"https:\/\/t.co\/uuyQUJ8Nhd","display_url":"pic.twitter.com\/uuyQUJ8Nhd","expanded_url":"https:\/\/twitter.com\/ramseur\/status\/787985759668170753\/photo\/1","type":"photo","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":1024,"h":555,"resize":"fit"},"large":{"w":1024,"h":555,"resize":"fit"},"small":{"w":680,"h":369,"resize":"fit"}}}]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705419829"}

{"created_at":"Mon Oct 17 11:57:00 +0000 2016","id":787985761232646144,"id_str":"787985761232646144","text":"RT @theryangeorge: Wait so\nTRUMP: i just kiss women, i don't even ask\n12 WOMEN: Trump kissed me without asking\nTRUMP: that's insane, you're\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":331208078,"id_str":"331208078","name":"Mary H","screen_name":"MARYlaand_","location":"md de","url":"http:\/\/instagram.com\/maryxhealey","description":"Bernie Sanders","protected":false,"verified":false,"followers_count":23836,"friends_count":499,"listed_count":44,"favourites_count":8257,"statuses_count":25832,"created_at":"Thu Jul 07 20:23:57 +0000 2011","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/553717814217818112\/lDf3jQLn.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/553717814217818112\/lDf3jQLn.jpeg","profile_background_tile":true,"profile_link_color":"000000","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/783475833902170112\/rrKCDWEm_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/783475833902170112\/rrKCDWEm_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/331208078\/1474422862","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Thu Oct 13 22:29:27 +0000 2016","id":786695373062569985,"id_str":"786695373062569985","text":"Wait so\nTRUMP: i just kiss women, i don't even ask\n12 WOMEN: Trump kissed me without asking\nTRUMP: that's insane, you're insane","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":17923412,"id_str":"17923412","name":"Ryan George","screen_name":"theryangeorge","location":"Montreal","url":"http:\/\/youtube.com\/movingmindstudio","description":"Host\/Producer at @screenrant, Comedian at @MovingMind. It's pronounced Rrr (the yan is silent)","protected":false,"verified":false,"followers_count":1124,"friends_count":568,"listed_count":20,"favourites_count":3212,"statuses_count":2881,"created_at":"Sat Dec 06 15:51:08 +0000 2008","utc_offset":-18000,"time_zone":"Quito","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"FFFFFF","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/378800000025064783\/406257ae1005f5b36e20525276b3e072.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/378800000025064783\/406257ae1005f5b36e20525276b3e072.jpeg","profile_background_tile":true,"profile_link_color":"1B95E0","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"C0DFEC","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/777925537050361857\/3kisVslj_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/777925537050361857\/3kisVslj_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/17923412\/1460463698","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":1140,"favorite_count":1753,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[],"user_mentions":[{"screen_name":"theryangeorge","name":"Ryan George","id":17923412,"id_str":"17923412","indices":[3,17]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705420202"}

{"created_at":"Mon Oct 17 11:57:00 +0000 2016","id":787985760850935808,"id_str":"787985760850935808","text":"I liked a @YouTube video from @mattconyt https:\/\/t.co\/N2jkmDkb9L #22 | Blonde Doxed, Trump Accusers, Podesta Emails | Beauty &amp; the","source":"\u003ca href=\"http:\/\/www.google.com\/\" rel=\"nofollow\"\u003eGoogle\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1907604540,"id_str":"1907604540","name":"\ud83c\udf85IRIS\ud83c\udf85","screen_name":"IRISnGEN","location":"New York","url":"http:\/\/youtube.com\/c\/IRISnGEN","description":"I\u2019m actually not funny. I\u2019m just really mean and people think I am joking. Also here to avoid Facebook friends.","protected":false,"verified":false,"followers_count":240,"friends_count":82,"listed_count":15,"favourites_count":375,"statuses_count":13586,"created_at":"Thu Sep 26 11:48:10 +0000 2013","utc_offset":-14400,"time_zone":"America\/New_York","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/378800000082752810\/76df28cbc0cc69be06e79a1901552bed.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/378800000082752810\/76df28cbc0cc69be06e79a1901552bed.jpeg","profile_background_tile":true,"profile_link_color":"0084B4","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/737637584839528448\/9KAhqc9K_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/737637584839528448\/9KAhqc9K_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/1907604540\/1430838410","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/N2jkmDkb9L","expanded_url":"http:\/\/youtu.be\/S7oUL7-sqrE?a","display_url":"youtu.be\/S7oUL7-sqrE?a","indices":[41,64]}],"user_mentions":[{"screen_name":"YouTube","name":"YouTube","id":10228272,"id_str":"10228272","indices":[10,18]},{"screen_name":"MattConYT","name":"Matt Christiansen","id":451746511,"id_str":"451746511","indices":[30,40]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705420111"}

{"created_at":"Mon Oct 17 11:57:00 +0000 2016","id":787985760511111168,"id_str":"787985760511111168","text":"RT @Twiggy164: Welcome to the Banana Republic of America. https:\/\/t.co\/fDoSlarnOc","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":750766596797247488,"id_str":"750766596797247488","name":"Jen","screen_name":"beforethedawn16","location":"Canada","url":null,"description":"38. Fannibal. World-weary INTJ. Not a fan of bullshit. Sweary as fuck. Canadian. Longtime writing partner to @destinyawakened, and animal enthusiast.","protected":false,"verified":false,"followers_count":122,"friends_count":131,"listed_count":5,"favourites_count":4087,"statuses_count":4319,"created_at":"Wed Jul 06 19:01:20 +0000 2016","utc_offset":-25200,"time_zone":"Pacific Time (US & Canada)","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"000000","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"E81C4F","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"000000","profile_text_color":"000000","profile_use_background_image":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/750769070262226944\/RO6qnE3D_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/750769070262226944\/RO6qnE3D_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/750766596797247488\/1470783560","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 03:31:09 +0000 2016","id":787858462948532224,"id_str":"787858462948532224","text":"Welcome to the Banana Republic of America. https:\/\/t.co\/fDoSlarnOc","display_text_range":[0,42],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":910854372,"id_str":"910854372","name":"Twiggy164","screen_name":"Twiggy164","location":"The Loon Star State","url":null,"description":"1960s SDS activist in Daley's Chicago, marched w\/Black Panthers. Grew up in PR, fluent Spanish\/teach it\/love it! Love all creatures & this good earth.#UniteBlue","protected":false,"verified":false,"followers_count":1837,"friends_count":1804,"listed_count":54,"favourites_count":18595,"statuses_count":23032,"created_at":"Sun Oct 28 18:35:06 +0000 2012","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/624427544767393792\/sCMLBp5j_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/624427544767393792\/sCMLBp5j_normal.png","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/910854372\/1433987199","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"quoted_status_id":787817046218199040,"quoted_status_id_str":"787817046218199040","quoted_status":{"created_at":"Mon Oct 17 00:46:35 +0000 2016","id":787817046218199040,"id_str":"787817046218199040","text":"Trump acts like Hillary's undermining our legitimate democratic system (\"its rigged\") while threatening to jail his\u2026 https:\/\/t.co\/Q2eoNlJTKK","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":435564722,"id_str":"435564722","name":"Jon Riley","screen_name":"JonRiley7","location":"Austin, TX","url":"http:\/\/politicsisfunandgames.blogspot.com","description":"If you like political cartoons flip through my media tab. I always #FollowBack on principle. RT\u2260endorsement. #ImWithHer","protected":false,"verified":false,"followers_count":15153,"friends_count":16021,"listed_count":143,"favourites_count":12766,"statuses_count":14005,"created_at":"Tue Dec 13 06:04:01 +0000 2011","utc_offset":-21600,"time_zone":"Mountain Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"9AE4E8","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme16\/bg.gif","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme16\/bg.gif","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"BDDCAD","profile_sidebar_fill_color":"DDFFCC","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/782074979445673984\/c2afRbbY_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/782074979445673984\/c2afRbbY_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/435564722\/1464120764","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"quoted_status_id":787267564405653505,"quoted_status_id_str":"787267564405653505","is_quote_status":true,"extended_tweet":{"full_text":"Trump acts like Hillary's undermining our legitimate democratic system (\"its rigged\") while threatening to jail his opponent like a dictator https:\/\/t.co\/MMi8A0gi2a","display_text_range":[0,140],"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/MMi8A0gi2a","expanded_url":"https:\/\/twitter.com\/realdonaldtrump\/status\/787267564405653505","display_url":"twitter.com\/realdonaldtrum\u2026","indices":[141,164]}],"user_mentions":[],"symbols":[]}},"retweet_count":43,"favorite_count":66,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/Q2eoNlJTKK","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787817046218199040","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[117,140]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":5,"favorite_count":9,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/fDoSlarnOc","expanded_url":"https:\/\/twitter.com\/jonriley7\/status\/787817046218199040","display_url":"twitter.com\/jonriley7\/stat\u2026","indices":[43,66]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"quoted_status_id":787817046218199040,"quoted_status_id_str":"787817046218199040","quoted_status":{"created_at":"Mon Oct 17 00:46:35 +0000 2016","id":787817046218199040,"id_str":"787817046218199040","text":"Trump acts like Hillary's undermining our legitimate democratic system (\"its rigged\") while threatening to jail his\u2026 https:\/\/t.co\/Q2eoNlJTKK","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":435564722,"id_str":"435564722","name":"Jon Riley","screen_name":"JonRiley7","location":"Austin, TX","url":"http:\/\/politicsisfunandgames.blogspot.com","description":"If you like political cartoons flip through my media tab. I always #FollowBack on principle. RT\u2260endorsement. #ImWithHer","protected":false,"verified":false,"followers_count":15153,"friends_count":16021,"listed_count":143,"favourites_count":12766,"statuses_count":14005,"created_at":"Tue Dec 13 06:04:01 +0000 2011","utc_offset":-21600,"time_zone":"Mountain Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"9AE4E8","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme16\/bg.gif","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme16\/bg.gif","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"BDDCAD","profile_sidebar_fill_color":"DDFFCC","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/782074979445673984\/c2afRbbY_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/782074979445673984\/c2afRbbY_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/435564722\/1464120764","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"quoted_status_id":787267564405653505,"quoted_status_id_str":"787267564405653505","is_quote_status":true,"extended_tweet":{"full_text":"Trump acts like Hillary's undermining our legitimate democratic system (\"its rigged\") while threatening to jail his opponent like a dictator https:\/\/t.co\/MMi8A0gi2a","display_text_range":[0,140],"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/MMi8A0gi2a","expanded_url":"https:\/\/twitter.com\/realdonaldtrump\/status\/787267564405653505","display_url":"twitter.com\/realdonaldtrum\u2026","indices":[141,164]}],"user_mentions":[],"symbols":[]}},"retweet_count":43,"favorite_count":66,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/Q2eoNlJTKK","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787817046218199040","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[117,140]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/fDoSlarnOc","expanded_url":"https:\/\/twitter.com\/jonriley7\/status\/787817046218199040","display_url":"twitter.com\/jonriley7\/stat\u2026","indices":[58,81]}],"user_mentions":[{"screen_name":"Twiggy164","name":"Twiggy164","id":910854372,"id_str":"910854372","indices":[3,13]}],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705420030"}

{"created_at":"Mon Oct 17 11:57:00 +0000 2016","id":787985761488343041,"id_str":"787985761488343041","text":"RT @LouDobbs: Wikileaks docs have revealed and proved the vast corruption of the  Clinton Cartel: Media blackout, now this\u2026 ","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":3306160258,"id_str":"3306160258","name":"SailfishMD","screen_name":"sailfishmd","location":"Indiana, USA","url":null,"description":"Fiesty Conservative. Physician. Lover of chess, travel, gardening, wine, & fishing. Political junkie. #AllLivesMatter #NeverHillary #EndMOC","protected":false,"verified":false,"followers_count":369,"friends_count":1240,"listed_count":22,"favourites_count":8278,"statuses_count":9493,"created_at":"Mon Jun 01 23:10:55 +0000 2015","utc_offset":null,"time_zone":null,"geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/772111433232445440\/hDX8Wj1W_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/772111433232445440\/hDX8Wj1W_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/3306160258\/1452009102","default_profile":true,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"retweeted_status":{"created_at":"Mon Oct 17 11:19:50 +0000 2016","id":787976407972450304,"id_str":"787976407972450304","text":"Wikileaks docs have revealed and proved the vast corruption of the  Clinton Cartel: Media blackout, now this\u2026 https:\/\/t.co\/AiPdPMpAfu","display_text_range":[0,140],"source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":26487169,"id_str":"26487169","name":"Lou Dobbs","screen_name":"LouDobbs","location":"New York, NY","url":"http:\/\/loudobbs.com","description":"Lou Dobbs Tonight, Fox Business Network, 7 & 11 pm    \nIG: http:\/\/instagram.com\/loudobbstonight\/","protected":false,"verified":true,"followers_count":809360,"friends_count":2477,"listed_count":3116,"favourites_count":3574,"statuses_count":23721,"created_at":"Wed Mar 25 12:39:59 +0000 2009","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/575734940228190209\/iXHsvsM7.jpeg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/575734940228190209\/iXHsvsM7.jpeg","profile_background_tile":false,"profile_link_color":"0084B4","profile_sidebar_border_color":"000000","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/663851941571641344\/OqqkE56l_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/663851941571641344\/OqqkE56l_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/26487169\/1426100831","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"quoted_status_id":787889195507417088,"quoted_status_id_str":"787889195507417088","quoted_status":{"created_at":"Mon Oct 17 05:33:17 +0000 2016","id":787889195507417088,"id_str":"787889195507417088","text":"Julian Assange's internet link has been intentionally severed by a state party. We have activated the appropriate contingency plans.","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":16589206,"id_str":"16589206","name":"WikiLeaks","screen_name":"wikileaks","location":"Everywhere","url":"http:\/\/wikileaks.org","description":"We open governments. \nContact: https:\/\/wikileaks.org\/#submit\n\nPGP: A04C 5E09 ED02 B328 03EB 6116 93ED 732E 9231 8DBA","protected":false,"verified":true,"followers_count":3708046,"friends_count":7597,"listed_count":53987,"favourites_count":4,"statuses_count":44147,"created_at":"Sat Oct 04 06:41:05 +0000 2008","utc_offset":3600,"time_zone":"Dublin","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"9AE4E8","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/3147857\/WL_Hour_Glass.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/3147857\/WL_Hour_Glass.jpg","profile_background_tile":true,"profile_link_color":"0084B4","profile_sidebar_border_color":"BDDCAD","profile_sidebar_fill_color":"DDFFCC","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/512138307870785536\/Fe00yVS2_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/512138307870785536\/Fe00yVS2_normal.png","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/16589206\/1402313050","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":20573,"favorite_count":19890,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"extended_tweet":{"full_text":"Wikileaks docs have revealed and proved the vast corruption of the  Clinton Cartel: Media blackout, now this    #MAGA #TrumpPence 16 #Dobbs https:\/\/t.co\/CNLzNXFezN","display_text_range":[0,139],"entities":{"hashtags":[{"text":"MAGA","indices":[112,117]},{"text":"TrumpPence","indices":[118,129]},{"text":"Dobbs","indices":[133,139]}],"urls":[{"url":"https:\/\/t.co\/CNLzNXFezN","expanded_url":"https:\/\/twitter.com\/wikileaks\/status\/787889195507417088","display_url":"twitter.com\/wikileaks\/stat\u2026","indices":[140,163]}],"user_mentions":[],"symbols":[]}},"retweet_count":802,"favorite_count":839,"entities":{"hashtags":[],"urls":[{"url":"https:\/\/t.co\/AiPdPMpAfu","expanded_url":"https:\/\/twitter.com\/i\/web\/status\/787976407972450304","display_url":"twitter.com\/i\/web\/status\/7\u2026","indices":[110,133]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"en"},"quoted_status_id":787889195507417088,"quoted_status_id_str":"787889195507417088","quoted_status":{"created_at":"Mon Oct 17 05:33:17 +0000 2016","id":787889195507417088,"id_str":"787889195507417088","text":"Julian Assange's internet link has been intentionally severed by a state party. We have activated the appropriate contingency plans.","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":16589206,"id_str":"16589206","name":"WikiLeaks","screen_name":"wikileaks","location":"Everywhere","url":"http:\/\/wikileaks.org","description":"We open governments. \nContact: https:\/\/wikileaks.org\/#submit\n\nPGP: A04C 5E09 ED02 B328 03EB 6116 93ED 732E 9231 8DBA","protected":false,"verified":true,"followers_count":3708046,"friends_count":7597,"listed_count":53987,"favourites_count":4,"statuses_count":44147,"created_at":"Sat Oct 04 06:41:05 +0000 2008","utc_offset":3600,"time_zone":"Dublin","geo_enabled":false,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"9AE4E8","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/3147857\/WL_Hour_Glass.jpg","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/3147857\/WL_Hour_Glass.jpg","profile_background_tile":true,"profile_link_color":"0084B4","profile_sidebar_border_color":"BDDCAD","profile_sidebar_fill_color":"DDFFCC","profile_text_color":"333333","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/512138307870785536\/Fe00yVS2_normal.png","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/512138307870785536\/Fe00yVS2_normal.png","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/16589206\/1402313050","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":20573,"favorite_count":19890,"entities":{"hashtags":[],"urls":[],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en"},"is_quote_status":true,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[],"urls":[{"url":"","expanded_url":null,"indices":[124,124]}],"user_mentions":[{"screen_name":"LouDobbs","name":"Lou Dobbs","id":26487169,"id_str":"26487169","indices":[3,12]}],"symbols":[]},"favorited":false,"retweeted":false,"filter_level":"low","lang":"en","timestamp_ms":"1476705420263"}

{"created_at":"Mon Oct 17 11:57:00 +0000 2016","id":787985761320767492,"id_str":"787985761320767492","text":"Wie k\u00e4uflich ist Hillary #Clinton? | Was dieser Mann f\u00fcr 10 Millionen Dollar will https:\/\/t.co\/cD5WIoKR9u - Haim Saban","source":"\u003ca href=\"http:\/\/bufferapp.com\" rel=\"nofollow\"\u003eBuffer\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":40532269,"id_str":"40532269","name":"Flensburg online","screen_name":"FlensburgOnline","location":"Flensburg, Germany","url":"http:\/\/www.flensburg-online.de\/blog\/","description":"Zeitung war gestern. Internet ist heute. Bibel ist immer.\r\nhttp:\/\/www.flensburg-online.de\/impressum.html","protected":false,"verified":false,"followers_count":6209,"friends_count":6817,"listed_count":109,"favourites_count":0,"statuses_count":223113,"created_at":"Sat May 16 20:07:58 +0000 2009","utc_offset":7200,"time_zone":"Berlin","geo_enabled":false,"lang":"de","contributors_enabled":false,"is_translator":false,"profile_background_color":"FFFFFF","profile_background_image_url":"http:\/\/pbs.twimg.com\/profile_background_images\/23289230\/flensburg.gif","profile_background_image_url_https":"https:\/\/pbs.twimg.com\/profile_background_images\/23289230\/flensburg.gif","profile_background_tile":false,"profile_link_color":"0099B9","profile_sidebar_border_color":"F5F5F5","profile_sidebar_fill_color":"F5F5F5","profile_text_color":"3C3940","profile_use_background_image":true,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/1760237451\/flo-twitter-buchstaben_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/1760237451\/flo-twitter-buchstaben_normal.jpg","profile_banner_url":"https:\/\/pbs.twimg.com\/profile_banners\/40532269\/1465766815","default_profile":false,"default_profile_image":false,"following":null,"follow_request_sent":null,"notifications":null},"geo":null,"coordinates":null,"place":null,"contributors":null,"is_quote_status":false,"retweet_count":0,"favorite_count":0,"entities":{"hashtags":[{"text":"Clinton","indices":[25,33]}],"urls":[{"url":"https:\/\/t.co\/cD5WIoKR9u","expanded_url":"http:\/\/buff.ly\/2dVvnJv","display_url":"buff.ly\/2dVvnJv","indices":[82,105]}],"user_mentions":[],"symbols":[]},"favorited":false,"retweeted":false,"possibly_sensitive":false,"filter_level":"low","lang":"de","timestamp_ms":"1476705420223"}


Writing /content/dataset/twitter_sample_small_records.json


# Running serial test code

In [ ]:
!make clean

In [ ]:
# Create serial test exec
!make bin/serialMain1

In [ ]:
!bin/serialMain1

# Running parallel test code

In [ ]:
!make clean

rm -rf bin


In [ ]:
# Create parallel test exec
!make bin/parallelMain1

mkdir bin
g++ -O3 -madx -msse4.1 -march=native -pthread -o bin/parallelMain1 general/*.cpp serialSpec/*.cpp parallelSpec/*.cpp testing/parallelMain1.cpp -mcmodel=medium -static-libstdc++ -lpthread
parallelSpec/ParallelBitmapIterator.cpp: In member function ‘virtual char* ParallelBitmapIterator::getValue()’:
parallelSpec/ParallelBitmapIterator.cpp:496:20: warning: ISO C++ forbids converting a string constant to ‘char*’ [-Wwrite-strings]
  496 |             return "";
      |                    ^~
parallelSpec/ParallelBitmapIterator.cpp:502:34: warning: ISO C++ forbids converting a string constant to ‘char*’ [-Wwrite-strings]
  502 |     if (text_length <= 0) return "";
      |                                  ^~


In [ ]:
!bin/parallelMain1

Running on 12 threads
Duration: 440789


# Parallel Thread Testing results

Measured in microseconds (average of 3 runs)

- 2 Threads: ~708509
- 4 Threads: ~550000
- 8 Threads: ~445000
- 12 Threads: ~432000
- 16 Threads: ~445000